In [1]:
import pandas as pd
from pathlib import Path

pd.options.mode.copy_on_write = True

Объединение наземных данных за 2021-2023

In [2]:
logs = list(Path(".").rglob("*.log"))
df_lst_21_23 = pd.concat([pd.read_csv(log, sep="|") for log in logs]).drop(columns=["TF", "SurfTermo", "SurfTermoEqu", "ConductSBE", "ScSBE", "ScSBEequ", "WaterIR", "AirIR", "Speed", "Cours"])
df_lst_21_23 = df_lst_21_23.rename(columns={"DateTime" : "Time", "TermoSBE": "T"})
df_lst_21_23["T"] = df_lst_21_23['T'].astype(str).str.replace(',', '.')
df_lst_21_23["Lat"] = df_lst_21_23['Lat'].astype(str).str.replace(',', '.')
df_lst_21_23["Lon"] = df_lst_21_23['Lon'].astype(str).str.replace(',', '.')
df_lst_21_23 = df_lst_21_23[df_lst_21_23["Lat"].astype(float) != 0] # какие-то лишние измерения
df_lst_21_23.to_csv("Наземеные_измерения/ready/Temp21-23.csv", index=False)

KeyboardInterrupt: 

Объединение наземных данных за 2024

In [9]:
df_june = pd.read_csv('Наземеные_измерения/ready/TempJune.csv')
df_july = pd.read_csv('Наземеные_измерения/ready/TempJuly.csv')
df_sept = pd.read_csv('Наземеные_измерения/ready/TempSept.csv')
df_lst_24 = pd.concat([df_june, df_july, df_sept]).drop(columns="Speed")
old_format = "%d.%m.%Y %H:%M"
new_format = "%d.%m.%Y %H:%M:%S"
df_lst_24["Time"] = df_lst_24["Time"].apply(lambda dt: pd.to_datetime(dt, format=old_format))
df_lst_24["Time"] = df_lst_24["Time"].dt.strftime(new_format)
df_lst_24["T"] = df_lst_24['T'].astype(str).str.replace(',', '.')
df_lst_24["Lat"] = df_lst_24['Lat'].astype(str).str.replace(',', '.')
df_lst_24["Lon"] = df_lst_24['Lon'].astype(str).str.replace(',', '.')
df_lst_24.to_csv('Наземеные_измерения/ready/Temp24.csv', index=False)

Объединение данных за 2021-2024

In [ ]:
df_lst = pd.concat([df_lst_21_23, df_lst_24])
df_lst.to_csv('Наземеные_измерения/ready/Temp21-24.csv', index=False)

Сокращение данных, чтобы они шли не чаще чем с определённой частотой

In [3]:
def parse_dateTime(df: pd.DataFrame) -> pd.DataFrame:
   format = "%d.%m.%Y  %H:%M:%S"
   return df["Time"].apply(lambda dt: pd.to_datetime(dt, format=format))

df_lst = pd.read_csv('Наземеные_измерения/ready/Temp21-24.csv')
df_lst["Time"] = parse_dateTime(df_lst)
df_lst = df_lst.sort_values(by=['Time'])
df_lst

,Time,T,Lat,Lon
731173,2021-05-25 18:42:01,2.56830,51.896156,105.098007
731174,2021-05-25 18:42:06,2.58880,51.896130,105.098320
731175,2021-05-25 18:42:14,2.61300,51.896072,105.098824
731176,2021-05-25 18:42:19,2.60840,51.896034,105.099136
731177,2021-05-25 18:42:27,2.62660,51.895969,105.099632
...,...,...,...,...
1733140,2024-10-03 14:37:00,9.79160,51.847950,104.870730
1733141,2024-10-03 14:38:00,9.79175,51.847950,104.870730
1733142,2024-10-03 14:39:00,9.79290,51.847950,104.870730
1733143,2024-10-03 14:40:00,9.79430,51.847950,104.870730


In [5]:
def shorten_data(df: pd.DataFrame, limit_frequency_minutes: int) -> pd.DataFrame:
   margin = pd.Timedelta(limit_frequency_minutes, 'minutes')
   i = 0
   while i<df.shape[0]:
      time=df["Time"].iloc[i]
      start_dt = time - margin
      end_dt = time + margin
      df = df[(df["Time"] == time) | (df["Time"] <= start_dt) | (end_dt <= df["Time"])]
      i = i + 1;
   return df

df_lst_15 = shorten_data(df_lst, 15)
df_lst_1 = shorten_data(df_lst, 1)

In [6]:
df_lst_15.to_csv('Наземеные_измерения/ready/Temp21-24_cut15.csv', index=False)
df_lst_1.to_csv('Наземеные_измерения/ready/Temp21-24_cut1.csv', index=False)

# Фильтрация для валидации

## Объединение L7 с L8-9

In [277]:
landsat89_df = pd.read_csv(f'Данные_валидации/raw/landsat_validation1.csv')
landsat7_df = pd.read_csv(f'Данные_валидации/raw/landsat_only7_validation1.csv')

In [278]:
landsat_merged_df = pd.concat([landsat7_df, landsat89_df], ignore_index=True, sort=False)
landsat_merged_df

,Time,T,Lat,Lon,Image Time,Image T,Landsat
0,2021-09-23 10:35:16+08:00,9.571900,53.143623,107.849472,2021-09-23 02:36:07.773845500+00:00,8.438689,L7
1,2022-08-04 10:13:45+08:00,14.132600,55.648727,109.369957,2022-08-04 02:13:44.191168+00:00,16.610226,L7
2,2021-07-12 10:47:50+08:00,16.130700,53.470188,108.915337,2021-07-12 02:48:42.341146400+00:00,2.354050,L7
3,2021-07-28 10:47:17+08:00,8.693400,53.017338,107.355103,2021-07-28 02:47:19.296578200+00:00,9.390626,L7
4,2021-08-13 10:45:21+08:00,13.084100,53.156502,107.977829,2021-08-13 02:45:54.146524+00:00,13.417129,L7
5,2021-10-16 10:40:37+08:00,4.826700,51.885227,105.564148,2021-10-16 02:40:40.300719600+00:00,-3.038646,L7
6,2021-08-20 10:51:51+08:00,14.136200,51.610619,104.894226,2021-08-20 02:51:51.804602+00:00,13.868249,L7
7,2021-10-23 10:46:19+08:00,7.491000,51.809681,104.891777,2021-10-23 02:46:13.069633200+00:00,4.255274,L7
8,2021-07-26 10:59:37+08:00,14.771800,51.628082,104.103561,2021-07-26 03:00:15.278371400+00:00,14.642000,L7
9,2022-07-28 10:27:03+08:00,15.792900,51.566982,104.228798,2022-07-28 02:27:07.235563500+00:00,11.230639,L7


In [279]:
landsat_merged_df.to_csv('Данные_валидации/raw/landsat_with7_validation1.csv', index=False)
landsat_merged_df.to_csv('Данные_валидации/ready/landsat_with7_validation1.csv', index=False)

## Чтение сырых значений для валидации

In [2]:
landsat_file_name_base = 'landsat_validation1'
landsat_with7_file_name_base = 'landsat_with7_validation1'
modis_file_name_base = 'modis_validation15'
l2sp_file_name_base = 'ls2p_validation1'

In [3]:
landsat_df = pd.read_csv(f'Данные_валидации/raw/{landsat_file_name_base}.csv')
landsat_in_situ = landsat_df['T']
landsat_image = landsat_df['Image T']

landsat_with7_df = pd.read_csv(f'Данные_валидации/raw/{landsat_with7_file_name_base}.csv')
landsat_with7_in_situ = landsat_with7_df['T']
landsat_with7_image = landsat_with7_df['Image T']

l2sp_df = pd.read_csv(f'Данные_валидации/raw/{l2sp_file_name_base}.csv')
l2sp_in_situ = l2sp_df['T']
l2sp_image = l2sp_df['Image T']

modis_df = pd.read_csv(f'Данные_валидации/raw/{modis_file_name_base}.csv')
modis_in_situ = modis_df['T']
modis_image = modis_df['Image T']

mod11_day_df = pd.read_csv('Данные_валидации/raw/mod11_day.csv')
mod11_day_in_situ = mod11_day_df['T']
mod11_day_image = mod11_day_df['Image T']

mod11_night_df = pd.read_csv('Данные_валидации/raw/mod11_night.csv')
mod11_night_in_situ = mod11_night_df['T']
mod11_night_image = mod11_night_df['Image T']

mod21_day_df = pd.read_csv('Данные_валидации/raw/mod21_day.csv')
mod21_day_in_situ = mod21_day_df['T']
mod21_day_image = mod21_day_df['Image T']

mod21_night_df = pd.read_csv('Данные_валидации/raw/mod21_night.csv')
mod21_night_in_situ = mod21_night_df['T']
mod21_night_image = mod21_night_df['Image T']

## Добавление дистанции до берега

In [283]:
import ee
import geemap

In [284]:
geemap.ee_initialize()
oeel = geemap.requireJS()
ee.Authenticate()
ee.Initialize(project='ee-amazyar-test1')
baikal_shape = ee.FeatureCollection('projects/ee-amazyar-test/assets/baikal')

baikal_geometry = baikal_shape.geometry()
baikal_polylines = baikal_geometry.coordinates()
baikal_combined_points = []
for point_list in baikal_polylines.getInfo():
   for point in point_list:
      baikal_combined_points.append(ee.Geometry.Point(point[0], point[1]))
baikal_points = ee.Geometry.MultiPoint(baikal_combined_points)

In [285]:
def add_distance_column(df):
   point_distance = df[['Lon','Lat']].apply(lambda x: (ee.Geometry.Point(*x)).distance(baikal_points).getInfo(), axis=1)
   return df.assign(distance_to_shore_m=pd.Series(point_distance).values)

In [ ]:
landsat_df_with_distance = add_distance_column(landsat_df)
landsat_with7_df_with_distance = add_distance_column(landsat_with7_df)
modis_df_with_distance = add_distance_column(modis_df)

In [ ]:
landsat_df_with_distance.to_csv(f'Данные_валидации/raw/{landsat_file_name_base}_distance.csv', index=False)
landsat_with7_df_with_distance.to_csv(f'Данные_валидации/raw/{landsat_with7_file_name_base}_distance.csv', index=False)
modis_df_with_distance.to_csv(f'Данные_валидации/raw/{modis_file_name_base}_distance.csv', index=False)

## Применения фильтра Хампеля

In [4]:
from sktime.transformations.series.outlier_detection import HampelFilter

In [5]:
def apply_hampel(df, window_length, do_shuffle=False, n_sigma=3):
   """
   Применяет фильтр Хампеля, возвращает профильтрованный датафрейм, где записи идут в случайном порядке.
   """
   if(do_shuffle):
      df = df.sample(frac=1).reset_index(drop=True)
   transformer = HampelFilter(window_length=window_length, n_sigma=n_sigma)
   t_diff = df["Image T"] - df["T"]
   result_filter_outliers = transformer.fit_transform(t_diff)
   result = df[result_filter_outliers.notnull()]
   t_diff = abs(t_diff[result_filter_outliers.notnull()])
   print(f"Max diff: {t_diff.max()}")
   print(f"Max diff idx: {t_diff.idxmax()}")
   return result

In [7]:
# Значения длины окна равны числу записей, т.к. выбросы мелкие, а данных мало
print("L8-9")
landsat_df_filtered = apply_hampel(landsat_df, landsat_df.shape[0], n_sigma=1) # граница - 1 стандартное отклонение, т.к. мало данных, просто Хампель не справляется
print("L7-9")
landsat_with7_df_filtered = apply_hampel(landsat_with7_df, landsat_df.shape[0], n_sigma=2) # граница - 2 стандартное отклонение, т.к. мало данных, хоть и побольше с Landsat 7

print("L2SP")
l2sp_df_filtered = apply_hampel(l2sp_df, 10)

# Значения длины окна подобраны "на глаз", чтобы убрать самые крупные выбросы
modis_window_length = 13
print("MOD11 Day")
mod11_day_df_filtered = apply_hampel(mod11_day_df, modis_window_length)
print("MOD11 Night")
mod11_night_df_filtered = apply_hampel(mod11_night_df, modis_window_length)
print("MOD21 Day")
mod21_day_df_filtered = apply_hampel(mod21_day_df, modis_window_length, do_shuffle=True)
print("MOD21 Night")
mod21_night_df_filtered = apply_hampel(mod21_night_df, modis_window_length, do_shuffle=True)

L8-9
Max diff: 2.9054905227814025
Max diff idx: 17
L7-9
Max diff: 3.235725843807259
Max diff idx: 7
L2SP
Max diff: 1.517252700000011
Max diff idx: 13
MOD11 Day
Max diff: 6.359899999999966
Max diff idx: 46
MOD11 Night
Max diff: 4.659699999999998
Max diff idx: 62
MOD21 Day
Max diff: 6.146700000000006
Max diff idx: 152
MOD21 Night
Max diff: 5.8544000000000445
Max diff idx: 26


In [8]:
l2sp_df_filtered

,Time,T,Lat,Lon,Image Time,Image T,Landsat
0,2021-09-23 10:35:16+08:00,9.57190,53.143623,107.849472,2021-09-23 02:36:07.773000+00:00,8.978461,Landsat 7 Tier 1
3,2021-07-28 10:47:17+08:00,8.69340,53.017338,107.355103,2021-07-28 02:47:19.296000+00:00,8.366635,Landsat 7 Tier 1
4,2021-08-13 10:45:21+08:00,13.08410,53.156502,107.977829,2021-08-13 02:45:54.146000+00:00,11.661607,Landsat 7 Tier 1
5,2021-08-20 10:51:51+08:00,14.13620,51.610619,104.894226,2021-08-20 02:51:51.804000+00:00,13.910664,Landsat 7 Tier 1
6,2021-07-26 10:59:37+08:00,14.77180,51.628082,104.103561,2021-07-26 03:00:15.278000+00:00,13.415051,Landsat 7 Tier 1
7,2021-07-13 11:38:04+08:00,7.07820,55.332798,109.608627,2021-07-13 03:38:29.542000+00:00,7.505294,Landsat 8 Tier 1
8,2021-07-13 11:38:04+08:00,7.07820,55.332798,109.608627,2021-07-13 03:38:53.420000+00:00,7.512130,Landsat 8 Tier 1
9,2021-07-29 11:39:00+08:00,11.98860,53.443645,108.509331,2021-07-29 03:39:24.750000+00:00,12.772463,Landsat 8 Tier 1
10,2021-08-14 11:38:37+08:00,15.30800,53.481445,108.645866,2021-08-14 03:39:31.126000+00:00,16.122123,Landsat 8 Tier 1
12,2021-09-22 11:45:11+08:00,9.04400,52.911846,107.182930,2021-09-22 03:45:51.413000+00:00,9.142526,Landsat 8 Tier 1


In [254]:
landsat_df_filtered

,Time,T,Lat,Lon,Image Time,Image T,Landsat
0,2021-07-13 11:38:04+08:00,7.078200,55.332798,109.608627,2021-07-13 03:38:29.542497+00:00,9.714929,L8
1,2021-07-13 11:38:04+08:00,7.078200,55.332798,109.608627,2021-07-13 03:38:53.420828900+00:00,9.716510,L8
2,2021-07-29 11:39:00+08:00,11.988600,53.443645,108.509331,2021-07-29 03:39:24.750487+00:00,13.214574,L8
3,2021-08-14 11:38:37+08:00,15.308000,53.481445,108.645866,2021-08-14 03:39:31.126727+00:00,16.345016,L8
4,2024-09-30 11:44:00+08:00,8.116000,54.242330,108.813370,2024-09-30 03:44:58.004715+00:00,4.723470,L8
5,2021-09-22 11:45:11+08:00,9.044000,52.911846,107.182930,2021-09-22 03:45:51.413924900+00:00,8.886449,L8
7,2022-09-25 11:45:34+08:00,10.208600,53.195732,107.793343,2022-09-25 03:46:12.624971+00:00,8.252914,L8
8,2024-07-28 11:45:00+08:00,19.927840,52.449190,106.141990,2024-07-28 03:45:34.798499900+00:00,20.237132,L8
9,2022-06-12 11:51:29+08:00,3.179000,53.250637,107.276199,2022-06-12 03:51:52.862325+00:00,4.722633,L8
10,2021-08-12 11:51:29+08:00,13.258200,51.912121,105.253105,2021-08-12 03:52:15.999436+00:00,14.475679,L8


In [261]:
landsat_with7_df_filtered

,Time,T,Lat,Lon,Image Time,Image T,Landsat
0,2021-09-23 10:35:16+08:00,9.571900,53.143623,107.849472,2021-09-23 02:36:07.773845500+00:00,8.438689,L7
1,2022-08-04 10:13:45+08:00,14.132600,55.648727,109.369957,2022-08-04 02:13:44.191168+00:00,16.610226,L7
3,2021-07-28 10:47:17+08:00,8.693400,53.017338,107.355103,2021-07-28 02:47:19.296578200+00:00,9.390626,L7
4,2021-08-13 10:45:21+08:00,13.084100,53.156502,107.977829,2021-08-13 02:45:54.146524+00:00,13.417129,L7
6,2021-08-20 10:51:51+08:00,14.136200,51.610619,104.894226,2021-08-20 02:51:51.804602+00:00,13.868249,L7
7,2021-10-23 10:46:19+08:00,7.491000,51.809681,104.891777,2021-10-23 02:46:13.069633200+00:00,4.255274,L7
8,2021-07-26 10:59:37+08:00,14.771800,51.628082,104.103561,2021-07-26 03:00:15.278371400+00:00,14.642000,L7
10,2021-07-13 11:38:04+08:00,7.078200,55.332798,109.608627,2021-07-13 03:38:29.542497+00:00,9.714929,L8
11,2021-07-13 11:38:04+08:00,7.078200,55.332798,109.608627,2021-07-13 03:38:53.420828900+00:00,9.716510,L8
12,2021-07-29 11:39:00+08:00,11.988600,53.443645,108.509331,2021-07-29 03:39:24.750487+00:00,13.214574,L8


In [12]:
mod11_day_df_filtered

,Time,T,Lat,Lon,Image Time,Image T,MODIS
0,2021-05-27 11:02:10+08:00,2.22800,52.465443,106.586891,2021-05-27 11:06:00+08:00,2.35,MOD11_Terra_day
1,2021-05-28 11:37:54+08:00,1.42000,52.052254,105.978180,2021-05-28 11:48:00+08:00,1.69,MOD11_Terra_day
2,2021-06-03 10:58:21+08:00,2.44340,51.606743,104.731491,2021-06-03 11:12:00+08:00,3.05,MOD11_Terra_day
3,2021-06-04 11:49:31+08:00,2.57930,51.626816,105.359306,2021-06-04 11:54:00+08:00,4.11,MOD11_Terra_day
4,2021-06-05 10:50:54+08:00,10.08810,52.292828,106.244370,2021-06-05 11:00:00+08:00,10.57,MOD11_Terra_day
...,...,...,...,...,...,...,...
207,2024-09-20 13:12:00+08:00,11.55735,51.756080,104.943670,2024-09-20 13:18:00+08:00,11.51,MOD11_Aqua_day
208,2024-09-21 13:57:00+08:00,9.19320,51.776960,105.368270,2024-09-21 14:00:00+08:00,8.85,MOD11_Aqua_day
209,2024-09-23 13:32:00+08:00,11.17130,52.954070,107.273180,2024-09-23 13:42:00+08:00,11.43,MOD11_Aqua_day
210,2024-09-24 12:28:00+08:00,11.51320,53.217890,107.853210,2024-09-24 12:42:00+08:00,11.55,MOD11_Aqua_day


In [11]:
mod11_night_df_filtered

,Time,T,Lat,Lon,Image Time,Image T,MODIS
1,2021-06-03 22:15:41+08:00,2.91520,51.717644,104.984360,2021-06-03 22:24:00+08:00,2.47,MOD11_Terra_night
2,2021-06-04 21:15:42+08:00,3.03760,51.996571,105.369530,2021-06-04 21:30:00+08:00,2.87,MOD11_Terra_night
3,2021-06-08 21:01:16+08:00,2.04260,53.457714,108.742622,2021-06-08 21:06:00+08:00,-0.41,MOD11_Terra_night
4,2021-06-09 21:37:02+08:00,1.84480,54.452457,109.069336,2021-06-09 21:48:00+08:00,1.33,MOD11_Terra_night
5,2021-06-10 20:50:08+08:00,2.18130,55.342632,109.308563,2021-06-10 20:54:00+08:00,1.97,MOD11_Terra_night
...,...,...,...,...,...,...,...
202,2024-09-23 03:01:00+08:00,11.58695,52.409230,106.441980,2024-09-23 03:12:00+08:00,11.31,MOD11_Aqua_night
203,2024-09-24 01:58:00+08:00,11.92380,52.742670,107.637250,2024-09-24 02:12:00+08:00,11.97,MOD11_Aqua_night
204,2024-09-26 03:24:00+08:00,11.96425,54.352880,109.483560,2024-09-26 03:30:00+08:00,9.75,MOD11_Aqua_night
205,2024-09-27 00:40:00+08:00,11.90755,55.138200,109.676810,2024-09-27 00:54:00+08:00,10.13,MOD11_Aqua_night


In [10]:
mod21_day_df_filtered

,Time,T,Lat,Lon,Image Time,Image T,MODIS
0,2023-09-26 01:00:19+08:00,12.43440,55.140530,109.348152,2023-09-26 01:14:00+08:00,10.17,MOD21_Aqua_day
1,2021-07-30 01:10:56+08:00,12.34190,54.424122,108.716713,2021-07-30 01:17:00+08:00,14.41,MOD21_Aqua_day
2,2021-06-08 00:56:41+08:00,2.07150,53.194977,107.806984,2021-06-08 01:08:00+08:00,2.73,MOD21_Terra_day
3,2024-07-28 01:28:00+08:00,18.03427,52.006840,105.864730,2024-07-28 01:17:00+08:00,22.01,MOD21_Aqua_day
4,2021-05-28 01:05:30+08:00,1.93390,52.298656,106.063141,2021-05-28 01:12:00+08:00,2.41,MOD21_Aqua_day
...,...,...,...,...,...,...,...
192,2022-09-24 01:05:16+08:00,7.23920,52.665291,106.851189,2022-09-24 01:06:00+08:00,6.73,MOD21_Terra_day
193,2021-07-30 00:55:51+08:00,14.33490,54.389412,108.722244,2021-07-30 01:06:00+08:00,13.67,MOD21_Terra_day
195,2021-10-18 00:59:34+08:00,6.02720,52.883705,107.152977,2021-10-18 01:06:00+08:00,6.31,MOD21_Terra_day
196,2021-06-12 01:16:46+08:00,2.45300,55.065163,109.483406,2021-06-12 01:17:00+08:00,3.79,MOD21_Aqua_day


In [13]:
mod21_night_df_filtered

,Time,T,Lat,Lon,Image Time,Image T,MODIS
0,2022-08-03 01:59:46+08:00,12.81340,54.396488,109.357117,2022-08-03 02:07:00+08:00,14.47,MOD21_Terra_night
1,2021-07-26 02:19:55+08:00,15.46290,51.564621,104.129318,2021-07-26 02:06:00+08:00,17.21,MOD21_Terra_night
2,2024-09-24 00:13:00+08:00,11.64630,52.775570,107.621770,2024-09-24 00:22:00+08:00,12.03,MOD21_Aqua_night
3,2021-06-03 01:56:13+08:00,2.47310,51.526257,104.238014,2021-06-03 02:04:00+08:00,2.83,MOD21_Terra_night
4,2023-07-18 00:03:00+08:00,13.42810,51.584373,105.059387,2023-07-18 00:12:00+08:00,11.69,MOD21_Aqua_night
...,...,...,...,...,...,...,...
120,2024-05-30 01:55:00+08:00,3.07393,51.899530,105.064300,2024-05-30 02:01:00+08:00,6.51,MOD21_Terra_night
121,2021-07-31 02:01:41+08:00,12.90220,55.324104,109.480698,2021-07-31 02:07:00+08:00,12.17,MOD21_Terra_night
122,2024-07-31 01:57:00+08:00,20.21280,53.992940,109.200040,2024-07-31 02:05:00+08:00,19.27,MOD21_Terra_night
124,2022-06-06 01:56:45+08:00,2.85890,52.832340,107.426132,2022-06-06 02:03:00+08:00,-0.17,MOD21_Terra_night


In [14]:
landsat_df_filtered.to_csv(f'Данные_валидации/ready/{landsat_file_name_base}_filtered.csv', index=False)
landsat_with7_df_filtered.to_csv(f'Данные_валидации/ready/{landsat_with7_file_name_base}_filtered.csv', index=False)
l2sp_df_filtered.to_csv(f'Данные_валидации/ready/{l2sp_file_name_base}_filtered.csv', index=False)
mod11_day_df_filtered.to_csv('Данные_валидации/ready/mod11_day_filtered.csv', index=False)
mod11_night_df_filtered.to_csv('Данные_валидации/ready/mod11_night_filtered.csv', index=False)
mod21_day_df_filtered.to_csv('Данные_валидации/ready/mod21_day_filtered.csv', index=False)
mod21_night_df_filtered.to_csv('Данные_валидации/ready/mod21_night_filtered.csv', index=False)

# Разделение MODIS

In [33]:
def split_modis(df):
   """
   Разбивает датафрейм на словарь "продукт":под-датафрейм 
   """
   modis_sub_df = df['MODIS'].str
   ## M11

   df_day_M11 = df[(modis_sub_df.contains("MOD11")) & (modis_sub_df.contains("day"))]
   df_night_M11 = df[(modis_sub_df.contains("MOD11")) & (modis_sub_df.contains("night"))]

   ## M21

   df_day_M21 = df[(modis_sub_df.contains("MOD21")) & (modis_sub_df.contains("day"))]
   df_night_M21 = df[(modis_sub_df.contains("MOD21")) & (modis_sub_df.contains("night"))]

   return {"mod11_day": df_day_M11, "mod11_night": df_night_M11, "mod21_day": df_day_M21, "mod21_night": df_night_M21}

In [ ]:
modis_df = pd.read_csv('Данные_валидации/raw/modis_validation15.csv')

In [50]:
modis_dfs = split_modis(modis_df)

mod11_day = modis_dfs["mod11_day"]
mod11_night = modis_dfs["mod11_night"]
mod21_day = modis_dfs["mod21_day"]
mod21_night = modis_dfs["mod21_night"]

In [ ]:
mod11_day.to_csv('Данные_валидации/raw/mod11_day.csv', index=False)
mod11_night.to_csv('Данные_валидации/raw/mod11_night.csv', index=False)
mod21_day.to_csv('Данные_валидации/raw/mod21_day.csv', index=False)
mod21_night.to_csv('Данные_валидации/raw/mod21_night.csv', index=False)

# Другое

Разница между модисами

In [2]:
mod11_day_df = pd.read_csv('Данные_валидации/raw/mod11_day.csv')
mod11_night_df = pd.read_csv('Данные_валидации/raw/mod11_night.csv')
mod21_day_df = pd.read_csv('Данные_валидации/raw/mod21_day.csv')
mod21_night_df = pd.read_csv('Данные_валидации/raw/mod21_night.csv')

In [ ]:
prep_m11_day = mod11_df[["Time", "Image T", "MODIS"]].rename(columns={"Image T": "Image T MOD11"})
prep_m21_day = mod21_df[["Time", "Image T", "MODIS"]].rename(columns={"Image T": "Image T MOD21"})
prep_m11_night = mod11_df[["Time", "Image T", "MODIS"]].rename(columns={"Image T": "Image T MOD11"})
prep_m21_night = mod21_df[["Time", "Image T", "MODIS"]].rename(columns={"Image T": "Image T MOD21"})
found_diffs = pd.merge(prep_m11, prep_m21, on='Time')
found_diffs

,Time,Image T MOD11,MODIS_x,Image T MOD21,MODIS_y
0,2021-06-04 11:49:31+08:00,4.11,MOD11_Terra,4.19,MOD21_Terra
1,2021-08-14 11:51:37+08:00,15.55,MOD11_Terra,20.19,MOD21_Terra
2,2024-06-09 13:55:00+08:00,13.49,MOD11_Aqua,3.49,MOD21_Aqua
3,2023-09-26 21:33:49+08:00,13.11,MOD11_Terra,13.29,MOD21_Terra
4,2021-06-03 02:26:20+08:00,1.99,MOD11_Aqua,2.53,MOD21_Aqua
5,2021-07-10 02:40:04+08:00,5.17,MOD11_Aqua,5.57,MOD21_Aqua
6,2021-07-26 02:35:00+08:00,15.33,MOD11_Aqua,14.85,MOD21_Aqua
7,2021-07-28 02:30:14+08:00,14.83,MOD11_Aqua,14.99,MOD21_Aqua
8,2021-08-14 03:19:38+08:00,15.75,MOD11_Aqua,17.21,MOD21_Aqua
9,2022-06-06 02:11:50+08:00,2.01,MOD11_Aqua,1.09,MOD21_Aqua


In [ ]:
found_diffs.to_csv('Данные_валидации/raw/mod_diffs_old.csv', index=False)

In [2]:
a = pd.read_csv('Данные_валидации/raw/mod_diffs_old.csv')
b = pd.read_csv('Данные_валидации/raw/mod_diffs_new.csv')

In [3]:
c = pd.concat([a,b], ignore_index=True, sort=False)
c

,Time,Image T MOD11,MODIS_x,Image T MOD21,MODIS_y
0,2021-06-04 11:49:31+08:00,4.11,MOD11_Terra,4.19,MOD21_Terra
1,2021-08-14 11:51:37+08:00,15.55,MOD11_Terra,20.19,MOD21_Terra
2,2024-06-09 13:55:00+08:00,13.49,MOD11_Aqua,3.49,MOD21_Aqua
3,2023-09-26 21:33:49+08:00,13.11,MOD11_Terra,13.29,MOD21_Terra
4,2021-06-03 02:26:20+08:00,1.99,MOD11_Aqua,2.53,MOD21_Aqua
5,2021-07-10 02:40:04+08:00,5.17,MOD11_Aqua,5.57,MOD21_Aqua
6,2021-07-26 02:35:00+08:00,15.33,MOD11_Aqua,14.85,MOD21_Aqua
7,2021-07-28 02:30:14+08:00,14.83,MOD11_Aqua,14.99,MOD21_Aqua
8,2021-08-14 03:19:38+08:00,15.75,MOD11_Aqua,17.21,MOD21_Aqua
9,2022-06-06 02:11:50+08:00,2.01,MOD11_Aqua,1.09,MOD21_Aqua


In [4]:
(c['Image T MOD21']-c['Image T MOD11']).mean()

0.3030769230769232

Вычисление температуры для каждого снимка для ускорения вычислений по датам

In [ ]:
import ee
import geemap
import datetime

In [ ]:
geemap.ee_initialize()
oeel = geemap.requireJS()
ee.Authenticate()
ee.Initialize(project='ee-amazyar-test1')

landsat_date_start = ee.Date("1986-03-16") # с 1986 начал предоставлять тепловую информацию Landsat-5
landsat_start_year = landsat_date_start.get("year").getInfo()
date_end = datetime.datetime.now()
end_year = date_end.date().year
openwater_month_start = 6 # июнь
openwater_month_end = 11 # ноябрь

baikal_shape = ee.FeatureCollection('projects/ee-amazyar-test/assets/baikal')

In [ ]:
LandsatLST = geemap.requireJS("../baikal_vkr_init/Landsat_LST.js")

In [ ]:
use_ndvi = True

def get_landsat_collection(satellite_name: str):
   coll = LandsatLST.collection(satellite_name, landsat_date_start, date_end, baikal_shape, use_ndvi)#.filter(ee.Filter.lt('CLOUD_COVER', 30))
   return coll.select(['LST'])

In [ ]:
landsats_collection = ee.ImageCollection.merge(get_landsat_collection('L8'),get_landsat_collection('L9'))
landsats_collection = ee.ImageCollection.merge(landsats_collection,get_landsat_collection('L7'))
landsats_collection = ee.ImageCollection.merge(landsats_collection,get_landsat_collection('L5'))

In [ ]:
def get_average_lst_landsat(landsat_collection: ee.ImageCollection):
   result = {}
   coll_size = landsat_collection.size().getInfo()
   if coll_size == 0:
      return result
   landsat_list = landsat_collection.toList(coll_size)
   for i in range(coll_size):
      image = ee.Image(landsat_list.get(i))
      temp_time = image.getString('DATE_ACQUIRED').cat(" ").cat(image.getString('SCENE_CENTER_TIME')).getInfo()
      time = pd.to_datetime(temp_time, format='%Y-%m-%d %H:%M:%S.%fZ')
      mean = image.reduceRegion(reducer=ee.Reducer.mean(), geometry=baikal_shape, scale=30, maxPixels=1000000000).getInfo()['LST']
      result[time] = mean
      print(f"{time}: {mean}")
   return result

In [ ]:
all_years_landsat = {}
for year in range(1996, 2026):
   year_coll = landsats_collection.filterDate(ee.Date(str(year)), ee.Date(str(year+1)))
   result = get_average_lst_landsat(year_coll)
   print(year, result)
   all_years_landsat[year] = result
   
all_years_landsat

2001-04-25 03:28:38.597242: -4.278918962109514
2001-06-12 03:28:11.705008500: 4.609386787428374
2001-07-30 03:27:45.435381600: 13.727172043541104
2001-08-31 03:27:16.502226700: 16.26646521084795
2001-10-18 03:26:51.641899600: 3.2325683612831857
2001-04-25 03:29:02.490194400: -5.779807935915401
2001-06-12 03:28:35.584338900: 3.5376763716642103
2001-07-30 03:28:09.315089600: 10.895842649891406
2001-08-31 03:27:40.382075100: 15.838761338748284
2001-10-02 03:27:16.746188200: 6.0221995423150405
2001-10-18 03:27:15.539730400: 4.59862249244397
2001-04-25 03:29:26.361585500: -12.39193873138029
2001-06-12 03:28:59.457871300: 4.239201909829961
2001-07-30 03:28:33.194409200: 10.29529378393163
2001-08-31 03:28:04.252812800: 16.849192443894584
2001-10-02 03:27:40.619599700: 2.772030184936525
2001-10-18 03:27:39.418556100: 4.751536887049733
2001-05-02 03:34:50.299372: -12.59809483692393
2001-06-03 03:34:27.922825100: 1.602774794311284
2001-06-19 03:34:16.722416100: 5.880714980810778
2001-07-21 03:34

{2001: {Timestamp('2001-04-25 03:28:38.597242'): -4.278918962109514,
  Timestamp('2001-06-12 03:28:11.705008500'): 4.609386787428374,
  Timestamp('2001-07-30 03:27:45.435381600'): 13.727172043541104,
  Timestamp('2001-08-31 03:27:16.502226700'): 16.26646521084795,
  Timestamp('2001-10-18 03:26:51.641899600'): 3.2325683612831857,
  Timestamp('2001-04-25 03:29:02.490194400'): -5.779807935915401,
  Timestamp('2001-06-12 03:28:35.584338900'): 3.5376763716642103,
  Timestamp('2001-07-30 03:28:09.315089600'): 10.895842649891406,
  Timestamp('2001-08-31 03:27:40.382075100'): 15.838761338748284,
  Timestamp('2001-10-02 03:27:16.746188200'): 6.0221995423150405,
  Timestamp('2001-10-18 03:27:15.539730400'): 4.59862249244397,
  Timestamp('2001-04-25 03:29:26.361585500'): -12.39193873138029,
  Timestamp('2001-06-12 03:28:59.457871300'): 4.239201909829961,
  Timestamp('2001-07-30 03:28:33.194409200'): 10.29529378393163,
  Timestamp('2001-08-31 03:28:04.252812800'): 16.849192443894584,
  Timestamp('

In [4]:
# Объединить все посчитанные данные
all_years_landsat = {'1986-07-13 03:01:24.924031': 16.27339768998559,
 '1986-07-29 03:00:51.344056': 16.573165869364228,
 '1986-11-02 02:57:32.929000': -13.227672344186969,
 '1986-07-13 03:01:48.785013': 12.091803148713247,
 '1986-07-29 03:01:15.206081': 15.148326739520522,
 '1986-11-02 02:57:56.660081': -18.596356014020028,
 '1986-07-13 03:02:12.788081': 7.4542387361959594,
 '1986-07-29 03:01:39.212006': 13.97481009402597,
 '1986-11-02 02:58:20.821050': -26.57226152621019,
 '1986-11-18 02:57:43.266088': -3.4592302068156173,
 '1986-06-11 03:03:39.572019': None,
 '1986-07-13 03:02:36.506075': None,
 '1986-11-02 02:58:44.553038': None,
 '1986-11-18 02:58:07': 2.5347998448313156,
 '1986-06-02 03:08:55.313094': -3.5995585078764716,
 '1986-06-18 03:08:25.396050': 6.847358187443933,
 '1986-11-09 03:03:27.547094': 0.30937971240183865,
 '1986-06-02 03:09:19.171038': -0.5565752948732023,
 '1986-06-18 03:08:49.255038': 2.6156504669638636,
 '1986-11-09 03:03:51.423019': -2.229586178007629,
 '1986-06-18 03:09:13.257025': 4.177805599950558,
 '1986-11-09 03:04:15.441050': 0.7690070096073587,
 '1986-06-18 03:09:36.973031': 7.274843632002761,
 '1986-11-09 03:04:39.173094': 2.5699043636144236,
 '1986-11-25 03:04:09.901063': 0.47438310284844043,
 '1986-07-27 03:13:41.122031': 10.994476261221505,
 '1986-10-31 03:10:22.699063': None,
 '1986-11-16 03:09:45.756038': -12.858406287744103,
 '1986-06-09 03:15:41.152044': None,
 '1986-07-27 03:14:05.127031': 13.916777285282565,
 '1986-10-31 03:10:46.717025': -7.904937742310696,
 '1986-06-09 03:16:04.867050': -0.163580782739512,
 '1986-07-27 03:14:28.846050': 13.00133318273807,
 '1986-10-31 03:11:10.449000': -12.16561142818517,
 '1986-11-16 03:10:33.508013': -5.45060148787239,
 '1986-04-29 03:23:24.899000': -1.5397001336750642,
 '1986-05-31 03:22:32.065063': 2.0580334116720014,
 '1986-06-16 03:22:02.443013': 5.4499889428164145,
 '1986-07-18 03:20:58.664038': 12.663749845634994,
 '1986-08-03 03:20:24.636063': 12.29559892737686,
 '1986-10-22 03:17:40.517081': None,
 '1986-11-07 03:17:05.518075': 3.9354982805085954,
 '1986-11-23 03:16:28.234056': -18.621886850289375,
'1987-09-18 03:05:29.994044': None,
 '1987-11-05 03:06:26.141094': -4.686794896999162,
 '1987-11-05 03:06:49.913063': -6.236807808685767,
 '1987-11-05 03:07:14.115000': -8.311005904337014,
 '1987-11-21 03:07:34.276019': None,
 '1987-02-13 03:05:28.337000': -27.54194845941271,
 '1987-09-25 03:11:49.937006': 9.407315114776772,
 '1987-02-13 03:05:52.077038': -25.94110678889002,
 '1987-03-17 03:07:05.507063': None,
 '1987-09-25 03:12:13.846006': 9.229384806263992,
 '1987-02-13 03:06:16.103081': -23.57706044407412,
 '1987-03-17 03:07:29.536006': -11.268020558022181,
 '1987-09-25 03:12:37.898025': 9.37873750347676,
 '1987-02-13 03:06:39.844031': -21.956009866166738,
 '1987-03-17 03:07:53.278038': -12.559052574196516,
 '1987-09-25 03:13:01.521000': 9.627027131073445,
 '1987-10-11 03:13:20.362025': 4.588773558784484,
 '1987-02-04 03:11:41.518044': -13.530588978925058,
 '1987-02-20 03:12:19.441069': None,
 '1987-04-09 03:14:04.717038': -4.539291041603977,
 '1987-10-02 03:18:33.208094': None,
 '1987-02-04 03:12:05.544025': -13.973192114599925,
 '1987-02-20 03:12:43.468044': -20.048931007542148,
 '1987-04-09 03:14:28.747094': -5.358725609738251,
 '1987-10-02 03:18:57.262031': -3.6621898356175215,
 '1987-11-19 03:19:53.731044': -11.598809874255549,
 '1987-02-04 03:12:29.284000': -12.518609215881831,
 '1987-02-20 03:13:07.209025': None,
 '1987-04-09 03:14:52.492050': -5.762070732241982,
 '1987-06-28 03:16:58.266025': 2.3059326347073523,
 '1987-10-02 03:19:21.029038': 2.2282123763101054,
 '1987-10-18 03:19:38.263006': 2.052230389794686,
 '1987-11-19 03:20:17.361050': None,
 '1987-02-11 03:18:33.055063': -18.70279996894706,
 '1987-03-31 03:20:21.254000': -5.275542830187344,
 '1987-09-23 03:24:56.922019': 0.8125911445736622,
 '1987-10-09 03:25:16.189019': -18.49079920191336,
 '1987-02-11 03:18:56.795081': -22.5052194068006,
 '1987-02-27 03:19:34.318000': -8.035610648370026,
 '1987-03-31 03:20:44.997044': -5.048465362514538,
 '1987-09-23 03:25:20.687088': None,
 '1987-10-09 03:25:39.814031': None,
 '1987-02-02 03:24:46.164094': -14.38057945049292,
 '1987-03-22 03:26:36.740088': -21.510586765790634,
 '1987-05-25 03:28:34.133000': 1.99554741495954,
 '1987-06-26 03:29:17.437094': -1.9484985479628487,
 '1987-07-12 03:29:36.241000': 10.482010840406327,
'1988-02-25 03:08:19.059056': -24.11670468755274,
 '1988-04-13 03:08:41.172056': None,
 '1988-02-25 03:08:42.841088': -21.93969630319511,
 '1988-04-13 03:09:04.961019': -5.688147509728328,
 '1988-04-13 03:09:29.036050': -9.553063583311873,
 '1988-04-20 03:14:54.705019': -4.1751755125638494,
 '1988-04-20 03:15:18.638031': -5.0682288557166215,
 '1988-04-20 03:15:42.571038': -4.295748422748889,
 '1988-04-20 03:16:06.361019': -3.7934336994863,
 '1988-02-07 03:20:49.489038': -20.9961518271745,
 '1988-02-23 03:21:03.322006': -16.4642362824017,
 '1988-06-14 03:21:54.055044': 2.237344641626151,
 '1988-02-07 03:21:13.413063': -18.99870942271565,
 '1988-02-23 03:21:27.247038': -16.50852952793568,
 '1988-06-14 03:22:18.139069': 3.4218443200743884,
 '1988-02-07 03:21:37.194056': -17.728144964231582,
 '1988-02-23 03:21:51.029056': -17.19328071857983,
 '1988-06-14 03:22:41.937025': 2.666402504954483,
 '1988-03-17 03:27:51.690081': -11.046944551590189,
 '1988-04-18 03:28:03.396031': None,
 '1988-05-04 03:28:11.177063': 1.760168559500417,
 '1988-03-17 03:28:15.476038': -13.54890353786184,
 '1988-04-18 03:28:27.042025': None,
 '1988-05-04 03:28:34.969025': -0.8265469591208523,
 '1988-02-21 03:34:11.255069': -15.667888396655497,
 '1988-03-08 03:34:21.417075': -3.0747822644761547,
 '1988-04-25 03:34:40.397081': -3.9670504977357637,
 '1988-10-18 03:35:03.086031': 5.297881455727258,
 '1989-04-16 03:08:02.313056': -8.560510139674692,
 '1989-04-16 03:08:26.428013': None,
 '1989-03-22 03:13:44.238019': -10.821806991112226,
 '1989-03-06 03:14:48.560006': -20.82492576184915,
 '1989-03-22 03:14:08.063031': -11.030517149538266,
 '1989-03-06 03:15:12.526094': -13.574461342257047,
 '1989-03-22 03:14:32.031094': -7.918887482325119,
 '1989-09-14 03:11:24.750056': 8.042493731148829,
 '1989-09-30 03:11:15.908075': 8.673107224672291,
 '1989-03-06 03:15:36.350025': -11.229552619718916,
 '1989-03-22 03:14:55.713056': -7.931363355961236,
 '1989-08-29 03:12:22.968050': 10.36825897631971,
 '1989-09-14 03:11:48.591038': 7.976297759527806,
 '1989-09-30 03:11:39.751056': 9.005899369492216,
 '1989-02-25 03:21:11.685038': -22.135969864046388,
 '1989-02-25 03:21:35.651094': -20.832024481150533,
 '1989-02-25 03:21:59.331038': -15.597762780254756,
 '1989-09-05 03:18:23.866056': 8.556239607702443,
 '1989-01-31 03:28:20.287000': -16.368265038149456,
 '1989-04-21 03:27:12.402069': -1.8618910725720905,
 '1989-07-10 03:25:48.256013': 3.8536190187099595,
 '1989-09-12 03:24:17.475006': 6.776338667977455,
 '1989-09-28 03:24:02.278044': 8.723383282828696,
 '1989-04-12 03:33:37.828056': -6.830586397509487,
'1990-03-18 02:59:03.268031': -6.400559008746804,
 '1990-04-03 02:58:36.568088': -7.941343356533656,
 '1990-04-19 02:58:49.242000': -4.482496308456788,
 '1990-05-05 02:58:43.473044': -5.721190326455852,
 '1990-06-06 02:58:45.855019': None,
 '1990-06-22 02:58:44.472088': 4.520755359221826,
 '1990-07-08 02:58:43.353069': 10.639718664336565,
 '1990-08-25 02:58:28.895006': None,
 '1990-03-18 02:59:27.124088': -6.128954889938374,
 '1990-04-03 02:59:00.427088': -7.233252325920392,
 '1990-04-19 02:59:13.103025': -3.20732121289873,
 '1990-05-05 02:59:07.192075': -1.9544272219229202,
 '1990-06-06 02:59:09.576038': None,
 '1990-06-22 02:59:08.338075': 0.9214761556256026,
 '1990-07-08 02:59:07.220069': 7.439423186964023,
 '1990-08-25 02:58:52.622038': 10.607681010741315,
 '1990-10-12 02:58:35.472025': 6.004331910691392,
 '1990-03-18 02:59:51.125013': -4.988957795875801,
 '1990-04-03 02:59:24.430056': -6.420421722462246,
 '1990-04-19 02:59:37.108019': -5.726252986822419,
 '1990-05-05 02:59:31.343031': 0.10087326438567137,
 '1990-06-06 02:59:33.728088': None,
 '1990-06-22 02:59:32.348031': 3.7484781913050913,
 '1990-07-08 02:59:31.231038': 11.203730249745481,
 '1990-08-25 02:59:16.781013': 13.28807901585858,
 '1990-10-12 02:58:59.490075': 6.237990320569312,
 '1990-03-18 03:00:14.837094': -5.062564411990448,
 '1990-03-09 03:05:30.846069': -10.964473019352639,
 '1990-05-12 03:05:00.907006': -1.394186499710966,
 '1990-05-28 03:04:56.473019': 0.9145639366436907,
 '1990-06-13 03:04:55.721050': 2.6704794803773426,
 '1990-07-15 03:04:52.318075': None,
 '1990-08-16 03:04:43.443013': 17.237412472295468,
 '1990-09-01 03:04:36.535063': 12.92645523668761,
 '1990-10-19 03:04:17.405075': 4.56344859280574,
 '1990-11-04 03:04:11.912081': -10.653357865384324,
 '1990-03-09 03:05:54.701031': -12.81547768149489,
 '1990-05-12 03:05:24.770056': -0.7285702198379524,
 '1990-05-28 03:05:20.193081': 1.6387655897116473,
 '1990-06-13 03:05:19.586081': 3.381107152427587,
 '1990-08-16 03:05:07.313013': 15.548830763065093,
 '1990-09-01 03:05:00.407006': 14.549058709442217,
 '1990-10-19 03:04:41.280050': 6.238692499405151,
 '1990-03-09 03:06:18.699069': -12.817391571235547,
 '1990-05-12 03:05:48.777075': 0.5355086391345961,
 '1990-05-28 03:05:44.345069': 3.3985635136141594,
 '1990-06-13 03:05:43.595081': 5.288270885808789,
 '1990-08-16 03:05:31.326088': 15.752457001317348,
 '1990-09-01 03:05:24.422038': 13.061105236921298,
 '1990-10-19 03:05:05.299013': 6.42175751833634,
 '1990-11-04 03:04:59.808025': 1.776038071832865,
 '1990-11-20 03:04:59.978094': -4.577826387121071,
 '1990-03-09 03:06:42.410063': -9.28470494877216,
 '1990-05-12 03:06:12.497044': 1.3219583928999714,
 '1990-05-28 03:06:08.066044': 3.9394201731018508,
 '1990-06-13 03:06:07.317031': 7.03165792285082,
 '1990-08-16 03:05:55.053013': 10.05688645675155,
 '1990-09-01 03:05:48.149094': 12.225442235627844,
 '1990-09-17 03:05:46.901075': 8.392129239180505,
 '1990-10-19 03:05:29.030019': 6.020509138126207,
 '1990-11-04 03:05:23.540025': -1.4887943674476722,
 '1990-11-20 03:05:23.568006': 0.3983329112270806,
 '1990-03-16 03:11:52.431013': -10.701039737437624,
 '1990-04-17 03:11:34.757075': -17.01042885297766,
 '1990-05-19 03:11:35.923094': 1.1038845376210908,
 '1990-06-20 03:11:30.285038': None,
 '1990-07-06 03:11:28.390006': None,
 '1990-08-23 03:11:15.003000': 15.396295020616243,
 '1990-09-08 03:11:09.815094': 5.965516449434541,
 '1990-03-16 03:12:16.430094': -9.600175287334304,
 '1990-04-17 03:11:58.762019': None,
 '1990-05-03 03:11:50.544094': 0.4719827942735978,
 '1990-05-19 03:11:59.931075': 0.2936907911846134,
 '1990-06-20 03:11:54.294075': 3.1966920505387724,
 '1990-07-06 03:11:52.400050': None,
 '1990-08-23 03:11:39.017081': 13.674989381862304,
 '1990-09-08 03:11:33.831069': 10.478517648283567,
 '1990-10-10 03:11:20.852056': 6.309244334568138,
 '1990-03-16 03:12:40.143019': -10.507573791138695,
 '1990-05-03 03:12:14.264013': 0.4627262030727493,
 '1990-05-19 03:12:23.652000': 0.4987796070778144,
 '1990-06-04 03:12:18.949044': 2.7147172028880573,
 '1990-06-20 03:12:18.016050': 3.6262391357967014,
 '1990-08-23 03:12:02.744094': 11.24682730961826,
 '1990-09-08 03:11:57.559081': 8.78889520036886,
 '1990-10-10 03:11:44.583013': 5.7640776571211685,
 '1990-11-27 03:11:26.128063': -0.4273128116365727,
 '1990-03-07 03:19:08.622044': -8.508774061718755,
 '1990-03-23 03:18:34.710031': -4.812755200171414,
 '1990-04-08 03:18:25.863075': -0.5131764398260309,
 '1990-04-24 03:18:33.269075': -2.980504428289924,
 '1990-05-10 03:18:32.855025': 0.9047777586652563,
 '1990-06-11 03:18:29.829013': 3.6139169693948334,
 '1990-06-27 03:18:28.739013': 7.449423454675185,
 '1990-07-13 03:18:26.431088': 4.780876110103247,
 '1990-08-14 03:18:17.582038': 14.32211018856863,
 '1990-08-30 03:18:10.503038': 9.251515298855628,
 '1990-10-17 03:17:52.587038': 7.800218078570257,
 '1990-03-30 03:24:35.350019': -32.505120577475246,
 '1990-05-01 03:24:34.060081': -18.857304693641797,
 '1990-05-17 03:24:45.458044': 2.6879525161455575,
 '1990-06-02 03:24:40.242013': -5.725402011347232,
 '1990-06-18 03:24:39.541019': 6.008796161464059,
 '1990-07-04 03:24:38.383019': None,
 '1990-08-05 03:24:29.962094': 11.447230540074385,
 '1990-08-21 03:24:25.990006': 13.385409256638441,
 '1990-09-06 03:24:18.504006': 4.784772475580243,
 '1990-10-24 03:24:01.851056': -7.456376857453875,
'1991-02-17 02:59:27.095013': -21.21348460039714,
 '1991-03-21 03:00:06.133031': None,
 '1991-04-06 03:00:17.789075': -6.034652891767264,
 '1991-04-22 03:00:35.278081': -5.473885078545852,
 '1991-05-08 03:00:55.241069': -1.6324699594642802,
 '1991-05-24 03:01:11.089013': -0.9315461022014906,
 '1991-06-09 03:01:23.024081': 3.6410938883221324,
 '1991-06-25 03:01:37.578031': None,
 '1991-07-11 03:01:47.002088': 10.154959430319034,
 '1991-07-27 03:02:02.858006': 11.503590199456072,
 '1991-08-28 03:02:14.779013': 10.835874976735793,
 '1991-09-13 03:02:27.013038': 11.694551853133891,
 '1991-09-29 03:02:32.413038': -17.994505277852724,
 '1991-10-15 03:02:43.243006': 0.2519645777759778,
 '1991-10-31 03:02:41.798063': -11.515048532891655,
 '1991-02-01 02:59:32.919056': -19.01616860140176,
 '1991-02-17 02:59:50.834050': -22.5753869040599,
 '1991-03-21 03:00:30.018063': -14.664662105002146,
 '1991-04-06 03:00:41.532006': -5.518516065478365,
 '1991-04-22 03:00:59.165075': -6.174878046933009,
 '1991-05-08 03:01:18.985050': -2.026337439656128,
 '1991-05-24 03:01:34.834006': -0.9443376499591732,
 '1991-06-09 03:01:46.770075': 3.2488846919632812,
 '1991-07-11 03:02:10.751025': 6.693788224615853,
 '1991-07-27 03:02:26.751031': 9.614355262521519,
 '1991-08-12 03:02:29.811019': 11.219966543900739,
 '1991-08-28 03:02:38.531031': 7.367750366144318,
 '1991-09-13 03:02:50.910025': 12.44784183353703,
 '1991-09-29 03:02:56.167088': -12.060544317480577,
 '1991-10-15 03:03:06.998063': 5.632055200188293,
 '1991-10-31 03:03:05.555044': -12.48677654487956,
 '1991-02-01 02:59:56.945075': -18.165169765270548,
 '1991-02-17 03:00:14.861050': -22.941399221386384,
 '1991-03-21 03:00:54.047081': -13.757834745002526,
 '1991-04-06 03:01:05.706006': -4.485199998449871,
 '1991-04-22 03:01:23.196056': -6.26134184730932,
 '1991-05-08 03:01:43.017019': -1.7332840621192847,
 '1991-05-24 03:01:58.866088': -0.14737279254668934,
 '1991-06-09 03:02:10.804069': 3.8628281829588356,
 '1991-07-11 03:02:34.787031': 8.615803784615634,
 '1991-07-27 03:02:50.788063': None,
 '1991-08-12 03:02:53.993075': 12.75989691748937,
 '1991-08-28 03:03:02.571050': 6.711146192350465,
 '1991-09-13 03:03:14.951006': 12.512360801390415,
 '1991-09-29 03:03:20.210038': None,
 '1991-10-15 03:03:31.042025': 6.7589018370338,
 '1991-10-31 03:03:29.600013': -2.2431083814034682,
 '1991-11-16 03:03:35.118019': None,
 '1991-03-28 03:06:19.065069': -14.322738489017523,
 '1991-04-13 03:06:39.304031': -5.690674814451637,
 '1991-04-29 03:06:55.161081': -4.2684433613340635,
 '1991-05-31 03:07:26.301013': 1.470971428468766,
 '1991-06-16 03:07:35.919019': 1.9114585416426904,
 '1991-07-02 03:07:55.755081': 6.426411270010334,
 '1991-07-18 03:08:06.571013': -0.17804462620283037,
 '1991-08-03 03:08:14.861063': 15.989953548798315,
 '1991-08-19 03:08:19.596038': None,
 '1991-09-04 03:08:33.130044': 12.18525644927412,
 '1991-09-20 03:08:43.608031': 10.355081074483637,
 '1991-10-22 03:08:54.468038': 3.1701248767096355,
 '1991-11-07 03:08:55.060081': 1.0694081423886275,
 '1991-02-24 03:06:10.191050': -19.955806919023146,
 '1991-03-28 03:06:42.951031': -13.772683337128854,
 '1991-04-13 03:07:03.190088': -4.825974522348159,
 '1991-04-29 03:07:19.048094': -4.1693155633707235,
 '1991-05-31 03:07:50.190038': 1.2180737384570786,
 '1991-06-16 03:07:59.809056': 2.8087002997159236,
 '1991-07-02 03:08:19.503038': 4.627259813937497,
 '1991-08-03 03:08:38.611056': 14.314479791327201,
 '1991-08-19 03:08:43.491088': 11.955337964353543,
 '1991-09-04 03:08:56.883006': 12.070448510049378,
 '1991-09-20 03:09:07.361094': 10.65421109451578,
 '1991-10-22 03:09:18.368025': 5.155683483678918,
 '1991-11-07 03:09:18.962013': -2.4543322168201946,
 '1991-02-24 03:06:34.219006': -20.0417262917953,
 '1991-03-28 03:07:06.980069': -11.02552647149883,
 '1991-04-13 03:07:27.221019': -2.6423325359217045,
 '1991-04-29 03:07:43.080000': -4.127849141297367,
 '1991-05-31 03:08:14.223050': 1.6863803996168922,
 '1991-06-16 03:08:23.843088': 3.1383466427399256,
 '1991-07-02 03:08:43.538075': 5.368144041376739,
 '1991-08-03 03:09:02.649038': 16.06092195240745,
 '1991-08-19 03:09:07.531025': 12.750532752982757,
 '1991-09-04 03:09:20.923056': 12.273551646470262,
 '1991-09-20 03:09:31.403044': 7.263241055919207,
 '1991-10-06 03:09:34.785081': 7.779088390664108,
 '1991-10-22 03:09:42.268025': 5.853387064600079,
 '1991-11-07 03:09:43.007038': -3.8010490107563424,
 '1991-11-23 03:09:46.682013': -3.73689420370636,
 '1991-01-23 03:06:25.292025': -12.302593698267703,
 '1991-02-24 03:06:57.958094': -21.661525519692294,
 '1991-03-12 03:07:16.773063': -14.856969631071477,
 '1991-03-28 03:07:30.722031': -14.817778387826428,
 '1991-04-13 03:07:50.963075': -6.571746790653305,
 '1991-04-29 03:08:06.679038': -4.599827835306764,
 '1991-05-31 03:08:37.968075': 2.1083054233426983,
 '1991-06-16 03:08:47.590044': 3.0211586490193256,
 '1991-07-18 03:09:18.249050': None,
 '1991-08-03 03:09:26.399044': 13.361449108881347,
 '1991-08-19 03:09:31.282069': 12.305132611718003,
 '1991-09-04 03:09:44.676013': 11.453445999254326,
 '1991-09-20 03:09:55.157013': 6.556502826889538,
 '1991-10-06 03:09:58.540088': 7.677311389590234,
 '1991-10-22 03:10:06.024025': 5.297951696260437,
 '1991-11-07 03:10:06.620075': -10.925214666594428,
 '1991-11-23 03:10:10.296025': -0.8947269849998053,
 '1991-03-03 03:12:26.700075': -13.252037747620285,
 '1991-03-19 03:12:49.094025': -12.490639582729818,
 '1991-04-04 03:12:59.916006': -2.2615372189967005,
 '1991-04-20 03:13:18.331038': -0.0030445873327413736,
 '1991-05-22 03:13:54.048044': None,
 '1991-06-23 03:14:20.943031': 5.401406804790313,
 '1991-07-09 03:14:33.141013': 1.302948646998398,
 '1991-07-25 03:14:47.386094': 10.923160964796999,
 '1991-08-10 03:14:51.712025': -8.002814005489116,
 '1991-09-11 03:15:12.112044': 12.29799355698267,
 '1991-09-27 03:15:15.960038': 4.848046129560619,
 '1991-10-13 03:15:27.058025': None,
 '1991-11-14 03:15:32.291069': None,
 '1991-01-30 03:12:18.896050': -14.973416855906624,
 '1991-03-03 03:12:50.728094': -13.04745591914392,
 '1991-03-19 03:13:13.123025': -11.361486263729416,
 '1991-04-04 03:13:23.946013': -2.4567319868676525,
 '1991-04-20 03:13:42.362019': -0.49043049076748196,
 '1991-05-22 03:14:18.081019': None,
 '1991-06-23 03:14:44.977094': 9.029808415032091,
 '1991-07-09 03:14:57.177031': 4.313329349291852,
 '1991-07-25 03:15:11.424013': 13.30125058244372,
 '1991-08-10 03:15:15.750075': 14.012447803543003,
 '1991-08-26 03:15:23.229019': None,
 '1991-09-11 03:15:36.153013': 12.010473451845856,
 '1991-09-27 03:15:40.002063': 9.629270066551324,
 '1991-10-13 03:15:51.101069': None,
 '1991-10-29 03:15:52.992019': 3.3917363196882606,
 '1991-11-14 03:15:56.337044': None,
 '1991-01-30 03:12:42.634094': -11.705471474714006,
 '1991-03-03 03:13:14.469038': -12.029690631836878,
 '1991-03-19 03:13:36.864050': -9.235161851229893,
 '1991-04-04 03:13:47.688038': -3.616056091883299,
 '1991-04-20 03:14:06.105013': -0.5995106691994839,
 '1991-05-22 03:14:41.826013': None,
 '1991-06-23 03:15:08.724081': 7.774450707901453,
 '1991-07-09 03:15:20.925063': 5.3806895078114065,
 '1991-07-25 03:15:35.173038': 12.541347129987846,
 '1991-08-10 03:15:39.501038': 9.300084017287492,
 '1991-08-26 03:15:46.981050': None,
 '1991-09-11 03:15:59.906000': 11.769684421571732,
 '1991-09-27 03:16:03.612094': 9.38794445768072,
 '1991-10-13 03:16:14.713019': None,
 '1991-10-29 03:16:16.604081': 3.489673726211161,
 '1991-11-14 03:16:19.951019': -4.813025646210589,
 '1991-11-30 03:16:31.988075': -0.7262951668211611,
 '1991-02-22 03:18:52.704044': -18.194171494679132,
 '1991-03-26 03:19:30.951050': -9.023218618458607,
 '1991-04-11 03:19:45.768019': -3.250249613484252,
 '1991-04-27 03:20:00.966025': 1.632642360819899,
 '1991-08-17 03:21:26.973025': 15.074904166834624,
 '1991-02-22 03:19:16.444019': -19.733996821919327,
 '1991-03-10 03:19:35.933038': -14.812901268852816,
 '1991-03-26 03:19:54.693013': -12.27005113641012,
 '1991-04-11 03:20:09.510081': -1.6952548931453444,
 '1991-04-27 03:20:24.565050': -1.5114634878247795,
 '1991-05-13 03:20:43.013056': 1.7332671872429726,
 '1991-05-29 03:20:58.595025': 0.5128368889236709,
 '1991-08-01 03:21:47.052019': 13.644554657893368,
 '1991-08-17 03:21:50.724063': 15.333383507518068,
 '1991-09-02 03:22:03.845069': None,
 '1991-09-18 03:22:15.648069': 10.49673142739351,
 '1991-10-04 03:22:18.248013': 2.6152619025635375,
 '1991-10-20 03:22:28.520038': -0.907173550022119,
 '1991-11-21 03:22:29.627025': -0.9557646779650463,
 '1991-01-28 03:25:03.404038': -12.319739525304577,
 '1991-03-17 03:25:54.214013': -13.516735291019673,
 '1991-04-18 03:26:26.285006': -5.205039434580625,
 '1991-05-20 03:27:00.455056': -18.902845050192298,
 '1991-06-05 03:27:16.270075': -17.367614399111677,
 '1991-06-21 03:27:27.856025': 10.856540384350927,
 '1991-07-07 03:27:42.455000': None,
 '1991-07-23 03:27:55.528069': 14.394756681201127,
 '1991-08-08 03:28:00.975081': 16.338353912415652,
 '1991-08-24 03:28:06.650094': 9.02980353403347,
 '1991-09-09 03:28:20.704063': 3.8840597346033765,
 '1991-09-25 03:28:26.280025': 2.080503960053911,
 '1991-10-11 03:28:34.059056': 6.669517794831446,
 '1991-10-27 03:28:38.249063': 4.9557658352601,
 '1991-11-12 03:28:41.722056': -9.351005242792073,
'1992-02-04 03:02:51.653044': -21.106130379033516,
 '1992-02-20 03:02:43.021050': -20.09401307605012,
 '1992-03-07 03:02:45.306088': -13.987664913217015,
 '1992-03-23 03:02:44.067094': -13.566052202934992,
 '1992-04-08 03:02:34.530038': -10.443641853055667,
 '1992-04-24 03:02:26.190063': -30.63797406873165,
 '1992-05-10 03:02:19.311038': -1.4808049034106776,
 '1992-05-26 03:02:10.301031': 1.084715170984978,
 '1992-06-27 03:01:48.543031': 4.573758979342803,
 '1992-07-29 03:01:24.760050': 15.911762604368418,
 '1992-08-14 03:01:10.703006': 1.5963274939737042,
 '1992-09-15 03:00:41.204094': None,
 '1992-10-01 03:00:19.764000': -8.542414617673638,
 '1992-10-17 03:00:03.893094': 3.5120102349333857,
 '1992-11-02 02:59:51.827031': None,
 '1992-02-04 03:03:15.416019': -18.231196621383415,
 '1992-02-20 03:03:06.929006': -18.915102461635804,
 '1992-03-07 03:03:09.071038': -13.996092640676348,
 '1992-03-23 03:03:07.833031': -12.026045169971942,
 '1992-04-08 03:02:58.297019': -8.598884914906337,
 '1992-04-24 03:02:49.958031': -28.47255285399894,
 '1992-05-10 03:02:43.080044': -4.586119252258508,
 '1992-05-26 03:02:34.071063': -0.2601872285405068,
 '1992-06-11 03:02:24.058088': -0.13922966011798243,
 '1992-06-27 03:02:12.316025': 1.830365982941076,
 '1992-07-13 03:02:00.614075': 1.994464158722673,
 '1992-07-29 03:01:48.537006': 11.69304669448921,
 '1992-08-14 03:01:34.625050': 11.600376710783124,
 '1992-09-15 03:01:04.986063': 5.6184164101411,
 '1992-10-01 03:00:43.546088': -1.1486345728940062,
 '1992-10-17 03:00:27.678019': 4.559347749972727,
 '1992-02-04 03:03:39.467000': -15.58243083252093,
 '1992-02-20 03:03:30.980056': -15.966953600880027,
 '1992-03-07 03:03:33.124006': None,
 '1992-03-23 03:03:31.886081': -9.072406936468196,
 '1992-04-08 03:03:22.351094': -7.136467030493817,
 '1992-04-24 03:03:14.014013': -21.83880130622505,
 '1992-05-10 03:03:07.137063': -3.676646497578922,
 '1992-05-26 03:02:58.130000': -0.10727642002023889,
 '1992-06-11 03:02:48.118069': 2.4068923222076926,
 '1992-06-27 03:02:36.377025': 3.9931176333485277,
 '1992-07-13 03:02:24.677044': 7.107717201934344,
 '1992-07-29 03:02:12.602000': 11.000944816889163,
 '1992-08-14 03:01:58.692000': 0.6743981960432314,
 '1992-09-15 03:01:29.056063': 6.366397290320389,
 '1992-10-01 03:01:07.618013': 0.043073120158134706,
 '1992-10-17 03:00:51.750081': 5.572214242531494,
 '1992-11-18 03:00:23.194025': -6.340225559944381,
 '1992-11-02 03:01:03.471088': None,
 '1992-02-11 03:08:54.930019': -19.531094549960706,
 '1992-05-17 03:08:26.107013': -0.8320127377171647,
 '1992-07-20 03:07:42.140000': -30.134721269163474,
 '1992-08-05 03:07:30.388094': 10.556642339432274,
 '1992-09-22 03:06:43.470081': 4.850769555791162,
 '1992-11-09 03:05:52.890006': -0.8559703862437981,
 '1992-02-11 03:09:18.837025': -18.260255814775423,
 '1992-03-14 03:09:19.772025': -14.904715163205768,
 '1992-03-30 03:09:14.151056': -6.229800426073007,
 '1992-04-15 03:09:08.406050': -15.815174917822178,
 '1992-05-17 03:08:50.020081': -1.9297977810601685,
 '1992-06-02 03:08:41.323056': 1.235039795019876,
 '1992-08-05 03:07:54.310031': -2.8209565685493927,
 '1992-11-09 03:06:16.819094': 2.03226281291897,
 '1992-01-26 03:09:51.916075': -16.22487348690835,
 '1992-02-11 03:09:42.888044': -19.597090582923084,
 '1992-03-14 03:09:43.825006': -12.426507130794167,
 '1992-03-30 03:09:38.205056': -5.0440003998713925,
 '1992-04-15 03:09:32.461075': -8.245977362160083,
 '1992-05-01 03:09:23.351050': -22.11664456033407,
 '1992-05-17 03:09:14.078038': -1.824000838369016,
 '1992-06-02 03:09:05.382044': 3.0904212429480884,
 '1992-07-20 03:08:30.122056': -2.2799918142236386,
 '1992-08-05 03:08:18.231075': 6.182869823454441,
 '1992-09-22 03:07:31.467019': 2.1733984578963765,
 '1992-11-09 03:06:40.749069': 2.8912366338755167,
 '1992-01-26 03:10:15.534069': -20.521867588524977,
 '1992-02-11 03:10:06.651044': -17.15597112535463,
 '1992-02-27 03:10:07.766006': -15.216619162173902,
 '1992-03-14 03:10:07.589094': -14.293462404669388,
 '1992-03-30 03:10:01.971044': -3.895157860743267,
 '1992-04-15 03:09:56.228081': -10.205874738683642,
 '1992-05-01 03:09:47.119063': -3.98607772776598,
 '1992-05-17 03:09:37.703081': -2.569171042847255,
 '1992-06-02 03:09:29.153025': 4.918052714846867,
 '1992-06-18 03:09:15.436081': 0.5354526942530796,
 '1992-07-04 03:09:05.647031': -8.884147372568583,
 '1992-08-05 03:08:42.009006': 13.599098070096854,
 '1992-08-21 03:08:25.294031': 5.761261788710746,
 '1992-10-08 03:07:29.530094': None,
 '1992-11-09 03:07:04.535025': 2.722281669273347,
 '1992-11-25 03:06:59.410013': -1.5756752278666313,
 '1992-02-02 03:15:39.528044': None,
 '1992-02-18 03:15:28.595031': -16.908502523664225,
 '1992-03-05 03:15:29.739013': -25.454075712222828,
 '1992-03-21 03:15:30.788063': -10.209126559241067,
 '1992-04-22 03:15:14.248069': -2.6939937494373454,
 '1992-05-24 03:14:56.470038': -4.527775789212789,
 '1992-07-11 03:14:23.431094': 4.375548067239566,
 '1992-07-27 03:14:11.395025': 9.543599121889766,
 '1992-08-28 03:13:42.605050': 9.978430423837068,
 '1992-09-13 03:13:28.683075': 7.506747583327347,
 '1992-09-29 03:13:08.267013': -2.9458634368619436,
 '1992-10-31 03:12:39.628000': -1.642639464121429,
 '1992-11-16 03:12:21.807050': -0.02329691280201901,
 '1992-02-02 03:16:03.435013': -15.586007206777598,
 '1992-02-18 03:15:52.646088': -15.440143116371177,
 '1992-03-05 03:15:53.791063': None,
 '1992-03-21 03:15:54.841094': -9.245489877222035,
 '1992-04-22 03:15:38.304044': -6.028964942749806,
 '1992-05-08 03:15:28.960050': -1.0514582606513825,
 '1992-05-24 03:15:20.384063': 2.768005060353999,
 '1992-07-11 03:14:47.494050': 8.856762053721454,
 '1992-07-27 03:14:35.459088': 12.156016459813165,
 '1992-08-28 03:14:06.674000': 8.385481302714751,
 '1992-09-13 03:13:52.753075': 9.112550123565534,
 '1992-09-29 03:13:32.338013': -0.2562197649829603,
 '1992-10-15 03:13:14.147056': 5.634814617199834,
 '1992-10-31 03:13:03.701069': 3.092125117338679,
 '1992-11-16 03:12:45.882038': -10.39947353937968,
 '1992-02-02 03:16:27.197075': -19.57364900546587,
 '1992-02-18 03:16:16.410038': -18.558090653785705,
 '1992-03-05 03:16:17.556000': None,
 '1992-03-21 03:16:18.607019': -10.227643575929326,
 '1992-04-06 03:16:09.558069': -17.731541054009668,
 '1992-04-22 03:16:01.928006': -10.85878132310923,
 '1992-05-08 03:15:52.585031': -1.1026705404282768,
 '1992-05-24 03:15:44.154075': 1.6416409575131592,
 '1992-06-09 03:15:35.182000': -17.859146157995216,
 '1992-06-25 03:15:22.834075': 2.7715090501094797,
 '1992-07-11 03:15:11.124081': 7.579395789786485,
 '1992-07-27 03:14:59.236031': 14.713007337745955,
 '1992-08-28 03:14:30.310013': 12.395339755649717,
 '1992-09-29 03:13:55.976069': None,
 '1992-10-15 03:13:37.931081': 6.0105385185121305,
 '1992-10-31 03:13:27.342088': 2.6832375395386574,
 '1992-11-16 03:13:09.524081': -10.99766887029969,
 '1992-01-24 03:22:35.146088': -18.37841621494073,
 '1992-02-09 03:22:29.886050': -21.817177278979344,
 '1992-02-25 03:22:30.122075': -11.488749119681898,
 '1992-03-12 03:22:28.823038': -11.877670808819087,
 '1992-03-28 03:22:22.931081': None,
 '1992-04-13 03:22:18.181075': -4.593786931222838,
 '1992-05-31 03:21:51.983044': -3.7076834667978393,
 '1992-06-16 03:21:39.247056': -19.63856729540285,
 '1992-07-02 03:21:28.477088': 8.882354633577334,
 '1992-08-03 03:21:05.261088': -22.27277366599521,
 '1992-09-20 03:20:19.228031': 7.108478500350862,
 '1992-10-06 03:19:54.863075': 6.156891255036474,
 '1992-11-07 03:19:29.265006': 1.86506537189019,
 '1992-11-23 03:19:19.647069': -0.3275382532751915,
 '1992-01-31 03:28:49.856044': None,
 '1992-02-16 03:28:37.221069': -12.385082163110189,
 '1992-03-03 03:28:38.773075': -15.178830517278824,
 '1992-03-19 03:28:41.154056': -12.317501080423055,
 '1992-04-04 03:28:32.823056': -9.086588598849156,
 '1992-04-20 03:28:25.811088': -1.3650126726077743,
 '1992-05-06 03:28:15.866069': -7.344108259103845,
 '1992-05-22 03:28:06.142094': 6.754255272718328,
 '1992-06-07 03:27:58.299088': 2.5170816763936825,
 '1992-06-23 03:27:45.362044': 4.8469761941140055,
 '1992-07-09 03:27:33.955094': 5.004463133499716,
 '1992-07-25 03:27:21.905075': 11.291527907348186,
 '1992-08-10 03:27:09.879081': 14.115403998311532,
 '1992-08-26 03:26:53.729006': -4.987823928168842,
 '1992-09-27 03:26:20.582000': None,
 '1992-10-13 03:26:00.156050': 6.837755120390337,
 '1992-10-29 03:25:51.025000': -1.415636409495975,
 '1992-11-14 03:25:32.092075': -2.6787780176627716,
'1993-02-06 03:00:27.445075': -26.633583429961416,
 '1993-02-22 03:00:32.198025': -23.00494620420298,
 '1993-03-26 03:00:44.966081': None,
 '1993-04-11 03:00:53.253019': -5.253300851817696,
 '1993-04-27 03:01:01.252038': -4.366278428511648,
 '1993-05-13 03:01:03.667050': -2.8356188296805342,
 '1993-05-29 03:01:06.831069': 1.6011239956722725,
 '1993-06-30 03:00:59.689088': 8.043167617768502,
 '1993-07-16 03:00:59.544031': 12.307697490879704,
 '1993-08-01 03:00:58.820006': 23.857921702529588,
 '1993-10-04 03:00:50.170006': 3.403150895908089,
 '1993-02-06 03:00:51.237050': -21.93819362106919,
 '1993-02-22 03:00:55.991031': -19.526148327419847,
 '1993-03-26 03:01:08.762019': -14.364698045015906,
 '1993-04-11 03:01:17.050019': -7.236847636007425,
 '1993-04-27 03:01:24.906063': -3.052034623015737,
 '1993-05-13 03:01:27.467013': -5.202290666264962,
 '1993-05-29 03:01:30.632056': 0.8157415658596006,
 '1993-06-14 03:01:29.677019': 2.903213720267415,
 '1993-06-30 03:01:23.494000': 3.730700854895071,
 '1993-07-16 03:01:23.349088': 8.43183558704628,
 '1993-08-01 03:01:22.627050': 20.125312437575797,
 '1993-10-04 03:01:13.983000': 4.571042426254769,
 '1993-02-06 03:01:15.317081': -18.434356134841902,
 '1993-02-22 03:01:20.072088': -18.46862060928427,
 '1993-03-26 03:01:32.846000': -10.081434259917186,
 '1993-04-11 03:01:41.135069': -7.960444131474467,
 '1993-04-27 03:01:48.993056': -2.230850230023303,
 '1993-05-13 03:01:51.555006': -0.35775406375054647,
 '1993-05-29 03:01:54.722000': 1.9346985668275438,
 '1993-06-14 03:01:53.767094': 5.061932874841751,
 '1993-06-30 03:01:47.586063': 5.1982851844780535,
 '1993-07-16 03:01:47.299069': 12.387008342005531,
 '1993-08-01 03:01:46.723044': 15.246746546451567,
 '1993-10-04 03:01:38.084063': 6.656197371043251,
 '1993-05-04 03:07:12.689075': -6.768974205358809,
 '1993-05-20 03:07:14.688094': -2.817282727484553,
 '1993-06-05 03:07:18.143000': 2.7377206096122135,
 '1993-09-09 03:07:06.661006': 8.717943774192573,
 '1993-10-11 03:06:58.808000': None,
 '1993-03-01 03:07:11.551050': -14.848636440828228,
 '1993-03-17 03:07:20.403031': -10.867273484995414,
 '1993-05-04 03:07:36.488050': -7.124973957418413,
 '1993-05-20 03:07:38.633031': -1.6933552209960758,
 '1993-06-05 03:07:42.088063': 2.4431719935600915,
 '1993-09-09 03:07:30.616025': 8.740741694023438,
 '1993-09-25 03:07:26.894025': None,
 '1993-01-28 03:07:19.367013': -15.828828435774128,
 '1993-02-13 03:07:29.414025': -15.611072359984867,
 '1993-03-01 03:07:35.633013': -12.51233926525272,
 '1993-03-17 03:07:44.486025': -6.536772972214787,
 '1993-05-04 03:08:00.575069': -8.378479252274854,
 '1993-05-20 03:08:02.577081': -1.6795391864838565,
 '1993-06-05 03:08:06.178056': 2.9859381404908607,
 '1993-09-09 03:07:54.571038': 9.673749489497133,
 '1993-10-11 03:07:46.724038': 4.382591797696964,
 '1993-01-28 03:07:43.158044': -15.412861080933247,
 '1993-03-01 03:07:59.426044': -13.10506171963408,
 '1993-03-17 03:08:08.136056': -7.352669895277526,
 '1993-05-04 03:08:24.230025': -1.4949988793150626,
 '1993-05-20 03:08:26.378000': -12.809541320629483,
 '1993-06-05 03:08:29.835069': 4.454234859380904,
 '1993-09-09 03:08:18.238006': 8.335535459069176,
 '1993-10-11 03:08:10.393094': 5.188431342565369,
 '1993-11-12 03:07:56.670050': -1.1287589073022746,
 '1993-11-28 03:07:52.115050': -12.832487615457444,
 '1993-02-04 03:13:11.586075': -20.420792752800335,
 '1993-03-08 03:13:28.417081': -9.832402113261073,
 '1993-03-24 03:13:29.682088': -8.191711386553788,
 '1993-04-09 03:13:37.115088': -6.5189162289635885,
 '1993-04-25 03:13:46.109088': -11.284943516478233,
 '1993-05-11 03:13:48.833063': -28.52358583071548,
 '1993-05-27 03:13:51.691006': 2.0530026418012532,
 '1993-06-12 03:13:51.788081': 1.7783440537462176,
 '1993-06-28 03:13:46.022094': None,
 '1993-07-14 03:13:44.909025': 14.335119241003497,
 '1993-08-15 03:13:44.834013': 10.301348587732267,
 '1993-08-31 03:13:42.728044': 11.979831165448873,
 '1993-10-02 03:13:36.040038': -25.467640436376925,
 '1993-10-18 03:13:30.243056': 4.220312116486312,
 '1993-02-04 03:13:35.666075': -14.510783060580085,
 '1993-03-08 03:13:52.500019': -12.463280312352804,
 '1993-03-24 03:13:53.622038': -6.618906022376074,
 '1993-04-09 03:14:01.201019': -6.527597124828798,
 '1993-04-25 03:14:10.052056': -4.170798407967656,
 '1993-05-11 03:14:12.777038': -0.2702345223962518,
 '1993-05-27 03:14:15.636000': 1.3316224863988804,
 '1993-06-12 03:14:15.879044': 5.193459992933125,
 '1993-06-28 03:14:10.115044': 2.363502701191095,
 '1993-07-14 03:14:08.858088': 13.314554895277334,
 '1993-08-15 03:14:08.787031': 12.803686614056332,
 '1993-08-31 03:14:06.827025': 11.800761129732285,
 '1993-10-02 03:14:00.141081': 4.380937202795727,
 '1993-10-18 03:13:54.346081': 6.278264892033822,
 '1993-02-04 03:13:59.314031': -13.202851476134207,
 '1993-03-08 03:14:16.294013': -11.451179691680146,
 '1993-03-24 03:14:17.417063': -6.811595203067092,
 '1993-04-09 03:14:24.853069': -6.938966789496525,
 '1993-04-25 03:14:33.851006': -1.553638650137526,
 '1993-05-11 03:14:36.576094': -2.0236605790968905,
 '1993-05-27 03:14:39.436063': None,
 '1993-06-12 03:14:39.537031': 4.249859392005634,
 '1993-06-28 03:14:33.919044': 1.6414056183013683,
 '1993-07-14 03:14:32.664038': None,
 '1993-08-15 03:14:32.452000': 13.95373855362779,
 '1993-08-31 03:14:30.493025': 12.996962340469963,
 '1993-10-02 03:14:23.810038': 7.174072998389596,
 '1993-10-18 03:14:18.017013': 7.009328765064662,
 '1993-01-26 03:19:40.022019': -18.40214131395433,
 '1993-02-11 03:19:50.325075': -14.539012067973413,
 '1993-03-31 03:20:08.614025': None,
 '1993-09-07 03:20:16.550025': 11.151283508611778,
 '1993-09-23 03:20:12.674094': 11.782436958234523,
 '1993-01-26 03:20:03.813031': -21.62894433080329,
 '1993-02-11 03:20:13.973081': -17.344421826775708,
 '1993-02-27 03:20:18.746044': -20.616712541015307,
 '1993-03-15 03:20:29.656025': None,
 '1993-03-31 03:20:32.410019': -16.22685427341192,
 '1993-04-16 03:20:40.404025': -4.1215287950911685,
 '1993-05-02 03:20:45.909019': -0.8444563179992723,
 '1993-07-05 03:20:42.721019': 9.537578930294508,
 '1993-09-07 03:20:40.361000': 10.723787417574135,
 '1993-09-23 03:20:36.487044': 10.502555563981701,
 '1993-02-02 03:26:19.650056': -22.763495245824128,
 '1993-02-18 03:26:26.816025': -33.692971448624014,
 '1993-03-06 03:26:36.485056': -9.322263558619115,
 '1993-03-22 03:26:39.655094': -4.138601916249128,
 '1993-04-07 03:26:45.168050': -5.701472686170366,
 '1993-04-23 03:26:54.752019': -3.7515396278880564,
 '1993-05-09 03:26:57.796025': -30.545380435745315,
 '1993-05-25 03:27:00.508031': 5.013806571891049,
 '1993-06-10 03:27:01.630075': 6.988258169766439,
 '1993-06-26 03:26:56.646031': None,
 '1993-07-12 03:26:53.920025': -0.724086448843426,
 '1993-09-30 03:26:46.066000': -5.127834888041045,
 '1993-11-17 03:26:25.423081': -4.226080715628773,
'1994-02-09 02:59:41.834006': -23.148379059522462,
 '1994-03-29 02:58:59.577000': -8.218426319109335,
 '1994-04-14 02:58:39.905038': None,
 '1994-04-30 02:58:23.020006': -2.856283276200338,
 '1994-05-16 02:58:04.512050': -1.17871129483999,
 '1994-07-03 02:57:03.934006': 4.267816857187177,
 '1994-07-19 02:56:38.290056': 11.91171381224394,
 '1994-08-04 02:56:10.284063': 2.3292408062386243,
 '1994-09-05 02:55:23.219075': None,
 '1994-10-23 02:53:59.379094': 3.269996844220329,
 '1994-11-08 02:53:25.918088': -10.952187288807476,
 '1994-02-09 03:00:05.658081': -20.81475454939475,
 '1994-03-29 02:59:23.262063': -6.63196485729665,
 '1994-04-14 02:59:03.736025': None,
 '1994-04-30 02:58:46.852000': -4.320478314473971,
 '1994-05-16 02:58:28.345081': -0.41508247009788163,
 '1994-07-03 02:57:27.770075': 0.9980148003176839,
 '1994-07-19 02:57:02.128031': 9.136051528831638,
 '1994-08-04 02:56:34.123031': 6.978299574619968,
 '1994-09-05 02:55:47.060094': 8.065500435029039,
 '1994-10-23 02:54:23.225019': 4.929478180188716,
 '1994-11-08 02:53:49.620044': None,
 '1994-02-09 03:00:29.772038': -18.39885718405182,
 '1994-03-29 02:59:47.381050': -8.894659818517548,
 '1994-04-14 02:59:27.856000': None,
 '1994-04-30 02:59:10.828050': -2.494931477318534,
 '1994-05-16 02:58:52.468000': 0.957907090645594,
 '1994-07-03 02:57:51.751081': None,
 '1994-07-19 02:57:26.255000': 8.791900867419326,
 '1994-09-05 02:56:11.046069': 9.60422581859395,
 '1994-10-23 02:54:47.214088': 6.283814815364182,
 '1994-03-20 03:05:19.194025': -13.15098940312644,
 '1994-04-21 03:04:43.695031': None,
 '1994-05-07 03:04:25.259031': None,
 '1994-05-23 03:04:08.230000': -3.0310796639949427,
 '1994-06-24 03:03:28.893094': 7.437882399987729,
 '1994-07-10 03:03:03.845069': 13.586749358587433,
 '1994-08-27 03:01:47.608006': 8.534129571156855,
 '1994-09-12 03:01:22.967031': 3.5189337530477,
 '1994-09-28 03:00:56.564063': -0.7769561189750029,
 '1994-01-31 03:06:22.010019': -18.669258039098835,
 '1994-03-04 03:05:56.817056': -19.336788850114036,
 '1994-03-20 03:05:43.023031': -11.964316219822729,
 '1994-04-21 03:05:07.526069': -8.90070555592189,
 '1994-05-07 03:04:49.236025': -1.2968465406312304,
 '1994-05-23 03:04:32.208000': -3.8786245985724195,
 '1994-06-24 03:03:52.874013': 3.9939177121443503,
 '1994-07-10 03:03:27.682081': 10.38677164833943,
 '1994-08-27 03:02:11.448081': 11.222379706904492,
 '1994-09-12 03:01:46.809013': 8.93941579271685,
 '1994-09-28 03:01:20.407063': 2.9177443099320537,
 '1994-11-15 02:59:46.016063': -0.2612253163593088,
 '1994-03-04 03:06:20.789006': -16.998038002345833,
 '1994-03-20 03:06:06.996088': -11.690988993234603,
 '1994-04-21 03:05:31.502050': -8.47734349077036,
 '1994-05-07 03:05:13.213013': -3.1132785433302264,
 '1994-05-23 03:04:56.185088': None,
 '1994-07-10 03:03:51.664038': 11.323985397331418,
 '1994-07-26 03:03:24.830031': None,
 '1994-08-27 03:02:35.433094': 9.831561641372025,
 '1994-09-12 03:02:10.795063': 11.710330123479322,
 '1994-09-28 03:01:44.539069': 0.38012997517573577,
 '1994-10-30 03:00:43.108056': -2.307435316873447,
 '1994-11-15 03:00:10.007075': 2.506820855515858,
 '1994-01-31 03:07:09.803000': -12.806132472129185,
 '1994-03-04 03:06:44.471075': -17.65521651898376,
 '1994-03-20 03:06:30.825094': -9.710725678600188,
 '1994-04-21 03:05:55.189044': -9.18207981129451,
 '1994-05-07 03:05:36.901000': -1.5108690058257341,
 '1994-05-23 03:05:19.874094': None,
 '1994-07-10 03:04:15.357006': 14.442646716940311,
 '1994-08-27 03:02:59.274063': 10.052712234719124,
 '1994-09-12 03:02:34.637044': 12.246478099215128,
 '1994-09-28 03:02:08.238019': 1.5678671262651906,
 '1994-10-30 03:01:06.954025': 1.896173736012274,
 '1994-11-15 03:00:33.709088': 2.504225373396471,
 '1994-02-07 03:12:28.621094': -23.811091749287144,
 '1994-02-23 03:12:14.957063': -20.33248025921784,
 '1994-03-11 03:12:00.782025': -14.252349400439481,
 '1994-04-28 03:11:10.708013': None,
 '1994-06-15 03:10:15.952094': 3.5300464793465416,
 '1994-08-02 03:08:59.147031': 13.996690122850369,
 '1994-10-21 03:06:48.794050': 5.798782733587713,
 '1994-11-06 03:06:15.235063': -14.323932561554074,
 '1994-02-23 03:12:39.071050': -21.474673270889987,
 '1994-03-11 03:12:24.754069': -12.7404491026868,
 '1994-03-27 03:12:11.118069': -3.0425404756337215,
 '1994-04-12 03:11:52.145081': None,
 '1994-04-28 03:11:34.828075': None,
 '1994-06-15 03:10:40.077013': 4.919856052633324,
 '1994-08-02 03:09:23.130044': 14.080445646780072,
 '1994-09-19 03:08:10.251013': 8.451433828927724,
 '1994-10-21 03:07:12.928063': 5.6403098617628205,
 '1994-11-06 03:06:39.370081': 0.4419926760664789,
 '1994-02-07 03:13:16.560006': -14.157943990846892,
 '1994-02-23 03:13:02.752019': None,
 '1994-03-11 03:12:48.438025': -13.631461143355482,
 '1994-03-27 03:12:34.804038': -1.702538531285024,
 '1994-04-12 03:12:15.832013': -12.29481866404091,
 '1994-06-15 03:11:03.767088': 5.76648363556824,
 '1994-08-02 03:09:46.824056': 10.481300795700287,
 '1994-09-19 03:08:34.093044': 7.044009300119027,
 '1994-10-21 03:07:36.629031': 6.3796944649416805,
 '1994-11-06 03:07:03.072056': 3.1498218826090567,
 '1994-11-22 03:06:29.238094': -1.3360695390594663,
 '1994-01-29 03:19:32.517038': -12.935396531510905,
 '1994-02-14 03:19:21.896006': -17.332926828421765,
 '1994-03-02 03:19:07.909069': -14.739224420559704,
 '1994-03-18 03:18:54.118013': -9.238643791536433,
 '1994-04-19 03:18:19.229081': -0.11563898310229884,
 '1994-05-05 03:18:01.041094': 1.0661169624636875,
 '1994-05-21 03:17:43.664094': 1.7587782421761262,
 '1994-07-08 03:16:40.225063': 9.928457724647396,
 '1994-07-24 03:16:13.548025': 11.529437311214002,
 '1994-08-25 03:15:23.853013': 14.006865262869889,
 '1994-09-10 03:14:59.371044': 11.324274966494182,
 '1994-10-28 03:13:32.858025': -11.103269022776997,
 '1994-02-21 03:25:26.448019': None,
 '1994-04-10 03:24:40.096013': -1.95483138839598,
 '1994-04-26 03:24:22.361050': -0.5777859599002992,
 '1994-05-28 03:23:47.139031': -17.473857319049788,
 '1994-06-29 03:23:05.764081': None,
 '1994-07-31 03:22:12.135031': 10.72029053204261,
 '1994-09-17 03:20:58.970063': 6.005182534369042,
 '1994-11-20 03:18:55.250081': 3.0054804706954794,
'1995-04-01 02:48:07.375000': -11.826925425008318,
 '1995-04-17 02:47:26.701069': -7.719552387433407,
 '1995-06-04 02:45:16.269044': -1.3114556437139624,
 '1995-06-20 02:44:34.065025': None,
 '1995-07-06 02:43:50.459050': 5.432263118336761,
 '1995-08-07 02:42:22.887000': 0.2489399006940613,
 '1995-10-26 02:38:21.530050': -0.7429724353311489,
 '1995-03-16 02:49:10.921044': None,
 '1995-04-01 02:48:31.233019': -10.60446538773757,
 '1995-04-17 02:47:50.561038': -7.182423540074354,
 '1995-05-19 02:46:24.689025': None,
 '1995-06-04 02:45:40.132006': -0.04567147338467222,
 '1995-06-20 02:44:57.784038': None,
 '1995-07-06 02:44:14.324056': 4.031058384172037,
 '1995-07-22 02:43:30.513044': None,
 '1995-08-07 02:42:46.754094': 4.530477329836351,
 '1995-08-23 02:42:02.084038': None,
 '1995-10-26 02:38:45.404006': 1.4080914581518245,
 '1995-11-11 02:38:26.798006': -11.393207305229197,
 '1995-03-16 02:49:35.065063': None,
 '1995-04-01 02:48:55.236000': -8.436980068161311,
 '1995-04-17 02:48:14.710025': -4.104589372320121,
 '1995-05-19 02:46:48.840019': -0.47789590350112127,
 '1995-06-04 02:46:04.284000': -0.12512673654206494,
 '1995-07-06 02:44:38.334019': 5.377412593839791,
 '1995-07-22 02:43:54.524063': 5.819107902148277,
 '1995-08-07 02:43:10.767056': 4.628350917331805,
 '1995-10-26 02:39:09.422038': 0.4604862158772096,
 '1995-11-11 02:38:50.817063': -9.958267244351358,
 '1995-08-07 02:43:34.490094': None,
 '1995-08-23 02:42:49.822081': 7.262278871848338,
 '1995-10-26 02:39:33.151038': -7.6295743894674315,
 '1995-02-19 02:55:57.725038': -24.73267841926522,
 '1995-03-23 02:54:39.837050': -15.957681271671026,
 '1995-04-08 02:54:00.666013': -8.338122807838548,
 '1995-04-24 02:53:19.132081': None,
 '1995-05-10 02:52:35.623069': -3.5804193871794667,
 '1995-05-26 02:51:52.498044': -5.880212092909301,
 '1995-06-11 02:51:08.147031': 2.6452608169543597,
 '1995-07-13 02:49:41.592069': 11.096652714875805,
 '1995-07-29 02:48:58.490063': 11.96611700233517,
 '1995-08-14 02:48:14.307075': 10.423579445345519,
 '1995-02-03 02:57:00.411019': -19.56767889420545,
 '1995-02-19 02:56:21.577063': -20.004266043356026,
 '1995-03-07 02:55:44.350019': -9.794427108999745,
 '1995-03-23 02:55:03.549038': -15.075696438577184,
 '1995-04-08 02:54:24.524088': -9.857979568040443,
 '1995-04-24 02:53:42.992081': -11.08491895866165,
 '1995-05-10 02:52:59.484081': -6.741321662994375,
 '1995-05-26 02:52:16.360038': -3.048371083106001,
 '1995-06-11 02:51:32.010044': 2.255795245227884,
 '1995-07-13 02:50:05.313069': 8.23046019182113,
 '1995-07-29 02:49:22.213019': 9.78238168912262,
 '1995-08-14 02:48:38.176000': 12.530850703622525,
 '1995-08-30 02:47:53.060063': 12.592378602473353,
 '1995-02-03 02:57:24.406081': -18.20669733830003,
 '1995-02-19 02:56:45.719000': -16.06906811145614,
 '1995-03-07 02:56:08.347056': -9.792203139602226,
 '1995-03-23 02:55:27.550031': -12.517342269716385,
 '1995-04-08 02:54:48.528044': -10.255477832994737,
 '1995-04-24 02:54:06.997044': -3.4462622787076675,
 '1995-05-10 02:53:23.635019': -15.140009905967492,
 '1995-05-26 02:52:40.366094': None,
 '1995-06-11 02:51:56.018019': 1.9357075397750754,
 '1995-07-13 02:50:29.324000': 9.376697013064561,
 '1995-07-29 02:49:46.224081': 11.494509463664645,
 '1995-08-14 02:49:02.189000': 12.042964813896024,
 '1995-08-30 02:48:17.075019': 13.020703728834675,
 '1995-02-03 02:57:48.113038': -17.79286546538819,
 '1995-02-19 02:57:09.426069': -16.861077077995343,
 '1995-03-07 02:56:32.055075': None,
 '1995-03-23 02:55:51.262006': -11.037543600726849,
 '1995-04-08 02:55:12.242081': -6.474999891401025,
 '1995-04-24 02:54:30.857050': -1.861461237697821,
 '1995-05-10 02:53:47.351063': None,
 '1995-05-26 02:53:04.228075': -1.0728687645019372,
 '1995-06-11 02:52:19.736075': 4.38344713072937,
 '1995-06-27 02:51:37.569063': None,
 '1995-07-13 02:50:53.044088': 11.662277636135823,
 '1995-07-29 02:50:09.947025': 13.417940087370392,
 '1995-08-14 02:49:25.912069': 13.362173702099046,
 '1995-08-30 02:48:40.800031': 13.886678089801064,
 '1995-02-26 03:02:16.603013': -15.466287945392045,
 '1995-03-14 03:01:37.699056': None,
 '1995-05-01 02:59:35.124075': -10.246198314289952,
 '1995-05-17 02:58:51.771063': -0.23810670275360019,
 '1995-06-18 02:57:24.698044': 4.7062026892563456,
 '1995-07-04 02:56:41.540094': 5.171254031178595,
 '1995-08-21 02:54:29.286056': 10.407304539685839,
 '1995-10-08 02:52:06.791044': -6.836507206909415,
 '1995-10-24 02:51:13.932000': -35.60156946893735,
 '1995-02-26 03:02:40.600069': -12.190420845006246,
 '1995-03-14 03:02:01.698088': -18.778715313773745,
 '1995-04-15 03:00:41.457025': -5.585650695488142,
 '1995-05-01 02:59:59.129088': -4.348051842775503,
 '1995-05-17 02:59:15.777069': -1.137537812272627,
 '1995-06-18 02:57:48.706063': 3.6469409741106835,
 '1995-07-04 02:57:05.550050': 8.6673075892424,
 '1995-08-21 02:54:53.300025': 15.40342955483835,
 '1995-10-08 02:52:30.808063': -0.6956265537887746,
 '1995-10-24 02:51:37.950006': -0.37172374285608234,
 '1995-11-09 02:51:13.546088': -7.2350205475786575,
 '1995-01-25 03:04:20.116000': -13.195013486047664,
 '1995-02-26 03:03:04.309006': -12.314340072060585,
 '1995-03-14 03:02:25.408094': -17.409843990071074,
 '1995-04-15 03:01:05.172025': -4.668973620411293,
 '1995-05-01 03:00:22.845081': None,
 '1995-05-17 02:59:39.639019': 1.7571348935862012,
 '1995-06-18 02:58:12.425063': 2.7296514097159488,
 '1995-07-04 02:57:29.270075': 10.776736041814635,
 '1995-08-21 02:55:17.024050': 14.15648973500548,
 '1995-10-08 02:52:54.536050': 4.804944745522249,
 '1995-10-24 02:52:01.534013': 4.472141817580878,
 '1995-11-09 02:51:37.276088': 1.0870033989566878,
 '1995-02-01 03:10:14.587069': -12.56552971121328,
 '1995-03-05 03:08:58.611006': -12.806559181403241,
 '1995-03-21 03:08:18.174075': -10.407884600312869,
 '1995-05-08 03:06:14.568013': -3.547353941168058,
 '1995-05-24 03:05:31.302050': -1.2327755709351154,
 '1995-06-25 03:04:04.695006': 5.6346022634778015,
 '1995-07-11 03:03:20.408050': 8.246014902153178,
 '1995-08-12 03:01:53.116075': 12.724063986359765,
 '1995-08-28 03:01:08.160088': 13.713353241988363,
 '1995-10-15 02:58:42.608038': None,
 '1995-11-16 02:57:48.884013': -0.07647400625398355,
 '1995-02-08 03:16:08.331025': -5.860854698258215,
 '1995-03-12 03:14:52.157025': -18.252967740360113,
 '1995-03-28 03:14:11.942038': -1.1329328913313463,
 '1995-04-13 03:13:31.955069': None,
 '1995-05-15 03:12:06.566038': None,
 '1995-05-31 03:11:22.600013': -35.77707609845995,
 '1995-11-07 03:04:00.027019': 4.21730762744494,
 '1995-11-23 03:04:12.549094': -25.213852405886932,
'1996-02-15 02:44:00.342031': -28.36873539374858,
 '1996-04-19 02:48:07.977088': -6.172183668628766,
 '1996-05-05 02:49:04.827013': -9.667157366075006,
 '1996-05-21 02:49:59.903063': -5.104927067466572,
 '1996-06-22 02:51:45.406050': 3.097755526854493,
 '1996-09-26 02:56:53.141094': -1.6897708555248214,
 '1996-11-13 02:59:17.068069': -19.975515822189802,
 '1996-02-15 02:44:24.077031': -27.874870953355156,
 '1996-04-03 02:47:32.815056': -12.773631220905221,
 '1996-04-19 02:48:31.720063': -5.209815974747305,
 '1996-05-05 02:49:28.570081': -8.683085243577208,
 '1996-05-21 02:50:23.793013': -2.3380336771029793,
 '1996-06-06 02:51:17.261031': None,
 '1996-06-22 02:52:09.153019': 2.406653070960133,
 '1996-09-26 02:57:17.040056': 2.48772457955174,
 '1996-11-13 02:59:40.826019': -11.020557295405915,
 '1996-02-15 02:44:48.246056': -23.957370631660357,
 '1996-04-03 02:47:56.846088': -9.799675882298018,
 '1996-04-19 02:48:55.753006': -4.527767289705448,
 '1996-05-05 02:49:52.603088': -11.663947256116142,
 '1996-05-21 02:50:47.827038': 0.40517792836595073,
 '1996-06-06 02:51:41.296044': 1.5204429843045235,
 '1996-06-22 02:52:33.334031': 3.179968578283658,
 '1996-09-26 02:57:41.084006': 5.432976577148997,
 '1996-11-13 03:00:05.018025': -4.2846596514815545,
 '1996-03-25 02:52:45.782044': -13.571061544428803,
 '1996-04-26 02:54:43.715013': -5.419403230987922,
 '1996-05-12 02:55:39.967019': -10.151290454705972,
 '1996-06-13 02:57:27.089044': 3.006540228797728,
 '1996-06-29 02:58:18.377081': -1.6862938600551547,
 '1996-07-15 02:59:08.778056': 14.456131580834148,
 '1996-07-31 03:00:00.219019': 13.579923758677632,
 '1996-09-01 03:01:42.204006': None,
 '1996-09-17 03:02:34.966044': 6.956017831010478,
 '1996-10-19 03:04:15.643069': -0.04749502298158533,
 '1996-03-25 02:53:09.523069': -12.872173680406418,
 '1996-04-26 02:55:07.603000': -9.626126724273092,
 '1996-05-12 02:56:03.711013': -8.792834058048326,
 '1996-05-28 02:56:58.081081': None,
 '1996-06-13 02:57:50.980038': 2.544825543555945,
 '1996-06-29 02:58:42.269088': 4.070636257689404,
 '1996-07-15 02:59:32.671088': 10.874951369045268,
 '1996-07-31 03:00:24.113019': 12.953521179777606,
 '1996-09-01 03:02:06.100063': 9.772670430135559,
 '1996-09-17 03:02:58.719063': 4.103549008424245,
 '1996-10-19 03:04:39.544006': -2.5784057067309902,
 '1996-02-22 02:51:27.505025': -15.656201454304673,
 '1996-03-25 02:53:33.554050': -10.152153412491337,
 '1996-04-26 02:55:31.635075': -6.423750502279584,
 '1996-05-12 02:56:27.744069': -3.017649403869346,
 '1996-05-28 02:57:22.116025': -6.615640455165648,
 '1996-06-13 02:58:15.016019': 1.1457074416214614,
 '1996-06-29 02:59:06.306063': 5.218184976426177,
 '1996-07-15 02:59:56.709088': 14.573133899616138,
 '1996-07-31 03:00:48.152000': 12.893630832187785,
 '1996-09-01 03:02:30.142006': 9.66547925528154,
 '1996-09-17 03:03:22.762038': 0.6741535482600495,
 '1996-10-19 03:05:03.589025': -0.7617279997071766,
 '1996-02-06 02:50:45.757006': None,
 '1996-02-22 02:51:51.241019': -16.12455832095217,
 '1996-03-09 02:52:55.160075': -10.902815911544373,
 '1996-03-25 02:53:57.295081': -8.390873175540314,
 '1996-04-26 02:55:55.378088': -1.7445998983745477,
 '1996-05-12 02:56:51.488075': -2.5736901729280555,
 '1996-05-28 02:57:45.861019': None,
 '1996-06-13 02:58:38.762038': 2.618185840640192,
 '1996-06-29 02:59:30.053094': 6.418594802044918,
 '1996-07-15 03:00:20.458038': 14.725630799899072,
 '1996-07-31 03:01:11.756050': 14.692929814420495,
 '1996-08-16 03:02:01.585038': 11.717596361987596,
 '1996-09-01 03:02:53.749000': 10.484826489190562,
 '1996-09-17 03:03:46.515044': None,
 '1996-10-03 03:04:37.796088': 6.6670849121549205,
 '1996-10-19 03:05:27.344069': -0.11576069158121538,
 '1996-03-16 02:58:45.632019': -16.102077686235834,
 '1996-04-01 02:59:46.876000': -5.554800271406833,
 '1996-04-17 03:00:46.211088': -5.49283005597674,
 '1996-05-03 03:01:43.199031': -3.998783161124131,
 '1996-05-19 03:02:38.588031': 0.08929357300506048,
 '1996-06-04 03:03:32.343094': None,
 '1996-07-06 03:05:14.927081': 5.208743369863654,
 '1996-07-22 03:06:06.218044': -21.826296452636377,
 '1996-08-07 03:06:56.785044': 13.197755372952013,
 '1996-09-08 03:08:40.036056': None,
 '1996-09-24 03:09:32.258081': 5.612741528148051,
 '1996-10-10 03:10:22.813063': 0.7162736261437705,
 '1996-10-26 03:11:11.191044': None,
 '1996-03-16 02:59:09.806094': -15.825811129299733,
 '1996-04-01 03:00:10.907019': -4.831681254957864,
 '1996-04-17 03:01:10.099025': -5.336760828125166,
 '1996-05-03 03:02:07.232038': -5.0025427825560635,
 '1996-05-19 03:03:02.622050': 1.4735522542568043,
 '1996-06-04 03:03:56.378094': None,
 '1996-06-20 03:04:48.573069': None,
 '1996-07-06 03:05:38.965019': 7.589351238586883,
 '1996-07-22 03:06:30.256088': 13.258087434006947,
 '1996-08-07 03:07:20.825006': 14.592089250836938,
 '1996-09-08 03:09:04.078075': 7.014653500488132,
 '1996-09-24 03:09:56.302019': 7.400155255931244,
 '1996-10-10 03:10:46.858006': 0.3554993134130169,
 '1996-10-26 03:11:35.237006': 2.4998834695451713,
 '1996-01-28 02:56:19.101031': -22.71201124997191,
 '1996-03-16 02:59:33.402075': -14.82004769602284,
 '1996-04-01 03:00:34.648081': -5.429550914545587,
 '1996-04-17 03:01:33.841088': -5.024465549994873,
 '1996-05-03 03:02:30.975094': -0.6489184983112481,
 '1996-05-19 03:03:26.367006': 1.0658004217454935,
 '1996-06-20 03:05:12.320031': 3.172470781653807,
 '1996-07-06 03:06:02.713006': 8.230656564058462,
 '1996-07-22 03:06:53.860088': 15.120498088912552,
 '1996-08-07 03:07:44.575013': 14.775431713033939,
 '1996-09-08 03:09:27.831013': 7.390522203722288,
 '1996-09-24 03:10:20.055094': 8.65638558610383,
 '1996-10-10 03:11:10.467094': None,
 '1996-10-26 03:11:58.993000': 4.365006251310656,
 '1996-02-04 03:02:35.360081': -15.880585095730465,
 '1996-02-20 03:03:41.097094': -12.849654872027871,
 '1996-03-07 03:04:45.177094': -10.698943931608232,
 '1996-03-23 03:05:47.599031': -32.33076576582518,
 '1996-04-08 03:06:47.959063': -11.604222822475668,
 '1996-05-10 03:08:42.529075': -16.34761325193864,
 '1996-05-26 03:09:37.191044': None,
 '1996-06-11 03:10:30.231088': 3.536048669091166,
 '1996-06-27 03:11:21.666081': 1.3875692305029939,
 '1996-08-14 03:13:53.350075': 11.826280283189808,
 '1996-09-15 03:15:37.969038': 9.189179102251616,
 '1996-11-18 03:18:51.025013': -3.700508638969168,
 '1996-02-04 03:02:59.095069': -16.81251225775194,
 '1996-02-20 03:04:04.833038': -14.762511302531765,
 '1996-03-07 03:05:08.917038': -11.884249971612679,
 '1996-03-23 03:06:11.340038': None,
 '1996-04-08 03:07:11.701050': -9.702886372240611,
 '1996-05-10 03:09:06.273075': -3.9049370948634796,
 '1996-05-26 03:10:00.936038': -3.2297819398872276,
 '1996-06-11 03:10:53.977088': 2.0024580592959236,
 '1996-08-14 03:14:17.101019': 12.883092212592489,
 '1996-09-15 03:16:01.722050': 8.539472915041141,
 '1996-11-18 03:19:14.783019': -1.22098104538187,
 '1996-02-11 03:09:38.766025': -11.715967739608534,
 '1996-02-27 03:10:43.842031': -13.627852383943809,
 '1996-03-14 03:11:47.306025': -13.15046806400941,
 '1996-03-30 03:12:48.854044': -9.512254800349487,
 '1996-05-17 03:15:41.293094': -1.7629940337794545,
 '1996-06-02 03:16:35.061094': None,
 '1996-07-04 03:18:18.231081': 5.0879709793404055,
 '1996-08-21 03:20:49.352019': 13.302117467668177,
 '1996-09-22 03:22:35.284069': -0.3482268502868553,
 '1996-10-08 03:23:25.994088': 8.343053561912415,
 '1996-10-24 03:24:14.800006': 5.834426181668079,
 '1996-11-25 03:25:45.813031': None,
'1997-02-17 03:03:42.985094': -26.049895378614988,
 '1997-03-05 03:04:22.442050': -17.106440997093518,
 '1997-03-21 03:04:59.554044': -11.618410494408431,
 '1997-04-06 03:05:34.751069': -5.269609566631004,
 '1997-04-22 03:06:08.476075': -0.6547779402851901,
 '1997-05-24 03:07:15.193094': None,
 '1997-06-09 03:07:48.799069': 3.428550248578207,
 '1997-07-27 03:09:27.568056': 11.282622208271839,
 '1997-08-28 03:10:27.825056': None,
 '1997-09-29 03:11:23.230081': 6.3888311251395775,
 '1997-10-15 03:11:49.259056': None,
 '1997-10-31 03:12:16.100031': -5.955406413372584,
 '1997-02-01 03:03:25.359056': -26.86790069660372,
 '1997-02-17 03:04:06.750050': -26.633985350920796,
 '1997-03-05 03:04:46.209056': -17.81103561899449,
 '1997-03-21 03:05:23.323031': -11.13538530357055,
 '1997-04-06 03:05:58.521056': -4.996434968389056,
 '1997-04-22 03:06:32.247069': -1.287807045238375,
 '1997-05-24 03:07:38.966056': 1.6612800979186433,
 '1997-06-09 03:08:12.573013': 2.857969738665754,
 '1997-07-11 03:09:19.057050': None,
 '1997-07-27 03:09:51.346063': 6.885894286199377,
 '1997-08-28 03:10:51.605038': 11.330503913413422,
 '1997-09-29 03:11:47.012081': 7.703133586300335,
 '1997-10-15 03:12:13.042069': None,
 '1997-10-31 03:12:39.884013': -5.725610588635878,
 '1997-11-16 03:13:01.920050': -13.825337072774419,
 '1997-02-01 03:03:49.411025': -22.897898747846295,
 '1997-02-17 03:04:30.804094': -21.824760611584175,
 '1997-03-05 03:05:10.266044': -11.699378983884204,
 '1997-03-21 03:05:47.527006': -9.357346853187837,
 '1997-04-06 03:06:22.581038': None,
 '1997-04-22 03:06:56.308056': -0.957847825820056,
 '1997-05-24 03:08:03.029006': 0.6895655291694894,
 '1997-06-09 03:08:36.636056': 3.0348901368304104,
 '1997-07-11 03:09:43.124050': 3.437288756278259,
 '1997-07-27 03:10:15.414075': 10.959838460818482,
 '1997-08-28 03:11:15.675025': 12.874126756726806,
 '1997-10-15 03:12:37.115088': None,
 '1997-10-31 03:13:03.958000': -1.6417656993121077,
 '1997-11-16 03:13:25.994088': None,
 '1997-02-08 03:09:30.686094': -23.857904642497406,
 '1997-03-12 03:10:49.713000': -15.073534503513399,
 '1997-03-28 03:11:25.948000': -10.586465009804682,
 '1997-04-13 03:12:00.117088': -0.34112673477896965,
 '1997-05-15 03:13:05.961044': -1.0332008416446832,
 '1997-05-31 03:13:40.945069': None,
 '1997-06-16 03:14:13.414013': 4.999953936618628,
 '1997-07-18 03:15:20.358025': 7.85696360639773,
 '1997-08-03 03:15:52.031056': 14.406470157365092,
 '1997-08-19 03:16:22.211075': 9.0305515847227,
 '1997-09-04 03:16:50.864056': None,
 '1997-09-20 03:17:17.724013': 9.255400076415535,
 '1997-10-22 03:18:12.244088': -1.514353629346405,
 '1997-11-07 03:18:37.220038': -0.0203198088720222,
 '1997-02-08 03:09:54.593075': -23.205903665280182,
 '1997-03-12 03:11:13.626006': -12.448453782816522,
 '1997-03-28 03:11:49.862031': -9.467270030450866,
 '1997-04-13 03:12:24.033019': -1.1508660091440504,
 '1997-04-29 03:12:57.746094': None,
 '1997-05-15 03:13:29.878056': -0.8267262046664596,
 '1997-05-31 03:14:04.863075': -4.4302817872254785,
 '1997-06-16 03:14:37.332088': 4.149445943492047,
 '1997-07-18 03:15:44.280081': 5.75511187708802,
 '1997-08-03 03:16:15.955013': 9.40665139084732,
 '1997-08-19 03:16:46.136013': 11.683092964143812,
 '1997-09-20 03:17:41.505069': 8.948156634910738,
 '1997-10-06 03:18:09.608006': 4.665289946235915,
 '1997-10-22 03:18:36.028013': -7.413314715293022,
 '1997-11-07 03:19:01.004044': 2.799900573489303,
 '1997-02-08 03:10:18.645056': -18.849530140375162,
 '1997-03-12 03:11:37.539013': -9.869037834399988,
 '1997-03-28 03:12:13.921063': -6.917380453600821,
 '1997-04-13 03:12:47.948050': -0.9878713048746779,
 '1997-04-29 03:13:21.808000': None,
 '1997-05-15 03:13:53.940056': 0.3280425984395732,
 '1997-05-31 03:14:28.926075': None,
 '1997-06-16 03:15:01.251063': 4.087079737630148,
 '1997-08-03 03:16:40.023056': 11.35919061908456,
 '1997-08-19 03:17:10.205038': 12.12856232154179,
 '1997-09-04 03:17:38.860038': 11.002462021289416,
 '1997-09-20 03:18:05.577025': 8.837744704788117,
 '1997-10-06 03:18:33.680063': 1.3698448022377903,
 '1997-10-22 03:19:00.101056': -8.008670506767192,
 '1997-11-07 03:19:25.078056': 4.13081837035946,
 '1997-02-08 03:10:42.407063': -17.116898610956696,
 '1997-03-12 03:12:01.307019': -8.362201018707204,
 '1997-03-28 03:12:37.546019': -4.12667564344821,
 '1997-04-13 03:13:11.718088': 0.0468674088110608,
 '1997-04-29 03:13:45.579019': None,
 '1997-05-15 03:14:17.712069': 2.2058561827036938,
 '1997-05-31 03:14:52.699088': 1.6608666773929528,
 '1997-06-16 03:15:25.025031': 4.19659623225589,
 '1997-07-18 03:16:31.981006': None,
 '1997-08-03 03:17:03.657013': 14.029581877050056,
 '1997-08-19 03:17:33.839063': 12.470699769015955,
 '1997-09-04 03:18:02.495069': 10.888461145268092,
 '1997-09-20 03:18:29.358075': 8.209267122521716,
 '1997-10-22 03:19:23.739081': -14.58744437239568,
 '1997-11-07 03:19:48.862069': 3.1146058738175393,
 '1997-01-30 03:15:41.624006': -22.2005381345987,
 '1997-03-19 03:17:40.454044': -7.71536301256736,
 '1997-04-04 03:18:15.931044': -0.7937745202696119,
 '1997-04-20 03:18:49.649013': -0.677535284576932,
 '1997-05-06 03:19:22.935056': 0.19171807433880467,
 '1997-05-22 03:19:56.241050': 3.2709769933285906,
 '1997-06-07 03:20:30.279063': 2.8001022610284667,
 '1997-06-23 03:21:03.008069': 4.385490834137687,
 '1997-07-25 03:22:09.057050': 4.7113837520982305,
 '1997-08-10 03:22:40.145025': 12.262058614300853,
 '1997-08-26 03:23:09.765019': 9.029612182905579,
 '1997-09-27 03:24:05.144088': 4.297065820109412,
 '1997-10-13 03:24:31.411013': None,
 '1997-10-29 03:24:58.408031': -2.267933385914552,
 '1997-11-14 03:25:21.171063': -0.01691711178817254,
 '1997-01-30 03:16:05.675075': -21.660740069044667,
 '1997-03-03 03:17:27.085063': -10.315045677221677,
 '1997-03-19 03:18:04.513006': -6.7489549842747,
 '1997-04-04 03:18:39.991013': -2.180752481071697,
 '1997-04-20 03:19:13.709088': -0.052413168667618616,
 '1997-05-06 03:19:46.996094': -1.1667777455639874,
 '1997-05-22 03:20:20.304006': 4.420926755709185,
 '1997-06-07 03:20:54.197094': 3.2655548598553903,
 '1997-06-23 03:21:27.075013': 5.059927777573895,
 '1997-07-25 03:22:33.125050': 5.641435966462975,
 '1997-08-10 03:23:04.214025': 13.866982436067396,
 '1997-08-26 03:23:33.690006': 4.9707781448890564,
 '1997-09-11 03:24:01.448044': None,
 '1997-09-27 03:24:29.216081': 5.692235209239583,
 '1997-10-13 03:24:55.484025': 5.084197537830112,
 '1997-10-29 03:25:22.482019': 2.5596483274847373,
 '1997-11-14 03:25:45.246019': -0.05939054481858147,
 '1997-01-30 03:16:29.437063': -17.776369872712873,
 '1997-02-15 03:17:11.104019': -21.645669763832128,
 '1997-03-03 03:17:50.852044': -12.830791512517877,
 '1997-03-19 03:18:28.281069': -8.838694677183241,
 '1997-04-04 03:19:03.616000': -5.766034832493325,
 '1997-04-20 03:19:37.480069': -0.10161018459372624,
 '1997-05-06 03:20:10.768050': -0.891810002236139,
 '1997-05-22 03:20:43.931063': 4.228296007204733,
 '1997-06-23 03:21:50.851056': 6.1019087523595354,
 '1997-07-25 03:22:56.903056': 4.19316906737221,
 '1997-08-10 03:23:27.848031': 13.027553105950245,
 '1997-08-26 03:23:57.470006': None,
 '1997-09-11 03:24:25.084044': 5.968316186887683,
 '1997-09-27 03:24:52.853075': 9.047104614589411,
 '1997-10-13 03:25:19.267044': 6.5069798762081925,
 '1997-10-29 03:25:46.121000': 3.1622327189103463,
 '1997-11-14 03:26:08.885063': -2.24737734187947,
 '1997-11-30 03:26:32.045075': -4.575254874387676,
 '1997-02-06 03:22:35.050069': -14.98600835541081,
 '1997-02-22 03:23:15.932088': -13.870077025511087,
 '1997-03-10 03:23:54.513006': -10.970280016031936,
 '1997-03-26 03:24:31.187069': -0.1223637952969763,
 '1997-04-11 03:25:05.503075': 0.5266794638086176,
 '1997-04-27 03:25:39.064044': -0.2931158423549645,
 '1997-05-13 03:26:11.641000': None,
 '1997-06-14 03:27:19.119069': 4.267241719327625,
 '1997-06-30 03:27:53.284038': -0.7482921927006541,
 '1997-07-16 03:28:25.760063': -27.96712568000505,
 '1997-08-01 03:28:57.747031': 17.06751554230986,
 '1997-08-17 03:29:28.098081': 12.638119325371425,
 '1997-09-02 03:29:57.007000': -29.677459442749598,
 '1997-10-04 03:30:52.017063': None,
 '1997-10-20 03:31:18.333094': None,
 '1997-11-05 03:31:43.889006': 3.796349909113965,
 '1997-11-21 03:32:05.317075': -4.801319938024958,
 '1997-02-06 03:22:58.813013': -17.465793495717033,
 '1997-02-22 03:23:39.553050': -16.62975984991381,
 '1997-03-10 03:24:18.280094': -12.33927225191661,
 '1997-03-26 03:24:54.812000': -0.3334612067354201,
 '1997-04-11 03:25:29.274006': -0.006257207195608589,
 '1997-04-27 03:26:02.690050': -0.8122631555978964,
 '1997-05-13 03:26:35.413000': None,
 '1997-06-14 03:27:42.748088': 4.50278703256553,
 '1997-06-30 03:28:16.915081': 0.45493939605306477,
 '1997-07-16 03:28:49.538006': 9.048207945117326,
 '1997-08-01 03:29:21.380075': 14.808663816686009,
 '1997-08-17 03:29:51.878019': 12.551422789757995,
 '1997-09-02 03:30:20.787044': 8.351248426376122,
 '1997-10-04 03:31:15.800000': 4.829186788567988,
 '1997-10-20 03:31:41.972025': 0.9986695627226668,
 '1997-11-05 03:32:07.673000': 3.229276521071513,
 '1997-11-21 03:32:29.102050': 0.14151375013014023,
 '1997-01-28 03:28:45.523075': -16.71749504830935,
 '1997-03-01 03:30:07.684075': -15.750368488561472,
 '1997-03-17 03:30:45.398031': -4.246564833995832,
 '1997-04-02 03:31:21.037038': -3.751911183101297,
 '1997-04-18 03:31:54.768044': -0.9107057406527254,
 '1997-05-04 03:32:28.344081': -14.147693775450453,
 '1997-06-21 03:34:08.107056': 5.5578211753450395,
 '1997-07-23 03:35:14.595050': 10.06480562612208,
 '1997-08-08 03:35:45.848056': -12.445765726500197,
 '1997-09-09 03:36:43.516063': 10.32192710730068,
 '1997-09-25 03:37:10.997081': 7.141828955000743,
 '1997-10-11 03:37:37.824075': None,
 '1997-11-12 03:38:28.064063': 3.1898035552520096,
 '1997-11-28 03:38:50.480006': None,
'1998-02-04 03:14:29.915019': -20.538842680754048,
 '1998-02-20 03:14:48.782056': None,
 '1998-03-08 03:15:04.080013': -11.756219593626895,
 '1998-03-24 03:15:18.393094': -14.871222600420387,
 '1998-04-09 03:15:29.727044': -5.096651372080654,
 '1998-04-25 03:15:41.277094': -3.302571388760145,
 '1998-05-11 03:15:57.259031': 0.7054124965060194,
 '1998-06-28 03:16:37.184094': 5.941134069725861,
 '1998-07-14 03:16:49.577019': None,
 '1998-08-15 03:17:02.258088': 7.292362329911311,
 '1998-08-31 03:17:13.825088': 12.739808219358968,
 '1998-09-16 03:17:21.169050': None,
 '1998-10-02 03:17:24.410031': 5.589504514724456,
 '1998-10-18 03:17:24.862031': -4.459842580114836,
 '1998-02-04 03:14:53.705088': -17.95342094410373,
 '1998-02-20 03:15:12.574056': -17.72307115515669,
 '1998-03-08 03:15:27.873031': -10.294304061094449,
 '1998-03-24 03:15:42.188050': -18.54978970788958,
 '1998-04-09 03:15:53.524088': -7.244249695999211,
 '1998-04-25 03:16:05.077050': None,
 '1998-05-11 03:16:21.059094': -0.7099976357061328,
 '1998-06-28 03:17:00.987063': 3.612817024503052,
 '1998-07-14 03:17:13.380081': -3.939894121441109,
 '1998-08-15 03:17:26.064050': 13.732133912341762,
 '1998-08-31 03:17:37.631088': 13.92365461770745,
 '1998-10-02 03:17:48.218031': 6.077373718380959,
 '1998-10-18 03:17:48.671013': 1.612875004584732,
 '1998-11-19 03:18:00.915025': -12.057951503429782,
 '1998-02-04 03:15:17.786063': -17.201065045316348,
 '1998-02-20 03:15:36.656081': -13.635662214062146,
 '1998-03-08 03:15:51.811044': -9.37257921479894,
 '1998-03-24 03:16:06.273031': -9.138002504534228,
 '1998-04-09 03:16:17.612050': -7.298502686645628,
 '1998-04-25 03:16:29.167031': None,
 '1998-05-11 03:16:45.150088': None,
 '1998-06-28 03:17:24.935050': 4.314884719320779,
 '1998-07-14 03:17:37.474081': 4.158841212310653,
 '1998-08-15 03:17:50.160038': 15.18380207505395,
 '1998-08-31 03:18:01.728031': 15.128745802934255,
 '1998-10-02 03:18:12.171050': 6.520465955701491,
 '1998-10-18 03:18:12.770044': 3.7298492323434047,
 '1998-10-02 03:18:35.979050': 6.760826411861781,
 '1998-04-16 03:21:44.468063': -1.8811612572953984,
 '1998-05-02 03:21:58.225050': -2.506070563639074,
 '1998-06-03 03:22:30.218013': 2.837707565203335,
 '1998-07-21 03:23:03.675081': 7.770899757893209,
 '1998-08-22 03:23:18.010050': 17.5602432102115,
 '1998-02-27 03:21:30.026031': -9.7457123795127,
 '1998-04-16 03:22:08.412019': -2.2948443430929917,
 '1998-05-02 03:22:22.025050': -0.27463385498111326,
 '1998-06-03 03:22:54.019075': 2.3120461910930374,
 '1998-07-05 03:23:18.270056': 4.329142281025493,
 '1998-07-21 03:23:27.479075': 4.631941964639081,
 '1998-08-22 03:23:41.816044': 14.579160253276797,
 '1998-11-10 03:24:09.542031': -6.7782252755825665,
 '1998-01-26 03:21:16.399088': -20.77194680832893,
 '1998-02-27 03:21:54.109025': -8.916270678036534,
 '1998-04-16 03:22:32.355088': -1.4563371102747358,
 '1998-05-02 03:22:46.115094': -1.8634117477844587,
 '1998-06-03 03:23:18.111063': 3.286027124906882,
 '1998-07-05 03:23:42.219006': 6.603676097948281,
 '1998-07-21 03:23:51.574006': 5.96417653645887,
 '1998-08-22 03:24:05.912069': 16.984870241739888,
 '1998-11-10 03:24:33.497056': -4.642498348213076,
 '1998-11-26 03:24:34.702038': -3.234771364632782,
 '1998-01-26 03:21:40.044025': -19.98725060505038,
 '1998-02-27 03:22:17.901094': -9.587906348483985,
 '1998-04-16 03:22:56.154031': -1.7136346454170344,
 '1998-05-02 03:23:09.916006': -0.30956329923696657,
 '1998-06-03 03:23:41.768006': 4.67664793686305,
 '1998-07-05 03:24:06.022038': 8.844577706723832,
 '1998-07-21 03:24:15.232088': 7.176517883935435,
 '1998-08-22 03:24:29.573038': 15.333328376954062,
 '1998-09-07 03:24:38.740000': 7.5142471237987065,
 '1998-09-23 03:24:46.737013': None,
 '1998-11-10 03:24:57.307069': -1.1152282312959252,
 '1998-11-26 03:24:58.513025': -4.8022834533412295,
 '1998-02-02 03:27:12.737081': -14.855703122539166,
 '1998-03-06 03:27:46.773056': -8.91920450705979,
 '1998-03-22 03:28:02.463013': -14.11801827052155,
 '1998-04-07 03:28:13.588038': None,
 '1998-04-23 03:28:25.268088': -3.70907377406705,
 '1998-05-09 03:28:40.687088': 1.4381681494146223,
 '1998-05-25 03:28:55.996063': None,
 '1998-06-10 03:29:10.449094': 3.5105279660725026,
 '1998-07-12 03:29:33.879056': 6.973384486245443,
 '1998-08-29 03:29:58.319094': 11.78039274708476,
 '1998-09-14 03:30:05.482038': 7.187864541824963,
 '1998-11-01 03:30:14.098088': -0.1245004426039757,
 '1998-11-17 03:30:22.434044': -6.509690430396217,
 '1998-02-02 03:27:36.818038': -14.691574078609403,
 '1998-02-18 03:27:56.126019': None,
 '1998-03-06 03:28:10.856094': -7.097981170074424,
 '1998-03-22 03:28:26.402050': -12.75482026665459,
 '1998-04-07 03:28:37.675081': None,
 '1998-04-23 03:28:49.358056': -2.6209768209236586,
 '1998-05-09 03:29:04.778075': 1.6222728559868107,
 '1998-06-10 03:29:34.542038': 4.314762659425352,
 '1998-06-26 03:29:44.561088': None,
 '1998-07-12 03:29:57.973044': 9.945209266236638,
 '1998-07-28 03:30:04.489000': 10.265983412691222,
 '1998-08-29 03:30:22.416019': 11.579295238011404,
 '1998-09-14 03:30:29.434075': 11.346191511410211,
 '1998-11-01 03:30:38.199025': -2.066257689064722,
 '1998-11-17 03:30:46.535031': -17.109930169805583,
 '1998-01-17 03:27:39.014006': None,
 '1998-02-02 03:28:00.463081': -14.806489657698526,
 '1998-02-18 03:28:19.773013': -16.07158076204812,
 '1998-03-06 03:28:34.505000': -7.556926530434803,
 '1998-03-22 03:28:50.196075': -11.10505916037806,
 '1998-04-07 03:29:01.327081': -9.508927802791701,
 '1998-04-23 03:29:13.012094': -1.9883952304345849,
 '1998-05-09 03:29:28.579038': 1.3058091348409004,
 '1998-05-25 03:29:43.744050': 0.7744125761824011,
 '1998-06-10 03:29:58.199031': 3.7710689911771715,
 '1998-06-26 03:30:08.219044': 2.451476400918596,
 '1998-07-12 03:30:21.631088': 10.992680781985719,
 '1998-07-28 03:30:28.148044': 10.20693433810692,
 '1998-08-29 03:30:46.077006': 12.884336090763943,
 '1998-09-14 03:30:53.241094': 10.853124620570783,
 '1998-11-17 03:31:10.200056': -5.653644644533832,
 '1998-12-03 03:31:05.460019': -6.273541809743533,
 '1998-02-09 03:33:56.524019': -14.525301694627474,
 '1998-02-25 03:34:13.859075': -8.53896795930859,
 '1998-03-29 03:34:41.612088': -7.042347004348323,
 '1998-04-14 03:34:53.118094': -10.459706649949904,
 '1998-04-30 03:35:04.874031': 0.15772948054597763,
 '1998-06-01 03:35:37.872038': -22.02825572295822,
 '1998-07-19 03:36:12.316081': None,
 '1998-08-04 03:36:16.244063': 5.889338736958736,
 '1998-08-20 03:36:25.540050': 14.479062830019878,
 '1998-09-05 03:36:36.164006': 15.104631259482721,
 '1998-10-07 03:36:45.988019': 6.081498928489119,
 '1998-11-08 03:36:53.860025': 3.5039438777898573,
 '1998-11-24 03:36:56.941013': -6.793233737156961,
 '1998-02-09 03:34:20.170038': -15.357262186219723,
 '1998-02-25 03:34:37.652050': -13.020352102748697,
 '1998-03-29 03:35:05.408044': -7.698203376226901,
 '1998-04-14 03:35:16.772006': -9.918077737193146,
 '1998-04-30 03:35:28.529025': -0.1488532001467473,
 '1998-06-01 03:36:01.674000': 0.22266382700566517,
 '1998-07-03 03:36:25.931000': None,
 '1998-07-19 03:36:35.975063': 6.256027482583469,
 '1998-08-04 03:36:40.049063': 11.128585255139821,
 '1998-08-20 03:36:49.201019': 13.297401283096816,
 '1998-09-05 03:36:59.825050': 13.102572642437632,
 '1998-09-21 03:37:07.666000': 5.965787214278313,
 '1998-10-07 03:37:09.651056': 5.7173947019916245,
 '1998-10-23 03:37:09.864025': None,
 '1998-11-08 03:37:17.525019': 2.627705806620555,
 '1998-11-24 03:37:20.606069': -9.264800773604431,
 '1998-01-31 03:40:19.551038': -14.937602739576398,
 '1998-03-04 03:40:54.551044': -6.64225653787103,
 '1998-03-20 03:41:10.243063': -12.943532465367921,
 '1998-04-05 03:41:21.370006': -4.436392892410943,
 '1998-04-21 03:41:33.069063': -14.145719954562848,
 '1998-05-07 03:41:48.058075': 0.7380161111476609,
 '1998-06-08 03:42:18.393031': None,
 '1998-06-24 03:42:27.405031': None,
 '1998-07-10 03:42:42.128019': None,
 '1998-08-27 03:43:06.448019': None,
 '1998-09-28 03:43:19.723088': 2.72824304821502,
 '1998-10-14 03:43:19.040025': 3.3346798894734166,
 '1998-11-15 03:43:31.712050': -9.569567015130826,
 '1998-12-01 03:43:28.578063': -0.2657447784804814,
'1999-07-09 03:31:21.764208800': 6.365575103223241,
 '1999-08-10 03:31:28.858656500': 6.451349275757448,
 '1999-09-27 03:31:34.537791200': 4.943573802174719,
 '1999-10-29 03:31:28.038374900': -1.349705580266875,
 '1999-07-09 03:31:45.656057800': 5.987844903206589,
 '1999-08-10 03:31:52.745661400': 7.484467751199012,
 '1999-09-27 03:31:58.431504800': 7.200582778407877,
 '1999-10-29 03:31:51.923746': -4.487520895045436,
 '1999-08-10 03:32:16.624295900': 10.003267074805658,
 '1999-09-27 03:32:22.307043400': 7.145025312761682,
 '1999-10-29 03:32:15.796894600': 1.389649411725797,
 '1999-11-14 03:32:16.104150900': 0.11698079328514122,
 '1999-06-30 03:37:26.731091500': 4.398800551288412,
 '1999-08-01 03:37:39.639629100': 14.787217682230295,
 '1999-08-17 03:37:36.700383600': 8.536822674956078,
 '1999-10-04 03:37:48.041844500': 3.842715556007938,
 '1999-06-30 03:37:50.617299800': 1.8932434509533327,
 '1999-08-01 03:38:03.525284100': 13.152334097157635,
 '1999-08-17 03:38:00.582596300': 6.57163165060037,
 '1999-10-04 03:38:11.930602600': 5.807726571634979,
 '1999-06-30 03:38:14.498297600': -1.1004559679030708,
 '1999-08-01 03:38:27.410485': 14.291005215181796,
 '1999-08-17 03:38:24.459996400': 9.458326441706394,
 '1999-10-04 03:38:35.809944300': 8.157165188746188,
 '1999-08-01 03:38:51.279414900': 16.390289184269125,
 '1999-08-17 03:38:48.334468600': 8.162014183527916,
 '1999-10-04 03:38:59.681460': 8.364654899430526,
 '1999-12-07 03:38:50.914740': -8.30738633604844,
 '1999-07-07 03:44:06.523083800': 8.4041816117721,
 '1999-07-23 03:44:08.553019400': 18.26656485137677,
 '1999-10-11 03:44:22.160686800': None,
 '1999-11-12 03:44:13.184108700': -12.243448989355194,
 '1999-07-07 03:44:30.401040700': 8.748000463492936,
 '1999-07-23 03:44:32.438292200': 18.57415053186333,
 '1999-09-09 03:44:38.148380600': 12.262719080654,
 '1999-10-11 03:44:46.039610400': 6.09499182684986,
 '1999-10-27 03:44:37.044517700': -1.6849975191371698,
 '1999-11-12 03:44:37.066433600': 0.09063415213047815,
 '1999-07-07 03:44:54.280339900': None,
 '1999-07-23 03:44:56.312883700': 15.11370620884092,
 '1999-09-09 03:45:02.021330200': None,
 '1999-10-11 03:45:09.913549200': 7.246495831731937,
 '1999-10-27 03:45:00.932873700': -5.660271790217639,
 '1999-11-12 03:45:00.940924100': -2.8553740762787285,
 '1999-07-14 03:50:42.934347400': None,
 '1999-09-16 03:50:43.976848400': -10.041152374330533,
 '1999-07-14 03:51:06.805207100': 13.697372379439953,
 '1999-09-16 03:51:07.855455100': 11.193292712001513,
 '1999-10-02 03:51:20.826025': 8.556146598035228,
 '1999-12-05 03:51:10.684773300': 2.047515510122158,
 '1999-08-22 03:57:22.594107200': 12.802893417763865,
 '1999-10-25 03:57:21.799593800': 5.53419294146607,
 '1999-02-07 03:17:44.946044': -22.742150182170054,
 '1999-02-23 03:17:50.362019': -18.488679528621628,
 '1999-04-12 03:17:17.258038': -6.655790021285972,
 '1999-04-28 03:17:17.368088': None,
 '1999-07-17 03:16:18.538063': 13.824739439198105,
 '1999-08-02 03:16:17.146019': 4.933333155087838,
 '1999-08-18 03:16:07.974013': 8.612228186352796,
 '1999-09-03 03:15:42.687075': 11.055986822030098,
 '1999-09-19 03:15:12.867069': 5.296238242202109,
 '1999-10-05 03:15:10.589081': -19.99693727145494,
 '1999-02-07 03:18:08.762038': -19.471284146964887,
 '1999-02-23 03:18:14.178088': -20.26057948040697,
 '1999-04-12 03:17:41.076081': -9.32506004810521,
 '1999-04-28 03:17:41.188031': -14.907873928741125,
 '1999-05-30 03:17:06.785019': -2.0835814199874347,
 '1999-06-15 03:16:56.876069': 2.7901192664063834,
 '1999-07-17 03:16:42.362038': 11.611278173640981,
 '1999-08-02 03:16:40.970063': 12.099195483646909,
 '1999-08-18 03:16:31.799050': 7.40996414099231,
 '1999-09-03 03:16:06.513075': 11.292296065297199,
 '1999-09-19 03:15:36.694063': 6.2618775292476725,
 '1999-10-05 03:15:34.417056': -3.9009663465392523,
 '1999-01-22 03:18:24.739088': -19.375212001273386,
 '1999-02-07 03:18:32.868069': -14.415431969637332,
 '1999-02-23 03:18:38.286013': -18.902579104134194,
 '1999-04-12 03:18:05.185069': -3.625874297909788,
 '1999-04-28 03:18:05.298019': -12.06902285706959,
 '1999-05-30 03:17:30.896075': 1.448565928304907,
 '1999-06-15 03:17:20.843081': 3.6673965959346657,
 '1999-07-17 03:17:06.331038': 13.648652422985675,
 '1999-08-02 03:17:04.940031': 15.608254459069457,
 '1999-08-18 03:16:55.915044': 5.68189121886394,
 '1999-09-03 03:16:30.630044': 9.954796302711145,
 '1999-09-19 03:16:00.812019': 7.878293685506953,
 '1999-10-05 03:15:58.535075': None,
 '1999-11-22 03:14:48.082063': -2.92264272264118,
 '1999-03-02 03:23:57.996088': -22.27598042071614,
 '1999-04-03 03:23:27.624025': -8.851157574455584,
 '1999-05-05 03:23:25.147081': -1.9419275566366028,
 '1999-05-21 03:23:05.961044': None,
 '1999-06-22 03:22:40.504050': 1.9056866390617808,
 '1999-07-08 03:22:32.129031': 7.0421583274402835,
 '1999-08-09 03:22:25.737025': None,
 '1999-10-12 03:21:24.032088': -9.45849501074782,
 '1999-10-28 03:21:07.395069': 1.7778629928886145,
 '1999-11-13 03:20:32.870094': -2.1829685993666628,
 '1999-01-29 03:24:12.178094': -25.87935098143636,
 '1999-03-02 03:24:21.813094': -21.447282300098415,
 '1999-04-03 03:23:51.442050': -8.829515815313899,
 '1999-05-05 03:23:48.967031': -1.7091702319161965,
 '1999-06-22 03:23:04.326075': 3.0566420097972373,
 '1999-07-08 03:22:55.952038': 5.6034055490569035,
 '1999-07-24 03:22:53.136031': 10.007751260356436,
 '1999-08-09 03:22:49.707050': None,
 '1999-08-25 03:22:34.406019': 5.977827152534109,
 '1999-09-26 03:21:40.405044': 6.456551044110156,
 '1999-10-28 03:21:31.224013': -0.005137864274558592,
 '1999-11-13 03:20:56.700000': -5.749425682945575,
 '1999-01-29 03:24:36.284056': -24.104971453301044,
 '1999-02-14 03:24:48.219081': -15.107698797055642,
 '1999-03-02 03:24:45.776019': -17.487898654642713,
 '1999-04-03 03:24:15.551019': -5.206559583565542,
 '1999-05-05 03:24:12.932013': 0.6360527713980387,
 '1999-05-21 03:23:53.893025': 0.22281708954217486,
 '1999-06-22 03:23:28.294038': 4.724273104098292,
 '1999-07-08 03:23:19.920081': 8.042767714462599,
 '1999-07-24 03:23:17.251000': 13.596326896638582,
 '1999-08-09 03:23:13.677081': 10.100912869287285,
 '1999-08-25 03:22:58.377013': 8.385018258796444,
 '1999-09-10 03:22:25.145063': None,
 '1999-09-26 03:22:04.378019': 8.196407676254367,
 '1999-10-12 03:22:11.833075': 2.6606528136258,
 '1999-10-28 03:21:55.343006': 2.704846092055374,
 '1999-11-13 03:21:20.819069': -0.5146610681681892,
 '1999-01-29 03:24:59.954056': -23.889285818302476,
 '1999-02-14 03:25:11.890081': -18.371037441431366,
 '1999-03-02 03:25:09.593019': -17.506446824422472,
 '1999-04-03 03:24:39.224013': -3.3008899624938683,
 '1999-05-05 03:24:36.751069': 0.35937190552517534,
 '1999-05-21 03:24:17.568075': 0.7847557421267839,
 '1999-06-22 03:23:51.971050': 6.739167835383371,
 '1999-07-08 03:23:43.743094': 11.146837693133136,
 '1999-07-24 03:23:40.929088': 13.152036395098492,
 '1999-08-09 03:23:37.357050': 11.304061632908914,
 '1999-08-25 03:23:22.202075': 8.77305318373745,
 '1999-09-10 03:22:48.827006': 10.22448698908968,
 '1999-09-26 03:22:28.205069': 9.086316886169806,
 '1999-10-12 03:22:35.516019': 4.445694188527656,
 '1999-10-28 03:22:19.026013': 3.792174776947399,
 '1999-11-13 03:21:44.503044': -0.09791661397673128,
 '1999-11-29 03:21:10.940013': 1.4252273439413867,
 '1999-02-05 03:30:28.817013': -16.607515394820332,
 '1999-04-10 03:30:01.751013': -0.8794020935957981,
 '1999-05-12 03:29:53.720031': -2.2335174108660873,
 '1999-06-13 03:29:18.970069': None,
 '1999-06-29 03:29:08.467000': 6.332790322714706,
 '1999-07-15 03:29:03.623006': -22.857522640143028,
 '1999-07-31 03:29:02.647069': 11.516805397792403,
 '1999-08-16 03:28:55.232038': 10.252828279944683,
 '1999-10-03 03:27:54.925019': 7.085968203699309,
 '1999-10-19 03:27:54.655031': 3.6682030314403837,
 '1999-02-05 03:30:52.923025': -17.179158877618395,
 '1999-04-10 03:30:25.715006': -1.282460734546959,
 '1999-04-26 03:30:27.388056': -2.4810489817547934,
 '1999-05-12 03:30:17.831019': -0.5398098879070159,
 '1999-06-13 03:29:42.937094': 2.7637302136711135,
 '1999-06-29 03:29:32.580050': 7.319749890030651,
 '1999-07-15 03:29:27.592000': 15.535660415647707,
 '1999-07-31 03:29:26.617050': 15.169833666207174,
 '1999-08-16 03:29:19.348038': 11.541305247272426,
 '1999-09-17 03:28:25.835081': 10.257831323547789,
 '1999-10-03 03:28:18.898025': 7.954654328512618,
 '1999-10-19 03:28:18.774013': 4.459623824171344,
 '1999-02-05 03:31:16.593081': -15.489209143963777,
 '1999-04-10 03:30:49.533075': -1.475079327473502,
 '1999-04-26 03:30:51.062094': -0.7731560616753101,
 '1999-05-12 03:30:41.506038': 1.202917534760591,
 '1999-06-13 03:30:06.614081': 3.718580803203848,
 '1999-06-29 03:29:56.258013': 8.437221912091474,
 '1999-07-15 03:29:51.270050': 13.03346148315898,
 '1999-07-31 03:29:50.296075': 14.092003095205467,
 '1999-08-16 03:29:43.028050': 11.669734512441476,
 '1999-09-17 03:28:49.517044': 10.47343355355739,
 '1999-10-03 03:28:42.726000': 7.638832141434461,
 '1999-10-19 03:28:42.457013': 5.720820344156646,
 '1999-06-04 03:35:59.278013': 3.8208048058860005,
 '1999-09-24 03:34:26.646044': 10.047114995492459,
 '1999-01-27 03:37:21.777038': -22.226089501567323,
 '1999-02-12 03:37:32.537044': -15.243282803980495,
 '1999-02-28 03:37:32.308075': -14.68649890273254,
 '1999-05-19 03:36:42.579006': 1.7215980208088528,
 '1999-06-04 03:36:22.954025': 3.2485708440164585,
 '1999-06-20 03:36:14.933094': 5.138279113500592,
 '1999-07-06 03:36:06.152075': 7.38719714583665,
 '1999-07-22 03:36:02.747031': None,
 '1999-08-07 03:35:59.958063': 11.100054047879363,
 '1999-09-08 03:35:13.737025': 14.24079605193571,
 '1999-09-24 03:34:50.328056': 4.563357495131197,
 '1999-10-10 03:34:56.967019': 6.875259570277308,
 '1999-10-26 03:34:44.213075': -35.33940706273461,
 '1999-11-11 03:34:11.442006': 0.45207010842922496,
 '1999-11-27 03:33:36.363019': -1.5827872593838286,
 '1999-02-03 03:43:36.521044': -13.72777006068029,
 '1999-02-19 03:43:45.893000': -9.127970504504841,
 '1999-03-07 03:43:37.994063': None,
 '1999-04-24 03:43:13.178081': -23.18107500961162,
 '1999-07-13 03:42:12.977038': 6.752664532941373,
 '1999-07-29 03:42:12.106006': None,
 '1999-08-30 03:41:46.105069': -20.06158724520343,
 '1999-10-01 03:41:02.390050': 8.77457582667348,
 '1999-11-02 03:40:40.771050': -3.739606132650079,
 '1999-12-04 03:39:37.646081': 1.75995660637218,
'2000-02-18 03:31:15.990165700': -21.722750455768526,
 '2000-03-21 03:31:11.318153600': -13.199899143511601,
 '2000-05-24 03:30:32.831402': -1.7983308258247646,
 '2000-06-09 03:30:26.112859500': 2.0499606723762405,
 '2000-06-25 03:30:19.334040400': 7.553617853576587,
 '2000-08-12 03:29:40.147816400': 21.682417182420608,
 '2000-08-28 03:29:31.211791600': 14.48074125131216,
 '2000-11-16 03:28:55.123272300': -13.192495295651318,
 '2000-02-18 03:31:39.869025200': -20.801136390904546,
 '2000-05-08 03:31:08.392280400': -1.972616338515585,
 '2000-05-24 03:30:56.712687100': 0.03716062908475694,
 '2000-06-09 03:30:49.999270900': 2.471630962613481,
 '2000-06-25 03:30:43.212423700': 4.261479314036795,
 '2000-08-28 03:29:55.096650400': 14.05914459267119,
 '2000-09-13 03:29:37.344184': 12.556041679253477,
 '2000-02-18 03:32:03.748540900': -19.404346479065843,
 '2000-03-21 03:31:59.083299500': -13.262262621283863,
 '2000-05-08 03:31:32.272108100': -3.1296160270866404,
 '2000-06-25 03:31:07.086250600': 5.982737118135257,
 '2000-09-13 03:30:01.214542900': 13.364155986017987,
 '2000-05-15 03:36:52.892715400': 0.7934232984663419,
 '2000-08-19 03:35:48.872222200': 12.566246458517467,
 '2000-09-04 03:35:34.095824800': 9.152561962892666,
 '2000-11-07 03:35:04.483224400': -0.4953786517622807,
 '2000-05-15 03:37:16.777699900': -1.1647208134142364,
 '2000-08-19 03:36:12.757343900': 7.99002059524667,
 '2000-09-04 03:35:57.974072300': 10.552704880457386,
 '2000-11-07 03:35:28.370012400': -7.1143421810221446,
 '2000-11-23 03:35:27.111480900': -10.943035270144552,
 '2000-04-29 03:37:52.017072900': None,
 '2000-05-15 03:37:40.652877100': -0.598274311503042,
 '2000-08-19 03:36:36.628682500': 8.071000254956843,
 '2000-09-04 03:36:21.848296900': 11.523936365863463,
 '2000-09-20 03:36:01.091897200': 7.489863340000759,
 '2000-11-23 03:35:50.995549900': -2.4900987632037457,
 '2000-01-24 03:38:45.510570': -17.414731276371576,
 '2000-04-13 03:38:16.212155900': -2.4195122715698627,
 '2000-08-19 03:37:00.502957500': 4.998069309635707,
 '2000-09-04 03:36:45.718253300': 10.676152820256627,
 '2000-09-20 03:36:24.958977200': -1.1971821531791358,
 '2000-10-22 03:36:11.396893800': -0.8100750152118824,
 '2000-11-23 03:36:14.871124200': -4.105939522189061,
 '2000-01-31 03:44:08.246100600': -19.106480750607812,
 '2000-06-07 03:43:10.596284800': 3.399308032741821,
 '2000-06-23 03:43:07.081646600': 4.589735695109813,
 '2000-07-25 03:42:33.924568500': 6.200983282706346,
 '2000-08-10 03:42:24.847798600': 18.113945114602128,
 '2000-11-14 03:41:40.735523300': 2.109625638395893,
 '2000-01-31 03:44:32.127497400': -18.161000982254862,
 '2000-06-07 03:43:34.466098300': 3.4925700480863227,
 '2000-06-23 03:43:30.958589900': 6.653018867275914,
 '2000-07-25 03:42:57.794771100': 9.952702816668465,
 '2000-08-10 03:42:48.720784400': 17.773767652958437,
 '2000-11-14 03:42:04.615706500': -4.19098385245065,
 '2000-01-31 03:44:55.997148800': -17.934218420067065,
 '2000-03-03 03:44:45.828678100': -12.308893698007036,
 '2000-04-04 03:44:35.363476700': -3.7706170164987176,
 '2000-06-07 03:43:58.342506400': 4.952519008546243,
 '2000-07-25 03:43:21.671860900': 15.152754833330919,
 '2000-08-10 03:43:12.596027700': 18.02312514770467,
 '2000-09-11 03:42:49.171474600': 12.080027091531987,
 '2000-11-14 03:42:28.495034': -3.6475160602652266,
 '2000-11-30 03:42:30.438969': None,
 '2000-10-20 03:48:07.465332': 5.845926680924447,
 '2000-02-07 03:51:03.835753400': -20.523365959883503,
 '2000-04-11 03:50:39.290292600': -2.509543131341006,
 '2000-04-27 03:50:38.112069700': -1.5246067065249567,
 '2000-07-16 03:49:46.699046': 12.266318045904699,
 '2000-08-17 03:49:23.338911700': 10.567774843391005,
 '2000-09-18 03:48:49.926630300': 6.594996715009404,
 '2000-10-20 03:48:31.339090900': 2.2363694211700236,
 '2000-12-07 03:48:38.004134300': -14.880106207945795,
 '2000-01-13 03:57:19.741266900': -7.374370882830582,
 '2000-02-14 03:57:13.295233100': None,
 '2000-05-04 03:56:44.426727900': 1.375191461098297,
 '2000-08-08 03:55:34.220556200': 15.765423656701293,
 '2000-10-27 03:54:48.445080700': 5.821134698757021,
 '2000-11-28 03:54:52.095654800': -2.0382137011215917,
 '2000-03-13 03:12:13.224063': -15.62582173883401,
 '2000-03-29 03:12:56.259075': -7.921499967251292,
 '2000-04-30 03:13:58.121006': -4.9595991509510675,
 '2000-06-01 03:14:37.951044': -0.4702258718991203,
 '2000-06-17 03:14:58.773094': 7.737678671772507,
 '2000-07-03 03:15:20.805000': 9.259507910246658,
 '2000-07-19 03:15:38.783019': 13.182878302131265,
 '2000-08-04 03:15:52.266063': 16.418612619625645,
 '2000-10-07 03:17:04.217063': 4.104616401492018,
 '2000-10-23 03:17:16.004081': -14.494666693823643,
 '2000-03-13 03:12:36.914013': -14.135441027619587,
 '2000-03-29 03:13:20.096019': -5.857015170352374,
 '2000-04-30 03:14:21.958094': -3.7083218662605915,
 '2000-06-01 03:15:01.790069': 2.1925142740683268,
 '2000-06-17 03:15:22.613056': 5.140619432706713,
 '2000-07-03 03:15:44.645044': 3.5456891373842607,
 '2000-07-19 03:16:02.624019': 11.803349410183666,
 '2000-08-04 03:16:15.963019': 14.299463453732804,
 '2000-10-07 03:17:28.061050': 6.120101007704306,
 '2000-10-23 03:17:39.849025': None,
 '2000-01-25 03:13:57.781075': -20.228833988121963,
 '2000-03-13 03:13:01.039075': -11.162089459491401,
 '2000-03-29 03:13:44.223031': -4.162872997913145,
 '2000-06-01 03:15:25.775038': 3.7504651668177256,
 '2000-06-17 03:15:46.598063': 3.871167874559235,
 '2000-07-03 03:16:08.631019': 3.4672989026223484,
 '2000-08-04 03:16:40.095081': 13.9157601712139,
 '2000-09-05 03:17:28.936038': None,
 '2000-10-07 03:17:52.196006': 6.5935633990974996,
 '2000-11-24 03:18:33.104038': -10.596711837062687,
 '2000-01-25 03:14:21.468031': -18.75071607836709,
 '2000-02-17 03:18:35.838019': -19.71906151535832,
 '2000-03-20 03:18:45.195075': -11.00322085527517,
 '2000-04-21 03:19:41.523025': -9.65334448459172,
 '2000-05-07 03:20:21.850081': -0.9526296311625533,
 '2000-06-08 03:20:57.555019': 1.4569360999503105,
 '2000-06-24 03:21:19.617013': 5.477754857044064,
 '2000-08-11 03:22:17.172069': 20.304829735647868,
 '2000-08-27 03:22:37.227094': 15.602920603692155,
 '2000-02-01 03:19:29.555075': -20.490512564484845,
 '2000-02-17 03:18:59.671019': -19.536351061676555,
 '2000-03-20 03:19:09.031031': -11.176313627241983,
 '2000-04-21 03:20:05.360081': -8.802198390559067,
 '2000-05-07 03:20:45.689000': -0.6714789930204043,
 '2000-06-08 03:21:21.394056': -0.6939592592620216,
 '2000-06-24 03:21:43.457013': 3.73859606784889,
 '2000-08-11 03:22:41.014063': 19.693169647362836,
 '2000-08-27 03:23:01.070056': 15.889177144500076,
 '2000-10-30 03:23:56.644056': 2.772218004626536,
 '2000-11-15 03:24:08.837038': -8.708286609212832,
 '2000-02-01 03:19:53.532069': -18.855191481157064,
 '2000-02-17 03:19:23.649038': -21.450061830323,
 '2000-03-20 03:19:33.012019': -7.7581824467039295,
 '2000-04-21 03:20:29.343063': None,
 '2000-05-07 03:21:09.672063': -0.415235567513099,
 '2000-05-23 03:21:24.900000': 1.9307257556894097,
 '2000-06-08 03:21:45.524063': 1.9871278740943241,
 '2000-06-24 03:22:07.587069': 5.922947990943048,
 '2000-07-26 03:22:39.772081': 11.889680022097567,
 '2000-08-11 03:23:05.147038': 18.361669529298023,
 '2000-08-27 03:23:25.058056': 15.279226177823881,
 '2000-10-30 03:24:20.634056': 2.998088835091697,
 '2000-11-15 03:24:32.973025': -1.757345289087482,
 '2000-02-01 03:20:17.219000': -17.50080027734665,
 '2000-02-17 03:19:47.482025': -23.59135222623043,
 '2000-03-20 03:19:56.847081': -10.707856503627415,
 '2000-04-21 03:20:53.035081': None,
 '2000-05-07 03:21:33.365056': 0.38209032104690527,
 '2000-05-23 03:21:48.593025': 4.2019088110695835,
 '2000-06-08 03:22:09.218075': 3.932099727210138,
 '2000-06-24 03:22:31.282031': 9.66400087965431,
 '2000-07-26 03:23:03.614006': 13.443051019264349,
 '2000-08-11 03:23:28.843094': 17.508452120390885,
 '2000-08-27 03:23:48.755069': 14.158510502419889,
 '2000-10-14 03:24:30.335038': 5.500366008507565,
 '2000-10-30 03:24:44.333088': 1.1082682375699822,
 '2000-11-15 03:24:56.673006': -0.9155426672859407,
 '2000-12-01 03:25:09.720081': -3.447666491661242,
 '2000-02-08 03:25:28.803094': -17.698140534247198,
 '2000-03-11 03:24:51.826050': -10.234067426735965,
 '2000-03-27 03:25:38.105056': -4.0559888372670905,
 '2000-04-28 03:26:38.525094': -30.937491241158128,
 '2000-05-30 03:27:20.743006': -1.9309061428901395,
 '2000-06-15 03:27:41.075019': 6.339199897038966,
 '2000-08-02 03:28:33.735069': -20.3858627834055,
 '2000-08-18 03:29:01.379063': 11.521677410305424,
 '2000-09-19 03:29:34.429019': 10.462831390831488,
 '2000-10-05 03:29:48.145044': 5.562966937618309,
 '2000-10-21 03:29:58.981094': 1.2105814935279327,
 '2000-01-23 03:26:24.160000': -19.28804631193872,
 '2000-02-08 03:25:52.780081': -15.799798925748389,
 '2000-02-24 03:25:16.107013': -14.37724141798529,
 '2000-03-11 03:25:15.951094': -8.63814552395371,
 '2000-03-27 03:26:02.087006': -4.545828856411345,
 '2000-04-12 03:26:09.655075': -1.7543752011010911,
 '2000-04-28 03:27:02.509019': -9.596720564681378,
 '2000-05-14 03:27:30.264063': -2.8797686990217715,
 '2000-05-30 03:27:44.727050': 3.924408979712407,
 '2000-06-15 03:28:05.205038': 7.281452109791463,
 '2000-07-01 03:28:28.221019': None,
 '2000-08-18 03:29:25.367000': 11.488763770989735,
 '2000-09-19 03:29:58.563031': 11.06374789783177,
 '2000-10-05 03:30:12.134088': 7.6329399495342285,
 '2000-10-21 03:30:22.971069': 3.5339459657274466,
 '2000-11-22 03:30:52.132013': -0.9701808047864949,
 '2000-01-23 03:26:47.846025': -20.393344392626645,
 '2000-02-08 03:26:16.467006': -16.160208629909146,
 '2000-02-24 03:25:39.795050': None,
 '2000-03-11 03:25:39.641025': -10.43193567738195,
 '2000-03-27 03:26:25.923025': -3.409048514492929,
 '2000-04-12 03:26:33.492081': -4.691752713946161,
 '2000-05-14 03:27:53.957069': -2.772529835242004,
 '2000-05-30 03:28:08.421019': 3.71082931546119,
 '2000-06-15 03:28:28.899044': 4.4439843650572906,
 '2000-07-01 03:28:51.915094': 3.2059604330098996,
 '2000-08-02 03:29:21.564056': None,
 '2000-08-18 03:29:49.209019': 11.066504548119843,
 '2000-09-19 03:30:22.261044': 9.670157663536266,
 '2000-10-05 03:30:35.833056': 6.519636343754205,
 '2000-10-21 03:30:46.670069': 4.316301393129727,
 '2000-11-06 03:30:56.168013': None,
 '2000-11-22 03:31:15.831069': -5.305150523276553,
 '2000-03-18 03:31:48.513094': -14.870821067075044,
 '2000-04-19 03:32:43.768038': -1.108838710024564,
 '2000-06-06 03:34:05.023019': -0.13683178325209677,
 '2000-06-22 03:34:25.751069': -11.343163193903827,
 '2000-07-08 03:34:47.089069': -29.948796800777455,
 '2000-07-24 03:35:01.589094': 7.658614770966553,
 '2000-08-09 03:35:22.591006': 16.17479787048059,
 '2000-08-25 03:35:42.931094': None,
 '2000-09-10 03:36:06.588019': 12.986150792404786,
 '2000-09-26 03:36:16.835050': 10.880039799077277,
 '2000-10-12 03:36:25.686063': 7.912702377060213,
 '2000-10-28 03:36:41.702000': -0.0620380654350411,
 '2000-11-13 03:36:52.027019': -28.518034461714322,
 '2000-01-30 03:32:42.275075': -20.58776883749178,
 '2000-02-15 03:32:14.105038': -22.58495255809908,
 '2000-03-18 03:32:12.349031': -12.476588797013955,
 '2000-04-19 03:33:07.460063': -0.5423943552148301,
 '2000-05-05 03:33:51.794069': -7.881270692200845,
 '2000-06-06 03:34:28.717025': 3.639222586467754,
 '2000-06-22 03:34:49.446019': 3.512371187233848,
 '2000-07-08 03:35:10.784075': 4.732772115907198,
 '2000-07-24 03:35:25.285081': 13.722294081496186,
 '2000-08-09 03:35:46.287069': 16.947041244961028,
 '2000-08-25 03:36:06.629013': 11.13878619702546,
 '2000-09-10 03:36:30.286006': 10.944433394321008,
 '2000-09-26 03:36:40.533094': 9.240172585766901,
 '2000-10-12 03:36:49.385013': 5.946223330749835,
 '2000-10-28 03:37:05.546063': -0.054152356732919654,
 '2000-11-29 03:37:32.534088': -9.355070258098525,
 '2000-02-22 03:38:05.411081': -15.93077204056817,
 '2000-03-09 03:37:56.397019': -8.10280172438914,
 '2000-04-10 03:38:53.858050': -1.27542362706056,
 '2000-04-26 03:39:42.467081': 0.0662993787551494,
 '2000-05-12 03:40:13.422038': -2.257035275263299,
 '2000-05-28 03:40:27.070019': None,
 '2000-08-16 03:42:08.837013': 9.719091733798683,
 '2000-09-01 03:42:29.815031': 9.002559238049049,
 '2000-10-03 03:42:57.361000': 4.365626764395905,
 '2000-10-19 03:43:05.009013': None,
 '2000-11-04 03:43:17.436038': 4.577924346045811,
 '2000-11-20 03:43:34.143088': 2.1450315247179566,
'2001-04-25 03:28:38.597242': -4.278918962109514,
 '2001-06-12 03:28:11.705008500': 4.609386787428374,
 '2001-07-30 03:27:45.435381600': 13.727172043541104,
 '2001-08-31 03:27:16.502226700': 16.26646521084795,
 '2001-10-18 03:26:51.641899600': 3.2325683612831857,
 '2001-04-25 03:29:02.490194400': -5.779807935915401,
 '2001-06-12 03:28:35.584338900': 3.5376763716642103,
 '2001-07-30 03:28:09.315089600': 10.895842649891406,
 '2001-08-31 03:27:40.382075100': 15.838761338748284,
 '2001-10-02 03:27:16.746188200': 6.0221995423150405,
 '2001-10-18 03:27:15.539730400': 4.59862249244397,
 '2001-04-25 03:29:26.361585500': -12.39193873138029,
 '2001-06-12 03:28:59.457871300': 4.239201909829961,
 '2001-07-30 03:28:33.194409200': 10.29529378393163,
 '2001-08-31 03:28:04.252812800': 16.849192443894584,
 '2001-10-02 03:27:40.619599700': 2.772030184936525,
 '2001-10-18 03:27:39.418556100': 4.751536887049733,
 '2001-05-02 03:34:50.299372': -12.59809483692393,
 '2001-06-03 03:34:27.922825100': 1.602774794311284,
 '2001-06-19 03:34:16.722416100': 5.880714980810778,
 '2001-07-21 03:34:03.633909100': 7.15133126996204,
 '2001-08-22 03:33:33.966771900': 15.200982107071574,
 '2001-06-03 03:34:51.798099900': -1.2432413473114308,
 '2001-06-19 03:34:40.599726500': 4.6106395080757725,
 '2001-07-21 03:34:27.517674500': 9.428359114782426,
 '2001-08-22 03:33:57.863430800': 15.980925362604463,
 '2001-02-27 03:35:52.918031800': -12.615707742305641,
 '2001-05-02 03:35:38.056540600': None,
 '2001-06-03 03:35:15.672391300': 3.195806116159527,
 '2001-06-19 03:35:04.469879500': 6.029521834432094,
 '2001-07-21 03:34:51.394476400': 12.191986171586532,
 '2001-08-22 03:34:21.723239400': 15.492120596881161,
 '2001-09-23 03:34:01.276538500': 8.102298825366807,
 '2001-01-10 03:36:25.238443400': -24.981989313538314,
 '2001-02-27 03:36:16.810317700': -11.528192437809258,
 '2001-05-02 03:36:01.928116800': None,
 '2001-06-03 03:35:39.543940900': 6.350370051867676,
 '2001-06-19 03:35:28.343653500': 7.530078525220905,
 '2001-07-21 03:35:15.265489': 11.952799991528758,
 '2001-08-22 03:34:45.594205100': 14.231983353809175,
 '2001-09-23 03:34:25.146022900': 8.728318749235187,
 '2001-02-02 03:41:48.483624600': None,
 '2001-04-07 03:41:22.567843800': -12.445037582851782,
 '2001-05-25 03:41:10.021099200': -21.192069451093083,
 '2001-06-10 03:40:58.192526300': None,
 '2001-06-26 03:40:41.140583600': 4.1153573150079445,
 '2001-07-12 03:40:42.152843400': 5.184169045150595,
 '2001-07-28 03:40:32.843483': 11.415432946694796,
 '2001-08-13 03:40:13.165145500': 16.195113159817854,
 '2001-11-01 03:39:38.527011400': 3.4029987594981033,
 '2001-02-02 03:42:12.366465100': -29.463351318440406,
 '2001-03-06 03:42:04.728879300': -15.662173783419856,
 '2001-04-07 03:41:46.451756900': -5.157078657915005,
 '2001-05-25 03:41:33.889287400': -0.47573449480336344,
 '2001-06-26 03:41:05.009422': 6.546641337086945,
 '2001-07-12 03:41:06.023503700': 10.056072905380962,
 '2001-08-13 03:40:37.038027400': 16.650955779103448,
 '2001-11-17 03:40:15.392793500': 1.659071599447945,
 '2001-02-18 03:42:33.130963200': -12.921426368814945,
 '2001-03-06 03:42:28.592551400': -15.479977864861302,
 '2001-04-07 03:42:10.325021900': -2.0496908895481303,
 '2001-05-09 03:42:08.149900800': 0.35465730606100937,
 '2001-06-26 03:41:28.878677800': 6.785036749056154,
 '2001-07-12 03:41:29.908217': 10.612482182699889,
 '2001-08-13 03:41:00.906810800': 14.890390024286296,
 '2001-10-16 03:40:22.891179900': 4.439836162574802,
 '2001-11-17 03:40:39.272234400': 3.956545723754789,
 '2001-10-23 03:46:12.352145200': 6.663896562156162,
 '2001-01-24 03:48:50.537566900': -21.769746551818493,
 '2001-04-14 03:48:26.177844600': -17.839611689718648,
 '2001-06-01 03:48:03.255014900': 0.17651884977172422,
 '2001-06-17 03:47:52.018958500': 5.156237971601699,
 '2001-08-20 03:47:07.954985200': 14.251504024799596,
 '2001-10-23 03:46:36.226481400': 5.796852431038294,
 '2001-01-15 03:54:58.565926400': -23.23739971125277,
 '2001-04-05 03:54:32.320076400': -2.234836730025248,
 '2001-05-07 03:54:31.563060200': -0.9133776566774449,
 '2001-06-08 03:54:08.200565400': 6.883930679203368,
 '2001-07-10 03:53:51.869923900': 6.136480014671527,
 '2001-07-26 03:53:43.996155900': 10.879954322009239,
 '2001-08-27 03:53:15.040270500': 12.674105801641595,
 '2001-09-28 03:52:54.654831200': 5.873995081250703,
 '2001-02-12 03:18:35.843038': -28.81879078675256,
 '2001-02-28 03:18:43.033019': -21.505254663447705,
 '2001-04-01 03:18:41.469025': -8.362171405231479,
 '2001-05-03 03:18:47.246006': None,
 '2001-06-04 03:18:56.425013': 2.0245712388125137,
 '2001-06-20 03:18:53.868075': 6.365536735825074,
 '2001-07-06 03:18:58.588056': 7.967435410674761,
 '2001-07-22 03:18:55.695094': 10.44402890325835,
 '2001-08-07 03:18:54.451063': 11.956795924470358,
 '2001-08-23 03:18:50.539075': None,
 '2001-09-08 03:18:45.040038': 10.799249940977681,
 '2001-09-24 03:18:37.819006': 5.2399750230594515,
 '2001-10-10 03:18:30.128038': 4.436925893825565,
 '2001-02-12 03:18:59.546031': -26.491102395031348,
 '2001-02-28 03:19:06.737063': -18.92355624720315,
 '2001-04-01 03:19:05.320038': -7.428376588605686,
 '2001-05-03 03:19:11.098069': -7.760212407449979,
 '2001-05-19 03:19:09.173031': -0.6308058390640491,
 '2001-06-04 03:19:20.279056': 1.6269143859470612,
 '2001-06-20 03:19:17.723075': 4.53332716774424,
 '2001-07-06 03:19:22.444081': 3.469039556856811,
 '2001-07-22 03:19:19.552063': None,
 '2001-08-07 03:19:18.163069': 13.00974684185319,
 '2001-08-23 03:19:14.252069': 13.64878184074878,
 '2001-09-08 03:19:08.754006': 10.344813835875497,
 '2001-09-24 03:19:01.679038': 4.61818594978691,
 '2001-10-10 03:18:53.989069': 6.055694624929292,
 '2001-10-26 03:18:45.354075': 1.362167769365036,
 '2001-01-27 03:19:18.195056': None,
 '2001-02-12 03:19:23.685038': -26.916559554322497,
 '2001-02-28 03:19:30.878019': -15.73194105524214,
 '2001-04-01 03:19:29.317000': -5.6700224869500016,
 '2001-05-19 03:19:33.172000': 0.6038988385545618,
 '2001-06-04 03:19:44.279050': 1.6502238618296325,
 '2001-06-20 03:19:41.724019': 4.283890550007697,
 '2001-07-06 03:19:46.446056': 6.347076737290019,
 '2001-07-22 03:19:43.554081': -1.7160517633256338,
 '2001-08-07 03:19:42.312006': 13.757854856688416,
 '2001-08-23 03:19:38.402006': 13.053202214886818,
 '2001-09-08 03:19:32.904031': 11.147971029842363,
 '2001-09-24 03:19:25.685025': 3.917498542554897,
 '2001-10-10 03:19:18.141094': 6.41311502538694,
 '2001-10-26 03:19:09.508019': 4.144524247164789,
 '2001-11-27 03:18:56.558044': -10.200805127928419,
 '2001-02-03 03:24:45.125019': -32.12007427804324,
 '2001-02-19 03:24:48.039050': None,
 '2001-03-07 03:24:54.186000': -20.75117579700408,
 '2001-03-23 03:24:54.604013': -17.974563318882115,
 '2001-04-08 03:24:40.310063': -1.9889579600738136,
 '2001-05-10 03:24:56.863031': -4.962139527440743,
 '2001-05-26 03:25:01.206063': -0.28199507515034,
 '2001-06-11 03:25:07.614056': 3.1809042291144425,
 '2001-06-27 03:25:06.841006': 4.380780221954479,
 '2001-07-13 03:25:09.829050': 12.652994195786265,
 '2001-07-29 03:25:04.935063': 13.066458547825736,
 '2001-08-14 03:25:04.306050': 9.375733911521234,
 '2001-08-30 03:24:57.219056': 14.686623762887914,
 '2001-09-15 03:24:51.944038': 5.004010447071322,
 '2001-10-17 03:24:42.956044': -1.1869997016232203,
 '2001-11-02 03:24:25.925013': 0.7357880353749044,
 '2001-02-03 03:25:08.973013': -31.822810304352483,
 '2001-02-19 03:25:11.888056': None,
 '2001-03-07 03:25:18.035069': -22.50788270618022,
 '2001-03-23 03:25:18.454075': -18.170628136556306,
 '2001-04-08 03:25:04.162000': -3.418835846300605,
 '2001-04-24 03:25:14.912006': -5.749672465035155,
 '2001-05-10 03:25:20.716031': -5.1106985904044135,
 '2001-05-26 03:25:25.060056': -32.365604844415294,
 '2001-06-11 03:25:31.469019': 2.658826478852328,
 '2001-06-27 03:25:30.696063': 4.034333954763628,
 '2001-07-13 03:25:33.685081': 9.62890642254131,
 '2001-07-29 03:25:28.792075': 9.44607386238257,
 '2001-08-14 03:25:28.164038': 8.166450543553681,
 '2001-08-30 03:25:21.078025': None,
 '2001-09-15 03:25:15.804025': None,
 '2001-10-17 03:25:06.818000': 3.425061644330518,
 '2001-11-02 03:24:49.787081': -4.840021272573138,
 '2001-02-03 03:25:32.966044': -30.019654210806255,
 '2001-02-19 03:25:35.882094': -13.550629135702252,
 '2001-03-07 03:25:42.030088': None,
 '2001-03-23 03:25:42.596044': -14.602953055101567,
 '2001-04-08 03:25:28.304019': -6.496380832114163,
 '2001-04-24 03:25:38.909044': -1.3080048904882347,
 '2001-05-10 03:25:44.860025': -6.322152679570053,
 '2001-05-26 03:25:49.060000': 58.208587103698186,
 '2001-06-11 03:25:55.469025': 1.8273716560670683,
 '2001-06-27 03:25:54.697075': 4.914061478547414,
 '2001-07-13 03:25:57.687063': 10.999245807325956,
 '2001-07-29 03:25:52.795050': 12.057714943455094,
 '2001-08-14 03:25:52.167081': None,
 '2001-08-30 03:25:45.082044': 13.151136119571348,
 '2001-10-01 03:25:31.080044': 3.8614474349144556,
 '2001-10-17 03:25:30.825013': 4.363199963315565,
 '2001-11-02 03:25:13.796000': 2.2897296033620744,
 '2001-11-18 03:25:05.550063': 1.9276407426775155,
 '2001-01-18 03:25:49.476044': -18.301055190848878,
 '2001-02-03 03:25:56.668088': -29.57920984452539,
 '2001-02-19 03:25:59.732006': -11.761809516982384,
 '2001-03-07 03:26:05.735019': None,
 '2001-03-23 03:26:06.301075': -13.014722243016363,
 '2001-04-08 03:25:52.010019': None,
 '2001-04-24 03:26:02.616000': -0.8078112808822717,
 '2001-05-10 03:26:08.567088': -0.23184430430870873,
 '2001-05-26 03:26:12.914000': -46.171733123817255,
 '2001-06-11 03:26:19.324000': 4.565550959490713,
 '2001-06-27 03:26:18.407094': 5.7401408341101785,
 '2001-07-13 03:26:21.398044': 11.813357096128078,
 '2001-07-29 03:26:16.507025': 14.238834965941226,
 '2001-08-14 03:26:15.880031': 11.044402206794372,
 '2001-08-30 03:26:08.795063': 12.370319050496104,
 '2001-10-01 03:25:54.795069': 5.800701022697226,
 '2001-10-17 03:25:54.541031': 3.935734486566435,
 '2001-11-02 03:25:37.513031': 4.334070346458884,
 '2001-11-18 03:25:29.414038': 2.463777364321665,
 '2001-12-04 03:25:26.440013': None,
 '2001-02-10 03:31:21.540006': -25.689757930639868,
 '2001-02-26 03:31:27.531069': -28.489422926017884,
 '2001-03-14 03:31:27.590031': None,
 '2001-04-15 03:31:18.198006': -0.7835638324323604,
 '2001-06-02 03:31:41.013025': 4.140738310697056,
 '2001-06-18 03:31:38.678063': 5.186957737950949,
 '2001-07-04 03:31:43.537050': None,
 '2001-07-20 03:31:42.229006': 7.188777944144591,
 '2001-08-05 03:31:40.181038': 14.46438351152214,
 '2001-08-21 03:31:37.090044': 13.652121917908422,
 '2001-09-06 03:31:31.020006': 1.6219933476054165,
 '2001-09-22 03:31:23.936000': 40.65221130798953,
 '2001-10-08 03:31:13.676038': -53.21147511962055,
 '2001-10-24 03:31:10.498031': None,
 '2001-01-25 03:31:37.963013': None,
 '2001-02-10 03:31:45.679031': -23.593417152969838,
 '2001-02-26 03:31:51.526069': -12.259371292811316,
 '2001-03-14 03:31:51.731044': -9.874049121167134,
 '2001-04-15 03:31:42.195031': -0.2677837041038015,
 '2001-06-02 03:32:05.158050': 3.766737135587771,
 '2001-06-18 03:32:02.679006': 6.359014938451794,
 '2001-07-04 03:32:07.539013': 6.14062148446323,
 '2001-07-20 03:32:06.231013': 12.002795318161784,
 '2001-08-05 03:32:04.184038': 12.794942092849226,
 '2001-08-21 03:32:01.094038': 14.746468456693602,
 '2001-09-06 03:31:55.024069': 5.921400202830753,
 '2001-09-22 03:31:48.087019': -3.0106992156119143,
 '2001-10-08 03:31:37.828056': -43.77495508982828,
 '2001-10-24 03:31:34.506000': None,
 '2001-11-25 03:31:18.274069': 1.4642560848879482,
 '2001-01-25 03:32:01.810063': -23.407114447067343,
 '2001-02-10 03:32:09.382031': -20.946504509003702,
 '2001-02-26 03:32:15.376025': -11.08793432663982,
 '2001-03-14 03:32:15.436019': -9.674216722815263,
 '2001-04-15 03:32:05.901069': -0.31500625846738245,
 '2001-05-17 03:32:17.678056': None,
 '2001-06-02 03:32:28.867038': 4.4421385186494255,
 '2001-06-18 03:32:26.388044': 6.586109601941429,
 '2001-07-04 03:32:31.249088': 5.574780830196409,
 '2001-07-20 03:32:30.087075': 13.306008840475572,
 '2001-08-05 03:32:27.896050': 8.757992487079765,
 '2001-08-21 03:32:24.807031': 13.980402749095612,
 '2001-09-06 03:32:18.738025': 8.734435296165852,
 '2001-09-22 03:32:11.802000': -2.6995035032465706,
 '2001-10-08 03:32:01.544019': -17.8215459016387,
 '2001-10-24 03:31:58.222063': 1.131417874622419,
 '2001-11-25 03:31:41.993069': 0.34715043411395263,
 '2001-02-17 03:37:56.487063': -38.42942299625713,
 '2001-03-05 03:38:04.126094': -12.406300765362564,
 '2001-03-21 03:38:04.855081': -9.98388297020562,
 '2001-04-22 03:37:57.790038': -0.9158478398511903,
 '2001-05-08 03:38:07.260013': -2.018381142250016,
 '2001-05-24 03:38:09.248000': -11.577780501981536,
 '2001-01-16 03:38:10.421081': -20.686382043261393,
 '2001-02-01 03:38:17.451081': -28.080971821379496,
 '2001-02-17 03:38:20.190094': -14.309489092017651,
 '2001-03-05 03:38:27.831050': -14.58211377633576,
 '2001-03-21 03:38:28.561006': -7.609124970778638,
 '2001-04-22 03:38:21.642044': -0.8750207939346305,
 '2001-05-08 03:38:30.967056': -5.279879665674695,
 '2001-05-24 03:38:32.956013': -0.16329854181027093,
 '2001-06-09 03:38:41.248063': -1.3567106258681556,
 '2001-06-25 03:38:39.806038': 5.914061018402825,
 '2001-07-11 03:38:43.140069': 10.125964125481673,
 '2001-07-27 03:38:38.318031': 8.371037355000844,
 '2001-08-12 03:38:37.823094': None,
 '2001-09-13 03:38:26.951044': 7.70441682868338,
 '2001-09-29 03:38:17.889025': -25.39982787659135,
 '2001-10-15 03:38:16.657081': 48.2878333780291,
 '2001-10-31 03:37:59.198081': 3.8011085968429077,
 '2001-11-16 03:37:51.684063': 1.441482530468664,
 '2001-12-02 03:37:50.418000': -13.14730785313304,
 '2001-01-23 03:44:23.610088': -21.954542325675906,
 '2001-02-08 03:44:31.055088': -33.40790742789825,
 '2001-02-24 03:44:35.887069': -14.069340552107487,
 '2001-03-28 03:44:39.968000': -11.958988939884605,
 '2001-04-13 03:44:26.479069': None,
 '2001-04-29 03:44:40.241006': None,
 '2001-05-15 03:44:38.633094': 3.088214885017329,
 '2001-06-16 03:44:49.669094': 7.882735756868748,
 '2001-07-02 03:44:52.047044': 6.160935409710054,
 '2001-07-18 03:44:52.617044': 9.866309054118723,
 '2001-08-19 03:44:47.382000': 11.075579901979355,
 '2001-09-04 03:44:39.833088': -10.383062894774326,
 '2001-09-20 03:44:33.503013': 40.27920107186723,
 '2001-10-22 03:44:21.483069': -2.2579663777815586,
 '2001-11-07 03:44:09.959013': -9.873685491219632,
'2002-02-23 03:27:28.576731500': -14.895686049806347,
 '2002-03-27 03:27:23.779455400': -12.108060399301586,
 '2002-05-14 03:27:18.427891800': 0.9900107918374597,
 '2002-06-15 03:27:05.989146700': None,
 '2002-08-02 03:26:47.880437400': 13.637815918119484,
 '2002-08-18 03:26:44.305248300': 5.736533254618867,
 '2002-09-03 03:26:34.540765400': 10.619437099036162,
 '2002-10-05 03:26:17.900974400': 4.866625915535425,
 '2002-10-21 03:26:21.946572': 0.7258504240979918,
 '2002-11-06 03:26:27.531611': 0.975594450946762,
 '2002-02-23 03:27:52.471018700': -10.394170386786731,
 '2002-05-14 03:27:42.314675900': 1.0332511342052522,
 '2002-08-02 03:27:11.780175': 13.167171637663413,
 '2002-09-03 03:26:58.429443800': 11.626968896209064,
 '2002-10-05 03:26:41.778914300': 4.910022763846367,
 '2002-10-21 03:26:45.842466400': 1.9402805711311386,
 '2002-11-06 03:26:51.426820800': -0.42800837111854756,
 '2002-02-23 03:28:16.348245200': -8.469193148474918,
 '2002-05-14 03:28:06.189853700': 1.735871782332019,
 '2002-07-01 03:27:48.977160500': None,
 '2002-08-02 03:27:35.652030500': 14.492288115613073,
 '2002-09-03 03:27:22.298916100': 13.768929446574504,
 '2002-10-05 03:27:05.658822700': 6.685026234775262,
 '2002-10-21 03:27:09.731146200': 0.33462403756396075,
 '2002-03-18 03:33:33.495844700': -12.93465159560464,
 '2002-06-06 03:33:21.268005700': 2.1169235704606364,
 '2002-07-24 03:33:00.733051300': 20.574640604891638,
 '2002-08-25 03:32:48.065287200': 14.234220443868361,
 '2002-09-26 03:32:31.658971': 8.20808729688242,
 '2002-11-13 03:32:38.557984100': -1.9484483927681406,
 '2002-03-02 03:34:07.225455100': -14.099835419708446,
 '2002-04-19 03:33:55.886376600': -5.55214650287218,
 '2002-07-24 03:33:24.618476500': 18.259622837739425,
 '2002-09-26 03:32:55.535092100': 8.639318955430587,
 '2002-11-13 03:33:02.449562200': -6.209888289428356,
 '2002-03-02 03:34:31.126505900': -13.53870998657369,
 '2002-04-19 03:34:19.768070700': -5.460399692720441,
 '2002-06-22 03:34:01.727307700': None,
 '2002-07-24 03:33:48.499035600': 21.63257734164925,
 '2002-09-26 03:33:19.415228900': 9.865561664114871,
 '2002-11-13 03:33:26.341673700': -2.584395762038575,
 '2002-03-02 03:34:55.000864800': -11.107916136685777,
 '2002-03-18 03:34:45.135958900': -13.722632980330157,
 '2002-06-22 03:34:25.600214400': None,
 '2002-07-08 03:34:22.382452200': 8.56846033187888,
 '2002-07-24 03:34:12.374066700': 21.47562641862591,
 '2002-09-10 03:33:54.210211500': 7.899769559920831,
 '2002-11-13 03:33:50.225403700': 0.9278257646855858,
 '2002-04-10 03:40:09.679679700': -2.8019264655158787,
 '2002-04-26 03:40:07.984920600': -0.7632475966726885,
 '2002-06-13 03:39:51.924995200': 4.418147295172442,
 '2002-10-03 03:39:03.526930': None,
 '2002-10-19 03:39:06.411821500': 1.5749337921021767,
 '2002-11-04 03:39:12.071222900': 1.4091997650799069,
 '2002-01-20 03:40:35.020793400': -12.71883424513599,
 '2002-04-10 03:40:33.555052800': -3.3662943170236876,
 '2002-04-26 03:40:31.864399100': -0.48072187538951044,
 '2002-06-13 03:40:15.799515400': 6.422702497467393,
 '2002-11-04 03:39:35.953079900': 0.9681821210596708,
 '2002-01-20 03:40:58.900646900': -10.018085680026408,
 '2002-04-10 03:40:57.433269': -3.035493291257084,
 '2002-04-26 03:40:55.737219200': 0.006577129830854005,
 '2002-06-13 03:40:39.674478300': 6.357337538978249,
 '2002-07-31 03:40:22.097716800': 15.629920003987536,
 '2002-09-17 03:39:55.055141900': 8.512731629219324,
 '2002-11-04 03:39:59.835153400': 1.5302067925623346,
 '2002-11-20 03:40:13.658949500': 0.7816223208243834,
 '2002-04-17 03:47:04.458562300': -10.477721673131583,
 '2002-05-03 03:47:09.893068600': -6.694078573187882,
 '2002-07-22 03:46:35.502363800': 18.340384395389233,
 '2002-11-11 03:46:11.701064100': -5.884775088093402,
 '2002-04-08 03:53:18.728818500': -5.6356059054292595,
 '2002-06-11 03:53:01.985373500': 5.253637715899565,
 '2002-07-29 03:52:44.404679900': 16.992577464051028,
 '2002-09-15 03:52:20.252244200': 8.150578151630523,
 '2002-10-01 03:52:13.239324': 2.0935038444916008,
 '2002-10-17 03:52:14.134186900': 3.7731463135222496,
 '2002-11-02 03:52:19.890053300': 4.946747925238068,
 '2002-02-15 03:17:07.245075': -11.229485443135994,
 '2002-03-03 03:16:49.398056': -13.777747900313143,
 '2002-03-19 03:16:39.703094': -51.71678289685231,
 '2002-05-22 03:15:20.742038': 1.2707636553637027,
 '2002-06-23 03:14:36.917019': 1.6208412754619992,
 '2002-07-09 03:14:12.364038': 11.776882343899372,
 '2002-07-25 03:13:55.042025': 23.313577054614523,
 '2002-08-10 03:13:25.821031': 18.086293742113977,
 '2002-09-11 03:12:36.933056': None,
 '2002-09-27 03:12:08.399013': 6.442589740487337,
 '2002-10-13 03:11:34.940006': 4.544271829540121,
 '2002-10-29 03:11:10.705038': 1.4752633457917976,
 '2002-11-14 03:10:32.296050': -5.079860601447163,
 '2002-01-30 03:17:52.873081': 25.619808704748625,
 '2002-02-15 03:17:31.118088': -8.218564647768973,
 '2002-03-03 03:17:13.128031': -13.720175277468172,
 '2002-03-19 03:17:03.436013': -48.74850298754203,
 '2002-05-22 03:15:44.572075': 1.6289785549765334,
 '2002-06-07 03:15:27.656094': 3.280246515617393,
 '2002-06-23 03:15:00.604081': -0.24898385787723085,
 '2002-07-09 03:14:36.194069': 8.849154757549217,
 '2002-07-25 03:14:18.872056': 22.635773165248516,
 '2002-08-10 03:13:49.651063': 17.849628110547247,
 '2002-08-26 03:13:17.864031': 12.193982563979576,
 '2002-09-11 03:13:00.763088': 10.245414437438848,
 '2002-09-27 03:12:32.229044': 8.001165479045017,
 '2002-10-13 03:11:58.627063': 4.328262366058876,
 '2002-10-29 03:11:34.535069': 1.1891273746513296,
 '2002-11-14 03:10:56.126081': -3.581943383183138,
 '2002-01-30 03:18:17.036019': 25.623557231379362,
 '2002-02-15 03:17:55.137050': -6.086379753067882,
 '2002-03-03 03:17:37.294081': -11.102940938609569,
 '2002-03-19 03:17:27.459056': -48.87505668073747,
 '2002-05-22 03:16:08.688044': 2.1390603515718722,
 '2002-06-07 03:15:51.630000': 5.365078170157518,
 '2002-06-23 03:15:24.720056': None,
 '2002-07-09 03:15:00.310044': 10.939542737837787,
 '2002-07-25 03:14:42.988031': 23.89382936219499,
 '2002-08-10 03:14:13.767031': 18.977070285951296,
 '2002-08-26 03:13:41.980000': 10.694821215905836,
 '2002-09-11 03:13:24.879056': 11.961136483196272,
 '2002-09-27 03:12:56.345013': 8.904418936935917,
 '2002-10-13 03:12:22.743031': 4.188887661209108,
 '2002-10-29 03:11:58.651038': -5.680036499977807,
 '2002-11-14 03:11:20.242050': -3.0990517521121124,
 '2002-01-30 03:18:40.762006': None,
 '2002-05-22 03:16:32.376013': 3.5118655902898137,
 '2002-06-07 03:16:15.460031': 11.434890848228871,
 '2002-06-23 03:15:48.408019': None,
 '2002-07-25 03:15:06.675094': 17.595775089541345,
 '2002-08-26 03:14:05.667063': None,
 '2002-09-11 03:13:48.567019': 10.883424957072405,
 '2002-03-26 03:22:46.229000': -24.82843826174141,
 '2002-05-13 03:21:42.952013': 0.4399181540960347,
 '2002-06-14 03:21:00.852088': 5.856990045395889,
 '2002-06-30 03:20:38.576019': 6.092048892509785,
 '2002-07-16 03:20:16.656063': 19.46328444650791,
 '2002-08-01 03:19:53.637019': 17.601534295907097,
 '2002-08-17 03:19:24.056031': 18.04487025221908,
 '2002-09-02 03:18:56.017075': None,
 '2002-10-04 03:18:07.424019': 4.986585914509375,
 '2002-11-05 03:17:08.133050': -2.6356611907752057,
 '2002-02-22 03:23:31.955038': -11.185419615418636,
 '2002-03-26 03:23:10.107013': -1.3864760952046695,
 '2002-05-13 03:22:06.782050': 1.08701366561164,
 '2002-06-14 03:21:24.683025': 5.06112494164294,
 '2002-06-30 03:21:02.406056': None,
 '2002-07-16 03:20:40.487000': 12.014058405934906,
 '2002-08-01 03:20:17.467056': 17.615619250488084,
 '2002-08-17 03:19:47.886063': 17.10891736180251,
 '2002-09-02 03:19:19.848000': 12.252625497755936,
 '2002-10-04 03:18:31.254050': 6.000842687974496,
 '2002-11-05 03:17:31.963081': 1.28622625858292,
 '2002-01-21 03:24:33.885063': -12.87767478478415,
 '2002-02-06 03:24:19.279006': -52.91991969014689,
 '2002-02-22 03:23:56.120075': -9.450300469024022,
 '2002-03-10 03:23:40.228013': None,
 '2002-03-26 03:23:34.130088': -8.273590168479817,
 '2002-05-13 03:22:30.755050': 2.0918320585408514,
 '2002-06-14 03:21:48.799000': 6.171875774452334,
 '2002-06-30 03:21:26.379063': 1.3778565287288957,
 '2002-07-16 03:21:04.602069': None,
 '2002-08-01 03:20:41.440056': 18.50203459918284,
 '2002-08-17 03:20:12.002038': 18.884370468540315,
 '2002-09-02 03:19:43.963075': 13.303284789721912,
 '2002-09-18 03:19:24.942031': 9.54274792798594,
 '2002-10-04 03:18:55.370019': 7.337976004284559,
 '2002-11-05 03:17:56.079050': 0.5614166666750849,
 '2002-01-21 03:24:57.610031': -11.725792416722035,
 '2002-02-06 03:24:43.005069': -42.122695511793,
 '2002-02-22 03:24:19.849050': -10.33699556122024,
 '2002-03-10 03:24:03.959019': -8.486614408829567,
 '2002-03-26 03:23:57.863050': -9.661384875114237,
 '2002-05-13 03:22:54.443019': 3.4816909144559487,
 '2002-06-14 03:22:12.486063': 7.996724796142485,
 '2002-06-30 03:21:50.209094': 6.976470658514582,
 '2002-08-01 03:21:05.128019': 17.033057738081418,
 '2002-08-17 03:20:35.690000': 18.557453504070565,
 '2002-09-02 03:20:07.651038': 13.524597601946503,
 '2002-09-18 03:19:48.772069': 10.02207622511944,
 '2002-10-04 03:19:19.057081': 8.119882332445343,
 '2002-02-13 03:29:56.028088': -11.260640090669094,
 '2002-03-01 03:29:36.867044': -19.560006079915183,
 '2002-03-17 03:29:25.626019': -36.17071935643543,
 '2002-06-05 03:27:52.527050': -4.561212816113292,
 '2002-07-23 03:26:43.306075': 19.922930050651868,
 '2002-08-08 03:26:14.844050': 11.745011543252842,
 '2002-08-24 03:25:43.952088': 13.608336814827304,
 '2002-09-09 03:25:23.964031': 9.418424955555466,
 '2002-09-25 03:24:54.106000': 9.284839951629657,
 '2002-10-11 03:24:23.943044': -27.612504163476427,
 '2002-10-27 03:23:58.899038': -0.6360002575424104,
 '2002-03-01 03:30:00.888006': -17.11948071851458,
 '2002-03-17 03:29:49.649069': -0.15642265837182825,
 '2002-06-05 03:28:16.643013': 6.773090551888348,
 '2002-07-23 03:27:07.279075': 20.31588871903536,
 '2002-08-08 03:26:38.960019': 13.470980406962004,
 '2002-08-24 03:26:07.925088': 12.49175244061768,
 '2002-09-09 03:25:48.080000': 10.795658222102011,
 '2002-09-25 03:25:18.221069': 9.007913456649563,
 '2002-10-11 03:24:48.059013': None,
 '2002-10-27 03:24:22.872038': 2.5101585128879256,
 '2002-01-28 03:31:04.335006': -19.531160682933173,
 '2002-02-13 03:30:43.920063': -10.45415681513849,
 '2002-03-01 03:30:24.617056': -13.482692575930072,
 '2002-03-17 03:30:13.382006': -17.29030107558279,
 '2002-06-05 03:28:40.330069': 5.745718109441301,
 '2002-07-23 03:27:30.967031': 19.78395652568866,
 '2002-08-08 03:27:02.647081': 14.379104475945027,
 '2002-08-24 03:26:31.613044': None,
 '2002-09-09 03:26:11.767056': 8.671389215334642,
 '2002-09-25 03:25:41.909025': 4.691266852926073,
 '2002-10-27 03:24:46.702063': 1.8214757368865164,
 '2002-11-28 03:23:45.573044': -1.662986354946076,
 '2002-10-18 03:30:46.008088': 7.743980624574841,
 '2002-01-19 03:37:20.420019': -16.71871969748772,
 '2002-02-20 03:36:43.433025': -9.682168943071114,
 '2002-03-08 03:36:27.241000': -5.794690697996227,
 '2002-03-24 03:36:20.626031': 46.28162881803846,
 '2002-06-28 03:34:14.603038': 13.201274472529807,
 '2002-07-30 03:33:30.637056': 9.562472818442838,
 '2002-08-15 03:33:01.299038': 14.975222035269189,
 '2002-10-18 03:31:09.696050': 5.272200581847812,
 '2002-11-03 03:30:45.990094': -0.03802192668411407,
 '2002-01-26 03:43:26.759081': -11.529828261434826,
 '2002-02-11 03:43:08.110000': -31.67953275572245,
 '2002-02-27 03:42:47.761056': -12.7690779584477,
 '2002-03-15 03:42:34.901019': -61.16036820024246,
 '2002-05-18 03:41:21.208081': -1.908133199695062,
 '2002-07-21 03:39:55.148031': 18.845399337662,
 '2002-08-06 03:39:26.899063': 14.10304651756917,
 '2002-08-22 03:38:57.954075': 20.322284188546973,
 '2002-09-07 03:38:34.594000': 14.561725531823265,
 '2002-09-23 03:38:09.131006': 12.540773325478535,
 '2002-10-25 03:37:10.551013': 6.123959333143348,
'2003-03-14 03:27:19.286434800': -15.442665253181767,
 '2003-07-20 03:27:07.637945100': 16.858488090297726,
 '2003-03-14 03:27:43.166726100': -16.61873526429851,
 '2003-07-20 03:27:31.515978500': 10.482588168864176,
 '2003-08-05 03:27:22.188870700': 13.131949813348031,
 '2003-03-14 03:28:07.047662': -14.666241903351576,
 '2003-05-17 03:28:04.471081400': -1.0832528152075225,
 '2003-07-20 03:27:55.394454': 10.6473890159073,
 '2003-08-05 03:27:46.064191600': 12.955155389096168,
 '2003-11-25 03:28:03.140793500': -1.7281019932999655,
 '2003-03-21 03:33:26.639433100': -7.785177779699129,
 '2003-05-08 03:33:31.030515400': -6.198496751732424,
 '2003-05-24 03:33:26.795672600': -0.4851155185888166,
 '2003-08-28 03:33:07.962838800': 11.312086249914657,
 '2003-03-21 03:33:50.518227600': -7.005167664261931,
 '2003-05-08 03:33:54.906060300': -1.506297718953152,
 '2003-05-24 03:33:50.674279400': -0.13803627967914717,
 '2003-08-28 03:33:31.837757600': 12.326806782592072,
 '2003-03-21 03:34:14.397374100': -4.876496407637224,
 '2003-04-22 03:34:17.199693200': -0.5673097438757895,
 '2003-05-08 03:34:18.782144400': -0.6210488443069752,
 '2003-05-24 03:34:14.553441800': 1.5990081040256008,
 '2003-08-12 03:33:57.858589500': 9.34404981091893,
 '2003-08-28 03:33:55.713067800': 12.186291615436003,
 '2003-03-21 03:34:38.277245500': -2.858578013408223,
 '2003-04-22 03:34:41.075148800': -0.45431072192455246,
 '2003-05-08 03:34:42.658996200': 1.11924245972864,
 '2003-08-12 03:34:21.734757400': 6.641632174056388,
 '2003-08-28 03:34:19.589146100': 11.992587237752788,
 '2003-04-13 03:40:06.226694600': -9.709282708031099,
 '2003-05-15 03:40:03.192266500': -13.371293670583695,
 '2003-05-31 03:40:02.468662': 3.6284490686489463,
 '2003-08-03 03:39:45.365629700': 11.529998966564067,
 '2003-04-13 03:40:30.102222': -11.473883500291336,
 '2003-05-15 03:40:27.069271200': None,
 '2003-05-31 03:40:26.349061300': 3.93354927433195,
 '2003-07-18 03:40:17.635877200': None,
 '2003-08-03 03:40:09.241542600': 12.725062105559633,
 '2003-04-13 03:40:53.978332400': None,
 '2003-04-29 03:40:54.109324500': -5.273004888523212,
 '2003-05-31 03:40:50.230057500': 4.6894223260463415,
 '2003-08-03 03:40:33.117616400': 12.211245478295254,
 '2003-03-19 03:46:37.281894800': -6.406092217160331,
 '2003-03-03 03:47:01.545644': -14.46571359739731,
 '2003-03-19 03:47:01.162197400': -7.489448880222474,
 '2003-05-06 03:47:04.728500400': -0.9791160749839567,
 '2003-08-10 03:46:43.049082500': 7.096833453968531,
 '2003-10-29 03:46:43.695723800': 5.527847538214254,
 '2003-02-22 03:53:09.533254100': -21.78481689955569,
 '2003-04-11 03:53:15.697306800': -2.0913645359744018,
 '2003-05-13 03:53:13.418558100': 1.67645144644146,
 '2003-05-29 03:53:11.865366300': 4.154511440278501,
 '2003-09-18 03:52:43.227573500': 6.788081261525061,
 '2003-02-02 03:11:21.140094': -20.995369324834524,
 '2003-02-18 03:11:41.693038': -15.595976391710337,
 '2003-03-06 03:12:09.605075': -15.23134384647264,
 '2003-03-22 03:12:28.792069': -9.237897125265274,
 '2003-04-07 03:12:57.445056': -2.092678076512508,
 '2003-05-09 03:13:41.537063': -4.4740927025579245,
 '2003-06-10 03:14:15.796019': 5.869619544489944,
 '2003-06-26 03:14:38.507025': 5.040423127716443,
 '2003-07-12 03:14:55.800063': 13.959963303469024,
 '2003-08-13 03:15:29.975006': 12.841958301621723,
 '2003-08-29 03:15:48.420063': None,
 '2003-09-14 03:16:04.809038': 7.960410884511429,
 '2003-10-16 03:16:28.332044': None,
 '2003-11-01 03:16:36.198056': -0.6951853766980659,
 '2003-02-02 03:11:44.971019': -20.441279242021416,
 '2003-02-18 03:12:05.523063': -13.93329531476607,
 '2003-03-06 03:12:33.436000': -14.593099615008159,
 '2003-03-22 03:12:52.623000': -8.152725088119384,
 '2003-04-07 03:13:21.275081': -3.4071966000167753,
 '2003-05-09 03:14:05.367094': -0.7768640583954622,
 '2003-06-26 03:15:02.194088': 4.604805614689906,
 '2003-07-12 03:15:19.631006': 8.927474948945532,
 '2003-08-13 03:15:53.805050': 11.061372436444785,
 '2003-08-29 03:16:12.251000': 13.012503133452245,
 '2003-09-14 03:16:28.497006': 8.746309987401936,
 '2003-10-16 03:16:52.162088': -3.178409109038552,
 '2003-11-01 03:16:59.886025': -0.7337132300825433,
 '2003-11-17 03:17:17.730075': -3.54575225757948,
 '2003-02-02 03:12:08.944019': -17.88261752447165,
 '2003-02-18 03:12:29.639031': -12.304693639312479,
 '2003-03-06 03:12:57.551075': -12.680939094541495,
 '2003-03-22 03:13:16.596000': -5.727780236816308,
 '2003-04-07 03:13:45.391050': -3.289215763049206,
 '2003-05-09 03:14:29.483063': -0.0343739693771514,
 '2003-06-10 03:15:03.599050': 5.28930303620488,
 '2003-06-26 03:15:26.310063': 6.328944318475565,
 '2003-07-12 03:15:43.746094': 10.582304546068778,
 '2003-08-13 03:16:17.778056': 11.504888014371405,
 '2003-08-29 03:16:36.224013': 13.00083393604915,
 '2003-09-14 03:16:52.612088': 8.984914354279693,
 '2003-10-16 03:17:16.278069': 3.121412338732065,
 '2003-11-17 03:17:41.846056': -0.8300220774824993,
 '2003-02-09 03:17:41.488075': -29.020476764324187,
 '2003-02-25 03:18:03.345050': -17.98235498251761,
 '2003-06-01 03:20:16.900075': 1.4066323925921989,
 '2003-06-17 03:20:36.140056': 7.013333067664035,
 '2003-07-19 03:21:13.709000': 12.42289349657146,
 '2003-08-04 03:21:32.187038': 14.382138902034693,
 '2003-08-20 03:21:49.070069': 13.226146535952658,
 '2003-09-05 03:22:05.061063': 12.753512911700765,
 '2003-09-21 03:22:21.659094': 7.343705871048893,
 '2003-02-09 03:18:05.319006': -30.919943234664263,
 '2003-02-25 03:18:27.175081': -17.44869816964053,
 '2003-03-13 03:18:53.342013': -13.729061644052317,
 '2003-04-14 03:19:43.464094': -7.2070436430578075,
 '2003-06-01 03:20:40.731006': 2.0834926798842477,
 '2003-06-17 03:20:59.970088': 6.425748380683476,
 '2003-07-19 03:21:37.682019': 8.246187854312316,
 '2003-08-04 03:21:56.160050': 13.185052157902206,
 '2003-08-20 03:22:12.901013': 13.172012868520003,
 '2003-09-05 03:22:29.034075': 12.902721436007765,
 '2003-09-21 03:22:45.490031': 5.532580372658298,
 '2003-10-07 03:22:56.868031': None,
 '2003-11-08 03:23:14.453000': -12.129866124756836,
 '2003-01-24 03:18:06.592063': -19.11708977164062,
 '2003-02-25 03:18:51.148075': -13.120934400640495,
 '2003-03-13 03:19:17.315013': -12.118257140325122,
 '2003-03-29 03:19:41.082019': -5.519375242646466,
 '2003-04-30 03:20:29.760094': -7.138699153779128,
 '2003-06-01 03:21:04.704006': 3.6431518295005754,
 '2003-06-17 03:21:23.943094': 7.4328611324355185,
 '2003-07-03 03:21:45.728069': 5.7847384966000535,
 '2003-07-19 03:22:01.655031': 9.929305648517834,
 '2003-08-04 03:22:20.133063': 13.28083735770077,
 '2003-08-20 03:22:37.016094': 11.425533252166735,
 '2003-09-05 03:22:53.007081': None,
 '2003-09-21 03:23:09.463044': 7.272229517967478,
 '2003-10-07 03:23:20.984013': 2.881216524270549,
 '2003-11-08 03:23:38.568069': -9.70611313659805,
 '2003-11-24 03:24:00.440075': -3.311743916364741,
 '2003-01-24 03:18:30.280025': None,
 '2003-02-09 03:18:52.979063': None,
 '2003-02-25 03:19:14.979006': -12.788443061860328,
 '2003-03-13 03:19:41.145044': -11.290601548576118,
 '2003-03-29 03:20:04.769075': -7.731087860949124,
 '2003-04-30 03:20:53.448050': -1.8212012603357752,
 '2003-06-01 03:21:28.391063': 3.6764483370520913,
 '2003-06-17 03:21:47.631056': 16.06917757732108,
 '2003-07-03 03:22:09.416044': None,
 '2003-07-19 03:22:25.485081': 15.267335446086053,
 '2003-08-04 03:22:43.964006': 14.053078617216727,
 '2003-08-20 03:23:00.704063': 2.0611162251350934,
 '2003-09-05 03:23:16.695056': None,
 '2003-09-21 03:23:33.151013': 6.469082634079408,
 '2003-10-07 03:23:44.671081': 2.7953829504825953,
 '2003-11-08 03:24:02.256031': -11.40625691080433,
 '2003-11-24 03:24:24.128044': 0.06766908090860847,
 '2003-01-31 03:24:03.907050': -18.911725789847605,
 '2003-02-16 03:24:23.965006': -11.293354855108026,
 '2003-03-04 03:24:51.621013': -14.123938145706695,
 '2003-04-05 03:25:39.691069': -13.611462564529964,
 '2003-04-21 03:26:03.643019': -0.8563800996516311,
 '2003-05-23 03:26:43.844075': -13.170232760693228,
 '2003-06-24 03:27:21.158056': None,
 '2003-07-10 03:27:39.550088': 1.1602460107924597,
 '2003-08-11 03:28:13.286019': 5.501710033726165,
 '2003-08-27 03:28:31.900044': 12.814814713961965,
 '2003-09-12 03:28:48.250081': 6.022773007349873,
 '2003-09-28 03:29:00.501025': -9.29208826098357,
 '2003-11-15 03:29:36.422013': -3.7304989920166176,
 '2003-01-31 03:24:28.023019': -18.63505286128395,
 '2003-02-16 03:24:47.938000': -13.037237837840694,
 '2003-03-04 03:25:15.736081': -13.253994769215533,
 '2003-03-20 03:25:34.652081': -10.818535323652235,
 '2003-04-05 03:26:03.807031': None,
 '2003-04-21 03:26:27.758081': -0.16752561565479296,
 '2003-05-23 03:27:07.817075': 0.9017700259084228,
 '2003-06-24 03:27:45.131063': None,
 '2003-07-10 03:28:03.523094': 8.68259898194417,
 '2003-07-26 03:28:20.635088': None,
 '2003-08-11 03:28:37.401094': 10.42184821868305,
 '2003-08-27 03:28:55.873050': 12.07486654720317,
 '2003-09-12 03:29:12.366056': 9.007982728373483,
 '2003-09-28 03:29:24.617000': None,
 '2003-11-15 03:30:00.537088': 1.8805728378593276,
 '2003-01-31 03:24:51.710075': -18.139109509331323,
 '2003-02-16 03:25:11.625063': -12.650545273745195,
 '2003-03-04 03:25:39.424038': -12.749528355801996,
 '2003-03-20 03:25:58.340038': -8.066649947397595,
 '2003-04-05 03:26:27.494088': -7.556771920470572,
 '2003-04-21 03:26:51.446038': -3.6544667484724966,
 '2003-05-23 03:27:31.648006': -0.1238733892114647,
 '2003-07-10 03:28:27.211063': 12.50621210743577,
 '2003-08-11 03:29:01.089056': None,
 '2003-08-27 03:29:19.561019': 10.342938815649902,
 '2003-09-12 03:29:36.054019': None,
 '2003-09-28 03:29:48.304069': 6.200568422140572,
 '2003-11-15 03:30:24.225056': 3.1079007269920775,
 '2003-12-01 03:30:38.662088': -5.250424848885472,
 '2003-02-07 03:30:46.956081': -22.32734577905654,
 '2003-02-23 03:31:09.910013': -31.68300387097341,
 '2003-03-11 03:31:36.889081': -22.135779927099403,
 '2003-03-27 03:31:58.981069': -3.9549842028819415,
 '2003-04-12 03:32:26.072050': -5.478975174490257,
 '2003-04-28 03:32:48.386088': -1.2911322337423745,
 '2003-05-14 03:33:08.007063': None,
 '2003-05-30 03:33:24.744044': 4.2843189482490915,
 '2003-06-15 03:33:42.998031': 8.783127428208338,
 '2003-07-01 03:34:05.064081': 7.429471786257765,
 '2003-07-17 03:34:21.588094': 7.62286314984111,
 '2003-08-02 03:34:39.769025': 10.014588657369107,
 '2003-08-18 03:34:55.907038': None,
 '2003-09-03 03:35:13.202019': 12.061998683908916,
 '2003-09-19 03:35:29.633025': 10.56948755182089,
 '2003-10-05 03:35:40.979056': 2.4108991370297277,
 '2003-10-21 03:35:53.743019': 4.781463255040991,
 '2003-11-22 03:36:20.857013': -15.328869805346104,
 '2003-01-22 03:30:49.992031': None,
 '2003-02-07 03:31:10.644038': -24.7434309453112,
 '2003-02-23 03:31:33.597069': -16.05455553754095,
 '2003-03-11 03:32:00.577038': -16.1799447432161,
 '2003-03-27 03:32:22.811094': -2.3568950220536014,
 '2003-04-12 03:32:49.902075': -12.009201636833678,
 '2003-04-28 03:33:12.074044': -1.2967718722866068,
 '2003-05-30 03:33:48.432006': 4.713836088106703,
 '2003-06-15 03:34:06.685094': 7.873220874590685,
 '2003-07-01 03:34:28.895019': 9.01493344912153,
 '2003-08-18 03:35:19.595000': -4.456104144027027,
 '2003-09-03 03:35:36.889088': 11.458792286540199,
 '2003-09-19 03:35:53.320094': 9.446658831582472,
 '2003-10-21 03:36:17.430088': 5.417507912109084,
 '2003-01-29 03:37:10.455050': -16.930884033104512,
 '2003-02-14 03:37:29.811088': -9.264323717908638,
 '2003-03-02 03:37:57.414088': -12.552550175410873,
 '2003-03-18 03:38:17.966038': -6.920157352980452,
 '2003-04-03 03:38:44.931075': -12.269724995265781,
 '2003-04-19 03:39:10.682056': -0.04252799526291074,
 '2003-05-05 03:39:32.149019': -0.9428677254635324,
 '2003-06-06 03:40:04.783094': 5.347534529447404,
 '2003-07-08 03:40:47.073044': 10.638544886156064,
 '2003-07-24 03:41:03.547000': -24.203731600886968,
 '2003-08-09 03:41:21.061006': None,
 '2003-08-25 03:41:39.031019': 4.9545465527137065,
 '2003-09-10 03:41:55.562044': 6.325338960365717,
 '2003-09-26 03:42:08.929025': -2.8275448796480065,
 '2003-10-12 03:42:20.870050': 1.9412221878996483,
'2004-04-01 03:27:51.158456800': -11.656479121326495,
 '2004-07-06 03:27:35.221058900': 6.897005283241395,
 '2004-08-07 03:27:23.175106400': 12.185318819976288,
 '2004-09-08 03:27:16.239866400': 10.56582936879368,
 '2004-07-06 03:27:59.099178500': 2.50944039291763,
 '2004-08-07 03:27:47.051836200': 11.170820408498548,
 '2004-09-08 03:27:40.118294800': 10.801099455279397,
 '2004-02-29 03:28:30.822735600': None,
 '2004-05-19 03:28:33.830274500': 0.3915988492195676,
 '2004-06-04 03:28:29.452381400': -0.1650992864937856,
 '2004-07-06 03:28:22.977797200': 3.0076754688414287,
 '2004-07-22 03:28:18.598377400': 8.454913552995086,
 '2004-08-07 03:28:10.929062100': 12.094904341954534,
 '2004-09-08 03:28:03.997569300': 11.85383420026834,
 '2004-09-24 03:27:59.123315800': 8.798644775164393,
 '2004-04-24 03:33:53.569887300': -1.6865865801752538,
 '2004-05-10 03:33:57.078649600': -0.6661513826317926,
 '2004-05-26 03:33:55.551454600': -2.443443796286133,
 '2004-06-27 03:33:46.999279300': 4.289521391321255,
 '2004-07-13 03:33:44.650756100': 9.807614754089371,
 '2004-08-14 03:33:34.017513': 9.998846640187082,
 '2004-08-30 03:33:30.748068600': 10.711480742305605,
 '2004-10-17 03:33:24.227161500': None,
 '2004-11-02 03:33:38.124097700': 2.90971319250634,
 '2004-04-24 03:34:17.447017100': -1.7699981254869206,
 '2004-05-10 03:34:20.956832': -0.3558994303594474,
 '2004-05-26 03:34:19.430667800': -1.8481389458172377,
 '2004-06-11 03:34:13.015879400': -2.7320181226683147,
 '2004-06-27 03:34:10.878063400': 4.786868107235421,
 '2004-07-13 03:34:08.528507800': 8.766349125709658,
 '2004-07-29 03:34:01.581489900': 6.054861144928345,
 '2004-08-14 03:33:57.894412300': 10.492846693375657,
 '2004-08-30 03:33:54.626007': 9.174965911263255,
 '2004-10-17 03:33:48.106451900': -0.8010765161990616,
 '2004-04-24 03:34:41.324865300': -1.572036781752525,
 '2004-05-10 03:34:44.835620100': -2.885372217898238,
 '2004-05-26 03:34:43.310266600': -0.7774894622945825,
 '2004-06-11 03:34:36.895827300': 0.2122830213466288,
 '2004-06-27 03:34:34.757500400': 5.600173124649577,
 '2004-07-13 03:34:32.406614900': 11.630901821378558,
 '2004-07-29 03:34:25.458529600': 6.250365251730834,
 '2004-08-14 03:34:21.772145900': 10.48692096638538,
 '2004-08-30 03:34:18.504777500': None,
 '2004-10-17 03:34:11.986046600': None,
 '2004-03-23 03:35:14.296832600': -7.549430140482766,
 '2004-04-24 03:35:05.203260700': -4.449651429172364,
 '2004-05-10 03:35:08.715046400': -2.421412002386576,
 '2004-05-26 03:35:07.190727700': 0.8587522022405556,
 '2004-06-11 03:35:00.776207600': 3.6221533195685924,
 '2004-06-27 03:34:58.637491700': 7.136868995722893,
 '2004-07-13 03:34:56.285497': 13.6778220338945,
 '2004-07-29 03:34:49.336234200': None,
 '2004-08-14 03:34:45.650267600': 7.814004076231379,
 '2004-05-01 03:40:30.423020800': None,
 '2004-06-02 03:40:28.007716900': 2.3178179495070643,
 '2004-06-18 03:40:21.505007900': 3.0586880491792314,
 '2004-08-21 03:40:07.854165300': 14.353996120930457,
 '2004-09-22 03:39:57.291740': 10.141865297079383,
 '2004-03-30 03:41:00.934187800': -12.223898463040031,
 '2004-04-15 03:40:54.416820100': -2.510900302712919,
 '2004-06-02 03:40:51.887544500': 3.1212017474233926,
 '2004-06-18 03:40:45.384797500': 3.9304367459620857,
 '2004-08-21 03:40:31.731962500': 12.363499453527242,
 '2004-09-06 03:40:26.555785200': 10.067155760860487,
 '2004-09-22 03:40:21.171944300': 9.8975883900948,
 '2004-02-11 03:41:15.075665700': -13.479739973816145,
 '2004-03-30 03:41:24.812224800': -11.33677219599489,
 '2004-06-02 03:41:15.768049700': 3.033623273550453,
 '2004-06-18 03:41:09.265216400': 5.125994852803297,
 '2004-08-21 03:40:55.610668500': 10.309771543287141,
 '2004-09-06 03:40:50.435589400': 10.701979745882452,
 '2004-09-22 03:40:45.052630': 7.746940775161354,
 '2004-11-25 03:41:11.543137500': 1.8669935891207001,
 '2004-03-05 03:47:06.182426200': -14.791126518249317,
 '2004-04-22 03:47:02.089404200': None,
 '2004-10-31 03:46:46.261369500': 5.143979974212109,
 '2004-11-16 03:46:51.551941800': 1.5685285784539007,
 '2004-03-05 03:47:30.062761800': -14.133956623973695,
 '2004-03-21 03:47:35.720243500': -9.346493695846023,
 '2004-04-06 03:47:33.745965800': -12.524296599824465,
 '2004-04-22 03:47:25.967734500': -2.1111161735730746,
 '2004-05-08 03:47:30.198944600': -0.7969422878782085,
 '2004-05-24 03:47:29.304755500': 2.0907054507105785,
 '2004-07-11 03:47:18.352582800': 12.685705971839127,
 '2004-08-12 03:47:07.358427200': 13.597098606965586,
 '2004-09-13 03:46:57.694357600': 11.47735703859125,
 '2004-10-15 03:46:55.328746600': 6.775342605092478,
 '2004-10-31 03:47:10.140381900': 3.9334319146200456,
 '2004-11-16 03:47:15.429638300': 2.599242364250393,
 '2004-03-12 03:53:44.154435': -14.982849523867934,
 '2004-04-29 03:53:39.306168200': -1.402436407153983,
 '2004-05-31 03:53:38.116024200': 5.58852132186306,
 '2004-06-16 03:53:30.686084700': 7.278881368188113,
 '2004-07-18 03:53:27.295509500': 10.554194437638124,
 '2004-08-03 03:53:18.243948800': 13.360086911430274,
 '2004-08-19 03:53:17.591077400': 16.176750438323886,
 '2004-09-04 03:53:12.915298300': 3.385880178812315,
 '2004-09-20 03:53:07.037236500': 8.441181855670491,
 '2004-11-07 03:53:24.900068100': -8.71532557312437,
 '2004-02-05 03:17:26.502063': -22.65076838826188,
 '2004-02-21 03:17:32.101006': -21.446752085757684,
 '2004-04-09 03:18:05.401031': -4.745568447084839,
 '2004-05-11 03:19:01.329013': None,
 '2004-06-12 03:19:55.452056': -4.018579196927801,
 '2004-06-28 03:20:24.071006': 5.536146592093734,
 '2004-07-14 03:20:51.785044': 8.930111611588305,
 '2004-07-30 03:21:14.024075': None,
 '2004-08-15 03:21:35.903038': 12.385808965673128,
 '2004-08-31 03:22:01.713006': 10.992937563426995,
 '2004-10-18 03:23:07.751013': 5.065251271084211,
 '2004-02-05 03:17:50.333000': -18.886083214254956,
 '2004-02-21 03:17:55.931050': -19.506496776918414,
 '2004-04-09 03:18:29.231069': -1.0291642772622729,
 '2004-05-11 03:19:25.159050': -4.574433333678207,
 '2004-05-27 03:19:51.680044': 0.24099535458368684,
 '2004-06-12 03:20:19.140025': -1.5336569398781001,
 '2004-06-28 03:20:47.901050': 4.4279612654079985,
 '2004-07-14 03:21:15.473013': 3.099002160537189,
 '2004-07-30 03:21:37.855013': 7.767098455318125,
 '2004-08-15 03:21:59.591006': 12.282905473032127,
 '2004-08-31 03:22:25.543050': 10.150868919600207,
 '2004-10-02 03:23:10.936075': None,
 '2004-10-18 03:23:31.581050': 6.20719564901666,
 '2004-11-03 03:23:51.238025': None,
 '2004-11-19 03:24:04.697050': 2.345625596913023,
 '2004-02-05 03:18:14.448081': -18.313984820460785,
 '2004-02-21 03:18:19.904056': -16.165785984249244,
 '2004-04-09 03:18:53.347050': -1.5868884424377936,
 '2004-05-11 03:19:49.275031': -0.6633383531034892,
 '2004-05-27 03:20:15.653056': 0.6778508820298446,
 '2004-06-12 03:20:43.256006': -2.016900398082234,
 '2004-06-28 03:21:12.017025': 5.555244458734992,
 '2004-07-14 03:21:39.588094': 9.223768802703601,
 '2004-07-30 03:22:01.828025': 9.153876677654491,
 '2004-08-15 03:22:23.706088': 13.523381154550218,
 '2004-08-31 03:22:49.659025': 12.241709765818232,
 '2004-09-16 03:23:14.525056': None,
 '2004-10-18 03:23:55.697031': 6.721119231163768,
 '2004-11-19 03:24:28.670056': -0.10111475290122603,
 '2004-02-12 03:23:39.234006': -18.17204459201423,
 '2004-02-28 03:23:44.921038': -27.495749006353265,
 '2004-03-31 03:23:57.975031': -11.128763940441695,
 '2004-05-18 03:25:24.035056': -3.502649564195302,
 '2004-07-05 03:26:47.324044': 8.700268401460736,
 '2004-08-22 03:27:58.578044': 13.915504027571174,
 '2004-09-07 03:28:23.855038': 10.790816689073546,
 '2004-10-25 03:29:27.051044': -4.457812090550448,
 '2004-01-27 03:24:00.041088': None,
 '2004-02-12 03:24:03.064044': -13.384062432291627,
 '2004-02-28 03:24:08.751075': -20.03776208680661,
 '2004-03-15 03:24:14.775025': -16.824376940392273,
 '2004-03-31 03:24:21.805069': -17.91928189206557,
 '2004-05-02 03:25:20.147019': -9.470253053610627,
 '2004-05-18 03:25:47.865094': -0.6842302928205238,
 '2004-06-03 03:26:14.758031': None,
 '2004-07-05 03:27:11.154088': 6.270903601156581,
 '2004-07-21 03:27:37.162063': None,
 '2004-08-22 03:28:22.408081': 12.64462608493694,
 '2004-09-07 03:28:47.685075': 10.157561901443849,
 '2004-10-25 03:29:50.881081': 0.9355609033117999,
 '2004-01-27 03:24:24.157063': -20.684755044885772,
 '2004-02-12 03:24:27.180019': -12.437549505365576,
 '2004-02-28 03:24:32.724081': -18.924703234767488,
 '2004-03-15 03:24:38.891000': -17.06845859290104,
 '2004-03-31 03:24:45.921044': -16.305832455949638,
 '2004-05-18 03:26:11.839000': 1.179965936836467,
 '2004-06-03 03:26:38.874013': 0.5244615482519324,
 '2004-06-19 03:27:05.670050': 11.254201286003884,
 '2004-07-05 03:27:35.270063': 7.590032524205498,
 '2004-08-06 03:28:20.449044': 10.887801510294603,
 '2004-08-22 03:28:46.524056': 10.634964876512358,
 '2004-09-07 03:29:11.658081': 10.693371450032508,
 '2004-09-23 03:29:34.724063': 7.01990858342388,
 '2004-10-25 03:30:14.854088': 3.609993621278667,
 '2004-11-26 03:30:51.144025': 1.2440976190349813,
 '2004-01-27 03:24:47.845025': -13.554214511248611,
 '2004-02-12 03:24:50.867081': -14.689173665177423,
 '2004-02-28 03:24:56.555019': -19.703624999453915,
 '2004-03-15 03:25:02.578069': -20.335807147449085,
 '2004-03-31 03:25:09.609006': -14.86333533882901,
 '2004-05-02 03:26:07.807094': -4.340146184271074,
 '2004-05-18 03:26:35.526069': -2.9993564069344956,
 '2004-06-03 03:27:02.561081': -0.033789634328349326,
 '2004-06-19 03:27:29.500088': 3.6279628449664254,
 '2004-07-05 03:27:59.101000': 10.631835563367247,
 '2004-08-06 03:28:44.137013': 12.701336433939467,
 '2004-08-22 03:29:10.212025': 8.168876762673772,
 '2004-09-07 03:29:35.346044': 11.913342673499908,
 '2004-09-23 03:29:58.412031': None,
 '2004-10-25 03:30:38.685025': 3.7695727974717217,
 '2004-11-26 03:31:14.974063': 2.5753926637666806,
 '2004-02-03 03:30:11.849013': -17.99834832662226,
 '2004-02-19 03:30:16.864044': -10.636293520100399,
 '2004-03-06 03:30:21.791081': -14.792664298514532,
 '2004-03-22 03:30:27.294050': -6.057113213699133,
 '2004-04-07 03:30:46.859000': None,
 '2004-04-23 03:31:16.190094': -0.6001868849086064,
 '2004-05-09 03:31:43.318094': None,
 '2004-05-25 03:32:09.400000': 2.4798705573551123,
 '2004-06-10 03:32:37.573019': 0.4166043488061763,
 '2004-06-26 03:33:05.943025': 5.685534294254355,
 '2004-07-12 03:33:33.906000': 5.142333390497217,
 '2004-07-28 03:33:57.194088': -17.723978769943518,
 '2004-08-13 03:34:17.645094': 9.39537482938319,
 '2004-10-16 03:35:50.622063': 7.36721713480348,
 '2004-11-01 03:36:10.578050': None,
 '2004-11-17 03:36:25.774000': 3.085067058340604,
 '2004-02-03 03:30:35.822019': -17.653026766497703,
 '2004-02-19 03:30:40.837044': -11.564895268330611,
 '2004-03-06 03:30:45.764088': -15.335659911763338,
 '2004-03-22 03:30:51.410025': -6.361558998648552,
 '2004-04-07 03:31:10.974075': -5.895043647092895,
 '2004-04-23 03:31:40.164000': -2.095555906089938,
 '2004-05-09 03:32:07.292000': 0.5090626875534646,
 '2004-05-25 03:32:33.373006': 3.2401357159754838,
 '2004-06-10 03:33:01.689000': 3.4061450299458276,
 '2004-06-26 03:33:29.916031': 8.609927584571073,
 '2004-07-12 03:33:58.021075': 10.661805785774739,
 '2004-07-28 03:34:21.167094': 9.73104562090614,
 '2004-08-13 03:34:41.761069': 12.521718731983642,
 '2004-09-30 03:35:54.366075': 5.965023586569517,
 '2004-10-16 03:36:14.738038': 7.536000963479605,
 '2004-11-01 03:36:34.694031': 3.311226759410644,
 '2004-11-17 03:36:49.747006': 2.6228668296890936,
 '2004-02-03 03:30:59.509081': -21.41063144497404,
 '2004-02-19 03:31:04.525013': -9.888212034180222,
 '2004-03-06 03:31:09.595025': -15.287248410328084,
 '2004-03-22 03:31:15.097088': -5.124769872886592,
 '2004-04-07 03:31:34.662044': -9.22153830339475,
 '2004-04-23 03:32:03.851069': -5.016326182906403,
 '2004-05-09 03:32:30.979069': 0.237966920028202,
 '2004-05-25 03:32:57.060075': 2.1751374328954274,
 '2004-06-10 03:33:25.376063': 2.457952958142073,
 '2004-06-26 03:33:53.746069': 6.8290762579161095,
 '2004-07-12 03:34:21.709044': 12.950921577825211,
 '2004-07-28 03:34:44.998031': 11.062066915952864,
 '2004-08-13 03:35:05.449038': 13.140413781194034,
 '2004-09-14 03:35:56.930075': 7.3811755284532135,
 '2004-09-30 03:36:18.054044': None,
 '2004-10-16 03:36:38.426006': 7.276128001610248,
 '2004-11-01 03:36:58.381094': -1.7684846887606736,
 '2004-11-17 03:37:13.577038': 3.0772023215398727,
 '2004-12-03 03:37:36.030050': -6.47007295489749,
 '2004-01-25 03:36:45.298013': -14.114901386656,
 '2004-02-10 03:36:48.259044': -8.221699456339685,
 '2004-03-13 03:36:59.709000': -31.399829093567796,
 '2004-04-30 03:38:01.413000': -3.5504319053408766,
 '2004-06-17 03:39:23.676050': 6.105099911976134,
 '2004-07-03 03:39:53.361019': 5.839839895808721,
 '2004-07-19 03:40:19.892038': None,
 '2004-08-04 03:40:39.815031': -40.036213002043795,
 '2004-08-20 03:41:04.840044': 14.521066654176002,
 '2004-09-05 03:41:29.992056': 9.861266565459088,
 '2004-09-21 03:41:53.754013': -16.36205915161516,
 '2004-10-07 03:42:13.165019': 1.6432701702747026,
 '2004-10-23 03:42:34.330056': None,
 '2004-11-08 03:42:53.231044': 1.71559826252111,
 '2004-11-24 03:43:09.511031': 1.1858749365855084,
 '2004-01-25 03:37:08.985075': -10.47180867122315,
 '2004-02-10 03:37:11.947006': -11.161420075392744,
 '2004-03-13 03:37:23.396069': -15.028690184896519,
 '2004-04-30 03:38:25.100069': -2.478737615172422,
 '2004-06-17 03:39:47.364019': 6.681621005387442,
 '2004-07-03 03:40:17.191056': 8.659661501908978,
 '2004-08-20 03:41:28.528013': 14.173589178261887,
 '2004-09-05 03:41:53.680025': 9.741125446872797,
 '2004-09-21 03:42:17.441075': -16.14760152824509,
 '2004-10-07 03:42:36.995056': 4.872012678468712,
 '2004-11-24 03:43:33.341069': 2.040289038304129,
 '2004-01-16 03:43:15.991094': -0.32645193401360456,
 '2004-02-01 03:43:21.025025': -15.521977990744617,
 '2004-02-17 03:43:25.368069': None,
 '2004-03-04 03:43:29.961050': -24.95644960559797,
 '2004-03-20 03:43:36.472000': -2.297353576657096,
 '2004-04-05 03:43:52.266013': -5.797142344908833,
 '2004-05-07 03:44:49.011056': -0.488080597786955,
 '2004-06-08 03:45:43.680075': 8.152160796962187,
 '2004-06-24 03:46:11.493031': 12.964909736631089,
 '2004-07-10 03:46:40.094044': 1.5005937943347571,
 '2004-09-12 03:48:15.612081': 10.515687925061718,
 '2004-09-28 03:48:37.215019': None,
 '2004-10-14 03:48:57.456050': -13.183121116210405,
 '2004-10-30 03:49:17.488019': 3.9898429637580732,
 '2004-11-15 03:49:32.465088': 2.4396784498944513,
 '2004-12-01 03:49:54.944031': -2.5302654383139758,
'2005-04-20 03:28:06.268558800': -1.3304668250629135,
 '2005-05-22 03:28:10.965279900': -2.8861787547172715,
 '2005-07-09 03:28:00.420265800': 12.679888668476558,
 '2005-07-25 03:27:59.881960300': 18.302597015722768,
 '2005-08-26 03:27:50.112843300': 13.78132270951424,
 '2005-04-20 03:28:30.147128600': -2.105659878279431,
 '2005-05-22 03:28:34.844870500': -4.680623818428586,
 '2005-06-07 03:28:29.567818100': 1.376338241890925,
 '2005-07-09 03:28:24.296600500': 7.824046772649003,
 '2005-07-25 03:28:23.758645200': 14.178089258960181,
 '2005-08-26 03:28:13.991956400': 13.454096718041908,
 '2005-04-20 03:28:54.026469400': -2.430970138015603,
 '2005-05-22 03:28:58.725135': -7.0192769963306025,
 '2005-06-07 03:28:53.446685300': 1.6405610577065817,
 '2005-07-09 03:28:48.173404800': 9.96951024408592,
 '2005-07-25 03:28:47.635813200': 13.592302704375127,
 '2005-08-26 03:28:37.871933100': 13.195858310409507,
 '2005-09-11 03:28:28.012485300': 11.940373226442231,
 '2005-09-27 03:28:27.063150800': -16.42034951578762,
 '2005-10-29 03:28:41.468763100': 1.9860671756019643,
 '2005-03-10 03:34:27.959969300': -19.088056336526215,
 '2005-05-13 03:34:22.780102400': 0.2526027420748733,
 '2005-05-29 03:34:20.011951600': -3.4648475428021284,
 '2005-06-14 03:34:12.851667800': 5.197729451015226,
 '2005-06-30 03:34:09.740214100': None,
 '2005-07-16 03:34:11.355751': None,
 '2005-08-17 03:34:04.775935100': 10.313562552549373,
 '2005-09-02 03:33:57.043155100': 9.931183422937128,
 '2005-09-18 03:33:49.789701200': 6.3098691981327555,
 '2005-10-04 03:33:49.735873800': 1.2359548083455316,
 '2005-10-20 03:33:57.391006300': -2.313158203551415,
 '2005-11-05 03:34:09.408499400': -0.8884483518191535,
 '2005-03-10 03:34:51.836574400': -18.81064116800665,
 '2005-05-13 03:34:46.660189': 0.26406956103484597,
 '2005-05-29 03:34:43.891173500': 1.3056285810226493,
 '2005-06-14 03:34:36.729244200': 4.327570311688551,
 '2005-06-30 03:34:33.616980': None,
 '2005-08-17 03:34:28.654483': 11.766772654367534,
 '2005-09-02 03:34:20.922919700': 11.878934198577493,
 '2005-09-18 03:34:13.669741500': 4.662193710964591,
 '2005-10-04 03:34:13.614700900': -0.44940748499789995,
 '2005-10-20 03:34:21.270721300': 0.5869535313079729,
 '2005-11-05 03:34:33.286862700': -0.4254444339328279,
 '2005-03-10 03:35:15.713768800': -17.34533428366793,
 '2005-05-13 03:35:10.540811700': 0.0320467576074786,
 '2005-05-29 03:35:07.770983200': 0.7646204846891003,
 '2005-06-14 03:35:00.607279': 5.57280947834801,
 '2005-07-16 03:34:59.109030100': 9.190736715877923,
 '2005-08-01 03:34:57.369919500': 14.54429773467951,
 '2005-08-17 03:34:52.533778': 12.640864585608284,
 '2005-09-02 03:34:44.803016300': 13.07586636418807,
 '2005-09-18 03:34:37.550501200': None,
 '2005-10-04 03:34:37.494230100': 2.1174512850590097,
 '2005-10-20 03:34:45.150770900': 4.219425352933045,
 '2005-11-05 03:34:57.165528800': 2.28213503164653,
 '2005-05-29 03:35:31.651342400': -8.466309191198375,
 '2005-06-14 03:35:24.485924700': 6.977840663746413,
 '2005-07-16 03:35:22.986680200': 9.126038116029278,
 '2005-08-01 03:35:21.248378200': 14.020531967057694,
 '2005-08-17 03:35:16.413523600': 13.301697616080332,
 '2005-09-02 03:35:08.684051100': 13.312234442040522,
 '2005-10-04 03:35:01.374026600': 5.396444761034193,
 '2005-10-20 03:35:09.031593500': 5.997957162836207,
 '2005-11-05 03:35:21.044801700': 3.03654339381325,
 '2005-04-02 03:40:59.639634': -5.154947134345942,
 '2005-05-20 03:40:56.847007': -2.2538510330263524,
 '2005-07-23 03:40:45.610487900': 3.8400529590713597,
 '2005-08-24 03:40:36.606019400': None,
 '2005-09-25 03:40:24.824301300': 9.48546918010632,
 '2005-04-02 03:41:23.517012100': -3.1005014042357844,
 '2005-05-04 03:41:20.246982': -3.8837269849991602,
 '2005-05-20 03:41:20.727337800': 1.2996003204175723,
 '2005-07-23 03:41:09.487816200': 11.8807155803849,
 '2005-08-24 03:41:00.485625200': 13.294997308467801,
 '2005-09-09 03:40:51.047344500': 12.432127251072064,
 '2005-09-25 03:40:48.704439600': 7.161245519487183,
 '2005-03-17 03:41:50.488413900': -11.56750119404524,
 '2005-04-02 03:41:47.395124800': -4.623542953069699,
 '2005-05-04 03:41:44.128112300': -3.979495484953106,
 '2005-05-20 03:41:44.608286600': -5.102793883846596,
 '2005-07-23 03:41:33.365315': 11.907906092116786,
 '2005-08-24 03:41:24.366167100': 13.830145549300429,
 '2005-09-09 03:41:14.928180600': 12.211818104303495,
 '2005-09-25 03:41:12.585305': 7.318247250297659,
 '2005-04-09 03:47:31.831653800': -3.332240440450859,
 '2005-04-25 03:47:28.429796200': -5.428797801703903,
 '2005-11-03 03:47:17.503509800': 5.7992917788652125,
 '2005-04-09 03:47:55.710235200': -5.371606697410707,
 '2005-04-25 03:47:52.310220': -3.624185529845326,
 '2005-05-11 03:47:55.974318200': -4.049781581666179,
 '2005-07-14 03:47:44.670927400': 11.212387979469412,
 '2005-07-30 03:47:43.237629800': 17.422887527528644,
 '2005-08-15 03:47:38.770399500': 12.322637866648606,
 '2005-10-02 03:47:23.161688200': 9.629167507788706,
 '2005-10-18 03:47:28.942577100': -9.044198549477898,
 '2005-11-03 03:47:41.383145400': 5.702119565194012,
 '2005-11-19 03:47:50.208389100': -1.4142045076947947,
 '2005-01-26 03:53:53.344920600': -23.189295057075004,
 '2005-02-11 03:54:04.212527200': -19.823984420092273,
 '2005-04-16 03:54:03.116868300': -2.262889985767341,
 '2005-05-18 03:54:06.533606500': 2.530112979310135,
 '2005-06-19 03:53:55.850973700': 9.336590160895781,
 '2005-07-05 03:53:54.810542200': 10.226854618514157,
 '2005-07-21 03:53:55.176585600': None,
 '2005-08-22 03:53:46.863863': 6.373740891177917,
 '2005-09-23 03:53:34.191111200': 10.810859892136392,
 '2005-10-09 03:53:31.313015200': None,
 '2005-11-10 03:53:56.490347500': 3.132268306470953,
 '2005-02-07 03:24:59.262063': -28.222500141950057,
 '2005-02-23 03:25:16.052063': -24.372863161091,
 '2005-03-27 03:25:38.383038': -9.083790425845379,
 '2005-04-12 03:25:42.267050': -5.812838349462176,
 '2005-04-28 03:25:46.149081': -6.008435979689636,
 '2005-05-14 03:25:51.924081': -2.669344926229462,
 '2005-05-30 03:25:59.903056': 1.7636023393622398,
 '2005-06-15 03:26:07.251050': 6.515865052719756,
 '2005-07-01 03:26:11.679063': 6.6611896273438775,
 '2005-07-17 03:26:21.860000': 8.485468015360583,
 '2005-08-02 03:26:32.197006': 18.318606715954324,
 '2005-08-18 03:26:38.581006': 7.872768277032134,
 '2005-09-03 03:26:42.024069': 9.41130991401147,
 '2005-09-19 03:26:41.952013': 7.736740811250669,
 '2005-10-05 03:26:38.356019': 2.4648602112379225,
 '2005-10-21 03:26:37.249038': 0.5811706236215931,
 '2005-11-06 03:26:45.320069': -12.48210756139208,
 '2005-02-07 03:25:22.950031': -27.040924322682713,
 '2005-02-23 03:25:39.883000': -23.72616581774189,
 '2005-03-27 03:26:02.213081': -8.072517599797907,
 '2005-04-12 03:26:06.097094': -3.1289226455508246,
 '2005-04-28 03:26:09.837050': -9.262682200727344,
 '2005-05-14 03:26:15.755025': -9.35273905914926,
 '2005-05-30 03:26:23.591025': 1.5274078741923267,
 '2005-06-15 03:26:31.081088': 3.919352046333168,
 '2005-07-01 03:26:35.510000': 3.5803842813543265,
 '2005-07-17 03:26:45.690031': None,
 '2005-08-02 03:26:56.027038': 18.300417234379417,
 '2005-08-18 03:27:02.411038': 10.579492271113327,
 '2005-09-03 03:27:05.855000': 10.862936430602643,
 '2005-09-19 03:27:05.782050': 7.526766425548467,
 '2005-10-05 03:27:02.186056': 6.606834451614185,
 '2005-10-21 03:27:01.079075': 0.5404728304455855,
 '2005-11-06 03:27:09.151006': -9.890376526655382,
 '2005-11-22 03:27:33.537088': 0.696262100919561,
 '2005-02-07 03:25:47.066013': -24.98794548798393,
 '2005-02-23 03:26:03.998081': -22.288827142727566,
 '2005-03-27 03:26:26.329056': -5.691901860951351,
 '2005-04-28 03:26:33.953031': -6.234600584875612,
 '2005-05-14 03:26:39.871006': None,
 '2005-05-30 03:26:47.707006': 0.43136784113232024,
 '2005-06-15 03:26:55.197063': 4.949443060484218,
 '2005-07-01 03:26:59.625075': 4.318174626385905,
 '2005-07-17 03:27:09.806006': None,
 '2005-08-02 03:27:20.000044': 17.933081706642024,
 '2005-08-18 03:27:26.384044': 11.816738075127294,
 '2005-09-03 03:27:29.970075': 12.798158877582912,
 '2005-10-05 03:27:26.302031': 6.410455108520457,
 '2005-10-21 03:27:25.195050': 2.747684205171875,
 '2005-11-06 03:27:33.124013': -9.661499213906525,
 '2005-11-22 03:27:57.653056': 0.3566532575376086,
 '2005-02-14 03:31:17.407069': -22.254748625405917,
 '2005-03-02 03:31:33.150006': -17.81516410560228,
 '2005-04-03 03:31:51.412013': -2.4189348028411435,
 '2005-04-19 03:31:54.416081': -6.039242633226382,
 '2005-05-21 03:32:05.991094': -2.39477952586882,
 '2005-06-06 03:32:14.067069': -0.6015424752061113,
 '2005-07-08 03:32:25.188088': 12.21530857020887,
 '2005-09-10 03:32:53.173075': 9.358142690802698,
 '2005-09-26 03:32:51.483063': 7.708083213175769,
 '2005-10-12 03:32:47.123094': -4.329730347619205,
 '2005-10-28 03:32:46.540056': 3.193134760882781,
 '2005-11-13 03:33:07.189025': -3.390540831333872,
 '2005-02-14 03:31:41.238000': -21.940309268723112,
 '2005-03-18 03:32:08.898069': -10.81378732166926,
 '2005-04-03 03:32:15.242050': -2.279712013171296,
 '2005-04-19 03:32:18.247019': -3.8913460841847582,
 '2005-05-21 03:32:29.822038': -3.1338812065192037,
 '2005-06-06 03:32:37.898006': 1.43021849299109,
 '2005-07-08 03:32:49.019025': 6.391023026871987,
 '2005-07-24 03:33:01.533006': 7.66033214685736,
 '2005-09-10 03:33:17.004006': 9.469375516557104,
 '2005-09-26 03:33:15.313094': 6.7102837045357955,
 '2005-10-12 03:33:10.954025': None,
 '2005-10-28 03:33:10.370081': 4.249003895793494,
 '2005-11-13 03:33:31.019050': -3.7150026800727707,
 '2005-02-14 03:32:05.353075': -20.3546711329683,
 '2005-03-02 03:32:21.096019': -10.182171208957492,
 '2005-03-18 03:32:32.871075': -13.038122587076803,
 '2005-04-03 03:32:39.358025': -1.3199720742610834,
 '2005-04-19 03:32:42.220025': -2.0647363867810506,
 '2005-05-05 03:32:47.592063': -2.2712194034548707,
 '2005-05-21 03:32:53.795044': -0.8629436406100587,
 '2005-06-06 03:33:02.013075': 1.5965502809720857,
 '2005-06-22 03:33:08.254019': 4.734313186426435,
 '2005-07-08 03:33:12.992025': 8.473521646410655,
 '2005-07-24 03:33:25.506006': 9.66352742700366,
 '2005-08-09 03:33:33.937081': None,
 '2005-09-10 03:33:41.119075': 10.929985790190228,
 '2005-09-26 03:33:39.429069': 7.799843921462627,
 '2005-10-28 03:33:34.486038': 6.009788587189702,
 '2005-11-13 03:33:55.135019': -2.776867839404808,
 '2005-01-13 03:32:01.742063': -13.202234070066757,
 '2005-02-14 03:32:29.041044': -15.793659254110791,
 '2005-03-18 03:32:56.559038': -11.346347360874509,
 '2005-04-03 03:33:03.045094': -0.7885199523536508,
 '2005-04-19 03:33:06.050069': -1.3736220723083177,
 '2005-05-05 03:33:11.280031': -1.4564838046246993,
 '2005-05-21 03:33:17.625081': 1.197599146155981,
 '2005-06-06 03:33:25.701038': None,
 '2005-06-22 03:33:31.941081': 3.9538703143840532,
 '2005-07-08 03:33:36.822056': 10.980925573002432,
 '2005-07-24 03:33:49.193069': 11.560469073729656,
 '2005-08-09 03:33:57.625044': 8.001750652062885,
 '2005-08-25 03:34:02.962050': 13.207983342939983,
 '2005-09-10 03:34:04.807038': 10.159278104206921,
 '2005-09-26 03:34:03.117031': 8.791303503070749,
 '2005-10-12 03:33:58.757063': None,
 '2005-10-28 03:33:58.173094': 6.622270386986723,
 '2005-11-13 03:34:18.822081': -5.536535099498476,
 '2005-02-05 03:37:42.599025': -22.752097699858858,
 '2005-02-21 03:37:59.507031': -21.185827341336257,
 '2005-03-09 03:38:13.663081': -17.38428975427728,
 '2005-03-25 03:38:23.004094': -9.486912108275536,
 '2005-04-10 03:38:27.602019': -4.9459099413807825,
 '2005-05-12 03:38:36.236038': None,
 '2005-05-28 03:38:44.214038': -28.599435263768157,
 '2005-06-13 03:38:51.966056': 5.347321553158001,
 '2005-06-29 03:38:56.599075': -7.040662469518979,
 '2005-07-15 03:39:05.911044': 0.1311032185031672,
 '2005-07-31 03:39:16.508056': 18.826116614842636,
 '2005-09-01 03:39:27.192000': 13.590788548769632,
 '2005-09-17 03:39:27.572069': None,
 '2005-10-19 03:39:22.584013': 2.1942527851060305,
 '2005-01-20 03:37:55.221000': -17.85331156687286,
 '2005-02-05 03:38:06.714094': -22.136383464857452,
 '2005-02-21 03:38:23.623006': -22.030913775292138,
 '2005-03-09 03:38:37.779050': -16.191517057468683,
 '2005-03-25 03:38:47.120069': -5.391911785787843,
 '2005-04-10 03:38:51.575019': -6.2304437152960634,
 '2005-05-12 03:39:00.209044': -1.0640304331821617,
 '2005-05-28 03:39:08.330019': 3.4689777343911645,
 '2005-06-13 03:39:15.939063': 6.050247175451595,
 '2005-06-29 03:39:20.715044': -4.9966485088581205,
 '2005-07-15 03:39:29.884050': 10.911566412262372,
 '2005-07-31 03:39:40.481056': 19.40796613648612,
 '2005-09-01 03:39:51.307075': 12.166359183290227,
 '2005-09-17 03:39:51.688038': 8.473586879232444,
 '2005-10-03 03:39:48.521000': 9.244828875497216,
 '2005-10-19 03:39:46.699088': 0.6698252510719251,
 '2005-01-20 03:38:18.908063': -22.12460694127094,
 '2005-02-05 03:38:30.402063': -21.28714335542933,
 '2005-02-21 03:38:47.310075': -21.224659839190508,
 '2005-03-09 03:39:01.467019': -17.70473206514972,
 '2005-03-25 03:39:10.808031': -5.121188193132918,
 '2005-04-10 03:39:15.262088': -5.547016800100002,
 '2005-05-12 03:39:24.039088': None,
 '2005-05-28 03:39:32.017088': 2.758037917921617,
 '2005-06-13 03:39:39.769094': 5.050482321234997,
 '2005-06-29 03:39:44.403006': 5.148692382659026,
 '2005-07-15 03:39:53.572013': 14.310969556202638,
 '2005-07-31 03:40:04.169019': 17.729580613830922,
 '2005-08-16 03:40:11.110050': 12.165568917521199,
 '2005-09-01 03:40:14.995038': 13.221646998893826,
 '2005-09-17 03:40:15.376000': 10.710773871952098,
 '2005-10-03 03:40:12.208063': 9.669493011867246,
 '2005-10-19 03:40:10.387050': None,
 '2005-11-04 03:40:15.102013': -15.288350518768647,
 '2005-11-20 03:40:40.096069': -9.202568930651834,
 '2005-01-27 03:44:10.762056': -28.17515695459185,
 '2005-02-12 03:44:24.980019': -17.55411024433733,
 '2005-02-28 03:44:40.994050': -10.317545704184148,
 '2005-03-16 03:44:53.379038': -6.86793791563282,
 '2005-04-01 03:45:00.473031': -0.987289250484755,
 '2005-05-19 03:45:14.030069': -0.11685077167903711,
 '2005-06-04 03:45:22.378013': 3.3567404083798595,
 '2005-06-20 03:45:29.307013': 4.387664571303834,
 '2005-07-06 03:45:33.331081': -6.1421161732496135,
 '2005-07-22 03:45:45.845044': 8.450458577355716,
 '2005-08-07 03:45:54.873019': 18.092920345729418,
 '2005-08-23 03:46:00.352088': -8.328600286083525,
 '2005-09-24 03:46:01.388075': 11.038770339671077,
 '2005-10-10 03:45:57.197006': -6.534808218055439,
 '2005-11-11 03:46:13.569000': -22.576712743822686,
 '2005-01-11 03:44:21.679031': -9.695223643927243,
 '2005-01-27 03:44:34.450019': -25.579069908629577,
 '2005-02-12 03:44:48.810056': -20.419536199180936,
 '2005-02-28 03:45:04.682013': -17.126540618181448,
 '2005-03-16 03:45:17.067000': -8.712647116608354,
 '2005-04-01 03:45:24.160094': -1.3887577089384713,
 '2005-05-19 03:45:37.718038': 2.1398139708755943,
 '2005-06-04 03:45:46.208044': 3.006559563988231,
 '2005-06-20 03:45:52.994075': 7.004093530205421,
 '2005-07-06 03:45:57.019044': 10.58779543809694,
 '2005-07-22 03:46:09.533006': 13.17586590387717,
 '2005-08-07 03:46:18.560081': 16.030606824236333,
 '2005-08-23 03:46:24.040050': 10.58905018273823,
 '2005-09-24 03:46:25.219013': 8.8116314585711,
 '2005-10-10 03:46:20.884069': 3.087255148620557,
 '2005-11-11 03:46:37.256056': -10.558886070237145,
 '2005-01-18 03:50:38.870031': -9.19503035640697,
 '2005-02-03 03:50:50.170006': None,
 '2005-02-19 03:51:06.963019': -19.523623284957697,
 '2005-03-07 03:51:21.584075': -14.153620843280232,
 '2005-03-23 03:51:31.694088': -10.0997047202758,
 '2005-04-08 03:51:36.601075': -3.8268767545890285,
 '2005-04-24 03:51:39.784075': 1.485556177522212,
 '2005-05-10 03:51:45.262031': -0.9443979355265568,
 '2005-05-26 03:51:52.669063': 1.1251995039931226,
 '2005-07-13 03:52:13.770013': 12.62044463352979,
 '2005-10-01 03:52:34.466019': 11.080262805462247,
 '2005-10-17 03:52:31.895000': -25.110033304054507,
 '2005-11-02 03:52:34.300044': -40.77066934329452,
 '2005-11-18 03:52:58.817019': 3.6851973318434443,
'2006-04-23 03:28:35.594435400': -1.73397433328124,
 '2006-05-25 03:28:38.863881500': -0.955360150163181,
 '2006-06-26 03:28:31.276082900': 7.472732232890909,
 '2006-07-28 03:28:16.535798800': 12.467000438459001,
 '2006-04-23 03:28:59.472730100': -2.1957632655583232,
 '2006-05-25 03:29:02.743843400': -2.3732210446478583,
 '2006-06-26 03:28:55.152810300': 4.661423737507692,
 '2006-07-28 03:28:40.412109600': 8.629557943136884,
 '2006-09-14 03:28:31.529496700': 7.863832036473009,
 '2006-04-23 03:29:23.351908400': -2.0932354736903327,
 '2006-05-09 03:29:26.542669700': -1.5454512086198435,
 '2006-05-25 03:29:26.624358800': -4.631210190216882,
 '2006-06-26 03:29:19.030024600': 5.790533835362262,
 '2006-07-28 03:29:04.289187400': 13.213879332664058,
 '2006-08-29 03:29:00.344640900': 9.313315501743343,
 '2006-09-14 03:28:55.410146600': 8.563500711170805,
 '2006-10-16 03:28:52.258532400': 3.2794472877235328,
 '2006-03-13 03:34:51.950276': -14.446465135202164,
 '2006-03-29 03:34:50.619789300': -12.5975409836562,
 '2006-04-30 03:34:48.230533200': -2.154241993339827,
 '2006-07-03 03:34:39.227909900': 11.934342929788354,
 '2006-07-19 03:34:31.145534500': 13.840627668856657,
 '2006-08-04 03:34:27.061961600': 13.18197569127282,
 '2006-11-08 03:34:36.588024700': -8.914172708442786,
 '2006-03-13 03:35:15.826773400': -13.340648508192816,
 '2006-03-29 03:35:14.495806800': -9.595662147055684,
 '2006-04-30 03:35:12.109696400': -1.5826259468908477,
 '2006-05-16 03:35:13.827750900': -1.2133441900211985,
 '2006-07-03 03:35:03.104148200': 7.929918019811168,
 '2006-07-19 03:34:55.021345700': 6.62651961677306,
 '2006-08-04 03:34:50.939261400': 17.39444668213884,
 '2006-11-08 03:35:00.465708600': -14.324438149371247,
 '2006-03-13 03:35:39.703823800': -7.869261465169032,
 '2006-03-29 03:35:38.372614500': -5.837476265776403,
 '2006-04-30 03:35:35.989492500': -1.5242609779686527,
 '2006-05-16 03:35:37.708548600': -0.701848225504876,
 '2006-07-03 03:35:26.980912900': 8.785178127806383,
 '2006-07-19 03:35:18.897761800': 6.448329189750489,
 '2006-08-04 03:35:14.816954700': 16.058614348572025,
 '2006-10-07 03:34:53.225719700': 2.246176470052481,
 '2006-11-08 03:35:24.344090': -2.0616299315159403,
 '2006-11-24 03:35:35.099256800': -3.987469182036392,
 '2006-02-09 03:35:59.710856200': -11.757539945742156,
 '2006-02-25 03:36:02.783287100': -17.406956596582727,
 '2006-03-13 03:36:03.581377900': -6.972858973131475,
 '2006-03-29 03:36:02.250064200': -4.69628691150266,
 '2006-04-14 03:35:58.408830500': -7.716377182631039,
 '2006-04-30 03:35:59.870109200': -3.416763310456957,
 '2006-05-16 03:36:01.589766200': -4.5853396016470205,
 '2006-07-03 03:35:50.858295': 9.744967040655574,
 '2006-07-19 03:35:42.774845600': 8.904849083975884,
 '2006-08-04 03:35:38.695560400': 15.433317246517573,
 '2006-10-07 03:35:17.107320700': 4.86882167671868,
 '2006-11-08 03:35:48.222872300': 2.1993048737572147,
 '2006-11-24 03:35:58.977359300': -1.4396173598137427,
 '2006-05-23 03:41:24.488959400': 1.2574360165099538,
 '2006-06-24 03:41:17.511293100': 4.956243067598452,
 '2006-08-27 03:40:58.565389500': 7.329316404085062,
 '2006-09-12 03:40:53.898296400': 6.66239721873987,
 '2006-09-28 03:40:47.106519200': 2.8854360376617136,
 '2006-10-14 03:40:47.930543100': 2.6817312216755784,
 '2006-05-23 03:41:48.369574300': 1.3326786976014766,
 '2006-06-24 03:41:41.388939600': 5.533101002013931,
 '2006-07-10 03:41:34.513333500': 0.5251460823539946,
 '2006-07-26 03:41:25.922513800': 8.069291543478826,
 '2006-08-27 03:41:22.445774700': 9.398980224242651,
 '2006-09-12 03:41:17.779024200': 8.690206467625893,
 '2006-09-28 03:41:10.986289100': 5.394905619756643,
 '2006-10-14 03:41:11.811040700': 5.361640343266574,
 '2006-05-23 03:42:12.250804500': 0.43544078105149714,
 '2006-06-24 03:42:05.267135600': 4.2098239635713695,
 '2006-07-10 03:41:58.390339200': None,
 '2006-07-26 03:41:49.800126600': 6.958072052518911,
 '2006-08-27 03:41:46.326565600': 10.00070851720625,
 '2006-09-12 03:41:41.660464500': 8.341125313712434,
 '2006-09-28 03:41:34.866667400': 3.739421305233184,
 '2006-10-14 03:41:35.692342200': 5.5309576581275826,
 '2006-11-15 03:42:04.163861': 0.5099599108057453,
 '2006-12-01 03:42:13.407602900': -4.709948870914995,
 '2006-02-07 03:47:56.913867700': -21.479214218462946,
 '2006-03-27 03:48:00.297692400': -6.70234390779035,
 '2006-10-21 03:47:29.822772': 1.9724868147130334,
 '2006-02-07 03:48:20.794826300': -21.387862890033606,
 '2006-03-11 03:48:25.226540500': -14.21341155206484,
 '2006-03-27 03:48:24.175039100': -11.347675875062334,
 '2006-04-12 03:48:20.688646400': None,
 '2006-05-14 03:48:23.188587': 1.3166426322450735,
 '2006-06-15 03:48:19.075389800': 4.241791490888441,
 '2006-09-03 03:47:55.347671200': 9.746279552984255,
 '2006-09-19 03:47:49.754072200': 9.291781820638953,
 '2006-10-21 03:47:53.703424900': 3.810482993176603,
 '2006-11-06 03:48:08.257062300': 4.291823721428315,
 '2006-11-22 03:48:19.450232600': -4.569998088341191,
 '2006-03-02 03:54:35.713585200': -12.588883039779978,
 '2006-04-03 03:54:33.806431300': -11.514980258024574,
 '2006-05-21 03:54:33.971028': None,
 '2006-06-06 03:54:32.094223900': 3.6400103306718363,
 '2006-06-22 03:54:27.606212200': 7.305822536535954,
 '2006-07-24 03:54:12.363290700': None,
 '2006-09-10 03:54:04.004658100': 8.356294218306273,
 '2006-09-26 03:53:57.465297500': None,
 '2006-10-12 03:53:55.186929300': -1.633281189686865,
 '2006-10-28 03:54:11.217414300': -1.906928152833642,
 '2006-02-10 03:28:59.256088': -23.979526817407947,
 '2006-02-26 03:29:20.974094': -20.193100523296206,
 '2006-03-14 03:29:41.991069': -6.717054539249108,
 '2006-04-15 03:30:16.525006': -14.55288758983535,
 '2006-06-18 03:31:12.251013': None,
 '2006-07-04 03:31:28.422019': 14.455475520179771,
 '2006-07-20 03:31:41.973031': 13.503824300369258,
 '2006-08-05 03:31:53.696088': None,
 '2006-09-22 03:32:26.818013': 5.978326045950064,
 '2006-10-08 03:32:38.479031': 1.283436638693101,
 '2006-10-24 03:32:49.475094': 1.8696975209021807,
 '2006-01-25 03:29:00.165025': None,
 '2006-02-10 03:29:23.087019': -22.428177751847645,
 '2006-02-26 03:29:44.805025': -19.326296833186042,
 '2006-03-14 03:30:05.679031': -9.206754222791183,
 '2006-04-15 03:30:40.355038': -15.983216187025867,
 '2006-06-18 03:31:36.081050': 1.5760752453946818,
 '2006-07-04 03:31:52.252050': 7.61644010027586,
 '2006-07-20 03:32:05.803069': 7.271022760966081,
 '2006-08-05 03:32:17.527025': None,
 '2006-08-21 03:32:28.269031': 8.638644601616594,
 '2006-09-06 03:32:39.176038': None,
 '2006-09-22 03:32:50.648050': 7.202216844291052,
 '2006-10-08 03:33:02.309069': -1.2343449150011807,
 '2006-10-24 03:33:13.163056': 1.0407628827725477,
 '2006-01-25 03:29:24.280094': -24.71985314660985,
 '2006-02-10 03:29:47.202094': -16.21910049083102,
 '2006-02-26 03:30:08.921000': -16.682599706771434,
 '2006-03-14 03:30:29.795006': -9.90479044710488,
 '2006-04-15 03:31:04.471013': -11.078552518313865,
 '2006-06-18 03:32:00.197025': 2.4980685616565084,
 '2006-07-04 03:32:16.368025': 8.702525571750945,
 '2006-07-20 03:32:29.776075': 8.929540118771133,
 '2006-08-05 03:32:41.500031': 11.860624590672817,
 '2006-08-21 03:32:52.242044': 9.341382454934521,
 '2006-09-06 03:33:03.292013': 4.868952470029575,
 '2006-10-08 03:33:26.425044': 1.0948415176402182,
 '2006-10-24 03:33:37.279025': 2.941135227853532,
 '2006-11-25 03:33:54.759025': -14.143022026935332,
 '2006-02-01 03:34:57.308094': -28.035789280331294,
 '2006-02-17 03:35:19.647094': -20.405590983825498,
 '2006-03-05 03:35:41.223056': -16.718994101084274,
 '2006-04-22 03:36:33.534081': -2.38702612739531,
 '2006-05-08 03:36:46.334056': -2.2406117447079534,
 '2006-06-09 03:37:14.268094': 2.194638557885049,
 '2006-06-25 03:37:30.452025': 7.041008045764926,
 '2006-07-27 03:37:58.152094': 11.30275311565537,
 '2006-08-28 03:38:19.967000': 5.1314956521977235,
 '2006-09-13 03:38:31.163050': 6.803053535408439,
 '2006-09-29 03:38:42.824056': 6.350493521987402,
 '2006-10-15 03:38:54.217081': 1.2943646213765467,
 '2006-02-01 03:35:21.139019': -26.805840469642195,
 '2006-03-05 03:36:05.053081': -15.461480595479372,
 '2006-04-06 03:36:42.502050': -27.320377883997505,
 '2006-04-22 03:36:57.365013': -2.7068700359778477,
 '2006-05-08 03:37:10.164088': -1.2648276698588612,
 '2006-06-09 03:37:38.099025': 2.2129958315896037,
 '2006-06-25 03:37:54.282063': 5.494271943768316,
 '2006-07-27 03:38:21.840056': 7.253446856573904,
 '2006-08-28 03:38:43.797038': 8.77392980601569,
 '2006-09-29 03:39:06.654088': 6.1198381261559875,
 '2006-10-15 03:39:18.048006': 3.3745694996399287,
 '2006-10-31 03:39:28.433075': 3.667102335750773,
 '2006-02-01 03:35:45.254088': -25.721305687347556,
 '2006-02-17 03:36:07.593094': -12.853825489316883,
 '2006-03-05 03:36:29.026081': -14.060287614906125,
 '2006-04-06 03:37:06.475044': -15.11024279501379,
 '2006-04-22 03:37:21.338013': -2.524026099352561,
 '2006-05-08 03:37:34.137088': -0.677232107944603,
 '2006-06-09 03:38:02.214094': 3.6482350814091054,
 '2006-06-25 03:38:18.398031': 6.6618674753475045,
 '2006-07-27 03:38:45.956031': 9.815193505199955,
 '2006-08-28 03:39:07.770044': 9.900802809452362,
 '2006-09-13 03:39:18.966088': 8.661214034866438,
 '2006-09-29 03:39:30.770063': 7.54113669227326,
 '2006-10-15 03:39:42.163081': 3.7379698435228716,
 '2006-10-31 03:39:52.406075': 4.140275250381612,
 '2006-02-01 03:36:08.942050': -27.78430650091868,
 '2006-02-17 03:36:31.424019': -18.32342270307271,
 '2006-03-05 03:36:52.857013': -14.405364810349468,
 '2006-04-06 03:37:30.163006': -9.134910976013536,
 '2006-04-22 03:37:45.168044': -3.296362977861826,
 '2006-05-08 03:37:57.968019': -1.4110544839467567,
 '2006-06-09 03:38:25.902063': None,
 '2006-06-25 03:38:42.085094': 8.677008496728545,
 '2006-07-27 03:39:09.644000': 7.334309583618771,
 '2006-08-12 03:39:20.855044': None,
 '2006-08-28 03:39:31.458006': 10.61909326654625,
 '2006-09-13 03:39:42.797025': 6.297187078364759,
 '2006-09-29 03:39:54.458025': 4.980091094943608,
 '2006-10-15 03:40:05.851044': 4.142231471122661,
 '2006-10-31 03:40:16.237000': 3.946656441202651,
 '2006-02-24 03:42:03.560019': -19.010401805394075,
 '2006-03-12 03:42:24.968006': -15.099254382798792,
 '2006-04-13 03:43:00.253056': -7.535336198782435,
 '2006-04-29 03:43:13.948075': -0.7301596557622246,
 '2006-06-16 03:43:55.539063': 2.901447219608401,
 '2006-07-18 03:44:25.844038': 5.652876876494534,
 '2006-08-03 03:44:37.752013': -20.6408112096959,
 '2006-08-19 03:44:48.601081': 11.104907245511516,
 '2006-09-20 03:45:10.843013': 5.67926085039249,
 '2006-10-22 03:45:33.616069': 4.332838648158381,
 '2006-11-07 03:45:43.437031': 1.5700753404840335,
 '2006-02-08 03:42:05.920075': -18.646759824058464,
 '2006-02-24 03:42:27.675088': -20.079801691125475,
 '2006-03-12 03:42:48.941006': -24.80228654458317,
 '2006-04-13 03:43:24.226056': -9.403212169900428,
 '2006-04-29 03:43:38.064050': -1.0271107023049926,
 '2006-06-16 03:44:19.655031': 4.170845959786962,
 '2006-07-18 03:44:49.817044': 6.625804810457192,
 '2006-08-03 03:45:01.725019': None,
 '2006-08-19 03:45:12.574088': 11.677560401615063,
 '2006-09-04 03:45:23.448038': None,
 '2006-09-20 03:45:34.958088': 5.284674018292892,
 '2006-10-06 03:45:46.601088': -2.602534907242784,
 '2006-10-22 03:45:57.589075': 4.959859543319085,
 '2006-11-07 03:46:07.410031': 2.4937100657109865,
 '2006-02-08 03:42:29.751006': -19.92463630461395,
 '2006-02-24 03:42:51.363050': -18.67076893750786,
 '2006-03-12 03:43:12.628063': None,
 '2006-04-13 03:43:47.914025': -9.751557861656524,
 '2006-04-29 03:44:01.752013': -6.004670689488824,
 '2006-05-31 03:44:27.455094': None,
 '2006-06-16 03:44:43.343000': 4.306424665524713,
 '2006-07-18 03:45:13.505006': None,
 '2006-08-19 03:45:36.262056': 12.22385866560799,
 '2006-09-04 03:45:47.278075': 7.10760913284231,
 '2006-09-20 03:45:58.646050': 5.534355982802855,
 '2006-10-06 03:46:10.432025': 3.9892646066099378,
 '2006-10-22 03:46:21.420006': 5.084074939203019,
 '2006-11-07 03:46:31.097088': 2.645624871196391,
 '2006-01-30 03:48:04.000019': -18.916156870133268,
 '2006-02-15 03:48:26.599025': None,
 '2006-03-03 03:48:48.175000': -10.198456888970448,
 '2006-04-04 03:49:26.102013': -7.87902609001876,
 '2006-05-06 03:49:54.309063': -5.092737671824239,
 '2006-06-07 03:50:21.816056': 3.3754689117793433,
 '2006-06-23 03:50:37.906094': 6.459833438808086,
 '2006-07-09 03:50:53.289025': None,
 '2006-07-25 03:51:06.009063': 9.270022110587018,
 '2006-08-10 03:51:17.365044': 4.8364176276780375,
 '2006-08-26 03:51:28.108081': -11.879478789878837,
 '2006-09-11 03:51:39.163075': 11.93435450268783,
 '2006-09-27 03:51:50.943000': 8.710987006976605,
 '2006-10-13 03:52:02.327000': -8.41367006310229,
 '2006-10-29 03:52:12.874075': None,
 '2006-01-30 03:48:27.830050': -19.807181752674932,
 '2006-02-15 03:48:50.286088': None,
 '2006-03-03 03:49:11.862063': -11.208371672015204,
 '2006-04-04 03:49:49.789075': -7.07255719921063,
 '2006-05-06 03:50:17.997025': -1.737270692828225,
 '2006-05-22 03:50:29.618063': None,
 '2006-06-07 03:50:45.504019': 2.2489568082116036,
 '2006-06-23 03:51:01.737031': 5.817013937461566,
 '2006-07-25 03:51:29.840000': None,
 '2006-08-10 03:51:41.053013': 14.529589683973105,
 '2006-08-26 03:51:51.796044': 7.2249966106202015,
 '2006-09-11 03:52:02.994013': 11.070755775240933,
 '2006-09-27 03:52:14.630063': 8.520010464309456,
 '2006-10-13 03:52:26.157031': 1.3362592109346034,
 '2006-10-29 03:52:36.562031': -2.1273301644469504,
 '2006-11-14 03:52:45.616019': -2.317632507539347,
 '2006-11-30 03:52:53.067025': -11.526446122839534,
 '2006-02-06 03:54:48.651094': -18.532788377236738,
 '2006-02-22 03:55:10.512069': -14.939793169975482,
 '2006-03-10 03:55:31.802081': -23.526995155570297,
 '2006-04-11 03:56:07.657050': -9.685755875057602,
 '2006-04-27 03:56:21.805069': 0.5493449902591319,
 '2006-05-13 03:56:33.816019': -23.632022725036606,
 '2006-05-29 03:56:46.914081': None,
 '2006-06-14 03:57:03.111038': None,
 '2006-06-30 03:57:19.555050': 2.3072796157199433,
 '2006-08-01 03:57:45.685075': -16.86580673237053,
 '2006-08-17 03:57:56.610088': 14.071429989148752,
 '2006-09-18 03:58:18.842019': None,
 '2006-10-04 03:58:30.654000': 7.571804839675713,
 '2006-10-20 03:58:41.757056': None,
 '2006-11-05 03:58:51.691063': 3.322647156485333,
 '2006-11-21 03:58:59.906000': -2.3275057578071254,
'2007-03-25 03:29:07.220050300': -11.473699940497166,
 '2007-04-10 03:29:06.351552400': -4.424694424963939,
 '2007-05-12 03:28:58.259614100': -0.30385509902822494,
 '2007-05-28 03:28:50.876557900': 3.2489137139578332,
 '2007-06-13 03:28:50.840723700': 4.247373916003977,
 '2007-07-15 03:28:47.097569400': 0.3737828333587932,
 '2007-07-31 03:28:42.731397500': 14.680007609986959,
 '2007-08-16 03:28:36.913016600': 18.039929606669887,
 '2007-09-01 03:28:29.762014400': 13.80017084852301,
 '2007-04-10 03:29:30.228088600': -10.019564861745303,
 '2007-04-26 03:29:27.317923300': -0.38990131205783124,
 '2007-05-12 03:29:22.139230700': -3.8116246426610023,
 '2007-05-28 03:29:14.756235800': 2.717739261684474,
 '2007-06-13 03:29:14.719674200': 3.7926255065259062,
 '2007-07-31 03:29:06.607449200': 13.783803313066882,
 '2007-08-16 03:29:00.790390900': 16.601163985644767,
 '2007-09-01 03:28:53.641402200': 14.318332341320307,
 '2007-04-10 03:29:54.105374200': -5.089982358874463,
 '2007-04-26 03:29:51.196797600': -0.16083683485567674,
 '2007-05-12 03:29:46.019448800': -4.452786249846271,
 '2007-05-28 03:29:38.636572': 3.573098813621412,
 '2007-06-13 03:29:38.599310600': 2.4380376343384706,
 '2007-07-31 03:29:30.483951700': 13.228614784356287,
 '2007-08-16 03:29:24.668803700': 16.842428156008854,
 '2007-09-01 03:29:17.521334500': 14.844685025637272,
 '2007-10-19 03:29:12.694125': -0.4114399340978934,
 '2007-11-20 03:29:32.383680300': -3.2630264440460146,
 '2007-05-03 03:35:12.297301600': -4.0683876554055,
 '2007-06-04 03:35:01.352241': 1.6551308627538215,
 '2007-06-20 03:35:01.431146900': None,
 '2007-07-06 03:34:59.653420200': 10.77667519332782,
 '2007-07-22 03:34:56.157256800': 11.336521921622456,
 '2007-09-24 03:34:32.312920900': 1.6619424364034545,
 '2007-10-10 03:34:28.620157100': -2.0617063398828623,
 '2007-10-26 03:34:40.866766400': -4.081519052947446,
 '2007-05-03 03:35:36.176219200': -8.164264047302709,
 '2007-05-19 03:35:30.042375': 0.9771339344453308,
 '2007-06-04 03:35:25.232216400': 3.0713528426632335,
 '2007-07-22 03:35:20.033018': 8.71430627144032,
 '2007-08-07 03:35:15.029180100': 11.774535111427014,
 '2007-10-10 03:34:52.501220200': 3.3354144704980975,
 '2007-10-26 03:35:04.746491400': 1.9733917710523226,
 '2007-03-16 03:36:05.274610900': -10.034075109407011,
 '2007-04-17 03:36:03.896078200': -2.024831520926282,
 '2007-05-03 03:36:00.055719': -2.9084629101757997,
 '2007-05-19 03:35:53.922919400': 2.0736592258272464,
 '2007-06-04 03:35:49.112457300': 3.911316663820286,
 '2007-06-20 03:35:49.188267400': None,
 '2007-07-22 03:35:43.909410700': 10.586233377126097,
 '2007-08-07 03:35:38.906456100': 13.786208636084393,
 '2007-09-08 03:35:24.756220500': None,
 '2007-10-10 03:35:16.382749300': 5.384072022717841,
 '2007-10-26 03:35:28.626512800': 4.544414501263544,
 '2007-03-16 03:36:29.152073800': -9.62579829744884,
 '2007-04-17 03:36:27.774701800': -0.5079578459141442,
 '2007-05-03 03:36:23.935999600': -0.20433837681361108,
 '2007-05-19 03:36:17.803973': 3.5975817338687217,
 '2007-06-04 03:36:12.993462600': 5.597743774564599,
 '2007-06-20 03:36:13.067793400': 2.737772381672391,
 '2007-07-22 03:36:07.786399500': 12.002308302037703,
 '2007-08-07 03:36:02.784132400': 12.960977517800492,
 '2007-09-08 03:35:48.637176700': None,
 '2007-10-10 03:35:40.265067500': 3.0107561036894768,
 '2007-10-26 03:35:52.507353900': 4.356480697796827,
 '2007-04-08 03:41:52.091463800': -1.1649295848678198,
 '2007-04-24 03:41:49.414150100': -1.0374400272233666,
 '2007-05-10 03:41:44.546020900': None,
 '2007-06-11 03:41:36.331426500': None,
 '2007-07-29 03:41:28.848525300': 14.814928594731654,
 '2007-08-14 03:41:23.231262500': None,
 '2007-08-30 03:41:16.251717300': 13.795865500728029,
 '2007-04-08 03:42:15.968588700': -0.8583312617463109,
 '2007-04-24 03:42:13.292947500': -4.485720296522587,
 '2007-05-10 03:42:08.426149700': -0.1844585242253275,
 '2007-06-11 03:42:00.211320600': 2.944440286869256,
 '2007-07-29 03:41:52.725155400': 16.170568192367988,
 '2007-08-14 03:41:47.108998200': 5.3010384986717325,
 '2007-08-30 03:41:40.131726800': 14.405309317257192,
 '2007-03-23 03:42:40.431027': -4.194813840128561,
 '2007-04-08 03:42:39.846185200': -1.4175055516158899,
 '2007-04-24 03:42:37.172225800': -2.0646667350576413,
 '2007-05-10 03:42:32.306946400': -2.8485344321715753,
 '2007-06-11 03:42:24.091663400': 4.222790770238872,
 '2007-07-29 03:42:16.602410400': 16.79002245042705,
 '2007-04-15 03:48:25.858420': -0.6146401469734597,
 '2007-05-01 03:48:22.296353500': None,
 '2007-06-18 03:48:10.938543900': 4.389799721816428,
 '2007-07-20 03:48:06.068219500': 12.822182409731841,
 '2007-08-21 03:47:55.008374100': 13.482336060003858,
 '2007-09-06 03:47:47.431144500': 13.223090448263749,
 '2007-10-08 03:47:36.342637500': 5.4672968877263495,
 '2007-10-24 03:47:48.847742500': -2.576900690619691,
 '2007-11-09 03:47:59.113231200': -4.373373104741314,
 '2007-02-10 03:48:43.052807900': -12.625514777971139,
 '2007-02-26 03:48:47.848780100': -17.536417255806395,
 '2007-03-14 03:48:50.590258500': -12.190948435769393,
 '2007-04-15 03:48:49.736811200': -1.0862803823999376,
 '2007-05-01 03:48:46.176571100': -8.055815124612462,
 '2007-05-17 03:48:40.303074800': 1.161519577457375,
 '2007-06-18 03:48:34.818064900': 5.625642257245394,
 '2007-07-20 03:48:29.945145600': 13.1029781035774,
 '2007-09-06 03:48:11.312532400': 12.292261528386524,
 '2007-10-08 03:48:00.224864400': 7.3754002805919265,
 '2007-10-24 03:48:12.728750400': 1.6061978227384286,
 '2007-11-09 03:48:22.991896500': 1.5884992091708472,
 '2007-04-06 03:55:01.688466800': -1.3274158911902463,
 '2007-04-22 03:54:59.251432': -0.49054368986400915,
 '2007-05-08 03:54:54.671136': 2.9668919235549702,
 '2007-05-24 03:54:47.858639100': 2.6973168167097397,
 '2007-06-25 03:54:45.130203900': 10.616632868062322,
 '2007-07-27 03:54:38.821184400': 16.199859406115163,
 '2007-08-12 03:54:33.384018400': 13.838730103774209,
 '2007-09-29 03:54:15.050913500': 9.539785649056777,
 '2007-10-15 03:54:16.761751400': 8.196883205201638,
 '2007-12-02 03:54:44.420537400': -14.547656837732818,
 '2007-02-13 03:33:34.219025': -22.62107420623669,
 '2007-03-01 03:33:34.498006': -20.226655505127322,
 '2007-03-17 03:33:32.243081': -16.801561903528942,
 '2007-04-02 03:33:27.505069': -4.751945015236836,
 '2007-04-18 03:33:20.693031': -2.6029674885776384,
 '2007-05-04 03:33:12.217056': -6.435361217234669,
 '2007-07-23 03:32:11.406081': 13.606765386516114,
 '2007-08-08 03:32:01.238050': 13.641136193950683,
 '2007-08-24 03:31:50.059088': 14.35041638708637,
 '2007-09-09 03:31:42.406063': 14.377788755071004,
 '2007-02-13 03:33:57.906081': -20.054655596509516,
 '2007-03-01 03:33:58.328038': -17.542721930485996,
 '2007-03-17 03:33:55.931044': -15.403941469683161,
 '2007-04-02 03:33:51.336000': -3.7139522234200686,
 '2007-04-18 03:33:44.523063': -2.8721576898519587,
 '2007-05-04 03:33:36.047088': -3.040591658451629,
 '2007-06-21 03:32:56.050044': None,
 '2007-07-07 03:32:43.700056': None,
 '2007-07-23 03:32:35.094056': 11.199485587511898,
 '2007-08-08 03:32:25.068094': 15.06668878805239,
 '2007-08-24 03:32:13.890025': 13.706555563229875,
 '2007-09-09 03:32:06.237000': 14.07660609348371,
 '2007-01-28 03:34:19.071013': -12.207116406407915,
 '2007-02-13 03:34:22.022050': -15.854556602583855,
 '2007-03-01 03:34:22.444006': -14.777936718633477,
 '2007-03-17 03:34:20.047013': -12.957595462623221,
 '2007-04-02 03:34:15.309006': -3.5379010336602343,
 '2007-04-18 03:34:08.639031': -3.6994350873507886,
 '2007-05-04 03:34:00.163069': -0.2733181234294288,
 '2007-05-20 03:33:49.849075': -1.877328060057771,
 '2007-06-21 03:33:20.023050': 4.044539763094644,
 '2007-07-07 03:33:07.673063': 7.994236757086573,
 '2007-07-23 03:32:59.210038': 11.567364792636027,
 '2007-08-08 03:32:49.042006': 16.026046463109495,
 '2007-08-24 03:32:38.006006': 13.53892999495594,
 '2007-09-09 03:32:30.352081': 12.9727181969823,
 '2007-01-28 03:34:42.758069': None,
 '2007-02-13 03:34:45.852081': -16.67304666193332,
 '2007-03-01 03:34:46.131069': -13.470901698304209,
 '2007-03-17 03:34:43.734075': -10.412573638659499,
 '2007-04-02 03:34:38.996069': -3.3595595918391448,
 '2007-04-18 03:34:32.326094': None,
 '2007-05-04 03:34:23.994006': 0.8841559705528373,
 '2007-05-20 03:34:13.537044': -4.175695629336817,
 '2007-02-04 03:39:43.560056': -22.823753195997913,
 '2007-02-20 03:39:45.471031': -28.059419529682426,
 '2007-03-08 03:39:44.646081': -22.376136664945722,
 '2007-03-24 03:39:41.227075': -8.604161327329045,
 '2007-04-09 03:39:35.464075': -3.45064419530688,
 '2007-04-25 03:39:28.059056': -0.39628836817456,
 '2007-05-11 03:39:18.691025': -2.058685335098682,
 '2007-05-27 03:39:07.362025': 3.2612860562381303,
 '2007-06-12 03:38:52.788031': 5.786436337863849,
 '2007-06-28 03:38:34.751000': None,
 '2007-07-14 03:38:27.202044': 9.439364482365342,
 '2007-07-30 03:38:17.916031': 13.828017451384264,
 '2007-08-15 03:38:07.165075': 17.320819496480343,
 '2007-08-31 03:37:55.690075': 14.178128683044061,
 '2007-10-02 03:37:42.368088': 6.8926385892764195,
 '2007-02-04 03:40:07.390075': None,
 '2007-02-20 03:40:09.301056': -26.297420711076313,
 '2007-03-08 03:40:08.477013': -22.89360420114155,
 '2007-03-24 03:40:05.058006': -8.513616630810965,
 '2007-04-09 03:39:59.295006': -2.027582881389476,
 '2007-04-25 03:39:51.889088': -0.6099876344763138,
 '2007-05-11 03:39:42.664000': -0.8284325064452512,
 '2007-05-27 03:39:31.192056': 3.0422512780550073,
 '2007-06-12 03:39:16.618069': 4.55779366999317,
 '2007-07-14 03:38:51.032081': 7.864031147355369,
 '2007-07-30 03:38:41.746063': 14.250480963226682,
 '2007-08-15 03:38:31.138081': 17.047047957622873,
 '2007-08-31 03:38:19.521013': 13.307580796916929,
 '2007-10-02 03:38:06.199019': 7.438503604894921,
 '2007-02-04 03:40:31.506038': None,
 '2007-02-20 03:40:33.274056': None,
 '2007-03-08 03:40:32.450006': None,
 '2007-03-24 03:40:29.031000': -4.858118027621429,
 '2007-04-09 03:40:23.410075': -0.979715691695289,
 '2007-05-11 03:40:06.636075': 1.179948462940317,
 '2007-05-27 03:39:55.165063': 3.936829344654007,
 '2007-06-12 03:39:40.591075': 4.715653681504749,
 '2007-07-14 03:39:15.005088': 8.620718366565601,
 '2007-07-30 03:39:05.719069': 15.300919361972461,
 '2007-08-15 03:38:55.111088': 16.520259036856217,
 '2007-08-31 03:38:43.636088': 13.071904266266982,
 '2007-10-02 03:38:30.172025': 8.621379639710874,
 '2007-01-19 03:40:50.929013': -16.499285935084842,
 '2007-02-04 03:40:55.193088': None,
 '2007-03-08 03:40:56.137069': None,
 '2007-03-24 03:40:52.718063': -5.421982614006317,
 '2007-04-09 03:40:47.098031': -0.3518407628527141,
 '2007-04-25 03:40:39.693019': 0.03478981074197454,
 '2007-05-11 03:40:30.466081': 2.621524175131216,
 '2007-05-27 03:40:18.995094': 4.892093458391759,
 '2007-06-12 03:40:04.422006': 5.656316325221637,
 '2007-07-14 03:39:38.693056': 9.551493893245082,
 '2007-07-30 03:39:29.407038': 12.657937432588781,
 '2007-08-15 03:39:18.799056': 17.71013059248566,
 '2007-08-31 03:39:07.324056': 15.34089809859914,
 '2007-10-02 03:38:54.002063': 6.260428548123805,
 '2007-03-15 03:46:18.034094': -10.563267705240232,
 '2007-03-31 03:46:13.722088': -6.0935983734004555,
 '2007-04-16 03:46:07.066038': -2.772114637762882,
 '2007-05-02 03:45:58.944069': -4.368225773779011,
 '2007-06-19 03:45:19.889094': 3.351684033639359,
 '2007-07-05 03:45:06.194069': 8.760809097494151,
 '2007-07-21 03:44:57.947044': 8.161071132301753,
 '2007-08-06 03:44:47.947013': -27.855167654397405,
 '2007-08-22 03:44:36.885044': None,
 '2007-09-07 03:44:28.625006': None,
 '2007-09-23 03:44:21.816031': 9.782455289615545,
 '2007-01-26 03:46:40.124013': -21.19924859629979,
 '2007-02-27 03:46:44.128069': None,
 '2007-03-15 03:46:42.150063': -11.83178150508955,
 '2007-03-31 03:46:37.695088': -10.694553522243906,
 '2007-04-16 03:46:31.182013': -3.083371023839562,
 '2007-05-02 03:46:23.060038': -0.006455082239993335,
 '2007-06-19 03:45:44.005069': 5.405490965794874,
 '2007-07-05 03:45:30.167075': 11.075766737463276,
 '2007-07-21 03:45:21.920050': 11.193719616962097,
 '2007-08-06 03:45:11.920019': None,
 '2007-09-23 03:44:45.789038': 9.417233444867309,
 '2007-01-26 03:47:03.811069': -18.168052800206034,
 '2007-02-11 03:47:07.190006': -20.41487834378502,
 '2007-02-27 03:47:07.816025': None,
 '2007-03-15 03:47:05.838025': -9.732567407386593,
 '2007-03-31 03:47:01.383050': -18.149235079546276,
 '2007-04-16 03:46:54.869075': -3.2588411468047496,
 '2007-05-02 03:46:46.748000': -0.3345045609320344,
 '2007-05-18 03:46:36.637081': None,
 '2007-06-19 03:46:07.693038': 6.302634892335584,
 '2007-07-05 03:45:53.855044': 12.631900949154375,
 '2007-07-21 03:45:45.608019': 5.730442046075364,
 '2007-09-07 03:45:16.428050': 11.788346949105375,
 '2007-09-23 03:45:09.619075': 9.042087302089662,
 '2007-06-26 03:51:46.677056': 6.0735933840068315,
 '2007-07-12 03:51:37.679088': None,
 '2007-07-28 03:51:28.548038': 17.708015212656928,
 '2007-08-29 03:51:06.736075': 14.810717449576648,
 '2007-09-14 03:51:00.704006': 15.24826227188073,
 '2007-09-30 03:50:53.013050': 10.408513183575758,
 '2007-01-17 03:53:11.840031': -5.989685470513934,
 '2007-02-02 03:53:16.568056': -4.278994632588883,
 '2007-02-18 03:53:18.637000': -15.79093588803214,
 '2007-03-06 03:53:18.096094': -13.187032923880276,
 '2007-03-22 03:53:14.964038': -7.538524327583222,
 '2007-04-07 03:53:09.604031': -0.45622919528062394,
 '2007-04-23 03:53:02.366069': -3.418191063908461,
 '2007-05-25 03:52:42.264019': 0.1963439307685497,
 '2007-06-26 03:52:10.365025': 9.327014606888243,
 '2007-07-28 03:51:52.236006': 18.122492962423035,
 '2007-08-29 03:51:30.567013': 14.32876015481289,
 '2007-09-14 03:51:24.534044': 14.251069114569566,
 '2007-09-30 03:51:16.701019': 9.445359304576643,
 '2007-01-24 03:59:25.031075': -11.27476032932497,
 '2007-02-09 03:59:28.595000': -11.400742946260007,
 '2007-02-25 03:59:29.606075': -10.46457973612308,
 '2007-03-13 03:59:27.771069': -10.35088153273715,
 '2007-03-29 03:59:23.746075': -11.453622262161218,
 '2007-04-14 03:59:17.426000': -10.8317365898451,
 '2007-04-30 03:59:09.447063': -0.24496178014184822,
 '2007-05-16 03:58:59.632025': 3.3350217149901633,
 '2007-06-01 03:58:47.130081': None,
 '2007-07-03 03:58:16.423094': 12.126650709511132,
 '2007-07-19 03:58:08.424038': -5.103634168731073,
 '2007-08-04 03:57:58.568056': 15.625347336685152,
 '2007-08-20 03:57:47.672063': 14.301015978102454,
 '2007-09-05 03:57:38.671094': 17.288817310450447,
 '2007-09-21 03:57:32.187069': -2.9512826803803396,
'2008-04-28 03:28:39.919488100': -14.942959118484076,
 '2008-05-14 03:28:34.857648200': 0.3303692613613878,
 '2008-05-30 03:28:28.028209500': -0.6616507760051223,
 '2008-06-15 03:28:19.411515100': 2.7655161717474637,
 '2008-07-01 03:28:09.164309300': 9.775643002694697,
 '2008-07-17 03:27:57.460828400': 15.705862410157776,
 '2008-08-18 03:27:43.302071400': 13.475138821546365,
 '2008-11-06 03:27:36.713665700': 0.29872054054336733,
 '2008-04-28 03:29:03.797557300': None,
 '2008-05-14 03:28:58.737460': 0.44387810000573064,
 '2008-05-30 03:28:51.908364500': -1.1648700298440364,
 '2008-06-15 03:28:43.290224800': 2.8956533635350055,
 '2008-07-01 03:28:33.041119800': 5.610100217155873,
 '2008-07-17 03:28:21.336124500': 2.5194532907711684,
 '2008-10-05 03:27:41.024227700': 6.480413120223843,
 '2008-10-21 03:27:45.363838300': 3.4739939949032843,
 '2008-11-06 03:28:00.592192500': -4.8839225642071735,
 '2008-05-14 03:29:22.617874300': 0.8571778548463607,
 '2008-05-30 03:29:15.789117600': -0.8709599550139377,
 '2008-06-15 03:29:07.169581700': 2.742783339233362,
 '2008-07-01 03:28:56.918317900': 5.500865346257907,
 '2008-07-17 03:28:45.212064800': None,
 '2008-09-03 03:28:23.591004500': 10.256728948167634,
 '2008-09-19 03:28:14.938393800': 9.027584120506722,
 '2008-10-05 03:28:04.903977200': 6.886808354471853,
 '2008-11-06 03:28:24.470995600': -12.721695048691544,
 '2008-05-05 03:34:48.758739200': -2.8606590139731947,
 '2008-05-21 03:34:42.943982800': 0.42390584188304525,
 '2008-06-06 03:34:35.284387300': 2.6338976303344603,
 '2008-06-22 03:34:25.959780': 7.317335665656215,
 '2008-07-08 03:34:15.023583300': 11.456649191108346,
 '2008-07-24 03:34:03.415140200': None,
 '2008-09-26 03:33:33.767004200': 6.968700634774282,
 '2008-10-12 03:33:22.841520100': 5.629760281006851,
 '2008-05-21 03:35:06.824245900': 0.4207498991683582,
 '2008-06-06 03:34:59.164167900': 0.22318533291384537,
 '2008-07-08 03:34:38.899670600': 7.464110989949414,
 '2008-07-24 03:34:27.290627100': 3.2148647159481363,
 '2008-08-09 03:34:21.645346400': 10.3355361497968,
 '2008-09-26 03:33:57.647197300': 8.246390048289394,
 '2008-10-12 03:33:46.722418900': 6.267235299661379,
 '2008-10-28 03:34:03.122542800': 3.083874700956105,
 '2008-03-18 03:35:42.073348800': -11.000954872910548,
 '2008-05-21 03:35:30.704919500': 0.6566205068068672,
 '2008-06-06 03:35:23.044596800': -1.3165075099395702,
 '2008-07-08 03:35:02.776341700': 9.642267505308324,
 '2008-07-24 03:34:51.167184900': 11.980598771318798,
 '2008-08-09 03:34:45.522857100': 10.4860736536934,
 '2008-09-26 03:34:21.527752800': 8.56623104859536,
 '2008-10-12 03:34:10.603979': 7.602164032496466,
 '2008-10-28 03:34:27.002763600': 4.620860152264776,
 '2008-03-02 03:36:09.135483600': -8.661451097054613,
 '2008-03-18 03:36:05.950386200': -11.933906086154213,
 '2008-04-03 03:36:06.259642800': -3.0468747257220463,
 '2008-05-21 03:35:54.586488': -0.41277200332029346,
 '2008-06-06 03:35:46.925377700': 0.4482316702669945,
 '2008-07-08 03:35:26.653481600': 9.677780087193975,
 '2008-07-24 03:35:15.043962200': 14.98882132156993,
 '2008-08-09 03:35:09.400862600': 13.135806885935473,
 '2008-09-10 03:34:54.644962': 10.761067126279753,
 '2008-09-26 03:34:45.409096300': 9.039842219754446,
 '2008-10-12 03:34:34.486157400': 8.490150374091675,
 '2008-10-28 03:34:50.883482300': 5.619634007010668,
 '2008-11-29 03:35:17.123394700': 0.5909256758666641,
 '2008-02-06 03:41:39.209587200': -17.36105964123799,
 '2008-02-22 03:41:35.224245300': -16.442981185768186,
 '2008-05-12 03:41:21.099256100': None,
 '2008-06-29 03:40:56.032546500': 8.894434936981332,
 '2008-07-15 03:40:44.501650': 10.096857291276239,
 '2008-07-31 03:40:35.779404900': 9.412508703143589,
 '2008-09-01 03:40:22.340822500': 12.24685347441466,
 '2008-09-17 03:40:13.841457300': 8.259563647231472,
 '2008-03-09 03:41:53.320971200': -4.919732973225445,
 '2008-03-25 03:41:53.327776700': -5.407583145256967,
 '2008-04-10 03:41:52.588399500': -3.770107363396071,
 '2008-05-12 03:41:44.979471300': -0.32645692321791114,
 '2008-06-29 03:41:19.910210100': 10.057222027960675,
 '2008-07-15 03:41:08.377667200': 11.05734886109581,
 '2008-07-31 03:40:59.655953500': 13.729760017378732,
 '2008-09-01 03:40:46.221105900': 12.708324693397818,
 '2008-09-17 03:40:37.722178300': 9.670229736830729,
 '2008-10-03 03:40:27.877176200': 7.800529211978083,
 '2008-10-19 03:40:28.852966': -6.204010164536048,
 '2008-03-09 03:42:17.198506200': -4.996853938219509,
 '2008-03-25 03:42:17.204666': -5.597064059763778,
 '2008-04-10 03:42:16.465685': -4.937429754833282,
 '2008-04-26 03:42:13.693960700': -0.6745053524500878,
 '2008-05-12 03:42:08.860517200': -0.5366065474786201,
 '2008-06-13 03:41:53.836798': 5.013691381250811,
 '2008-06-29 03:41:43.788166700': 10.663273187770804,
 '2008-07-15 03:41:32.254229200': 11.768648594250589,
 '2008-07-31 03:41:23.533240': 14.333790389374352,
 '2008-08-16 03:41:17.416077100': 10.193310394649739,
 '2008-09-01 03:41:10.101683900': 12.011303860318662,
 '2008-09-17 03:41:01.603703400': 9.60675614114528,
 '2008-10-03 03:40:51.757853200': 6.889566438092991,
 '2008-10-19 03:40:52.734550800': 1.0902727948008026,
 '2008-11-04 03:41:08.210838400': -0.3789123354875971,
 '2008-03-16 03:48:03.521157500': -7.16501455439018,
 '2008-04-17 03:48:02.425979300': -6.745570084590795,
 '2008-08-23 03:47:01.274577900': None,
 '2008-10-10 03:46:30.062581500': 8.467855749558165,
 '2008-10-26 03:46:46.690001900': 5.606604992748302,
 '2008-11-11 03:47:01.246040': 3.3490148142729126,
 '2008-03-16 03:48:27.398498700': -7.798772145314759,
 '2008-04-17 03:48:26.304121300': -17.175898673252252,
 '2008-05-03 03:48:22.630010200': -0.7810541029398334,
 '2008-08-23 03:47:25.154203500': 12.609514340949543,
 '2008-10-10 03:46:53.944478500': 6.525034963359992,
 '2008-10-26 03:47:10.570794100': 4.827037765624438,
 '2008-11-11 03:47:25.124986600': 2.4986907408962464,
 '2008-03-07 03:54:39.654056500': -5.453641734218958,
 '2008-03-23 03:54:38.745987': -3.864781377173018,
 '2008-04-08 03:54:38.321492400': -4.840355813380423,
 '2008-04-24 03:54:35.793263100': -0.8035522721316797,
 '2008-05-10 03:54:31.189856500': 0.7047229699781487,
 '2008-06-11 03:54:16.605441900': 7.231139480381723,
 '2008-07-29 03:53:45.838610800': 10.033069409222271,
 '2008-08-30 03:53:32.690704200': None,
 '2008-09-15 03:53:24.359189800': 12.063699550539862,
 '2008-03-19 03:28:31.647063': -19.76130814480882,
 '2008-04-20 03:27:45.540056': -3.3739361495134523,
 '2008-05-06 03:27:19.883031': -1.5532991965349048,
 '2008-05-22 03:26:55.656094': None,
 '2008-06-07 03:26:30.011044': 2.2407738591872284,
 '2008-08-26 03:24:12.440069': None,
 '2008-09-11 03:23:46.043063': 8.643835126104904,
 '2008-09-27 03:23:17.346000': 6.6173736485696555,
 '2008-03-19 03:28:55.335019': -21.01548519952032,
 '2008-04-04 03:28:33.153075': -4.133305387026133,
 '2008-04-20 03:28:09.370088': -2.564930734811687,
 '2008-05-06 03:27:43.713063': -4.976071817659116,
 '2008-06-07 03:26:53.841075': 3.548476571960594,
 '2008-08-26 03:24:36.270094': 9.574466720000126,
 '2008-09-11 03:24:09.873094': 10.22334234641838,
 '2008-09-27 03:23:41.176025': 6.8092713105322495,
 '2008-03-03 03:29:40.052000': -13.081568902230288,
 '2008-03-19 03:29:19.450088': None,
 '2008-04-04 03:28:57.269044': -5.4516244208392965,
 '2008-04-20 03:28:33.486050': -2.030160580877386,
 '2008-05-06 03:28:07.829031': -8.086281710938591,
 '2008-05-22 03:27:43.603000': None,
 '2008-06-07 03:27:17.957044': 4.334717484411098,
 '2008-08-26 03:25:00.386069': 10.049493518449744,
 '2008-09-11 03:24:33.989069': 10.433893389320268,
 '2008-09-27 03:24:05.291094': 7.871435426279463,
 '2008-08-26 03:25:24.217000': None,
 '2008-09-11 03:24:57.820000': 9.95813697204357,
 '2008-09-27 03:24:28.979050': 5.735632540122846,
 '2008-03-26 03:34:32.855094': -5.603752084668196,
 '2008-04-11 03:34:09.957050': None,
 '2008-04-27 03:33:45.451063': -0.691491063295598,
 '2008-05-29 03:32:54.999000': -0.3381564335258052,
 '2008-08-17 03:30:37.426044': 11.065448412739785,
 '2008-09-02 03:30:11.925094': 12.145501143117098,
 '2008-09-18 03:29:44.630044': 1.1351329090650697,
 '2008-10-04 03:29:14.774056': -0.8551652541876212,
 '2008-03-26 03:34:56.686019': -6.5276716482815536,
 '2008-04-11 03:34:33.787075': None,
 '2008-04-27 03:34:09.281094': -0.643279050976865,
 '2008-05-13 03:33:44.185019': -0.9825815523256923,
 '2008-08-17 03:31:01.256075': 15.229854705582579,
 '2008-09-02 03:30:35.898094': 12.713755780453958,
 '2008-09-18 03:30:08.460069': 6.873769762361925,
 '2008-03-26 03:35:20.801088': -6.134439389853227,
 '2008-04-11 03:34:57.760075': -15.831148663245868,
 '2008-04-27 03:34:33.397063': -0.6611428394433744,
 '2008-05-13 03:34:08.300088': 0.8818855980807698,
 '2008-05-29 03:33:42.802038': 1.5357698713194057,
 '2008-08-17 03:31:25.372044': 11.92212005676924,
 '2008-09-02 03:30:59.871094': 11.059756461706323,
 '2008-09-18 03:30:32.576038': 2.6169723973001253,
 '2008-10-04 03:30:02.577069': 5.988259704857707,
 '2008-03-26 03:35:44.489050': -2.775081159532689,
 '2008-04-11 03:35:21.591006': -8.144765950244885,
 '2008-04-27 03:34:57.085019': -1.6036082437404684,
 '2008-05-13 03:34:31.988050': 2.331514483470801,
 '2008-05-29 03:34:06.490000': 3.130441019886231,
 '2008-08-17 03:31:49.060000': 12.672052805614983,
 '2008-09-02 03:31:23.702025': 7.587864107690894,
 '2008-09-18 03:30:56.263094': 2.938143820468848,
 '2008-10-04 03:30:26.407094': 7.91739221611026,
 '2008-03-17 03:41:19.647063': -12.050928826752019,
 '2008-04-18 03:40:34.116006': -6.550285845656855,
 '2008-05-04 03:40:08.849094': 0.012981532034047448,
 '2008-05-20 03:39:44.376063': 1.7513441793855091,
 '2008-06-05 03:39:18.731019': 2.649372937623598,
 '2008-08-08 03:37:27.542088': 7.704358974305403,
 '2008-08-24 03:37:01.158075': None,
 '2008-10-11 03:35:35.626063': 5.591433902670507,
 '2008-03-17 03:41:43.763025': -11.883663070439528,
 '2008-04-18 03:40:58.231075': -6.320195370018999,
 '2008-05-04 03:40:32.823000': 0.03986441068579523,
 '2008-05-20 03:40:08.349063': 3.3119884679361427,
 '2008-06-05 03:39:42.704019': 3.1543755199033785,
 '2008-08-08 03:37:51.658050': 8.662837684482106,
 '2008-08-24 03:37:25.131075': 11.737539878658307,
 '2008-09-09 03:36:58.995063': None,
 '2008-10-11 03:35:59.599056': 8.002721979782248,
 '2008-03-17 03:42:07.450081': -11.462854851894082,
 '2008-04-18 03:41:21.919031': -8.853613734994058,
 '2008-05-04 03:40:56.653031': -0.123668244622016,
 '2008-05-20 03:40:32.037025': 2.893248022677773,
 '2008-06-05 03:40:06.391088': 2.40845623234211,
 '2008-08-24 03:37:48.962006': 12.964435654991169,
 '2008-10-11 03:36:23.287006': 7.583617661291438,
 '2008-03-08 03:48:06.283031': -21.26168119223095,
 '2008-03-08 03:48:29.970081': -25.478197029856677,
 '2008-05-11 03:46:56.591006': 1.6019846758725047,
 '2008-05-27 03:46:31.522031': 2.026626756316314,
 '2008-08-15 03:44:13.211031': None,
 '2008-09-16 03:43:21.431013': None,
 '2008-10-02 03:42:51.890075': 7.115868544315346,
 '2008-02-28 03:54:51.803025': -11.321009198528843,
 '2008-03-31 03:54:09.980031': -1.0822387835853793,
 '2008-04-16 03:53:46.510075': -1.5663299279766005,
 '2008-05-02 03:53:21.409088': -4.630529257690994,
 '2008-05-18 03:52:56.937088': 3.4972005905826324,
 '2008-06-19 03:52:04.714031': -20.63250389253231,
 '2008-07-21 03:51:09.715031': 14.21140568049146,
 '2008-08-22 03:50:13.696019': -27.00484012297325,
 '2008-09-07 03:49:47.742019': 1.8990774894974454,
 '2008-10-09 03:48:49.005075': -9.031329712776248,
'2009-03-30 03:28:50.547943100': -7.42289434282184,
 '2009-04-15 03:28:55.657423300': -8.565639923485715,
 '2009-06-02 03:29:01.495381400': 1.4041713580522321,
 '2009-06-18 03:29:00.192757400': None,
 '2009-07-04 03:28:57.411324500': 7.963356100539524,
 '2009-07-20 03:28:53.178180300': 15.00923738730196,
 '2009-08-05 03:28:47.582268400': 15.760272383908038,
 '2009-10-24 03:28:52.331767300': 1.2832490491136108,
 '2009-03-30 03:29:14.423649100': -7.79690869736618,
 '2009-04-15 03:29:19.533765700': -4.892359270845477,
 '2009-06-18 03:29:24.071768800': 3.559166581888773,
 '2009-07-04 03:29:21.288282500': 4.564693639129198,
 '2009-07-20 03:29:17.053819800': 12.650103300747302,
 '2009-08-05 03:29:11.457918600': 15.245739590514164,
 '2009-10-24 03:29:16.211904400': 2.3639494295699697,
 '2009-03-30 03:29:38.299805600': -6.884797451935429,
 '2009-04-15 03:29:43.410836500': -3.395041823548935,
 '2009-05-01 03:29:46.966205700': -7.2632677596172925,
 '2009-05-17 03:29:48.970427600': 1.020221642347337,
 '2009-06-18 03:29:47.951208100': 4.339939719912919,
 '2009-07-04 03:29:45.165715900': 4.380318432949246,
 '2009-07-20 03:29:40.929977600': 13.866254716031014,
 '2009-08-05 03:29:35.334608700': 15.7988350120562,
 '2009-09-06 03:29:32.807265500': 9.09418271844019,
 '2009-10-24 03:29:40.092408': 3.383294959455669,
 '2009-02-17 03:34:41.403149100': -28.625571977644253,
 '2009-04-22 03:35:08.216880600': -1.052032116642241,
 '2009-05-08 03:35:11.118402200': -0.9231656407925228,
 '2009-05-24 03:35:12.366494700': -1.3680159041883941,
 '2009-06-09 03:35:11.962733200': -0.8391647760164869,
 '2009-06-25 03:35:09.995388': None,
 '2009-07-11 03:35:06.559257800': 8.985275313679127,
 '2009-08-12 03:34:55.761939200': 13.941289284624464,
 '2009-08-28 03:34:56.129707200': 10.074590070558823,
 '2009-10-15 03:34:53.877126': 1.8971414940748905,
 '2009-10-31 03:35:09.841692600': -0.8490863820264193,
 '2009-04-22 03:35:32.093950200': -3.4176731943214906,
 '2009-05-08 03:35:34.997501300': -1.2165494063729294,
 '2009-05-24 03:35:36.246867400': 0.3256989185365814,
 '2009-06-09 03:35:35.842789100': 1.0200954040665409,
 '2009-06-25 03:35:33.873574200': 3.940648318408345,
 '2009-07-11 03:35:30.435469900': 7.528035832785649,
 '2009-08-12 03:35:19.638775600': 14.138076274585769,
 '2009-08-28 03:35:20.008859300': 11.956107476915063,
 '2009-10-15 03:35:17.758193300': 0.6366050390483636,
 '2009-10-31 03:35:33.720907500': -3.968817225437066,
 '2009-04-06 03:35:51.531794': -0.7964824529486793,
 '2009-04-22 03:35:55.971796200': None,
 '2009-05-08 03:35:58.877130100': -1.508945749557957,
 '2009-05-24 03:36:00.127827500': 1.368414937291994,
 '2009-06-09 03:35:59.723288800': 3.652928524922139,
 '2009-06-25 03:35:57.752237700': 4.908702984556132,
 '2009-07-11 03:35:54.312234500': 10.356445038915927,
 '2009-08-12 03:35:43.516105200': 16.318827410140965,
 '2009-08-28 03:35:43.888306': 13.448762161166892,
 '2009-10-15 03:35:41.639428400': 4.159318372844844,
 '2009-10-31 03:35:57.600467': 0.48657323269400876,
 '2009-02-01 03:35:43.875071': -15.104846125987578,
 '2009-02-17 03:35:53.044956300': -23.751359201557165,
 '2009-04-22 03:36:19.850224100': -14.39983224381369,
 '2009-05-08 03:36:22.757599200': -2.133164399169524,
 '2009-05-24 03:36:24.009443500': 3.286080953698464,
 '2009-06-09 03:36:23.604396900': 5.231658664041237,
 '2009-06-25 03:36:21.631481700': 6.183150407897369,
 '2009-07-11 03:36:18.189359900': 12.513413833606961,
 '2009-08-12 03:36:07.394357200': 17.305489083577445,
 '2009-08-28 03:36:07.768632100': 13.823791302351633,
 '2009-09-13 03:36:06.996015500': 6.062446266481668,
 '2009-10-31 03:36:21.480496500': 3.7338087632300465,
 '2009-04-29 03:41:44.361056400': -0.5863587295089557,
 '2009-05-15 03:41:46.576795800': None,
 '2009-05-31 03:41:47.065788100': 3.349024760848918,
 '2009-07-02 03:41:43.336685700': 2.5599945141224882,
 '2009-07-18 03:41:39.280163500': 9.44638970903597,
 '2009-08-03 03:41:33.851591200': None,
 '2009-10-06 03:41:26.521433400': 7.752634493949653,
 '2009-04-29 03:42:08.239743': -0.12766215540813042,
 '2009-05-15 03:42:10.457200300': -2.0390481141199057,
 '2009-05-31 03:42:10.946905200': 4.344130089824023,
 '2009-07-18 03:42:03.156436900': 8.466604246036512,
 '2009-08-03 03:41:57.727863': 14.17700349103366,
 '2009-10-06 03:41:50.402066700': 7.405007964578963,
 '2009-11-23 03:42:27.101406': -9.746811686127835,
 '2009-03-12 03:42:16.269425800': -15.506916738618374,
 '2009-04-29 03:42:32.119002400': 0.13845646133642875,
 '2009-05-15 03:42:34.338337500': 0.4075833473002516,
 '2009-05-31 03:42:34.828478': 4.2979178958778705,
 '2009-06-16 03:42:33.717411700': 3.01871111091654,
 '2009-07-18 03:42:27.033446500': None,
 '2009-08-03 03:42:21.604896200': 14.11255263305375,
 '2009-08-19 03:42:18.510575900': 12.962219088220609,
 '2009-10-06 03:42:14.282939300': 7.702439676365671,
 '2009-02-15 03:47:49.590688500': -19.84541850637611,
 '2009-05-06 03:48:20.224813100': 1.527447504691136,
 '2009-05-22 03:48:21.694732': 1.908629194344485,
 '2009-06-07 03:48:21.499017100': None,
 '2009-06-23 03:48:19.702608700': 4.667243133293808,
 '2009-07-09 03:48:16.457988': 7.697643226957829,
 '2009-08-10 03:48:05.763940500': 16.87702883031687,
 '2009-09-11 03:48:04.918846700': 11.396187062863504,
 '2009-09-27 03:48:02.988030600': None,
 '2009-10-13 03:48:01.134287600': 7.5552991541220225,
 '2009-10-29 03:48:17.377514700': 4.570796820757616,
 '2009-02-15 03:48:13.472122300': -21.627772383234106,
 '2009-03-03 03:48:22.578045500': -10.323422290845869,
 '2009-04-04 03:48:36.380758200': -1.0215160674378205,
 '2009-04-20 03:48:41.008715800': -0.7665766687304529,
 '2009-05-06 03:48:44.104980700': 0.5278529750655376,
 '2009-05-22 03:48:45.576263400': 1.2362382840768227,
 '2009-06-23 03:48:43.582303800': 4.547795119205223,
 '2009-07-09 03:48:40.335609500': 8.353399601811807,
 '2009-08-10 03:48:29.641498600': 15.249404748903311,
 '2009-09-11 03:48:28.800575': 11.038427748306889,
 '2009-09-27 03:48:26.869674700': 6.287846869374919,
 '2009-10-13 03:48:25.016592800': 5.8648234971448865,
 '2009-10-29 03:48:41.257906600': 1.9344406730619619,
 '2009-03-10 03:54:36.941861800': -7.2561512237135,
 '2009-04-11 03:54:49.427937300': -0.48964517652271394,
 '2009-04-27 03:54:53.372802900': -8.268761244798668,
 '2009-05-13 03:54:55.795452100': 3.273686150166048,
 '2009-05-29 03:54:56.487115': -1.7716942177912158,
 '2009-08-01 03:54:43.969121200': 16.237663709611898,
 '2009-08-17 03:54:40.062836800': 5.548671590278446,
 '2009-09-02 03:54:40.121116800': 14.342123363579041,
 '2009-09-18 03:54:38.954029300': -17.063643199519245,
 '2009-02-18 03:24:32.079025': -28.281163003255195,
 '2009-03-22 03:25:16.381088': -17.29614647270347,
 '2009-04-07 03:25:36.606088': -22.55084938631589,
 '2009-04-23 03:25:55.135031': -1.1160493211092537,
 '2009-05-09 03:26:13.627094': None,
 '2009-05-25 03:26:31.416050': 0.054022661091961355,
 '2009-06-10 03:26:48.612019': 3.2831625365280313,
 '2009-06-26 03:27:06.148050': 6.425678655733393,
 '2009-07-12 03:27:22.463019': 8.883489826886567,
 '2009-07-28 03:27:37.452006': 6.413178391381984,
 '2009-08-13 03:27:51.148081': 6.450719817559932,
 '2009-08-29 03:28:06.440069': 10.071022351199318,
 '2009-09-14 03:28:19.955069': 6.333921105863205,
 '2009-09-30 03:28:31.778050': 4.26599635140677,
 '2009-10-16 03:28:41.739063': -7.46988319716979,
 '2009-02-18 03:24:55.909050': -25.98009984340642,
 '2009-03-22 03:25:40.212019': -16.635525755267597,
 '2009-04-07 03:26:00.437013': -17.544110360453228,
 '2009-04-23 03:26:18.965063': -2.2216144718574142,
 '2009-05-09 03:26:37.458031': -3.9330480422338763,
 '2009-05-25 03:26:55.246081': 0.9580954616554761,
 '2009-06-10 03:27:12.299081': 2.965295620955128,
 '2009-06-26 03:27:29.978088': 3.00260406499158,
 '2009-07-12 03:27:46.293056': 6.585769712948872,
 '2009-07-28 03:28:01.282044': 0.9322406546183973,
 '2009-08-13 03:28:14.979019': 2.5832833218350535,
 '2009-08-29 03:28:30.128038': 11.681947296575899,
 '2009-09-14 03:28:43.643038': 8.693879508669145,
 '2009-09-30 03:28:55.466019': 6.543325101649591,
 '2009-10-16 03:29:05.569094': -16.426323883413776,
 '2009-11-01 03:29:13.912056': -11.382207118483272,
 '2009-11-17 03:29:20.449038': -9.018949168040287,
 '2009-02-18 03:25:20.025013': -25.204087495695923,
 '2009-03-22 03:26:04.185025': -14.088637401357673,
 '2009-04-07 03:26:24.410019': -17.820320482412786,
 '2009-04-23 03:26:43.081031': -1.3729265964028246,
 '2009-05-09 03:27:01.574006': -0.8095883441889264,
 '2009-05-25 03:27:19.362050': 2.1483914923555467,
 '2009-06-10 03:27:36.415050': 4.027769466396137,
 '2009-07-12 03:28:10.266063': 11.330217625453114,
 '2009-07-28 03:28:25.255056': 8.331617363887347,
 '2009-08-13 03:28:39.094094': -2.926909829300786,
 '2009-08-29 03:28:54.244013': 13.078313282262478,
 '2009-09-14 03:29:07.759013': 9.562685036340259,
 '2009-09-30 03:29:19.581094': 6.984440927394842,
 '2009-02-09 03:30:29.596094': -17.231702634220945,
 '2009-02-25 03:30:53.019000': None,
 '2009-03-29 03:31:36.177075': -7.074814778314369,
 '2009-04-14 03:31:55.669013': -7.799191475149118,
 '2009-04-30 03:32:13.454056': -8.489472899131668,
 '2009-05-16 03:32:32.555019': -0.7321750704117167,
 '2009-06-01 03:32:49.308069': 0.3811741515624188,
 '2009-07-19 03:33:39.939094': 15.619403737175556,
 '2009-08-04 03:33:54.322063': 14.976126542507759,
 '2009-08-20 03:34:08.876000': None,
 '2009-09-05 03:34:23.309094': 3.7784793819465805,
 '2009-10-07 03:34:47.116019': 4.965893673877716,
 '2009-10-23 03:34:56.434044': 0.18552625865167055,
 '2009-02-09 03:30:53.427019': -15.280958863059302,
 '2009-03-13 03:31:39.002081': -16.861243912129975,
 '2009-03-29 03:32:00.008006': -7.66686060286285,
 '2009-04-14 03:32:19.642013': -7.27038541422192,
 '2009-05-16 03:32:56.385050': -1.2035874253989887,
 '2009-06-01 03:33:13.139000': 2.614034024349669,
 '2009-06-17 03:33:31.013056': None,
 '2009-07-19 03:34:03.770031': 12.44001682942085,
 '2009-08-04 03:34:18.295063': 14.694813454404978,
 '2009-08-20 03:34:32.706038': 11.663324849470738,
 '2009-09-05 03:34:47.140031': 8.701409681157356,
 '2009-09-21 03:34:59.967006': 6.0848574793105525,
 '2009-10-07 03:35:10.946050': 7.480750685821903,
 '2009-10-23 03:35:20.264069': 3.4244038997898536,
 '2009-11-08 03:35:27.689069': -12.347292027420567,
 '2009-02-09 03:31:17.400006': -14.81855124751448,
 '2009-02-25 03:31:40.964094': None,
 '2009-03-13 03:32:03.118050': -16.99903039741632,
 '2009-03-29 03:32:24.123075': -5.712601852264997,
 '2009-04-14 03:32:43.615013': -6.3938049186166985,
 '2009-04-30 03:33:01.400056': -0.1671865480891571,
 '2009-05-16 03:33:20.501019': -0.6624937672341615,
 '2009-06-01 03:33:37.112000': 3.5283945497627207,
 '2009-06-17 03:33:55.129031': 3.9721646956437064,
 '2009-07-03 03:34:12.142025': None,
 '2009-07-19 03:34:27.743038': 13.750879126692878,
 '2009-08-04 03:34:42.268069': 15.885814202829312,
 '2009-08-20 03:34:56.822013': 13.104664079301019,
 '2009-09-05 03:35:11.256006': 8.45262495602062,
 '2009-09-21 03:35:23.940013': 6.282306077566488,
 '2009-10-07 03:35:34.919044': 7.231666004982515,
 '2009-10-23 03:35:44.237063': 4.1164949112844536,
 '2009-11-08 03:35:51.805031': -7.07098729022745,
 '2009-11-24 03:35:57.926006': -13.00347181696894,
 '2009-02-09 03:31:41.230031': -12.95914100791325,
 '2009-02-25 03:32:04.652056': -18.38453021349346,
 '2009-03-13 03:32:26.806013': -15.992459994546541,
 '2009-03-29 03:32:47.811038': -5.9943276177762,
 '2009-04-14 03:33:07.302075': -3.86197807592539,
 '2009-04-30 03:33:25.088019': 0.27962584785835143,
 '2009-05-16 03:33:44.188081': 0.553616079636491,
 '2009-06-01 03:34:00.799063': 3.9013502165612532,
 '2009-06-17 03:34:18.817000': 4.639702948841183,
 '2009-07-03 03:34:35.829088': None,
 '2009-07-19 03:34:51.573075': 13.981958038966699,
 '2009-08-04 03:35:06.099006': 15.790711189915838,
 '2009-08-20 03:35:20.509081': 13.763337805058647,
 '2009-09-05 03:35:34.943075': 9.332162407649562,
 '2009-09-21 03:35:47.627075': 4.745389829503781,
 '2009-10-07 03:35:58.749075': 5.845488354223473,
 '2009-10-23 03:36:08.067094': 1.8124558807613016,
 '2009-11-08 03:36:15.492088': -1.5994827429687712,
 '2009-11-24 03:36:21.756025': -9.441504887992936,
 '2009-02-16 03:37:14.661013': -24.673481235440637,
 '2009-03-04 03:37:37.502031': -14.60406898753388,
 '2009-03-20 03:37:59.230013': -32.84982320694826,
 '2009-04-21 03:38:38.424019': None,
 '2009-05-07 03:38:56.771044': -0.009895493332537706,
 '2009-06-08 03:39:31.744050': 1.7650675139432839,
 '2009-06-24 03:39:49.476063': 4.029956844495472,
 '2009-07-10 03:40:05.918025': 10.629134225174488,
 '2009-07-26 03:40:21.065094': 14.58603119814575,
 '2009-08-11 03:40:35.054013': 15.443372944068601,
 '2009-08-27 03:40:50.002075': 11.090456881384267,
 '2009-09-12 03:41:03.839081': 10.188119580567372,
 '2009-09-28 03:41:15.820063': 7.650464333265223,
 '2009-02-16 03:37:38.634006': -24.03738839930851,
 '2009-03-04 03:38:01.618000': -16.11022304940939,
 '2009-03-20 03:38:23.203013': -26.135744448422336,
 '2009-05-07 03:39:20.744044': -1.7187028071497799,
 '2009-06-08 03:39:55.717050': 4.599439193187508,
 '2009-06-24 03:40:13.592044': 5.265609367216608,
 '2009-07-10 03:40:30.034000': 11.446022004148157,
 '2009-07-26 03:40:45.181069': 13.804918003734223,
 '2009-08-11 03:40:59.027019': 16.760698544854407,
 '2009-08-27 03:41:14.118050': 12.986897826188882,
 '2009-09-12 03:41:27.812088': 9.238611049952008,
 '2009-09-28 03:41:39.793069': 6.443837634873485,
 '2009-01-31 03:37:38.042063': -21.98261509613264,
 '2009-02-16 03:38:02.464031': -23.424905380510417,
 '2009-05-07 03:39:44.574075': -6.868499868494368,
 '2009-05-23 03:40:02.651056': None,
 '2009-06-08 03:40:19.405013': 3.264973978500771,
 '2009-06-24 03:40:37.280013': 2.5889078819934577,
 '2009-07-10 03:40:53.721069': 9.056603911591043,
 '2009-07-26 03:41:08.869038': 13.097881059953368,
 '2009-08-11 03:41:22.857056': 15.734388573252042,
 '2009-08-27 03:41:37.806019': 13.384480026846159,
 '2009-09-12 03:41:51.500056': 11.183123952936299,
 '2009-09-28 03:42:03.481031': 6.168550745829785,
 '2009-02-07 03:43:36.118069': -10.191110013196006,
 '2009-02-23 03:43:59.704013': -22.300101783001057,
 '2009-03-11 03:44:22.007038': -22.77251433179358,
 '2009-03-27 03:44:43.155006': None,
 '2009-04-12 03:45:02.926013': -4.586975986255524,
 '2009-05-14 03:45:39.804000': 3.1227004995210392,
 '2009-05-30 03:45:56.856050': -11.160265803411516,
 '2009-06-15 03:46:14.453019': None,
 '2009-07-01 03:46:31.769044': -10.437202881376715,
 '2009-07-17 03:46:47.633056': 3.7589329681438692,
 '2009-08-02 03:47:02.191081': 17.52364458960768,
 '2009-08-18 03:47:16.448094': 13.63378711302705,
 '2009-10-05 03:47:55.399094': 7.759163857431484,
 '2009-10-21 03:48:04.852050': 6.070912654979255,
 '2009-11-22 03:48:18.849006': -6.940306784798309,
 '2009-01-22 03:43:34.495044': -13.150674291107633,
 '2009-02-07 03:43:59.806025': -11.477421272324994,
 '2009-02-23 03:44:23.391069': -22.41165811619924,
 '2009-03-11 03:44:45.837069': -21.451597978128536,
 '2009-03-27 03:45:06.985038': -22.87020715617752,
 '2009-04-12 03:45:26.756044': -1.5545508639072454,
 '2009-04-28 03:45:44.690094': None,
 '2009-05-14 03:46:03.634031': 2.3852212876611194,
 '2009-05-30 03:46:20.544013': -2.7834457164507724,
 '2009-07-01 03:46:55.457013': 1.4234613338168978,
 '2009-07-17 03:47:11.321025': None,
 '2009-08-02 03:47:25.879050': 17.13770456528514,
 '2009-08-18 03:47:40.279031': 13.917181498334271,
 '2009-10-21 03:48:28.540013': 3.724687118168482,
 '2009-11-06 03:48:36.285081': -27.438646735676148,
 '2009-11-22 03:48:42.536056': -6.604658406155242,
 '2009-02-14 03:50:21.183094': None,
 '2009-03-02 03:50:44.191019': -6.6940717299962635,
 '2009-03-18 03:51:06.061094': -11.282753444615365,
 '2009-04-03 03:51:26.613075': -19.37357261837095,
 '2009-04-19 03:51:45.672000': -0.7416841160110027,
 '2009-05-05 03:52:03.748025': 1.347898710415111,
 '2009-05-21 03:52:22.110056': -2.3715449499753616,
 '2009-06-22 03:52:56.737056': 5.225888738352554,
 '2009-07-08 03:53:13.349075': None,
 '2009-07-24 03:53:28.780069': 15.602631988079315,
 '2009-08-09 03:53:42.751000': 7.714746286360413,
 '2009-08-25 03:53:57.718050': 2.7002117306275473,
 '2009-09-10 03:54:11.656019': -9.275701390906262,
 '2009-09-26 03:54:23.786056': 5.249635511243724,
 '2009-10-12 03:54:34.195050': 7.5499085440512745,
 '2009-10-28 03:54:43.094056': -9.369790734111424,
 '2009-11-13 03:54:49.948075': -1.9383121221414703,
 '2009-11-29 03:54:57.361056': -0.160049716498143,
'2010-03-17 03:30:43.700621900': -16.35787997989939,
 '2010-05-04 03:30:45.392718600': -5.037342361772064,
 '2010-05-20 03:30:41.027870800': -10.162263376252163,
 '2010-06-05 03:30:43.113545400': 1.008727836682423,
 '2010-06-21 03:30:44.609041200': 3.7403081994907175,
 '2010-07-07 03:30:47.527034700': 8.965853815207092,
 '2010-07-23 03:30:48.474372600': 16.690539487450316,
 '2010-08-24 03:30:50.156934300': 14.750548807115146,
 '2010-09-09 03:30:52.096302900': 11.375996213952613,
 '2010-03-17 03:31:07.576364800': -15.898244950834716,
 '2010-04-02 03:31:11.145678600': -9.577942373133746,
 '2010-05-04 03:31:09.271883800': -3.1845611545754227,
 '2010-06-05 03:31:06.993706300': 0.8857880852460397,
 '2010-06-21 03:31:08.487473300': 4.414439555486187,
 '2010-07-07 03:31:11.403318800': 2.0325650030415825,
 '2010-07-23 03:31:12.349641200': 10.992920023403842,
 '2010-08-24 03:31:14.035819400': 13.408624299907297,
 '2010-09-09 03:31:15.977182500': 11.184982024898497,
 '2010-03-17 03:31:31.452720800': -14.032595522999078,
 '2010-04-02 03:31:35.021770300': -7.7071452741117055,
 '2010-05-04 03:31:33.151676200': -0.4175618920093589,
 '2010-06-05 03:31:30.874635400': 3.0451412768772292,
 '2010-06-21 03:31:32.366477400': 6.1768737165341365,
 '2010-07-07 03:31:35.280081600': 1.2970984503491991,
 '2010-07-23 03:31:36.225563200': 13.522896991150898,
 '2010-08-24 03:31:37.915297100': 14.173115358535041,
 '2010-09-09 03:31:39.858459800': 11.989875869459267,
 '2010-05-27 03:36:53.078650400': -1.2198787810078746,
 '2010-06-12 03:36:54.069778600': 4.506520352657043,
 '2010-08-15 03:36:58.985602100': 10.68999517070269,
 '2010-08-31 03:37:02.055769400': 10.772082714244702,
 '2010-09-16 03:37:03.044450700': 8.807110387544222,
 '2010-10-18 03:37:10.494193700': 2.9620954479826347,
 '2010-03-08 03:37:15.113898900': -19.665513013656312,
 '2010-05-27 03:37:16.959418800': -6.828997099143365,
 '2010-06-12 03:37:17.949271900': 4.659093637901105,
 '2010-08-15 03:37:22.862918400': 10.80326882168128,
 '2010-08-31 03:37:25.935702900': 8.043362443713479,
 '2010-09-16 03:37:26.925435': 7.523381584816134,
 '2010-10-18 03:37:34.374215700': 3.787978390511045,
 '2010-11-19 03:37:56.353083': -13.98704254159285,
 '2010-03-08 03:37:38.991340700': -16.426075695052507,
 '2010-05-27 03:37:40.840808900': None,
 '2010-06-12 03:37:41.829342100': 3.546883026607153,
 '2010-07-30 03:37:46.839633500': 7.990506464082241,
 '2010-08-15 03:37:46.741211600': 11.378649505762075,
 '2010-08-31 03:37:49.815988100': 11.194871521350835,
 '2010-10-18 03:37:58.254635400': 4.356639871584447,
 '2010-11-03 03:38:06.347419': None,
 '2010-11-19 03:38:20.229454': -24.92062004287243,
 '2010-03-08 03:38:02.869302': -15.843701549864887,
 '2010-05-11 03:38:05.861047500': -2.7095717284516057,
 '2010-06-12 03:38:05.709948600': 5.881643251042794,
 '2010-07-30 03:38:10.716067100': None,
 '2010-08-15 03:38:10.619817': 11.857888592835995,
 '2010-08-31 03:38:13.697158200': 10.857482276542404,
 '2010-10-18 03:38:22.135688400': 3.8089274386188845,
 '2010-11-03 03:38:30.225981300': 1.641757787492538,
 '2010-04-16 03:43:33.342338': -5.781204075217026,
 '2010-06-19 03:43:29.621402300': -2.249509967426632,
 '2010-07-05 03:43:32.795111': 5.496077480910272,
 '2010-08-06 03:43:33.124603800': 7.404096258458411,
 '2010-08-22 03:43:35.277666': 8.546525498706142,
 '2010-09-07 03:43:37.470096400': 8.641882125398677,
 '2010-10-09 03:43:39.156594500': -0.9755903303800414,
 '2010-10-25 03:43:49.096810200': 3.4320251072285894,
 '2010-04-16 03:43:57.219666700': -6.47946308276904,
 '2010-05-18 03:43:50.333695800': -2.540357222956503,
 '2010-06-19 03:43:53.500565800': 0.8494357073095175,
 '2010-07-05 03:43:56.672057': 6.544553575164901,
 '2010-08-06 03:43:57.001357': 9.546220179033021,
 '2010-08-22 03:43:59.157029500': 10.873134281869534,
 '2010-09-07 03:44:01.351485700': 8.706303513947809,
 '2010-09-23 03:44:01.305928100': 8.549209549693607,
 '2010-10-09 03:44:03.038370': 5.553033290532704,
 '2010-10-25 03:44:12.976052': 3.6855558577919605,
 '2010-04-16 03:44:21.097803100': -5.774921335388843,
 '2010-05-02 03:44:19.120855': -1.2959752805835254,
 '2010-05-18 03:44:14.215481100': -5.48576399383049,
 '2010-06-19 03:44:17.380451400': 2.4595888798942735,
 '2010-07-05 03:44:20.549690200': 2.7914093231844856,
 '2010-08-06 03:44:20.878842900': 9.983084153994204,
 '2010-08-22 03:44:23.036844100': 10.613202022074598,
 '2010-09-07 03:44:25.233224800': 5.263060554085693,
 '2010-09-23 03:44:25.187927300': 9.078213216246139,
 '2010-10-09 03:44:26.920555100': 6.0039449599038655,
 '2010-10-25 03:44:36.856176200': 4.2373322561436915,
 '2010-05-09 03:50:04.235422700': 0.6996495778065377,
 '2010-05-25 03:50:02.149574600': 2.618467625839118,
 '2010-06-10 03:50:03.457195100': 2.6312089478648306,
 '2010-07-12 03:50:08.246737400': 6.698797370423856,
 '2010-08-29 03:50:11.179197600': 11.426437335962996,
 '2010-10-16 03:50:18.630423500': 5.904459194563424,
 '2010-11-01 03:50:26.932195100': 5.170700352783863,
 '2010-11-17 03:50:40.362115300': None,
 '2010-03-22 03:50:29.184917200': -11.207901115755028,
 '2010-05-09 03:50:28.116349800': 0.30959401396892194,
 '2010-05-25 03:50:26.031428': 1.8838213072667818,
 '2010-06-10 03:50:27.337998600': -0.3233621004994116,
 '2010-07-12 03:50:32.123628700': 5.4796767829119215,
 '2010-08-29 03:50:35.060175': 11.785787063735459,
 '2010-10-16 03:50:42.511573100': 5.850720991249293,
 '2010-11-01 03:50:50.810875500': 4.285573438451412,
 '2010-02-09 03:56:20.586088100': None,
 '2010-02-25 03:56:30.262797900': None,
 '2010-04-14 03:56:42.797678800': -14.407246503091015,
 '2010-04-30 03:56:41.147135200': 1.3978156361427383,
 '2010-05-16 03:56:36.586591900': None,
 '2010-06-01 03:56:37.765445500': 4.1479251430864394,
 '2010-07-03 03:56:41.908145600': 6.964484528009622,
 '2010-07-19 03:56:43.295262200': 11.986651979888807,
 '2010-08-04 03:56:42.746119400': 6.1593674234324025,
 '2010-08-20 03:56:44.238616200': 6.5173062998158295,
 '2010-11-24 03:57:20.066178200': -13.744359728837065,
 '2010-02-05 03:29:28.023063': -24.040111810252853,
 '2010-03-25 03:29:30.996044': -14.579175976358995,
 '2010-05-28 03:29:18.693081': 1.4331862545190521,
 '2010-06-13 03:29:14.119025': 5.437915463748227,
 '2010-07-15 03:29:05.168050': 4.4214979286277,
 '2010-09-01 03:28:51.371081': 11.820181212440664,
 '2010-09-17 03:28:46.601031': 8.325248882234,
 '2010-02-05 03:29:51.853088': -24.391984752783223,
 '2010-03-25 03:29:54.826050': -17.293396509252126,
 '2010-05-28 03:29:42.523088': 0.9000224613504023,
 '2010-06-13 03:29:37.806069': 4.617810522033659,
 '2010-07-15 03:29:28.998075': 1.2077638984339452,
 '2010-07-31 03:29:23.872081': 5.786931871032735,
 '2010-09-01 03:29:15.059031': 12.228966063164536,
 '2010-09-17 03:29:10.288094': 8.996646623579304,
 '2010-02-05 03:30:15.969050': -23.5115166017891,
 '2010-02-21 03:30:18.294088': -25.059152920848934,
 '2010-03-25 03:30:18.942000': None,
 '2010-05-28 03:30:06.639038': 0.23841832751032838,
 '2010-06-13 03:30:01.922019': 5.195006332170641,
 '2010-07-31 03:29:47.988025': 7.67834497475541,
 '2010-09-01 03:29:39.174088': 13.069874232394477,
 '2010-09-17 03:29:34.404063': 9.86363425439743,
 '2010-10-03 03:29:27.131044': 7.860938725791538,
 '2010-02-12 03:35:40.007088': -24.278390850883113,
 '2010-05-19 03:35:32.288069': 0.6239628275146516,
 '2010-06-04 03:35:26.834000': 1.0595113632712962,
 '2010-07-06 03:35:18.015031': 5.677726431366665,
 '2010-08-07 03:35:07.939000': None,
 '2010-08-23 03:35:03.833081': 13.264088521471022,
 '2010-09-08 03:35:00.335025': 10.214071136057505,
 '2010-02-12 03:36:03.838013': -23.283443789647976,
 '2010-05-19 03:35:56.118094': -0.45830805321279255,
 '2010-06-04 03:35:50.664006': 1.2085510477008552,
 '2010-06-20 03:35:46.490031': 2.999074482808066,
 '2010-07-06 03:35:41.845056': 5.052546181326854,
 '2010-08-23 03:35:27.664000': 12.21215792268318,
 '2010-09-08 03:35:24.165050': 8.306873073888156,
 '2010-01-27 03:36:24.742063': -25.91697607852229,
 '2010-02-12 03:36:27.953081': -22.28793949368129,
 '2010-05-19 03:36:20.234050': -1.5952895708174182,
 '2010-06-04 03:36:14.636088': 2.815888576483655,
 '2010-06-20 03:36:10.463013': 5.0449132413815,
 '2010-07-06 03:36:05.818050': 7.606601399682546,
 '2010-08-23 03:35:51.779056': 12.113296324898435,
 '2010-09-08 03:35:48.138044': 10.484853280718577,
 '2010-10-10 03:35:34.021081': 6.160362121472845,
 '2010-11-11 03:35:19.854088': 1.601091517343769,
 '2010-01-27 03:36:48.430013': -23.053636455184847,
 '2010-02-12 03:36:51.641038': -23.134030447243106,
 '2010-04-01 03:36:52.238025': -4.694429862927128,
 '2010-04-17 03:36:50.266075': -15.660802237491508,
 '2010-05-03 03:36:46.707056': -5.166999731043232,
 '2010-05-19 03:36:43.922006': None,
 '2010-06-04 03:36:38.324025': 3.8055042117340054,
 '2010-06-20 03:36:34.150056': 6.716801027207488,
 '2010-07-06 03:36:29.648075': 8.05364308874471,
 '2010-08-07 03:36:19.572031': None,
 '2010-08-23 03:36:15.467006': 11.587982175605722,
 '2010-09-08 03:36:11.968069': 11.313161885663396,
 '2010-10-10 03:35:57.709025': 6.753111186621104,
 '2010-11-11 03:35:43.685000': -6.313132291779602,
 '2010-02-19 03:42:15.407044': None,
 '2010-03-23 03:42:16.699094': -15.081252932887551,
 '2010-06-11 03:42:00.077069': -0.3416368701508063,
 '2010-08-14 03:41:40.405081': 5.617041279318183,
 '2010-08-30 03:41:37.221081': 13.007096114476946,
 '2010-09-15 03:41:32.788013': 9.10122655183222,
 '2010-10-01 03:41:25.796038': 4.211892709382879,
 '2010-10-17 03:41:16.490038': 0.9109614804165088,
 '2010-11-18 03:41:08.046038': 1.6426984596005723,
 '2010-02-19 03:42:39.380044': None,
 '2010-03-23 03:42:40.815038': -11.632395031273498,
 '2010-06-11 03:42:24.050056': -0.2936320987192569,
 '2010-08-14 03:42:04.378069': 10.956245387041763,
 '2010-08-30 03:42:01.194081': 13.051962742945479,
 '2010-09-15 03:41:56.761006': 9.735605593227747,
 '2010-10-01 03:41:49.769025': 6.206033747918296,
 '2010-10-17 03:41:40.463031': 4.553055190953572,
 '2010-02-03 03:43:00.854038': -22.42789218852726,
 '2010-02-19 03:43:03.068000': None,
 '2010-03-23 03:43:04.502081': -14.760620299224401,
 '2010-04-08 03:43:02.677050': -12.321245559405241,
 '2010-08-14 03:42:28.066019': 8.90123943103281,
 '2010-08-30 03:42:24.882038': 12.035445603824654,
 '2010-10-01 03:42:13.456075': 7.204539205115443,
 '2010-11-18 03:41:55.849044': 1.6625251575707534,
 '2010-01-25 03:48:45.918038': -21.17276878923446,
 '2010-03-30 03:48:50.601013': -5.506164865654216,
 '2010-05-01 03:48:44.754031': -2.0707649399650006,
 '2010-06-18 03:48:32.752063': -2.720389838118187,
 '2010-07-04 03:48:27.865006': 3.316001272607671,
 '2010-07-20 03:48:24.134056': 2.739769985724118,
 '2010-08-05 03:48:18.326006': 5.60393442829531,
 '2010-08-21 03:48:13.888056': 12.261547810713646,
 '2010-09-06 03:48:10.341038': 9.027362780527874,
 '2010-10-24 03:47:47.366044': None,
 '2010-01-25 03:49:09.605088': -16.868311634713436,
 '2010-03-30 03:49:14.288050': -5.937092800777749,
 '2010-04-15 03:49:12.508025': -7.012343748016319,
 '2010-05-01 03:49:08.441081': -1.2664951857152784,
 '2010-07-04 03:48:51.552063': 5.564378814740024,
 '2010-07-20 03:48:47.964075': 8.768058582856458,
 '2010-08-05 03:48:42.156025': 9.548967664747538,
 '2010-08-21 03:48:37.576013': 11.148997001572445,
 '2010-09-06 03:48:34.171063': 9.728022692528844,
 '2010-10-08 03:48:20.525050': None,
 '2010-10-24 03:48:11.054000': -7.952162786240101,
 '2010-01-16 03:55:18.412094': -17.55117548731431,
 '2010-02-01 03:55:22.080081': None,
 '2010-02-17 03:55:24.727019': -18.67423074762059,
 '2010-04-06 03:55:24.634075': -29.54124463843132,
 '2010-04-22 03:55:21.464000': -0.892504905554911,
 '2010-05-08 03:55:18.716000': -6.030767397507564,
 '2010-06-09 03:55:10.041063': 4.253261081831052,
 '2010-06-25 03:55:04.982075': 5.101538352155011,
 '2010-07-11 03:55:01.148088': 13.866158515826994,
 '2010-07-27 03:54:56.590038': 14.714689684429073,
 '2010-08-28 03:54:47.023006': 0.16587607322087866,
 '2010-09-13 03:54:42.879013': None,
 '2010-09-29 03:54:36.132044': 5.273496597392621,
 '2010-10-15 03:54:27.124069': 2.0723247770592224,
 '2010-10-31 03:54:18.974038': 4.438727962329209,
'2011-04-21 03:32:10.467343500': -1.168633181076229,
 '2011-05-07 03:32:11.289682300': -4.9744330014980065,
 '2011-05-23 03:32:09.678554600': 1.1934667542000683,
 '2011-06-08 03:32:11.192692300': 3.4418012631943955,
 '2011-06-24 03:32:07.795889': 5.077421758472471,
 '2011-07-10 03:32:06.530720100': 2.3127093356211055,
 '2011-07-26 03:32:04.331482200': 12.669983247218799,
 '2011-08-27 03:31:57.725997300': 14.458541897769061,
 '2011-09-28 03:31:49.271652400': 5.198020921211782,
 '2011-10-14 03:31:50.284400800': None,
 '2011-10-30 03:31:56.779099800': -4.671600432167067,
 '2011-04-05 03:32:36.323360400': -3.809329170735326,
 '2011-04-21 03:32:34.346061900': -1.2281304794277315,
 '2011-05-23 03:32:33.559759': 0.7833235199483265,
 '2011-06-08 03:32:35.072047700': 2.57642739941205,
 '2011-06-24 03:32:31.672503700': 1.7541306803477168,
 '2011-07-10 03:32:30.405427': 4.18333157708124,
 '2011-07-26 03:32:28.206673600': 8.912586494720918,
 '2011-08-11 03:32:23.268651100': 8.667579206447229,
 '2011-08-27 03:32:21.606586500': 13.92398984924186,
 '2011-09-28 03:32:13.152463100': 6.338970148813412,
 '2011-10-14 03:32:14.162941600': None,
 '2011-10-30 03:32:20.654819900': 3.0420381549245117,
 '2011-11-15 03:32:32.654842300': -8.460691898238013,
 '2011-04-05 03:33:00.200174400': None,
 '2011-04-21 03:32:58.225209200': -1.0043641883850871,
 '2011-05-23 03:32:57.441376': 1.2246596397668448,
 '2011-06-08 03:32:58.951800200': 2.437938235343497,
 '2011-06-24 03:32:55.549625700': 2.6542783989151233,
 '2011-07-10 03:32:54.280864800': 8.193447749288206,
 '2011-07-26 03:32:52.082734500': 9.195558925227504,
 '2011-08-11 03:32:47.147273400': 10.20620318577639,
 '2011-08-27 03:32:45.487890900': 0.5268055373066431,
 '2011-09-28 03:32:37.033935300': 5.273741304442992,
 '2011-10-30 03:32:44.531039100': 4.601944268284462,
 '2011-11-15 03:32:56.531235600': -2.310623902247375,
 '2011-03-27 03:38:20.887748700': -9.31500333413866,
 '2011-04-12 03:38:22.907757400': -1.764040218941971,
 '2011-05-14 03:38:20.261795100': 0.34092742950893795,
 '2011-05-30 03:38:21.826723600': 2.5798791605431313,
 '2011-06-15 03:38:21.005492300': 12.32828422067864,
 '2011-07-01 03:38:17.116531300': None,
 '2011-08-02 03:38:13.157004800': 12.871535733539394,
 '2011-08-18 03:38:09.865851100': 15.019109554515325,
 '2011-09-03 03:38:06.639570400': 11.043864072871825,
 '2011-09-19 03:38:02.052093200': 9.101285443863748,
 '2011-11-06 03:38:12.791498100': 2.9361183155946464,
 '2011-03-27 03:38:44.763136': -6.836076542216617,
 '2011-04-12 03:38:46.784658500': -2.5224611048188526,
 '2011-05-14 03:38:44.142533700': -0.2253105244422308,
 '2011-05-30 03:38:45.707182400': 2.8654827701474215,
 '2011-06-15 03:38:44.883380500': 4.158981341912796,
 '2011-07-01 03:38:40.992004800': 4.286639145873448,
 '2011-08-02 03:38:37.033048500': 10.25887684102022,
 '2011-08-18 03:38:33.745135300': 13.977784589966392,
 '2011-09-03 03:38:30.520864': 10.553178379195627,
 '2011-09-19 03:38:25.933452100': 9.285009379711335,
 '2011-10-21 03:38:29.821125300': 3.3903750872089744,
 '2011-11-06 03:38:36.667372900': 4.0975409436571635,
 '2011-02-23 03:39:03.050411100': -16.305072693700748,
 '2011-03-11 03:39:07.298466400': -9.268041159088318,
 '2011-03-27 03:39:08.639171600': -4.28739787466829,
 '2011-04-12 03:39:10.662187800': -1.7171373083117187,
 '2011-05-14 03:39:08.023964400': 0.8028089131958387,
 '2011-05-30 03:39:09.588191600': 3.638257602859378,
 '2011-06-15 03:39:08.762003': 5.182109133487148,
 '2011-07-01 03:39:04.868134300': 6.441785662291717,
 '2011-07-17 03:39:04.594378700': 7.722127513314364,
 '2011-08-02 03:39:00.909566': 9.45587543531951,
 '2011-08-18 03:38:57.624714100': 13.832139570479866,
 '2011-09-03 03:38:54.402809': 7.3148251925042995,
 '2011-09-19 03:38:49.815571700': 9.038092701071914,
 '2011-10-21 03:38:53.698630600': 5.883324385139024,
 '2011-11-06 03:39:00.543817100': 4.916563566424703,
 '2011-11-22 03:39:13.186100200': 0.2801369452477192,
 '2011-02-23 03:39:26.929305600': -15.616553240026622,
 '2011-03-27 03:39:32.515809100': -4.960382301933447,
 '2011-04-12 03:39:34.540443100': -0.07344734541324206,
 '2011-05-14 03:39:31.905830600': 1.6060115524225418,
 '2011-05-30 03:39:33.469673200': 4.332388820137284,
 '2011-06-15 03:39:32.640895300': 9.582290940019044,
 '2011-07-01 03:39:28.744724': 7.6608938231869335,
 '2011-07-17 03:39:28.470182800': 11.237018441728965,
 '2011-08-02 03:39:24.787088600': 14.043132410742466,
 '2011-08-18 03:39:21.505317800': 15.101025279978618,
 '2011-09-03 03:39:18.285271600': None,
 '2011-09-19 03:39:13.697988900': 9.310105127036294,
 '2011-10-21 03:39:17.576778400': 5.993569240318222,
 '2011-11-06 03:39:24.421038600': 4.660104061454807,
 '2011-11-22 03:39:37.064319700': 1.3632820710448703,
 '2011-04-03 03:44:57.703565700': -1.2584569288882739,
 '2011-05-05 03:44:57.113157300': -3.117414381754075,
 '2011-06-06 03:44:56.841066': 3.881462222306451,
 '2011-07-08 03:44:52.071115700': 5.537693723324613,
 '2011-07-24 03:44:50.311893700': 6.769690011431341,
 '2011-08-09 03:44:45.138951400': 10.729633445023582,
 '2011-09-10 03:44:38.581840500': 8.872141144637814,
 '2011-10-12 03:44:33.959615800': None,
 '2011-11-13 03:44:53.396634800': -10.779427101257072,
 '2011-04-03 03:45:21.580117900': -3.0853619259746603,
 '2011-04-19 03:45:19.655456': None,
 '2011-05-05 03:45:20.994219300': -1.9690240098680727,
 '2011-06-06 03:45:20.721285400': 5.016644623660131,
 '2011-07-08 03:45:15.946619200': 7.501124941758217,
 '2011-07-24 03:45:14.187473600': 11.127567443753028,
 '2011-08-09 03:45:09.017047400': 11.197060780982541,
 '2011-09-10 03:45:02.464163400': 9.547867524283514,
 '2011-10-12 03:44:57.838832600': 4.758471661152887,
 '2011-11-13 03:45:17.273120600': 0.8540204326393078,
 '2011-03-18 03:45:41.926189': -10.815547482817237,
 '2011-04-03 03:45:45.457533500': -1.9942867257744286,
 '2011-04-19 03:45:43.534837700': -1.1428537643437782,
 '2011-06-06 03:45:44.601995300': 3.5361210495046387,
 '2011-07-08 03:45:39.822667700': 5.717234543727507,
 '2011-07-24 03:45:38.063925100': 10.967011964778079,
 '2011-08-09 03:45:32.895712900': 8.70673197828538,
 '2011-09-10 03:45:26.346952400': 10.431726826187656,
 '2011-10-12 03:45:21.719070700': 2.6758691340708918,
 '2011-11-13 03:45:41.150419500': 0.4641607540385682,
 '2011-11-29 03:45:50.658130700': -5.014433259058349,
 '2011-03-25 03:51:29.471418200': -3.6432457558895996,
 '2011-04-10 03:51:32.566141300': -1.1935755310836327,
 '2011-04-26 03:51:31.672730400': 0.0005415188852683213,
 '2011-06-29 03:51:26.299040500': 3.7170354094908205,
 '2011-07-15 03:51:26.438087500': 10.68371215608317,
 '2011-07-31 03:51:23.180038600': 7.310095683214544,
 '2011-08-16 03:51:19.425113400': 9.69624652170198,
 '2011-09-01 03:51:16.671087500': 11.811468014035,
 '2011-09-17 03:51:11.565453800': None,
 '2011-10-03 03:51:11.083609700': 9.553045906814395,
 '2011-11-04 03:51:19.799770800': 5.410759999861301,
 '2011-11-20 03:51:33.376745800': -6.294596156443174,
 '2011-03-25 03:51:53.348099800': -8.53276051518351,
 '2011-04-10 03:51:56.444032600': -1.62010995875737,
 '2011-04-26 03:51:55.553535100': -0.4456105962207607,
 '2011-06-29 03:51:50.175814500': 5.803448023113723,
 '2011-07-15 03:51:50.313930500': 14.6169712255341,
 '2011-07-31 03:51:47.057061900': 9.20429513359462,
 '2011-08-16 03:51:43.305265500': 10.719140894665422,
 '2011-09-01 03:51:40.553585400': 10.070705266973183,
 '2011-09-17 03:51:35.448323': 6.8129809583177785,
 '2011-10-03 03:51:34.964681700': 7.716207828150516,
 '2011-10-19 03:51:38.110005600': -14.961511010418196,
 '2011-11-04 03:51:43.676924900': 4.076804241599262,
 '2011-11-20 03:51:57.254622800': -8.79405463373989,
 '2011-04-01 03:58:06.636762200': -2.808464600977541,
 '2011-04-17 03:58:05.802074300': -10.43485788272293,
 '2011-06-04 03:58:06.262051400': 5.392180925901337,
 '2011-09-08 03:57:48.832476700': 9.916291810444426,
 '2011-09-24 03:57:45.226813700': 6.140234601195976,
 '2011-10-10 03:57:42.912510100': 5.381286514874068,
 '2011-10-26 03:57:51.675195800': 2.7257680223614362,
 '2011-11-27 03:58:11.817013100': 1.066926718275856,
 '2011-04-29 03:28:13.165006': 0.6202077656576859,
 '2011-05-15 03:28:09.297050': -1.3031773226771473,
 '2011-05-31 03:28:03.794038': 2.598849917308867,
 '2011-06-16 03:27:53.840013': 7.369919201511371,
 '2011-07-02 03:27:47.762038': 8.366723781023184,
 '2011-07-18 03:27:41.688025': 6.5490287248076955,
 '2011-08-19 03:27:23.565031': 16.881623744056835,
 '2011-09-20 03:27:00.460081': 8.430952356295082,
 '2011-04-13 03:28:44.925038': -3.118131764621496,
 '2011-04-29 03:28:36.852056': -3.7602876659742503,
 '2011-05-15 03:28:33.127069': -1.0080060400850597,
 '2011-05-31 03:28:27.481081': 2.2662721369143006,
 '2011-06-16 03:28:17.670044': 5.516546805075591,
 '2011-07-02 03:28:11.592063': 6.166032328500528,
 '2011-07-18 03:28:05.518056': 2.9233905490521006,
 '2011-08-03 03:27:55.504013': 13.534807340700937,
 '2011-08-19 03:27:47.395050': 15.209687202719646,
 '2011-09-20 03:27:24.291000': 8.867928070980787,
 '2011-11-07 03:26:40.051069': 4.0539306118377665,
 '2011-04-13 03:29:08.898025': -1.9910999119842316,
 '2011-04-29 03:29:00.968025': -10.092871704997641,
 '2011-05-15 03:28:57.243025': 0.7128481584280817,
 '2011-05-31 03:28:51.597038': 2.6199921123543106,
 '2011-06-16 03:28:41.786019': 6.496550348779792,
 '2011-07-02 03:28:35.708025': 7.154528345060448,
 '2011-07-18 03:28:29.634019': 8.735711911587247,
 '2011-08-03 03:28:19.477006': 14.170039641085983,
 '2011-08-19 03:28:11.511006': 14.171113437659962,
 '2011-09-04 03:28:02.126013': 8.51469865553426,
 '2011-09-20 03:27:48.406056': 10.321592376878645,
 '2011-04-20 03:34:27.937025': -1.1995130741822115,
 '2011-05-06 03:34:22.270013': -1.3240141328963237,
 '2011-05-22 03:34:18.096050': -2.41065657281191,
 '2011-06-07 03:34:10.709069': 2.8764365120113387,
 '2011-06-23 03:34:02.554031': 3.564787633031279,
 '2011-07-25 03:33:48.472038': 10.851212693093853,
 '2011-08-10 03:33:39.329050': None,
 '2011-09-11 03:33:19.831081': 9.921807775962495,
 '2011-09-27 03:33:03.023088': 5.024133634540684,
 '2011-04-20 03:34:51.767044': -1.4597450989087983,
 '2011-05-06 03:34:46.100031': 0.26354639564652615,
 '2011-05-22 03:34:41.926069': 0.11250237356715259,
 '2011-06-07 03:34:34.539094': 2.8247073650558,
 '2011-06-23 03:34:26.384050': 2.2636072970427317,
 '2011-07-25 03:34:12.302056': None,
 '2011-08-10 03:34:03.159075': None,
 '2011-09-11 03:33:43.662000': 7.063655439525558,
 '2011-09-27 03:33:26.854013': 4.50251357107486,
 '2011-10-29 03:33:01.857006': 5.065287188382627,
 '2011-03-03 03:35:30.467063': -11.595086171733502,
 '2011-04-20 03:35:15.740038': -0.2908963511535932,
 '2011-05-06 03:35:10.073013': -0.6346862798697599,
 '2011-05-22 03:35:05.899050': 1.6416248640212798,
 '2011-06-07 03:34:58.655056': 4.14243761890007,
 '2011-06-23 03:34:50.500013': -0.4316452367283521,
 '2011-08-10 03:34:27.132075': 8.363604072889263,
 '2011-08-26 03:34:18.766069': 12.95222823610048,
 '2011-09-11 03:34:07.634088': 5.011408536119712,
 '2011-10-29 03:33:25.972056': 5.422000881184286,
 '2011-01-30 03:35:53.856038': -18.810018352753147,
 '2011-03-03 03:35:54.155000': -13.326274920743247,
 '2011-04-20 03:35:39.427088': -0.20032671753035391,
 '2011-05-06 03:35:33.760063': -6.785052109206403,
 '2011-05-22 03:35:29.729069': 2.533941689521027,
 '2011-06-07 03:35:22.343013': 5.441334624120729,
 '2011-06-23 03:35:14.187069': 1.9891694955594605,
 '2011-07-25 03:35:00.105063': None,
 '2011-08-10 03:34:50.963006': 9.957619636174021,
 '2011-08-26 03:34:42.454013': 9.215566586890494,
 '2011-09-11 03:34:31.322038': 8.295737608025664,
 '2011-09-27 03:34:14.657031': None,
 '2011-10-29 03:33:49.660000': -1.6605043352805962,
 '2011-03-26 03:41:10.067000': -11.762439265078905,
 '2011-05-13 03:40:54.918081': 1.3858964306359451,
 '2011-05-29 03:40:50.166081': 2.030982017896702,
 '2011-06-14 03:40:40.838044': 5.074361542357489,
 '2011-06-30 03:40:34.295069': 4.558067853012303,
 '2011-08-01 03:40:18.579013': 10.939835984106526,
 '2011-08-17 03:40:10.225063': 9.097648927590349,
 '2011-09-02 03:40:01.117019': None,
 '2011-09-18 03:39:48.000075': 7.979765955615531,
 '2011-10-04 03:39:27.983038': 8.065138176083948,
 '2011-02-22 03:41:42.353081': -13.822306240850448,
 '2011-03-26 03:41:34.039088': -5.4316246220023885,
 '2011-04-27 03:41:23.529069': -1.053528835754555,
 '2011-05-13 03:41:19.034038': 2.009081417050762,
 '2011-05-29 03:41:14.139069': 4.5033673202000575,
 '2011-06-14 03:41:04.811038': 6.328607785635872,
 '2011-06-30 03:40:58.268063': 6.412296755652494,
 '2011-07-16 03:40:52.221069': 11.873264782064568,
 '2011-08-01 03:40:42.552000': 12.506750772092353,
 '2011-08-17 03:40:34.198050': 13.630547223734542,
 '2011-09-02 03:40:25.232081': 4.796343948488288,
 '2011-09-18 03:40:12.116031': 8.790956919968064,
 '2011-10-04 03:39:51.956019': 6.65481154487051,
 '2011-11-05 03:39:28.624075': 5.241935788864502,
 '2011-02-22 03:42:06.041038': -13.762368275249047,
 '2011-03-26 03:41:57.727031': -5.894886659555651,
 '2011-05-13 03:41:42.721088': 1.7217062310232032,
 '2011-05-29 03:41:37.827013': 3.1497198472057666,
 '2011-06-14 03:41:28.498088': 5.012674253713842,
 '2011-06-30 03:41:22.098081': 5.863468318158048,
 '2011-07-16 03:41:15.909019': 13.451806508728152,
 '2011-08-01 03:41:06.382013': 11.658491556538646,
 '2011-08-17 03:40:58.028069': 13.691687386750175,
 '2011-09-02 03:40:48.920038': 5.781621178712722,
 '2011-09-18 03:40:35.803088': 8.451170554560083,
 '2011-10-04 03:40:15.786038': 7.170992357730233,
 '2011-11-05 03:39:52.312019': 4.82067392251849,
 '2011-04-02 03:47:44.818075': -1.7444514940712887,
 '2011-05-04 03:47:32.577050': None,
 '2011-05-20 03:47:28.309088': -19.882760053917085,
 '2011-06-05 03:47:21.505013': 4.448195622773322,
 '2011-06-21 03:47:12.844069': -17.756889793528014,
 '2011-08-08 03:46:49.807050': 10.688875387451075,
 '2011-08-24 03:46:41.494063': -5.975977666955632,
 '2011-09-09 03:46:30.932081': 9.370314298942931,
 '2011-09-25 03:46:15.105063': 4.204692837831547,
 '2011-10-11 03:45:57.935081': 8.533629093893138,
 '2011-10-27 03:45:49.585056': 6.377584914058053,
 '2011-04-02 03:48:08.506025': -2.1907857181962997,
 '2011-05-20 03:47:51.997044': -3.68043664122009,
 '2011-06-05 03:47:45.192069': 5.255529397160771,
 '2011-06-21 03:47:36.532025': None,
 '2011-08-08 03:47:13.495013': 11.14046423864335,
 '2011-08-24 03:47:05.182013': -7.361231314034894,
 '2011-09-09 03:46:54.620031': 9.64614840426522,
 '2011-09-25 03:46:38.793025': 8.672803135777839,
 '2011-10-11 03:46:21.765094': 7.360776233938323,
 '2011-10-27 03:46:13.273000': 5.902352998205551,
 '2011-04-09 03:54:17.617075': -0.6515301830582441,
 '2011-05-27 03:54:00.367025': 4.438458826006531,
 '2011-06-12 03:53:51.589056': 7.673955439911044,
 '2011-06-28 03:53:44.677000': -7.166988819210386,
 '2011-07-14 03:53:38.563031': 14.482753927748158,
 '2011-07-30 03:53:29.295094': None,
 '2011-08-15 03:53:20.700000': None,
 '2011-09-16 03:52:59.477031': 3.1602790133138967,
 '2011-10-02 03:52:40.258013': 6.695188670163089,
 '2011-10-18 03:52:29.925069': 6.650601999902749,
'2012-02-19 03:32:43.446476400': -23.339280467710818,
 '2012-03-06 03:32:40.651162300': -16.50216088880985,
 '2012-05-09 03:33:13.010569300': -2.704487497921245,
 '2012-05-25 03:33:10.077806800': None,
 '2012-06-10 03:32:56.906127800': 5.31260738199517,
 '2012-07-12 03:33:19.663716500': 12.263746438501231,
 '2012-07-28 03:33:29.421738700': 16.533962550392292,
 '2012-08-13 03:33:41.657091': 15.679669893263192,
 '2012-09-14 03:34:00.698848700': 10.55084261944778,
 '2012-09-30 03:34:10.914479800': 6.7846898971283895,
 '2012-10-16 03:34:19.599348500': -1.6596095919832061,
 '2012-02-19 03:33:07.322322100': -22.73176941236696,
 '2012-03-06 03:33:04.526027200': -16.36965016683942,
 '2012-05-09 03:33:36.892274100': -2.694211997263758,
 '2012-05-25 03:33:33.956224200': None,
 '2012-06-10 03:33:20.782726': 3.6575306018566276,
 '2012-07-28 03:33:53.300207800': 13.5115048445996,
 '2012-08-13 03:34:05.537767800': 15.109382970032012,
 '2012-09-14 03:34:24.578590900': 11.760927002436446,
 '2012-09-30 03:34:34.791826100': 7.292516442436252,
 '2012-10-16 03:34:43.475769600': 1.9651837426534409,
 '2012-11-17 03:34:56.581268300': -8.319633038293276,
 '2012-02-19 03:33:31.198690500': -19.738123831409677,
 '2012-03-06 03:33:28.401529700': -14.298182929421491,
 '2012-05-09 03:34:00.774465700': -1.5451075413709294,
 '2012-05-25 03:33:57.835334100': -0.682700706812745,
 '2012-06-10 03:33:44.659780300': 3.3809462500717453,
 '2012-07-28 03:34:17.179215300': 13.647885518476386,
 '2012-08-13 03:34:29.419330900': None,
 '2012-08-29 03:34:38.078485900': 9.922799952102833,
 '2012-09-14 03:34:48.458776400': 13.020006830514616,
 '2012-09-30 03:34:58.670100500': 5.347847142209858,
 '2012-10-16 03:35:07.352471800': None,
 '2012-11-17 03:35:20.460390200': -7.744548639387502,
 '2012-05-09 03:34:24.657238900': None,
 '2012-04-14 03:38:48.774699400': -5.244571917471739,
 '2012-04-30 03:39:11.432807300': -6.02280592823257,
 '2012-06-01 03:39:11.270227300': 3.7473813783808847,
 '2012-07-19 03:39:35.427576100': 14.340058065252272,
 '2012-08-04 03:39:45.222475500': None,
 '2012-08-20 03:39:56.769114900': None,
 '2012-09-05 03:40:07.106446900': 10.404032157866382,
 '2012-09-21 03:40:15.239253600': 8.395499899903966,
 '2012-03-13 03:39:15.932141600': -10.959062905540257,
 '2012-04-14 03:39:12.655244700': None,
 '2012-04-30 03:39:35.315272800': -1.9439948667246232,
 '2012-06-01 03:39:35.147049300': 2.2819509454437763,
 '2012-06-17 03:39:35.799603500': None,
 '2012-07-19 03:39:59.304694700': 11.671545886935947,
 '2012-08-04 03:40:09.102435800': None,
 '2012-08-20 03:40:20.650007400': 12.648535832836709,
 '2012-09-05 03:40:30.987088400': 12.0832996636971,
 '2012-09-21 03:40:39.118264900': 10.392003015949433,
 '2012-02-10 03:39:40.514645900': -17.85865735756054,
 '2012-02-26 03:39:41.670237100': -14.818314594567624,
 '2012-03-13 03:39:39.808411900': -7.930055544058258,
 '2012-03-29 03:39:40.690896200': -4.464253066330823,
 '2012-04-14 03:39:36.536524100': -4.956073545573695,
 '2012-04-30 03:39:59.198223900': -2.093369705362099,
 '2012-06-01 03:39:59.024448600': 2.4872854779772324,
 '2012-06-17 03:39:59.675822200': 0.1598610149560842,
 '2012-07-19 03:40:23.182530700': 10.81744428487007,
 '2012-08-04 03:40:32.982773': None,
 '2012-08-20 03:40:44.531859300': 12.353075291005743,
 '2012-09-05 03:40:54.868314800': 13.083960957377538,
 '2012-09-21 03:41:02.997511900': 10.941487711474672,
 '2012-11-08 03:41:24.829620300': 1.334825545750031,
 '2012-02-26 03:40:05.546435500': -12.564021837338537,
 '2012-03-13 03:40:03.685458300': -7.793604267112396,
 '2012-03-29 03:40:04.569871900': -8.129065233720652,
 '2012-04-14 03:40:00.418279100': -4.067936138362539,
 '2012-04-30 03:40:23.081891500': -2.0704377035959425,
 '2012-06-01 03:40:22.902427600': 3.5801912615451945,
 '2012-06-17 03:40:23.552503700': 4.950184534002344,
 '2012-07-03 03:40:33.497524': None,
 '2012-07-19 03:40:47.061042600': 10.417690599448411,
 '2012-08-20 03:41:08.414019100': 11.919134833050048,
 '2012-09-05 03:41:18.750199700': 12.48218137677761,
 '2012-09-21 03:41:26.877479700': 10.02570281846527,
 '2012-10-23 03:41:46.330528300': None,
 '2012-11-08 03:41:48.708368800': -5.634395569767418,
 '2012-11-24 03:41:56.825570700': None,
 '2012-02-01 03:45:23.917444800': -20.30491369220886,
 '2012-02-17 03:45:28.799659800': -17.9154263243563,
 '2012-03-04 03:45:26.924466900': -16.43936614810978,
 '2012-04-05 03:45:25.467084100': -3.3918841209868846,
 '2012-04-21 03:45:30.772758600': -1.133950256587094,
 '2012-05-07 03:45:55.993465600': 2.981183764767426,
 '2012-05-23 03:45:58.058031400': -0.6503990756153551,
 '2012-06-08 03:45:40.903183600': None,
 '2012-06-24 03:45:49.285377300': 0.7627867289819914,
 '2012-07-10 03:46:03.520546900': 7.6188406209049715,
 '2012-07-26 03:46:13.985336800': None,
 '2012-08-11 03:46:25.764154700': 16.885981648168936,
 '2012-08-27 03:46:34.790781600': None,
 '2012-09-12 03:46:45.423994700': 11.281216266728888,
 '2012-09-28 03:46:55.316733300': 11.3464170302959,
 '2012-02-01 03:45:47.796881900': -21.207744179266346,
 '2012-02-17 03:45:52.676279900': -16.38690521541234,
 '2012-03-04 03:45:50.799976100': -16.161687676827615,
 '2012-03-20 03:45:51.961245600': None,
 '2012-04-05 03:45:49.346807100': -3.119933362191703,
 '2012-04-21 03:45:54.655953400': -1.3303270473697346,
 '2012-05-07 03:46:19.875939100': 1.7730684018461713,
 '2012-05-23 03:46:21.937380400': 2.872679385958792,
 '2012-06-08 03:46:04.780531100': None,
 '2012-06-24 03:46:13.161208800': None,
 '2012-07-10 03:46:27.397459300': 12.55070727312791,
 '2012-07-26 03:46:37.864271200': None,
 '2012-08-11 03:46:49.645195600': 16.23769624059879,
 '2012-08-27 03:46:58.672630': 11.423481642783331,
 '2012-09-12 03:47:09.304336700': 10.85699542988191,
 '2012-09-28 03:47:19.195016800': 10.315598442858207,
 '2012-02-01 03:46:11.676829900': -21.663907921904837,
 '2012-02-17 03:46:16.553595600': -15.851492371904033,
 '2012-03-04 03:46:14.676117100': -12.929290811705211,
 '2012-04-05 03:46:13.227209200': -4.381474186725458,
 '2012-04-21 03:46:18.539756100': -8.253597629401089,
 '2012-05-07 03:46:43.758997200': 2.034261384575659,
 '2012-05-23 03:46:45.817453800': 2.7164867706783213,
 '2012-06-08 03:46:28.658518400': 3.2013663299110324,
 '2012-07-10 03:46:51.274853100': 12.023583671309117,
 '2012-08-11 03:47:13.527254700': 15.58070266109627,
 '2012-08-27 03:47:22.554735800': 11.353197158627758,
 '2012-09-12 03:47:33.185475900': 10.726841614275397,
 '2012-09-28 03:47:43.073814100': 7.995598494293819,
 '2012-11-15 03:48:04.968055400': 1.8582692176549842,
 '2012-03-11 03:52:00.717690800': -10.689185484097877,
 '2012-03-27 03:52:02.635388300': -0.683263839844757,
 '2012-04-12 03:51:57.649783800': None,
 '2012-05-30 03:52:23.522900400': None,
 '2012-06-15 03:52:20.278809': 4.83426254386971,
 '2012-07-01 03:52:30.009309400': 7.278007863729232,
 '2012-07-17 03:52:43.581167900': 8.585205048225285,
 '2012-09-03 03:53:15.068742800': 12.359972083407714,
 '2012-09-19 03:53:22.790940900': 12.059790887785917,
 '2012-10-05 03:53:32.875915800': 10.349239588915367,
 '2012-10-21 03:53:43.148689300': 4.855337863418799,
 '2012-03-11 03:52:24.594676800': -13.226517992988192,
 '2012-03-27 03:52:26.514383900': -0.9856697453602962,
 '2012-04-12 03:52:21.531543': -2.771990783083125,
 '2012-04-28 03:52:41.655238': -0.9368926037471601,
 '2012-05-14 03:53:02.845390300': 0.918296475262372,
 '2012-05-30 03:52:47.401345400': 2.7574781239652415,
 '2012-06-15 03:52:44.155874900': 5.711952199293961,
 '2012-07-01 03:52:53.886128400': 10.994541007798416,
 '2012-07-17 03:53:07.459232': 11.973524665378115,
 '2012-09-03 03:53:38.950671300': 11.672814910464917,
 '2012-09-19 03:53:46.671108900': 10.60078652157718,
 '2012-10-05 03:53:56.753649100': 9.773171089097445,
 '2012-10-21 03:54:07.026059200': 5.593658787495256,
 '2012-03-18 03:58:37.263286400': -11.287079261856716,
 '2012-04-03 03:58:35.651988500': -4.516357974350657,
 '2012-04-19 03:58:36.228072600': -3.1798033886358854,
 '2012-06-06 03:58:48.622302700': 0.3983322318519334,
 '2012-06-22 03:58:58.023439700': 6.597662751421824,
 '2012-07-08 03:59:11.070649600': 10.843006811299198,
 '2012-07-24 03:59:22.349539900': 10.538821908663504,
 '2012-08-09 03:59:33.650209': 13.805776026033763,
 '2012-08-25 03:59:43.339860': 13.26990930080354,
 '2012-09-10 03:59:53.948192700': 11.25447113554399,
 '2012-09-26 04:00:03.379635900': None,
 '2012-10-12 04:00:10.902181300': -2.4363983384676304,
 '2012-10-28 04:00:20.129918300': 5.194376003898822,
 '2012-11-13 04:00:25.277376500': -3.48443392606161,
 '2012-11-29 04:00:28.908319800': None,
'2013-04-01 03:36:33.783319': -14.321932489351575,
 '2013-04-01 03:36:57.564220': -9.02345160392157,
 '2013-04-11 03:36:36.244308': None,
 '2013-04-01 03:37:21.345122': -7.402184974480649,
 '2013-04-11 03:37:00.092985': None,
 '2013-04-18 03:40:23.372413': -5.731535373251515,
 '2013-05-04 03:40:23.223925': -1.7999400653392106,
 '2013-05-20 03:40:36.108798': -3.6644011714021643,
 '2013-06-05 03:40:37.513660': -2.1631147582316843,
 '2013-06-21 03:40:29.790719': 7.815687185266595,
 '2013-07-07 03:40:34.385641': 5.789309531688387,
 '2013-07-23 03:40:32.716585': 12.393303254124687,
 '2013-08-08 03:40:35.685948': 16.991886379009493,
 '2013-08-24 03:40:37.553718': 14.433756004292993,
 '2013-09-25 03:40:27.898246': None,
 '2013-10-11 03:40:27.976089': None,
 '2013-10-27 03:40:18.890800': 2.2141149501330024,
 '2013-11-12 03:40:18.738606': -8.430430628182503,
 '2013-04-18 03:40:47.254978': -5.514732247333109,
 '2013-05-04 03:40:47.110725': -1.141850858735406,
 '2013-05-20 03:40:59.995599': -7.139804556762111,
 '2013-06-05 03:41:01.400461': -0.5440388127025839,
 '2013-06-21 03:40:53.673284': 6.286331321698599,
 '2013-07-07 03:40:58.268206': 5.078088043312651,
 '2013-07-23 03:40:56.599150': 10.93967814957708,
 '2013-08-08 03:40:59.568513': 15.639150431644326,
 '2013-08-24 03:41:01.436283': 15.123135361166831,
 '2013-09-25 03:40:51.776575': None,
 '2013-10-11 03:40:51.858654': None,
 '2013-10-27 03:40:42.764893': 2.2241946906080137,
 '2013-11-12 03:40:42.612699': -1.2795928496943272,
 '2013-04-18 03:41:11.133308': -4.529897750029853,
 '2013-05-04 03:41:10.993290': 0.606211488443973,
 '2013-05-20 03:41:23.873928': -3.6348918509014694,
 '2013-06-05 03:41:25.278790': None,
 '2013-06-21 03:41:17.555849': 7.144061404810558,
 '2013-07-07 03:41:22.146535': 6.178647319205814,
 '2013-07-23 03:41:20.473243': 13.466224472514426,
 '2013-08-08 03:41:23.451078': 16.13784899427411,
 '2013-08-24 03:41:25.310376': 14.940994358886739,
 '2013-09-25 03:41:15.654904': 5.098028719705462,
 '2013-10-11 03:41:15.736983': None,
 '2013-10-27 03:41:06.643222': 1.3893839766562832,
 '2013-11-12 03:41:06.491028': -1.729905892762792,
 '2013-11-28 03:40:59.624732': 1.7741919754475273,
 '2013-07-07 03:41:46.020628': None,
 '2013-07-23 03:41:44.355808': 17.754911237583137,
 '2013-08-08 03:41:47.325171': 17.247085236374247,
 '2013-08-24 03:41:49.192941': 17.04740761502573,
 '2013-09-09 03:41:47.536270': None,
 '2013-09-25 03:41:39.537469': -4.111642722520594,
 '2013-10-11 03:41:39.619548': None,
 '2013-10-27 03:41:30.521551': None,
 '2013-11-12 03:41:30.365121': None,
 '2013-11-28 03:41:23.507297': 1.0951988700207351,
 '2013-05-27 03:46:48.962115': -0.20307539247869705,
 '2013-06-12 03:46:45.832950': 5.310814267130474,
 '2013-06-28 03:46:43.197512': 3.262389838068497,
 '2013-07-14 03:46:45.012560': 7.907961329753387,
 '2013-07-30 03:46:45.601955': 17.44145922178803,
 '2013-08-15 03:46:45.783993': 15.050696177226033,
 '2013-08-31 03:46:48.485833': 17.176413267169195,
 '2013-09-16 03:46:43.953419': 9.525590442648134,
 '2013-10-02 03:46:38.806612': 4.392786257868585,
 '2013-10-18 03:46:36.397210': 0.3639961239876277,
 '2013-11-19 03:46:24.638660': -4.556426338787149,
 '2013-05-27 03:47:12.848916': -0.6361333309451647,
 '2013-06-12 03:47:09.715515': 4.458479995804588,
 '2013-06-28 03:47:07.075840': 3.3494900919810555,
 '2013-07-14 03:47:08.886653': 4.7218938325715065,
 '2013-07-30 03:47:09.480284': 15.522238929513177,
 '2013-08-15 03:47:09.662322': 16.694550555501948,
 '2013-08-31 03:47:12.368398': 17.195370362717462,
 '2013-09-16 03:47:07.835984': 9.261359797072421,
 '2013-10-02 03:47:02.693413': 5.751196927841366,
 '2013-10-18 03:47:00.275539': 3.0543631177678368,
 '2013-11-03 03:46:54.320129': 4.332373011947837,
 '2013-11-19 03:46:48.516989': -1.400672714037234,
 '2013-05-27 03:47:36.727245': 0.8660452364517397,
 '2013-06-12 03:47:33.598080': 4.95887696493608,
 '2013-06-28 03:47:30.958405': 3.066890321339572,
 '2013-07-14 03:47:32.764982': 8.658777398963782,
 '2013-07-30 03:47:33.362849': 16.331385918392773,
 '2013-08-15 03:47:33.540651': 16.578406559321532,
 '2013-08-31 03:47:36.250963': 16.815175185540223,
 '2013-09-16 03:47:31.718549': 9.234490634407218,
 '2013-10-02 03:47:26.567506': 6.6236437944805875,
 '2013-10-18 03:47:24.153868': 4.025922156029829,
 '2013-11-03 03:47:18.198458': 4.863804441826771,
 '2013-11-19 03:47:12.395319': -2.3249790195734947,
 '2013-04-25 03:47:42.871288': -9.380974890791727,
 '2013-06-12 03:47:57.480645': 5.657924967008598,
 '2013-06-28 03:47:54.836734': None,
 '2013-07-14 03:47:56.643311': 10.575624405980015,
 '2013-07-30 03:47:57.236942': 16.643974551562955,
 '2013-08-15 03:47:57.427452': 14.508215632587241,
 '2013-08-31 03:48:00.133528': 15.62912674025318,
 '2013-09-16 03:47:55.609586': 1.0517650712037911,
 '2013-10-02 03:47:50.454307': 6.592571065868497,
 '2013-10-18 03:47:48.027961': 2.9413134746076754,
 '2013-11-03 03:47:42.072551': 5.088723060002888,
 '2013-11-19 03:47:36.265175': -4.348060416878036,
 '2013-12-05 03:47:34.745190': -0.10946189195964441,
 '2013-04-09 03:53:08.735060': -3.6390201843770766,
 '2013-04-16 03:53:09.521306': -3.574621872929298,
 '2013-05-18 03:53:20.569086': -2.15008113022384,
 '2013-06-03 03:53:23.439935': None,
 '2013-06-19 03:53:16.661928': 6.046156767499446,
 '2013-07-05 03:53:19.638895': 4.99884139660271,
 '2013-07-21 03:53:17.773471': 11.178673796312301,
 '2013-08-06 03:53:21.191995': None,
 '2013-08-22 03:53:22.699685': 14.635433038451959,
 '2013-09-07 03:53:21.979182': 0.032484077095879686,
 '2013-10-09 03:53:13.786980': 5.250859842437717,
 '2013-10-25 03:53:06.230796': None,
 '2013-11-10 03:53:04.947740': -0.5257936925219181,
 '2013-04-04 03:54:32.353953': -8.24123253089019,
 '2013-04-09 03:53:32.579502': -4.609453724075631,
 '2013-04-16 03:53:33.399635': -3.4245063389737678,
 '2013-06-03 03:53:47.326736': None,
 '2013-06-19 03:53:40.540257': 7.805236235596091,
 '2013-07-05 03:53:43.517224': 6.7880127372159045,
 '2013-07-21 03:53:41.647564': 12.422214759497782,
 '2013-08-06 03:53:45.070324': 10.203309288798179,
 '2013-08-22 03:53:46.578014': 13.78949494813514,
 '2013-09-23 03:53:38.662815': 5.115351344348832,
 '2013-10-09 03:53:37.661073': None,
 '2013-10-25 03:53:30.109125': 1.537320160364341,
 '2013-11-10 03:53:28.821832': -0.6469274603048922,
 '2013-11-26 03:53:20.752420': 0.3683791912747888,
 '2013-04-04 03:54:56.126382': -5.73943115031156,
 '2013-04-09 03:53:56.419707': -11.214339860276832,
 '2013-04-16 03:53:57.282200': -5.520484500382812,
 '2013-06-03 03:54:11.209301': None,
 '2013-06-19 03:54:04.418586': 7.979713342787565,
 '2013-07-05 03:54:07.395553': 5.588016325501324,
 '2013-07-21 03:54:05.525893': 12.553725834762147,
 '2013-08-22 03:54:10.460579': 13.218925484295232,
 '2013-09-07 03:54:09.740076': None,
 '2013-09-23 03:54:02.549616': 6.802347317602378,
 '2013-10-09 03:54:01.539402': None,
 '2013-10-25 03:53:53.987454': 1.254354949448963,
 '2013-11-10 03:53:52.700162': -1.9565516770469198,
 '2013-11-26 03:53:44.630749': -2.4800824062528286,
 '2013-12-12 03:53:42.911112': -3.0315865395591777,
 '2013-04-23 03:59:41.581597': -0.2760707761514081,
 '2013-06-10 03:59:56.103781': 5.439089790887699,
 '2013-07-28 03:59:54.537389': 12.885633610864485,
 '2013-08-13 03:59:55.502750': None,
 '2013-08-29 03:59:58.001681': 10.475730069017176,
 '2013-09-14 03:59:54.244030': None,
 '2013-09-30 03:59:47.796790': -0.65035459127447,
 '2013-10-16 03:59:46.721836': 6.882561094505008,
 '2013-11-01 03:59:39.317777': 5.172699001579592,
 '2013-11-17 03:59:35.891140': 2.735564699895595,
 '2013-04-23 04:00:05.468398': -4.5603492429230865,
 '2013-06-10 04:00:19.990582': 5.3316664080861775,
 '2013-06-26 04:00:15.715523': None,
 '2013-07-28 04:00:18.419954': 14.490216729103922,
 '2013-08-13 04:00:19.385315': 12.346451342775296,
 '2013-08-29 04:00:21.884246': 11.340071207879118,
 '2013-09-14 04:00:18.126595': 6.861092863164816,
 '2013-09-30 04:00:11.675119': 4.4845040963887755,
 '2013-10-16 04:00:10.600165': 5.911979945406895,
 '2013-11-01 04:00:03.196106': 4.468360452099639,
 '2013-11-17 03:59:59.765233': 3.08274977180254,
 '2013-12-03 03:59:56.520961': 1.2379843333573042,
 '2013-04-14 04:06:18.548725': None,
 '2013-04-30 04:06:16.478862': -5.241529502967232,
 '2013-05-16 04:06:28.649441': 2.507166832771588,
 '2013-06-01 04:06:33.104633': 2.831239892365758,
 '2013-06-17 04:06:27.284462': 7.961021596586246,
 '2013-07-03 04:06:28.647565': 8.730081717799797,
 '2013-07-19 04:06:27.857437': None,
 '2013-08-04 04:06:30.453604': 13.53761224129309,
 '2013-08-20 04:06:31.554804': None,
 '2013-09-05 04:06:31.846452': 13.153661263619759,
 '2013-10-07 04:06:23.251771': None,
 '2013-10-23 04:06:17.250149': -1.84826280442085,
 '2013-11-08 04:06:14.725945': 4.4911286516266875,
 '2013-11-24 04:06:05.593312': 0.250008461312771,
 '2013-12-10 04:06:05.665819': -15.755234070290118,
 '2013-03-09 03:34:43.616028700': -24.0311487298502,
 '2013-04-10 03:34:36.680952600': -6.6419424946310395,
 '2013-05-12 03:34:27.437077400': 0.385376714891095,
 '2013-05-28 03:34:22.633951500': 0.6485729928878032,
 '2013-06-13 03:34:15.256586600': 4.564993307054035,
 '2013-06-29 03:34:08.050953300': 5.759842842174836,
 '2013-07-15 03:33:58.031024200': 11.026882273167956,
 '2013-07-31 03:33:49.840681500': 19.112519117839582,
 '2013-08-16 03:33:52.492037100': 14.066258703502703,
 '2013-09-01 03:34:03.854433700': 17.304833147320835,
 '2013-09-17 03:34:14.990991800': 8.404873829984842,
 '2013-10-19 03:34:37.333840700': -0.9414920652861594,
 '2013-11-04 03:34:41.583422600': 1.4514140194964311,
 '2013-03-09 03:35:07.493422': -21.200327238911,
 '2013-04-10 03:35:00.561874900': -6.095639438217646,
 '2013-04-26 03:34:53.372558100': None,
 '2013-05-12 03:34:51.315073400': -2.134978722945401,
 '2013-05-28 03:34:46.509533600': 0.38689605031018165,
 '2013-06-13 03:34:39.132154900': 3.3947644943945425,
 '2013-07-15 03:34:21.910206400': 5.969378049821421,
 '2013-07-31 03:34:13.721640100': 17.286402447307257,
 '2013-08-16 03:34:16.372318200': 15.640420439195422,
 '2013-09-01 03:34:27.733380': 16.9974887129689,
 '2013-09-17 03:34:38.868169600': 7.658749523753746,
 '2013-10-19 03:35:01.211094700': 3.217679997441479,
 '2013-11-04 03:35:05.462619400': 1.4141959820190309,
 '2013-03-09 03:35:31.371489400': -17.23382923040121,
 '2013-04-10 03:35:24.443329': -4.638079759443102,
 '2013-04-26 03:35:17.253408300': None,
 '2013-05-12 03:35:15.193612400': -2.1963332375485405,
 '2013-05-28 03:35:10.385667300': 0.8858986105695968,
 '2013-06-13 03:35:03.008089100': 3.7885426094876595,
 '2013-07-15 03:34:45.790167500': 7.380218803262612,
 '2013-07-31 03:34:37.603211600': 14.516733492967353,
 '2013-08-16 03:34:40.253501700': 16.539074578281625,
 '2013-09-01 03:34:51.612742400': 17.455210524182537,
 '2013-09-17 03:35:02.745735700': 9.642171557318314,
 '2013-10-19 03:35:25.088779200': 4.1287479340537,
 '2013-11-04 03:35:29.342617600': 0.6117660864060339,
 '2013-11-20 03:35:33.009354700': -6.3909311142427025,
 '2013-02-28 03:40:54.119292900': -22.541412328053557,
 '2013-04-17 03:40:44.236188500': -6.486193575893298,
 '2013-05-03 03:40:40.262848900': -0.6212167611730263,
 '2013-05-19 03:40:37.826097900': -0.6818299059538553,
 '2013-06-04 03:40:27.697811300': -3.1111692971926526,
 '2013-06-20 03:40:23.877811900': 5.6395232526305055,
 '2013-07-06 03:40:13.401028100': 6.916906730602287,
 '2013-07-22 03:40:05.913667900': 13.493934242539831,
 '2013-08-23 03:40:08.298045100': None,
 '2013-09-08 03:40:20.212051900': 6.694251453220719,
 '2013-09-24 03:40:28.987713500': 5.8316793899196995,
 '2013-10-10 03:40:42.723861200': None,
 '2013-10-26 03:40:49.817377800': 2.392965603829558,
 '2013-02-28 03:41:17.995732900': -20.28381900748367,
 '2013-04-01 03:41:12.402059400': -11.171863482378864,
 '2013-04-17 03:41:08.116708': -5.121452554923462,
 '2013-05-03 03:41:04.142297500': 0.021741831689130825,
 '2013-05-19 03:41:01.702858': -0.6050865946338048,
 '2013-06-20 03:40:47.753410400': 5.228420202864157,
 '2013-07-06 03:40:37.278543800': 4.616581724780744,
 '2013-07-22 03:40:29.793484800': 9.317656151816998,
 '2013-09-24 03:40:52.863950700': 7.260642554735278,
 '2013-10-10 03:41:06.600454400': 3.7468304589082932,
 '2013-10-26 03:41:13.695255700': 0.9734810059610456,
 '2013-11-11 03:41:19.923243200': 1.8401406240865958,
 '2013-02-28 03:41:41.872754600': -16.94117589697262,
 '2013-03-16 03:41:40.878144': -6.500534656225137,
 '2013-04-01 03:41:36.283171800': -13.677980374360665,
 '2013-04-17 03:41:31.998060400': -3.681373791550538,
 '2013-05-03 03:41:28.022238800': 0.33981826709537294,
 '2013-05-19 03:41:25.580237800': 0.637001723199551,
 '2013-06-20 03:41:11.629853300': 6.535692426462138,
 '2013-07-06 03:41:01.156787600': 2.978560352014628,
 '2013-07-22 03:40:53.674032200': 12.194913424120827,
 '2013-09-24 03:41:16.740874100': 7.131624944214864,
 '2013-10-10 03:41:30.477761': 5.1398421558435885,
 '2013-10-26 03:41:37.573542700': None,
 '2013-11-11 03:41:43.803977': 2.570169974112911,
 '2013-02-28 03:42:05.750444200': -15.115034681970313,
 '2013-04-01 03:42:00.164699400': -15.558269207504921,
 '2013-04-17 03:41:55.879716': -4.610294580352752,
 '2013-05-03 03:41:51.902653500': 0.3787392491083003,
 '2013-05-19 03:41:49.458068200': 2.4966734112300326,
 '2013-06-04 03:41:39.324636900': 2.5497011214412617,
 '2013-06-20 03:41:35.506904100': 9.329373634156726,
 '2013-07-22 03:41:17.555281400': 12.826120524609811,
 '2013-08-23 03:41:19.938724100': None,
 '2013-09-24 03:41:40.618187400': 6.5736926934833475,
 '2013-10-10 03:41:54.355452100': 5.642762946053846,
 '2013-10-26 03:42:01.452530600': None,
 '2013-11-11 03:42:07.685235200': 0.8730676039461709,
 '2013-11-27 03:42:13.603477800': -21.474951633470766,
 '2013-03-07 03:47:29.236312500': -14.739920650322048,
 '2013-03-23 03:47:24.660820': -13.514002522756144,
 '2013-04-08 03:47:22.709950900': -3.049042058110722,
 '2013-04-24 03:47:14.651649200': -0.24206542978007112,
 '2013-05-10 03:47:12.673988600': -0.280249456645247,
 '2013-05-26 03:47:09.470966800': 2.421704981793653,
 '2013-06-11 03:47:01.187441200': 3.153784972907128,
 '2013-07-13 03:46:44.102532600': 6.254262320612425,
 '2013-07-29 03:46:36.455456300': 10.915487556451078,
 '2013-08-14 03:46:36.326146100': 15.970081200376274,
 '2013-08-30 03:46:47.575600900': 15.597524701664653,
 '2013-09-15 03:46:59.391373800': None,
 '2013-10-01 03:47:09.090650700': -0.517097509341713,
 '2013-11-18 03:47:30.944070400': -6.138303187220825,
 '2013-03-07 03:47:53.114203': -13.261118276217704,
 '2013-03-23 03:47:48.540709': -11.977896065438028,
 '2013-04-08 03:47:46.591424500': -3.635204058991138,
 '2013-04-24 03:47:38.532770900': -0.9838193001506553,
 '2013-05-10 03:47:36.552742700': -0.10526274313735,
 '2013-05-26 03:47:33.347386500': 1.9738395844469627,
 '2013-06-11 03:47:25.063487400': 4.83583526763058,
 '2013-06-27 03:47:18.770530100': None,
 '2013-07-13 03:47:07.982094600': 9.095262504016882,
 '2013-07-29 03:47:00.336540100': 11.418683862543025,
 '2013-08-14 03:47:00.207436200': 15.073715675243367,
 '2013-08-30 03:47:11.455405200': 15.350422279856234,
 '2013-09-15 03:47:23.269064900': 8.864492217130902,
 '2013-10-01 03:47:32.967677200': 6.532648177350286,
 '2013-11-02 03:47:49.406964800': 2.136979684249542,
 '2013-11-18 03:47:54.825122800': -3.9490753075960234,
 '2013-02-19 03:48:19.243829800': -20.714026116400415,
 '2013-03-07 03:48:16.992642900': -13.302858527441073,
 '2013-03-23 03:48:12.421191900': -9.7250444023943,
 '2013-04-08 03:48:10.473438900': -4.73833361853211,
 '2013-04-24 03:48:02.414466200': -0.3523202659791325,
 '2013-05-10 03:48:00.432189400': -1.0536572943012998,
 '2013-06-11 03:47:48.940247500': 5.226805214133162,
 '2013-06-27 03:47:42.648177500': 4.273801373478798,
 '2013-07-13 03:47:31.862476500': 10.092413576269333,
 '2013-07-29 03:47:24.218202100': 13.7300490828794,
 '2013-08-14 03:47:24.089026200': 12.325704591465453,
 '2013-08-30 03:47:35.335726400': 13.96378033591304,
 '2013-09-15 03:47:47.147355600': 9.150350623036946,
 '2013-10-01 03:47:56.845145200': 6.223460959742838,
 '2013-11-02 03:48:13.287287300': 2.890101031860383,
 '2013-11-18 03:48:18.706893300': 0.2812476470541656,
 '2013-02-10 03:54:08.013453400': -16.509213461063997,
 '2013-04-15 03:53:54.762533400': -4.236497603145003,
 '2013-05-01 03:53:49.944607100': -0.6301663424780751,
 '2013-05-17 03:53:47.587984300': None,
 '2013-06-02 03:53:38.904437': 3.589187657164423,
 '2013-06-18 03:53:34.004162400': 5.304905301387535,
 '2013-07-20 03:53:16.263993100': 7.942846546757942,
 '2013-08-05 03:53:10.446750600': -10.3980821682097,
 '2013-09-06 03:53:28.146072': 13.125593711927484,
 '2013-10-08 03:53:50.443348800': 1.768299717507924,
 '2013-10-24 03:53:58.968388600': 0.4920761236087884,
 '2013-11-09 03:54:04.732164300': 1.6377501541192674,
 '2013-11-25 03:54:09.930548': 2.029903513829297,
 '2013-02-10 03:54:31.890479600': -19.20320444581609,
 '2013-03-14 03:54:26.924779': -8.088455962953995,
 '2013-05-01 03:54:13.825358100': -0.2575105221221519,
 '2013-05-17 03:54:11.466056200': None,
 '2013-06-02 03:54:02.781003200': 3.3140721764641556,
 '2013-06-18 03:53:57.881140200': 5.865586979346811,
 '2013-07-04 03:53:48.324148100': 5.896775572435166,
 '2013-07-20 03:53:40.144935700': 12.71368863321251,
 '2013-09-06 03:53:52.025285100': 12.707612228480809,
 '2013-10-08 03:54:14.321118100': -6.691833710605589,
 '2013-10-24 03:54:22.847254900': 1.7837442756414223,
 '2013-11-09 03:54:28.613193500': -2.310961420295826,
 '2013-11-25 03:54:33.813514400': -1.0725126553798843,
 '2013-02-01 04:00:42.764714400': None,
 '2013-03-05 04:00:38.634279900': -1.1612644398637089,
 '2013-03-21 04:00:35.104043800': -13.544227766702976,
 '2013-04-06 04:00:32.440829300': None,
 '2013-04-22 04:00:25.067765200': -6.683311956680582,
 '2013-05-08 04:00:22.780149900': 1.0254939878580394,
 '2013-05-24 04:00:19.967966700': -5.671761390268262,
 '2013-06-09 04:00:10.904412800': 3.8567319486430076,
 '2013-06-25 04:00:05.503859700': 5.044425237829514,
 '2013-07-11 03:59:53.925400800': 9.946455364986225,
 '2013-07-27 03:59:47.141100600': 13.104185811042404,
 '2013-08-12 03:59:43.951547300': 11.314236850747742,
 '2013-08-28 03:59:55.091227600': 7.46431287933766,
 '2013-09-13 04:00:07.591730100': None,
 '2013-09-29 04:00:16.947239500': -26.908704466263508,
 '2013-10-15 04:00:30.287301800': 4.47453287226627,
 '2013-11-16 04:00:40.704204900': -8.659034358851756,
 '2013-12-02 04:00:48.988865100': 1.6411334323555204,
'2014-01-31 03:39:39.587779': -23.74606884396127,
 '2014-02-16 03:39:25.537847': -28.79295531597006,
 '2014-03-20 03:39:00.931233': -8.315881416511095,
 '2014-04-05 03:38:45.568221': 0.223006310540774,
 '2014-04-21 03:38:28.854995': -1.8473805170999857,
 '2014-05-07 03:38:12.554343': 0.017660494864752855,
 '2014-05-23 03:38:04.454991': 2.000104905349631,
 '2014-06-08 03:38:15.171553': 4.6259637699469005,
 '2014-06-24 03:38:17.712546': 6.283863425898216,
 '2014-07-10 03:38:26.096204': 16.293662062674812,
 '2014-07-26 03:38:28.482355': 16.382898516343293,
 '2014-08-11 03:38:37.754095': 15.397528359617088,
 '2014-08-27 03:38:39.553436': 16.524196994600903,
 '2014-09-28 03:38:42.301990': 5.545404363913867,
 '2014-10-14 03:38:48.529310': None,
 '2014-10-30 03:38:44.294114': -3.269812410002994,
 '2014-11-15 03:38:48.011691': -5.64940682591407,
 '2014-01-31 03:40:03.457636': -19.637171758434487,
 '2014-02-16 03:39:49.407704': -23.64101213944601,
 '2014-03-20 03:39:24.796854': -10.568625525211752,
 '2014-04-05 03:39:09.438078': -0.23534775597579924,
 '2014-04-21 03:38:52.724852': -0.16312847202605807,
 '2014-05-07 03:38:36.424200': -0.17649055524310664,
 '2014-05-23 03:38:28.337557': 1.1598104989734026,
 '2014-06-08 03:38:39.058354': 4.435175420658741,
 '2014-06-24 03:38:41.599347': 5.618562198557624,
 '2014-07-10 03:38:49.974533': 11.21045072294311,
 '2014-07-26 03:38:52.360684': 15.186576888782673,
 '2014-08-11 03:39:01.636660': 15.689099567094937,
 '2014-08-27 03:39:03.436002': 17.08141000987977,
 '2014-09-12 03:39:07.671211': None,
 '2014-09-28 03:39:06.188791': 6.584641436091375,
 '2014-10-14 03:39:12.407640': 1.126613673865102,
 '2014-10-30 03:39:08.172443': 2.7193328088568585,
 '2014-11-15 03:39:11.894256': -2.4848012554302827,
 '2014-01-15 03:40:31.995675': -7.736373251100501,
 '2014-01-31 03:40:27.335965': -16.86917452487815,
 '2014-02-16 03:40:13.286033': -19.518854094855467,
 '2014-03-20 03:39:48.670947': -5.050051353636061,
 '2014-04-05 03:39:33.316407': -0.018859945371163273,
 '2014-04-21 03:39:16.598945': 0.8172292847295995,
 '2014-05-07 03:39:00.302529': 1.19412962290981,
 '2014-05-23 03:38:52.220122': 2.5110698216093796,
 '2014-06-08 03:39:02.940919': 4.6569498043249435,
 '2014-06-24 03:39:05.477676': 7.589812052441675,
 '2014-07-10 03:39:13.865570': 6.842803756822226,
 '2014-07-26 03:39:16.247485': 12.86830645403509,
 '2014-08-11 03:39:25.523461': 14.984491724268825,
 '2014-08-27 03:39:27.322803': 16.869919019209934,
 '2014-09-12 03:39:31.553776': 10.83648461425386,
 '2014-09-28 03:39:30.071356': 7.325933371331518,
 '2014-10-14 03:39:36.294441': 2.044628951334515,
 '2014-10-30 03:39:32.055008': 3.4595411534076614,
 '2014-11-15 03:39:35.776821': -4.019289774039436,
 '2014-12-01 03:39:34.326483': -4.950423620339883,
 '2014-01-15 03:40:55.874004': -2.926015311195747,
 '2014-01-31 03:40:51.214294': -14.994048422962868,
 '2014-02-16 03:40:37.160126': -19.213582393185447,
 '2014-03-20 03:40:12.545040': -5.1331575257837585,
 '2014-04-05 03:39:57.194736': 0.9830223839550061,
 '2014-04-21 03:39:40.477274': 1.1947092162282364,
 '2014-05-07 03:39:24.176622': 0.38893776348754766,
 '2014-05-23 03:39:16.106923': None,
 '2014-06-08 03:39:26.823484': 5.637614957817993,
 '2014-06-24 03:39:29.360241': 11.915977095819866,
 '2014-07-10 03:39:37.743899': 9.709363528000559,
 '2014-07-26 03:39:40.130050': None,
 '2014-08-11 03:39:49.406026': 17.217285705307688,
 '2014-08-27 03:39:51.205368': 17.240333636582267,
 '2014-09-12 03:39:55.432105': None,
 '2014-09-28 03:39:53.953921': None,
 '2014-10-14 03:40:00.177006': 5.72460505552777,
 '2014-10-30 03:39:55.941809': 4.236310404714193,
 '2014-11-15 03:39:59.659386': 1.8845542006155143,
 '2014-12-01 03:39:58.209048': 0.6811310302421896,
 '2014-02-23 03:45:31.481835': -10.794782035411304,
 '2014-03-11 03:45:18.875842': -12.450076551131145,
 '2014-03-27 03:45:01.466507': None,
 '2014-04-12 03:44:48.700720': None,
 '2014-04-28 03:44:33.588204': 0.2572253322714274,
 '2014-05-14 03:44:19.722862': 0.808175128813464,
 '2014-05-30 03:44:20.599243': -4.663550229520541,
 '2014-06-15 03:44:28.244743': 4.953654563397007,
 '2014-07-01 03:44:31.768754': None,
 '2014-07-17 03:44:38.161977': 20.58278247357756,
 '2014-08-02 03:44:44.590326': 11.003737717256625,
 '2014-08-18 03:44:50.083366': 11.616704376019449,
 '2014-09-03 03:44:52.615825': 13.669700937764864,
 '2014-09-19 03:44:53.119215': 11.199834018405639,
 '2014-10-05 03:44:57.080572': 4.315812613383468,
 '2014-10-21 03:44:59.897756': -6.07683304999503,
 '2014-11-06 03:44:59.238288': -2.04857013530761,
 '2014-02-07 03:46:08.382834': -20.788612649943918,
 '2014-02-23 03:45:55.351692': -9.35713708066339,
 '2014-03-11 03:45:42.745699': -10.751742911142394,
 '2014-03-27 03:45:25.340600': -3.3014329125580644,
 '2014-04-12 03:45:12.574813': -6.817866070640467,
 '2014-04-28 03:44:57.462297': 1.2635435147124625,
 '2014-05-14 03:44:43.596955': 1.2352203268599948,
 '2014-05-30 03:44:44.477572': -1.0197474273821197,
 '2014-06-15 03:44:52.127308': 3.9676277022827353,
 '2014-07-17 03:45:02.044542': 19.174080421754944,
 '2014-08-02 03:45:08.472891': 15.25421244080946,
 '2014-08-18 03:45:13.965931': 14.120403003193186,
 '2014-09-03 03:45:16.498390': 13.903245559139929,
 '2014-09-19 03:45:16.993308': 11.479107117625418,
 '2014-10-05 03:45:20.967373': 7.239122875760777,
 '2014-10-21 03:45:23.776085': 1.4775855851936486,
 '2014-11-06 03:45:23.116617': 1.033051955621595,
 '2014-11-22 03:45:19.362116': -5.818347416103152,
 '2014-01-22 03:46:41.894297': None,
 '2014-02-07 03:46:32.256927': -20.958841439074913,
 '2014-02-23 03:46:19.221550': -10.221569116120582,
 '2014-03-11 03:46:06.619792': -8.187796965355117,
 '2014-03-27 03:45:49.218929': -1.5985471168137442,
 '2014-04-12 03:45:36.453142': -1.6169858872732725,
 '2014-04-28 03:45:21.336390': 0.7560138803820796,
 '2014-05-14 03:45:07.471048': 2.704429412283446,
 '2014-05-30 03:45:08.368609': 2.2887363735653916,
 '2014-06-15 03:45:16.009873': 4.533945009712572,
 '2014-07-17 03:45:25.922871': 14.401794381311914,
 '2014-08-02 03:45:32.359692': 11.74416485967374,
 '2014-08-18 03:45:37.852732': 15.761933132100124,
 '2014-09-03 03:45:40.385191': 12.560100667087612,
 '2014-09-19 03:45:40.880109': 11.494257390655253,
 '2014-10-05 03:45:44.849939': 7.249262284940729,
 '2014-10-21 03:45:47.658650': 4.925372774250324,
 '2014-11-06 03:45:46.994946': 1.5035987319462283,
 '2014-11-22 03:45:43.240445': -3.116934771956532,
 '2014-01-06 03:47:17.248787': -23.811218386188994,
 '2014-01-22 03:47:05.772626': -14.091193000092524,
 '2014-02-07 03:46:56.135256': -23.20747798701845,
 '2014-02-23 03:46:43.095643': -13.111723303389853,
 '2014-03-11 03:46:30.489649': -7.56638159849058,
 '2014-03-27 03:46:13.093022': -0.9111238684750377,
 '2014-04-12 03:46:00.327235': 1.0213440890973728,
 '2014-04-28 03:45:45.218956': 2.169499995376625,
 '2014-05-14 03:45:31.340905': 3.2354061523097397,
 '2014-05-30 03:45:32.251174': 5.360698138368212,
 '2014-06-15 03:45:39.888202': 6.921479461739801,
 '2014-07-17 03:45:49.809673': 14.126806775738858,
 '2014-08-18 03:46:01.735298': 16.416098104435385,
 '2014-09-03 03:46:04.267756': 11.026211749967429,
 '2014-09-19 03:46:04.762675': 11.633078743732748,
 '2014-10-05 03:46:08.728268': 5.9638508545897375,
 '2014-10-21 03:46:11.536979': 5.437326665095187,
 '2014-11-06 03:46:10.881747': 3.135550248638733,
 '2014-11-22 03:46:07.118774': None,
 '2014-12-08 03:46:05.236511': -19.465983566269006,
 '2014-01-29 03:52:26.271394': -8.65986650636211,
 '2014-02-14 03:52:11.810091': None,
 '2014-03-02 03:51:57.013506': -16.161028016409727,
 '2014-03-18 03:51:48.512131': -1.4800950072855072,
 '2014-04-03 03:51:32.556102': -1.836341940011873,
 '2014-04-19 03:51:15.321788': -7.942574941202649,
 '2014-05-05 03:50:59.517189': 0.20289879403794367,
 '2014-05-21 03:50:50.841672': -0.6423664994336232,
 '2014-06-22 03:51:03.332557': 9.09038857964013,
 '2014-07-08 03:51:10.834871': 7.55027893712383,
 '2014-07-24 03:51:13.000494': 16.7025199851183,
 '2014-08-09 03:51:22.627988': 11.93982442144997,
 '2014-08-25 03:51:25.210861': 8.701306655298428,
 '2014-09-10 03:51:29.187863': 12.070119142493597,
 '2014-10-12 03:51:33.373350': 6.244027907785252,
 '2014-10-28 03:51:31.277247': -8.796670326364408,
 '2014-11-13 03:51:34.210984': 2.8539869845981243,
 '2014-01-29 03:52:50.153959': -13.441400710669749,
 '2014-02-14 03:52:35.684184': -18.49414223067162,
 '2014-03-02 03:52:20.887600': -15.420775088765902,
 '2014-03-18 03:52:12.381988': -2.298347005780073,
 '2014-04-03 03:51:56.425959': -0.5212048650833319,
 '2014-04-19 03:51:39.204353': -6.942055426820644,
 '2014-05-05 03:51:23.391282': 1.767576694974527,
 '2014-05-21 03:51:14.728474': None,
 '2014-06-22 03:51:27.215122': 11.068058002085127,
 '2014-07-08 03:51:34.713201': 6.807049603723044,
 '2014-07-24 03:51:36.883059': 16.4388278278795,
 '2014-08-09 03:51:46.510553': 14.533127043595774,
 '2014-08-25 03:51:49.093426': 13.80388852388537,
 '2014-09-10 03:51:53.070428': 12.173561764372424,
 '2014-10-12 03:51:57.251679': 6.7199332767466124,
 '2014-10-28 03:51:55.155576': -8.24979431426166,
 '2014-11-13 03:51:58.093550': 2.8565773900972813,
 '2014-11-29 03:51:56.194126': None,
 '2014-01-13 03:53:20.225581': -12.39670371628908,
 '2014-01-29 03:53:14.032288': -15.236149794673036,
 '2014-02-14 03:52:59.562513': -16.402973552473462,
 '2014-03-02 03:52:44.757457': -13.671666879523624,
 '2014-03-18 03:52:36.256081': -1.92364868761206,
 '2014-04-03 03:52:20.300052': 0.44556145302303085,
 '2014-04-19 03:52:03.074210': -6.360196340958606,
 '2014-05-05 03:51:47.265375': 1.9653697089118671,
 '2014-05-21 03:51:38.615275': None,
 '2014-06-06 03:51:47.455299': None,
 '2014-06-22 03:51:51.101923': 10.654252043053368,
 '2014-07-08 03:51:58.595766': 9.393770424268022,
 '2014-07-24 03:52:00.769861': 15.234376539342627,
 '2014-08-09 03:52:10.397354': 13.986323849044258,
 '2014-08-25 03:52:12.980227': 14.135355974815527,
 '2014-09-10 03:52:16.944521': 10.05409382147451,
 '2014-09-26 03:52:13.797060': None,
 '2014-10-12 03:52:21.138480': 6.2394285637520825,
 '2014-11-13 03:52:21.976115': 3.3947119974717284,
 '2014-01-20 03:59:03.839674': -10.776697798041324,
 '2014-02-05 03:58:55.875885': -13.7776085544218,
 '2014-02-21 03:58:42.653209': -8.483298312830993,
 '2014-03-09 03:58:28.996766': None,
 '2014-04-10 03:58:00.621594': None,
 '2014-04-26 03:57:45.026436': 2.80318060657135,
 '2014-05-28 03:57:28.496251': 3.275128591298932,
 '2014-06-13 03:57:37.217634': 5.364189559591264,
 '2014-06-29 03:57:39.572393': None,
 '2014-07-15 03:57:47.388886': 12.192319557376958,
 '2014-07-31 03:57:52.643593': 15.677355368269039,
 '2014-08-16 03:57:59.153303': 18.130473097748123,
 '2014-09-01 03:58:01.168882': 13.01981133687434,
 '2014-09-17 03:58:03.175917': 12.20789022220135,
 '2014-10-03 03:58:05.743579': 9.743342469980478,
 '2014-10-19 03:58:09.450528': 6.462263740502477,
 '2014-11-04 03:58:07.803733': 3.3972060251762226,
 '2014-01-04 03:59:40.604914': -0.5192974176180433,
 '2014-01-20 03:59:27.722239': -6.285429102953887,
 '2014-02-05 03:59:19.745742': -13.972935353217057,
 '2014-02-21 03:59:06.531538': -10.721522571500067,
 '2014-03-09 03:58:52.875095': -11.873563765684256,
 '2014-03-25 03:58:38.125817': None,
 '2014-04-10 03:58:24.495687': -8.805356223170653,
 '2014-04-26 03:58:08.904765': 0.8419419485844665,
 '2014-05-12 03:57:54.310714': None,
 '2014-05-28 03:57:52.383052': 0.6059762260273135,
 '2014-06-13 03:58:01.095963': 6.3953574023237945,
 '2014-06-29 03:58:03.459194': 11.27260822194363,
 '2014-07-15 03:58:11.267215': 16.194239149418273,
 '2014-07-31 03:58:16.530394': 14.149062739017456,
 '2014-08-16 03:58:23.031632': 17.80452298844826,
 '2014-09-01 03:58:25.051447': 13.076953835895218,
 '2014-09-17 03:58:27.054246': 10.495962625390545,
 '2014-10-03 03:58:29.630380': 8.393319852476655,
 '2014-10-19 03:58:33.337329': 2.7537433662428956,
 '2014-11-04 03:58:31.690534': 1.7831343526551104,
 '2014-11-20 03:58:28.690051': None,
 '2014-12-06 03:58:28.379262': -6.621535079667073,
 '2014-01-11 04:05:44.356835': -3.4943074591442653,
 '2014-01-27 04:05:36.608845': -6.009911243816546,
 '2014-02-12 04:05:22.181045': -15.124461934175699,
 '2014-02-28 04:05:09.586909': -14.278157815808061,
 '2014-03-16 04:04:59.674999': None,
 '2014-04-01 04:04:43.075015': None,
 '2014-04-17 04:04:25.272887': None,
 '2014-05-03 04:04:11.612153': 0.9467881469189074,
 '2014-06-04 04:04:07.972508': 4.043800530436427,
 '2014-06-20 04:04:12.763006': 5.308163338347803,
 '2014-07-06 04:04:19.227234': None,
 '2014-07-22 04:04:22.413737': 15.639085159601308,
 '2014-08-23 04:04:34.623920': 10.41423717551662,
 '2014-09-08 04:04:38.253590': 12.543949242387631,
 '2014-09-24 04:04:34.716160': 10.987475681953157,
 '2014-10-26 04:04:42.083653': 4.145785054067282,
 '2014-11-11 04:04:43.975072': None,
 '2014-11-27 04:04:41.550442': None,
 '2014-12-13 04:04:34.936799': -14.017596005569674,
 '2014-02-08 03:35:27.432958900': -27.53174916171312,
 '2014-02-24 03:35:17.793633500': None,
 '2014-03-12 03:35:24.149183500': None,
 '2014-03-28 03:35:31.684537700': -9.371082761169223,
 '2014-04-29 03:35:47.907348': None,
 '2014-05-15 03:35:54.283290100': -1.2692330551038207,
 '2014-05-31 03:35:58.188036400': 1.6350346769016026,
 '2014-06-16 03:35:58.840582600': 4.93628387187572,
 '2014-07-02 03:36:05.330639600': None,
 '2014-07-18 03:36:06.869572': None,
 '2014-08-03 03:36:13.771411200': None,
 '2014-08-19 03:36:13.359802200': 15.976151546078947,
 '2014-09-04 03:36:13.712483800': -40.08243354217346,
 '2014-09-20 03:36:18.039541': 3.2385626754172896,
 '2014-10-06 03:36:30.349133600': 3.9565844016400633,
 '2014-11-07 03:36:48.987343500': -1.5078339937861753,
 '2014-02-08 03:35:51.309367700': -22.31121061165915,
 '2014-03-12 03:35:48.028187900': -19.278076881221903,
 '2014-03-28 03:35:55.564330900': -3.3422838107154536,
 '2014-04-29 03:36:11.786181600': None,
 '2014-05-15 03:36:18.160947': -0.5969054702492057,
 '2014-05-31 03:36:22.064698300': 2.9691925869301703,
 '2014-06-16 03:36:22.717039400': 4.088878778488226,
 '2014-07-02 03:36:29.208527900': 7.591547011156909,
 '2014-08-03 03:36:37.651528100': 15.173937898480345,
 '2014-08-19 03:36:37.239107600': 16.810039615509734,
 '2014-09-04 03:36:37.590674700': 1.9884704318231932,
 '2014-09-20 03:36:41.917524': 4.700003258295811,
 '2014-10-06 03:36:54.227497900': -0.30837258236938936,
 '2014-10-22 03:37:03.206666900': None,
 '2014-11-07 03:37:12.866677900': 2.215959460820186,
 '2014-01-23 03:36:06.978819200': -10.687307801817797,
 '2014-02-08 03:36:15.186644600': -17.411081502911795,
 '2014-03-12 03:36:11.907850300': -14.723416354840577,
 '2014-03-28 03:36:19.444583700': -1.6443839342474298,
 '2014-04-29 03:36:35.665703100': None,
 '2014-05-15 03:36:42.039077200': 1.7969198557985766,
 '2014-05-31 03:36:45.941826800': 1.3754245890940748,
 '2014-06-16 03:36:46.593860800': 5.226986063343119,
 '2014-07-02 03:36:53.087118200': 8.078906429512418,
 '2014-07-18 03:36:54.629244900': None,
 '2014-08-03 03:37:01.531986900': 14.577688703699659,
 '2014-08-19 03:37:01.118673800': 17.1088052250806,
 '2014-09-04 03:37:01.469058': 3.1373230684272357,
 '2014-09-20 03:37:05.796252900': 1.774057081353062,
 '2014-11-07 03:37:36.746583400': 3.074126855480897,
 '2014-11-23 03:37:39.620448900': 0.43583978751507807,
 '2014-02-15 03:41:31.597181500': -32.04325628857106,
 '2014-03-03 03:41:31.549642200': -20.91266740613934,
 '2014-03-19 03:41:39.685128800': -3.967757074207514,
 '2014-04-04 03:41:44.825574200': -2.3508484600673962,
 '2014-04-20 03:41:54.770089300': -5.534614993097576,
 '2014-05-06 03:41:58.885844200': 0.297628664669635,
 '2014-05-22 03:42:07.420626900': 1.8899706819455935,
 '2014-06-23 03:42:12.273518500': 9.718413844442168,
 '2014-07-09 03:42:17.354877200': 4.5581650040170425,
 '2014-07-25 03:42:21.243313100': 10.632356826114318,
 '2014-08-10 03:42:25.180948400': 15.292052240933506,
 '2014-08-26 03:42:22.274950600': 15.909230645764723,
 '2014-09-11 03:42:23.222605100': 11.389598383528604,
 '2014-09-27 03:42:33.147511700': 6.169275240786379,
 '2014-10-13 03:42:43.478852900': None,
 '2014-10-29 03:42:52.799068600': -4.033209365861061,
 '2014-11-14 03:43:01.030995800': -0.7292927740398655,
 '2014-01-30 03:42:00.549297300': -22.93441444733572,
 '2014-02-15 03:41:55.473976400': None,
 '2014-03-03 03:41:55.428035200': -19.06165054329108,
 '2014-03-19 03:42:03.564367800': -4.29162260556901,
 '2014-04-04 03:42:08.705365500': -2.830951442246992,
 '2014-04-20 03:42:18.649854500': -4.463250075598611,
 '2014-05-06 03:42:22.763849800': 0.3203716377990149,
 '2014-05-22 03:42:31.297603600': 0.9997842585459716,
 '2014-06-23 03:42:36.150847300': 6.138812549640811,
 '2014-07-09 03:42:41.233321300': 6.132545641008368,
 '2014-07-25 03:42:45.123159700': 10.701954123070063,
 '2014-08-10 03:42:49.061047300': 15.892442700878762,
 '2014-08-26 03:42:46.153466200': 14.575390518346982,
 '2014-09-11 03:42:47.099940100': 11.155675708943605,
 '2014-09-27 03:42:57.025700': 7.1058393298665194,
 '2014-10-13 03:43:07.357230400': 6.142881516498755,
 '2014-10-29 03:43:16.678308800': 1.7107737942399792,
 '2014-11-14 03:43:24.910436': 1.6523686887878177,
 '2014-01-30 03:42:24.426396300': -16.42407785344755,
 '2014-02-15 03:42:19.351520500': None,
 '2014-03-03 03:42:19.307236400': -16.982916764947582,
 '2014-03-19 03:42:27.444173200': -3.4340004402917526,
 '2014-04-04 03:42:32.585413200': -1.9500611267157573,
 '2014-04-20 03:42:42.530156100': -3.5221638630107748,
 '2014-05-06 03:42:46.642400300': 1.4100391597692479,
 '2014-05-22 03:42:55.175137900': 2.010707863920422,
 '2014-06-07 03:42:58.049641': None,
 '2014-06-23 03:43:00.028637': 8.585328084735265,
 '2014-07-09 03:43:05.112426500': 4.086781161498844,
 '2014-07-25 03:43:09.003815100': 10.496197620531705,
 '2014-08-10 03:43:12.941347900': 15.280839461689403,
 '2014-08-26 03:43:10.032697400': 15.065450169127374,
 '2014-09-11 03:43:10.977905500': 11.105396476650043,
 '2014-09-27 03:43:20.904787100': 8.077088396005019,
 '2014-10-13 03:43:31.235931700': 6.524522431812238,
 '2014-10-29 03:43:40.558438400': 3.328828418824621,
 '2014-11-14 03:43:48.790288500': 2.336373882361293,
 '2014-01-14 03:42:30.301457100': -6.1249547816985075,
 '2014-01-30 03:42:48.304195700': -13.463674744793988,
 '2014-02-15 03:42:43.229586700': None,
 '2014-03-03 03:42:43.186850900': -14.718453109132478,
 '2014-03-19 03:42:51.324693': -4.075292654012884,
 '2014-04-04 03:42:56.466291500': -0.6190124608804063,
 '2014-04-20 03:43:06.411144400': -1.9006215505796962,
 '2014-05-06 03:43:10.521638600': 1.8310795525363377,
 '2014-05-22 03:43:19.053182600': 3.5147263612879924,
 '2014-06-07 03:43:21.927211200': None,
 '2014-06-23 03:43:23.907288300': 11.276241929045305,
 '2014-07-09 03:43:28.991989900': -0.16504334273123894,
 '2014-07-25 03:43:32.884875200': 5.130640473294813,
 '2014-08-10 03:43:36.822359600': 15.332584103190703,
 '2014-08-26 03:43:33.912143800': 13.855172388712871,
 '2014-09-11 03:43:34.856283': 8.19164143416813,
 '2014-09-27 03:43:44.784412300': 8.285329130709767,
 '2014-10-13 03:43:55.115593': 6.52733041947583,
 '2014-10-29 03:44:04.438771': 3.414457677836401,
 '2014-11-14 03:44:12.670956300': 2.468692785131317,
 '2014-11-30 03:44:18.874540300': -3.3193821825038037,
 '2014-02-06 03:48:14.163527400': -15.700028884721672,
 '2014-02-22 03:48:02.969162900': -16.240159206007064,
 '2014-03-10 03:48:09.177570700': -8.168232780905122,
 '2014-03-26 03:48:15.691934700': -7.949111788970778,
 '2014-04-11 03:48:21.426634900': 0.037498506182121,
 '2014-04-27 03:48:32.968518600': 0.28973350940480386,
 '2014-05-13 03:48:38.664749100': 1.5381666475923128,
 '2014-05-29 03:48:43.110103400': None,
 '2014-06-14 03:48:44.769925300': 4.966872072398451,
 '2014-06-30 03:48:50.152952400': 9.927393253945821,
 '2014-07-16 03:48:51.278784300': 16.88639841627631,
 '2014-08-01 03:48:58.752930700': 11.323578837138378,
 '2014-08-17 03:48:59.208124600': 19.12794493331871,
 '2014-09-02 03:48:58.857170700': 12.770676426804632,
 '2014-09-18 03:49:02.330927400': None,
 '2014-10-04 03:49:14.589462100': 8.213133964039098,
 '2014-10-20 03:49:23.463467500': 3.816481191229238,
 '2014-11-05 03:49:33.449583800': 2.9657120843752045,
 '2014-01-21 03:48:26.359732500': -11.048735162314113,
 '2014-02-06 03:48:38.040576400': -18.71972715693957,
 '2014-02-22 03:48:26.847754700': -15.875878958547382,
 '2014-03-10 03:48:33.056675500': -8.543596891506924,
 '2014-03-26 03:48:39.572281900': -1.7175633432156439,
 '2014-04-11 03:48:45.307199700': 0.47048729840553716,
 '2014-04-27 03:48:56.848096100': 0.3542121027287759,
 '2014-05-13 03:49:02.543116200': 1.6376453952781171,
 '2014-05-29 03:49:06.987446700': None,
 '2014-06-14 03:49:08.646984800': 0.33477926907438993,
 '2014-06-30 03:49:14.031208900': 14.46409743888442,
 '2014-07-16 03:49:15.158647200': 15.158598670109939,
 '2014-08-01 03:49:22.633660100': 15.337466509082535,
 '2014-08-17 03:49:23.088129800': 19.005840887868867,
 '2014-09-02 03:49:22.736109400': 14.009496243841657,
 '2014-09-18 03:49:26.209681200': None,
 '2014-10-04 03:49:38.468386100': 6.623032802712013,
 '2014-10-20 03:49:47.342728300': 4.357579896401041,
 '2014-11-05 03:49:57.329665300': 2.1758669010458247,
 '2014-11-21 03:50:00.081707200': -1.1738627982986747,
 '2014-01-21 03:48:50.237840200': -8.703995710493029,
 '2014-02-06 03:49:01.918317100': -16.40553866065668,
 '2014-02-22 03:48:50.726932500': -15.096712122187279,
 '2014-03-10 03:48:56.936430800': -11.50579102833796,
 '2014-03-26 03:49:03.453255800': -2.2809221920416376,
 '2014-04-11 03:49:09.188309500': 0.4561091169655727,
 '2014-04-27 03:49:20.728417300': 1.0186841946052154,
 '2014-05-13 03:49:26.422040900': 1.3272401349620213,
 '2014-05-29 03:49:30.865144100': 2.171458301053375,
 '2014-06-14 03:49:32.524431400': None,
 '2014-06-30 03:49:37.910354800': 12.972021133524107,
 '2014-07-16 03:49:39.039408800': 12.877564723803784,
 '2014-08-01 03:49:46.514788100': 13.78957478124664,
 '2014-08-17 03:49:46.968414': 18.837907577928128,
 '2014-09-02 03:49:46.615213300': 13.981282322499082,
 '2014-09-18 03:49:50.088844100': 11.372780661092262,
 '2014-10-04 03:50:02.347867400': 2.3916379518600888,
 '2014-10-20 03:50:11.222697400': 4.420063351993602,
 '2014-11-05 03:50:21.210169100': 0.31412854617895797,
 '2014-11-21 03:50:23.962694': 0.9659829194744861,
 '2014-02-13 03:54:43.192519700': -13.312834212838675,
 '2014-03-01 03:54:39.307006900': -12.480560380132152,
 '2014-03-17 03:54:48.047284900': -3.66862219326365,
 '2014-04-02 03:54:53.998489400': -3.407321237003302,
 '2014-04-18 03:55:02.392648800': -10.096465269214809,
 '2014-05-04 03:55:08.441838800': 2.243817676844239,
 '2014-05-20 03:55:16.373993900': 2.523904370938518,
 '2014-06-05 03:55:19.430317200': 0.7400958993425831,
 '2014-06-21 03:55:20.625635100': 5.146202962704399,
 '2014-07-07 03:55:26.636753700': 7.215146296209217,
 '2014-07-23 03:55:29.675475900': 13.430247443093918,
 '2014-08-08 03:55:34.565490400': None,
 '2014-08-24 03:55:32.347990900': 10.620204647341163,
 '2014-09-09 03:55:34.294244200': 13.550339122003107,
 '2014-09-25 03:55:40.534702600': 11.311529327147653,
 '2014-10-11 03:55:52.534961300': 7.1496158267320595,
 '2014-10-27 03:56:01.589462300': 4.745613515495279,
 '2014-11-12 03:56:10.491148900': -2.676109769673147,
 '2014-01-28 03:55:08.265009700': -14.99061001838908,
 '2014-02-13 03:55:07.070491900': -15.98494399570778,
 '2014-03-01 03:55:03.186679100': -13.472700747801253,
 '2014-03-17 03:55:11.927800': -8.210735337314684,
 '2014-04-02 03:55:17.879343600': -1.9903466182134082,
 '2014-04-18 03:55:26.273798200': -5.120575349940643,
 '2014-05-04 03:55:32.321169': 1.1332103457122438,
 '2014-05-20 03:55:40.252362': 1.6009574333555219,
 '2014-06-05 03:55:43.307958900': 3.3399785463537435,
 '2014-06-21 03:55:44.504023300': 8.188578398988668,
 '2014-07-07 03:55:50.516458': 8.520870339500258,
 '2014-07-23 03:55:53.556728800': 16.26143447657526,
 '2014-08-24 03:55:56.228078800': 9.434587792338387,
 '2014-09-09 03:55:58.173087900': 12.648309251450542,
 '2014-09-25 03:56:04.413662700': 10.044336551095576,
 '2014-10-11 03:56:16.414512600': 5.186988924273577,
 '2014-10-27 03:56:25.469583600': 5.644520584056247,
 '2014-11-12 03:56:34.371548800': 3.2361425748076424,
 '2014-11-28 03:56:39.102221900': -2.9085333313535866,
 '2014-01-19 04:01:09.452754600': 0.786178398399931,
 '2014-02-04 04:01:24.065683600': -16.164661688560777,
 '2014-02-20 04:01:11.738698200': -13.695317202000423,
 '2014-03-08 04:01:18.128478600': -13.896720904838508,
 '2014-03-24 04:01:24.880761900': -0.28486443026139663,
 '2014-04-09 04:01:28.747132800': None,
 '2014-05-11 04:01:46.707819200': -21.394599177329827,
 '2014-05-27 04:01:51.963531900': -21.5929767157574,
 '2014-06-12 04:01:54.406387300': 6.466471146239727,
 '2014-06-28 04:01:58.722957200': 7.256360276040646,
 '2014-07-14 04:02:00.817627900': 15.56081058993822,
 '2014-07-30 04:02:07.470011100': 4.94473150244534,
 '2014-08-15 04:02:08.881798': 16.875533644368726,
 '2014-08-31 04:02:07.748644400': 13.013469246214676,
 '2014-09-16 04:02:10.330920900': 12.700318182882025,
 '2014-10-02 04:02:22.423643400': 8.740842934340451,
 '2014-10-18 04:02:31.156927400': -4.679609521672635,
 '2014-11-03 04:02:41.453316600': 4.376011939247796,
'2015-02-19 03:38:23.855532': -19.4814579949275,
 '2015-03-07 03:38:17.058443': -25.426106928876177,
 '2015-03-23 03:38:07.774381': -9.649936797721159,
 '2015-04-08 03:37:57.611254': -10.63859894631746,
 '2015-04-24 03:37:55.213011': None,
 '2015-05-10 03:37:39.328774': 1.121683165338293,
 '2015-05-26 03:37:38.946203': 2.199476731641713,
 '2015-06-11 03:37:50.049891': 5.272510626364477,
 '2015-06-27 03:37:55.482971': 4.92348176554914,
 '2015-07-13 03:38:06.820679': 3.103933682113574,
 '2015-07-29 03:38:11.735439': 19.822019582045275,
 '2015-08-14 03:38:16.992220': 14.468169980691078,
 '2015-08-30 03:38:23.189471': None,
 '2015-10-01 03:38:35.868860': 4.912165094006642,
 '2015-10-17 03:38:35.967502': 0.4718612983913577,
 '2015-11-18 03:38:42.929657': -5.240061217869374,
 '2015-02-19 03:38:47.733861': -18.245827699889478,
 '2015-03-07 03:38:40.928301': -20.755397035322268,
 '2015-03-23 03:38:31.652710': -7.767847961991729,
 '2015-04-08 03:38:21.485347': -14.273189471889832,
 '2015-04-24 03:38:19.087104': -12.540356006038692,
 '2015-05-10 03:38:03.202867': 1.1101137710882174,
 '2015-05-26 03:38:02.828768': 2.303589761153403,
 '2015-06-11 03:38:13.928220': 4.282083636394047,
 '2015-06-27 03:38:19.369772': 4.257839415221857,
 '2015-07-13 03:38:30.703244': 4.524359284892341,
 '2015-07-29 03:38:35.618004': 19.277767844564945,
 '2015-08-14 03:38:40.883257': 14.462404173474223,
 '2015-08-30 03:38:47.067800': 16.997886798249745,
 '2015-10-01 03:38:59.755661': 6.667481491951603,
 '2015-10-17 03:38:59.854303': 0.5569497023181906,
 '2015-11-02 03:39:05.694214': -0.8715903285707595,
 '2015-11-18 03:39:06.812223': -8.256461821973005,
 '2015-01-18 03:39:23.010146': -18.691773865336454,
 '2015-02-03 03:39:19.275664': -19.350186306430835,
 '2015-02-19 03:39:11.612190': -19.115756001741243,
 '2015-03-07 03:39:04.810866': -15.650856704379144,
 '2015-03-23 03:38:55.531039': -4.088663582811194,
 '2015-04-08 03:38:45.363677': -3.7501080140632617,
 '2015-04-24 03:38:42.969669': None,
 '2015-05-10 03:38:27.081196': 1.8755148069534218,
 '2015-05-26 03:38:26.715569': 3.56100898905144,
 '2015-06-11 03:38:37.819257': 3.7248356729242276,
 '2015-06-27 03:38:43.252337': 5.210244614555063,
 '2015-07-13 03:38:54.585810': 5.358852630348251,
 '2015-07-29 03:38:59.500569': 19.144039639226236,
 '2015-08-14 03:39:04.761587': 15.935525242548803,
 '2015-08-30 03:39:10.950365': 16.923824436613668,
 '2015-10-01 03:39:23.638227': 7.450206115400289,
 '2015-10-17 03:39:23.736869': 4.35145636432278,
 '2015-11-02 03:39:29.581016': -0.650470159616719,
 '2015-11-18 03:39:30.690552': -3.834164554543498,
 '2015-01-18 03:39:46.888475': -22.30733683187833,
 '2015-02-03 03:39:43.153993': None,
 '2015-02-19 03:39:35.490520': None,
 '2015-03-07 03:39:28.689196': -13.287516558264766,
 '2015-03-23 03:39:19.409368': -1.4874060292669486,
 '2015-04-08 03:39:09.237770': -17.702401408388276,
 '2015-04-24 03:39:06.843762': None,
 '2015-05-10 03:38:50.955289': 2.774065040228552,
 '2015-05-26 03:38:50.598134': 4.841078478678922,
 '2015-06-11 03:39:01.701822': 8.182115780627344,
 '2015-06-27 03:39:07.139138': 7.546663855262696,
 '2015-07-13 03:39:18.472611': None,
 '2015-07-29 03:39:23.387370': 19.978668286136994,
 '2015-08-14 03:39:28.644152': 18.33097898796826,
 '2015-08-30 03:39:34.832931': None,
 '2015-10-01 03:39:47.516556': None,
 '2015-10-17 03:39:47.615198': None,
 '2015-11-02 03:39:53.459345': None,
 '2015-11-18 03:39:54.573117': 0.06640651669771229,
 '2015-02-10 03:44:37.845384': -18.496808179307372,
 '2015-02-26 03:44:33.401910': -19.79580496338124,
 '2015-03-30 03:44:12.852645': -7.313701233849863,
 '2015-04-15 03:44:09.027995': -3.8800496441174452,
 '2015-05-01 03:43:59.732332': -1.4055951857335556,
 '2015-05-17 03:43:47.089456': -3.0538437438293218,
 '2015-06-02 03:43:53.722821': None,
 '2015-06-18 03:44:03.933628': 8.09008246118554,
 '2015-07-04 03:44:11.649075': 16.27218095596593,
 '2015-07-20 03:44:20.412890': 17.991513926031498,
 '2015-08-05 03:44:23.092652': 15.80737508825932,
 '2015-08-21 03:44:31.179151': 16.461904991748018,
 '2015-09-06 03:44:36.398554': 11.849566963588114,
 '2015-09-22 03:44:44.546476': 2.653305687439046,
 '2015-10-08 03:44:46.872047': 6.0538647892882915,
 '2015-10-24 03:44:50.500661': None,
 '2015-11-09 03:44:52.418285': -1.4056176817522827,
 '2015-01-25 03:45:06.643310': -23.798705333054098,
 '2015-02-10 03:45:01.719477': -17.865241293267555,
 '2015-02-26 03:44:57.276003': -18.036656007229745,
 '2015-03-14 03:44:45.573956': -13.779892968501887,
 '2015-03-30 03:44:36.722502': -5.690009568348562,
 '2015-04-15 03:44:32.897852': -1.6267745661201942,
 '2015-05-01 03:44:23.610661': -3.369543325637954,
 '2015-05-17 03:44:10.967785': -0.555940408168158,
 '2015-06-02 03:44:17.605386': None,
 '2015-06-18 03:44:27.811957': 7.090227740569778,
 '2015-07-04 03:44:35.531640': 11.106015643679639,
 '2015-07-20 03:44:44.299691': 17.006308567870473,
 '2015-08-05 03:44:46.979453': 15.016661548985253,
 '2015-08-21 03:44:55.061717': 17.00390010218571,
 '2015-09-06 03:45:00.281119': 13.659518577814024,
 '2015-09-22 03:45:08.424806': 8.651970978628635,
 '2015-10-08 03:45:10.750376': 6.7160582990479725,
 '2015-10-24 03:45:14.378990': None,
 '2015-11-09 03:45:16.300851': 0.7793866647090137,
 '2015-01-25 03:45:30.530112': -15.61606319559232,
 '2015-02-10 03:45:25.597807': -13.408115724594492,
 '2015-02-26 03:45:21.158568': -14.960086682196291,
 '2015-03-14 03:45:09.448050': -11.628217236124524,
 '2015-03-30 03:45:00.600831': -4.848880840183383,
 '2015-04-15 03:44:56.776182': -1.4664525134294295,
 '2015-05-01 03:44:47.484754': -4.775448004346567,
 '2015-05-17 03:44:34.858822': 0.7845332233765923,
 '2015-06-02 03:44:41.492187': 2.247127716921955,
 '2015-06-18 03:44:51.694522': 9.431950701827226,
 '2015-07-04 03:44:59.418441': 9.842009959640377,
 '2015-07-20 03:45:08.182256': 17.455722806179462,
 '2015-08-05 03:45:10.862019': 15.195821316296383,
 '2015-08-21 03:45:18.948518': 17.56340157367023,
 '2015-09-06 03:45:24.159448': 14.534245682470882,
 '2015-09-22 03:45:32.307371': 8.050092593273654,
 '2015-10-08 03:45:34.632942': 7.223850589619776,
 '2015-10-24 03:45:38.261555': -3.98190310278541,
 '2015-11-09 03:45:40.179180': -1.3473850232112505,
 '2015-11-25 03:45:42.721162': -1.0470301157422248,
 '2015-01-09 03:46:00.643286': -5.1623660235978095,
 '2015-01-25 03:45:54.408441': -16.443592994438497,
 '2015-02-10 03:45:49.480372': -10.618589695303056,
 '2015-02-26 03:45:45.032662': -13.113497832703812,
 '2015-03-14 03:45:33.330615': -6.649652681304432,
 '2015-03-30 03:45:24.483397': -4.096610644114761,
 '2015-04-15 03:45:20.658747': -1.925792457159621,
 '2015-05-01 03:45:11.358847': 0.5357577397056114,
 '2015-05-17 03:44:58.741387': 1.2975713206875992,
 '2015-06-02 03:45:05.374752': 4.0661765344270435,
 '2015-06-18 03:45:15.585559': 10.919533898953377,
 '2015-07-04 03:45:23.305242': 4.57014024531303,
 '2015-07-20 03:45:32.069057': 17.28656689021764,
 '2015-08-05 03:45:34.740348': 15.876701836542384,
 '2015-08-21 03:45:42.831083': 17.55203622211598,
 '2015-09-06 03:45:48.042013': 13.313077404234528,
 '2015-09-22 03:45:56.194172': 2.2455803229079216,
 '2015-10-08 03:45:58.519743': 7.0569552985200605,
 '2015-10-24 03:46:02.144120': 4.822902166196892,
 '2015-11-09 03:46:04.065981': -5.792900061028287,
 '2015-11-25 03:46:06.603728': -5.716790559096038,
 '2015-12-11 03:46:04.002776': -11.63187784179355,
 '2015-02-01 03:51:17.578010': -10.259383298894578,
 '2015-02-17 03:51:09.236674': -13.54668875708464,
 '2015-03-21 03:50:54.409024': -8.723870742781742,
 '2015-04-06 03:50:43.224668': -9.184491279417715,
 '2015-04-22 03:50:41.988894': -1.3018729104234348,
 '2015-05-08 03:50:26.232435': 2.035683670147599,
 '2015-05-24 03:50:24.112883': 2.712007525921447,
 '2015-06-09 03:50:34.288579': 3.481447046778679,
 '2015-06-25 03:50:39.814833': None,
 '2015-07-11 03:50:51.258049': 10.182876810608212,
 '2015-07-27 03:50:56.905402': 15.296997732535711,
 '2015-08-12 03:51:01.348567': None,
 '2015-08-28 03:51:08.265779': None,
 '2015-09-13 03:51:15.333122': 15.635618222842258,
 '2015-10-31 03:51:27.082195': -1.8531602834606271,
 '2015-11-16 03:51:27.755212': -18.508769262780124,
 '2015-01-16 03:51:45.703576': -13.502978621523534,
 '2015-02-01 03:51:41.460575': -10.288141742264164,
 '2015-02-17 03:51:33.115003': -11.042412245822113,
 '2015-03-05 03:51:28.083899': -9.669558833960636,
 '2015-03-21 03:51:18.287354': -8.126445381541627,
 '2015-04-06 03:51:07.098761': -6.896127960842066,
 '2015-04-22 03:51:05.858751': -0.8546741526616777,
 '2015-05-08 03:50:50.110764': 1.9008416835052555,
 '2015-05-24 03:50:47.999684': 3.3518893688797493,
 '2015-06-09 03:50:58.166909': 4.453490908744249,
 '2015-06-25 03:51:03.697398': 5.458902480860287,
 '2015-07-11 03:51:15.140614': 9.515237907416738,
 '2015-07-27 03:51:20.787967': 14.882975550839726,
 '2015-08-12 03:51:25.231133': None,
 '2015-08-28 03:51:32.148344': 12.759620385018508,
 '2015-09-13 03:51:39.215687': 15.94518273307584,
 '2015-09-29 03:51:44.967208': None,
 '2015-10-31 03:51:50.968997': -0.11163245282926144,
 '2015-11-16 03:51:51.637777': -6.466642845534952,
 '2015-12-02 03:51:53.184128': -0.3557602843976256,
 '2015-01-16 03:52:09.581905': -8.547301120330447,
 '2015-02-01 03:52:05.343141': -10.870161801823551,
 '2015-02-17 03:51:56.993332': -9.960156776291052,
 '2015-03-05 03:51:51.970700': -21.407891353092456,
 '2015-03-21 03:51:42.169919': -7.907243797049468,
 '2015-04-06 03:51:30.977091': -4.3921222937065,
 '2015-04-22 03:51:29.741317': -0.47912039700308945,
 '2015-05-08 03:51:13.984857': 1.951891053464369,
 '2015-05-24 03:51:11.886485': 0.5033219490154197,
 '2015-06-09 03:51:22.053710': 4.181685777266733,
 '2015-06-25 03:51:27.579963': 6.694251669020718,
 '2015-07-11 03:51:39.027416': 10.084785453053078,
 '2015-07-27 03:51:44.666296': 12.10600729432078,
 '2015-08-28 03:51:56.026673': None,
 '2015-09-13 03:52:03.098252': 16.47161110301284,
 '2015-10-15 03:52:08.376207': None,
 '2015-10-31 03:52:14.851562': -2.0416374323794333,
 '2015-11-16 03:52:15.520342': -6.353711220865679,
 '2015-12-02 03:52:17.062457': -1.464694088945622,
 '2015-01-23 03:57:52.020419': -7.349429560388823,
 '2015-02-08 03:57:48.913648': -12.187823658710604,
 '2015-02-24 03:57:43.436269': -9.500598700156573,
 '2015-03-12 03:57:32.081766': -7.663844924549316,
 '2015-03-28 03:57:24.195770': None,
 '2015-04-13 03:57:18.223032': -6.44699693910096,
 '2015-04-29 03:57:11.145525': 2.2797457150509515,
 '2015-05-15 03:56:55.355126': 1.2077413816775329,
 '2015-05-31 03:57:01.204552': 4.041864088958083,
 '2015-06-16 03:57:12.690109': -3.0182088769716553,
 '2015-07-02 03:57:19.350964': 10.490899012815065,
 '2015-07-18 03:57:29.148035': None,
 '2015-08-03 03:57:32.475586': 14.897127440626283,
 '2015-08-19 03:57:39.735864': 15.225407815976675,
 '2015-09-04 03:57:44.400215': None,
 '2015-09-20 03:57:53.234050': 10.509870026418477,
 '2015-10-06 03:57:56.334707': -2.6609045616650624,
 '2015-10-22 03:57:59.043793': 5.769574038717682,
 '2015-11-07 03:58:02.071195': 1.3118807213534927,
 '2015-11-23 03:58:04.176741': -10.979850076703546,
 '2015-01-07 03:58:22.222743': -11.540527792780525,
 '2015-01-23 03:58:15.898748': -4.893006010572026,
 '2015-02-08 03:58:12.796214': -12.4150712275442,
 '2015-02-24 03:58:07.314598': -13.531811958115236,
 '2015-03-12 03:57:55.960096': -9.864268690527787,
 '2015-03-28 03:57:48.074100': None,
 '2015-04-13 03:57:42.097125': -2.678961579111371,
 '2015-04-29 03:57:35.019618': 0.381880341148451,
 '2015-05-15 03:57:19.241927': 0.1650667604155532,
 '2015-05-31 03:57:25.091353': 3.424081372360334,
 '2015-06-16 03:57:36.572674': 4.15603829546164,
 '2015-07-02 03:57:43.237765': 14.717865218255579,
 '2015-07-18 03:57:53.034836': 7.435132670448388,
 '2015-08-03 03:57:56.353916': 15.357578306201184,
 '2015-08-19 03:58:03.614193': 14.455545530281249,
 '2015-09-04 03:58:08.278544': 10.424986997766016,
 '2015-09-20 03:58:17.116615': 8.386954336479263,
 '2015-10-06 03:58:20.221508': 3.744226248107991,
 '2015-10-22 03:58:22.930594': 6.081991419229398,
 '2015-11-07 03:58:25.949525': 3.053670922070406,
 '2015-11-23 03:58:28.059306': -7.650449791835567,
 '2015-12-09 03:58:26.136018': 0.8081709331092414,
 '2015-01-14 04:04:32.063546': -5.7698226041817255,
 '2015-01-30 04:04:27.218880': -2.3636248067061016,
 '2015-02-15 04:04:18.237739': None,
 '2015-03-19 04:04:04.744123': -7.480109439957903,
 '2015-04-04 04:03:52.496285': -5.99835809574403,
 '2015-04-20 04:03:52.255809': None,
 '2015-05-06 04:03:38.056505': -0.9424399963572248,
 '2015-05-22 04:03:33.060483': 4.735994394885047,
 '2015-06-07 04:03:42.193907': 3.5774004633483223,
 '2015-06-23 04:03:49.071240': 6.493917396081688,
 '2015-07-09 04:03:59.429860': 11.161699412365664,
 '2015-07-25 04:04:05.881785': 12.145862243985903,
 '2015-08-10 04:04:09.502756': None,
 '2015-08-26 04:04:17.085074': 12.044845879430692,
 '2015-09-11 04:04:23.635419': 9.309651648986744,
 '2015-09-27 04:04:30.072821': 3.489918466754202,
 '2015-10-13 04:04:30.391116': 9.566789413204813,
 '2015-10-29 04:04:36.103969': 6.083193339034437,
 '2015-11-14 04:04:36.285433': None,
 '2015-03-15 03:37:32.227641800': -19.917650151694982,
 '2015-03-31 03:37:38.395454100': -12.391897111430373,
 '2015-04-16 03:37:48.243805800': -1.0207438178993882,
 '2015-05-02 03:37:53.901722': -1.416525236229206,
 '2015-05-18 03:38:02.228420400': 0.5455519596516631,
 '2015-06-03 03:38:07.288202600': 3.123601576909508,
 '2015-06-19 03:38:11.790158300': 4.866472431281503,
 '2015-07-05 03:38:15.732631900': 8.496988884096563,
 '2015-07-21 03:38:20.012173': 19.358383876227855,
 '2015-08-06 03:38:23.567538700': 17.46062101130508,
 '2015-08-22 03:38:25.957679300': 15.83865275854883,
 '2015-09-23 03:38:32.747634400': 6.438242198889817,
 '2015-10-09 03:38:39.636702': 5.002683251034709,
 '2015-10-25 03:38:57.076950400': None,
 '2015-11-10 03:39:12.049489': -3.404122799893017,
 '2015-01-26 03:37:35.906772200': -22.43442667976504,
 '2015-03-15 03:37:56.106832700': -14.919033104315215,
 '2015-03-31 03:38:02.274448900': -5.787133295792924,
 '2015-04-16 03:38:12.122780400': -1.3178637584332948,
 '2015-05-02 03:38:17.780046500': -3.7723144708357386,
 '2015-05-18 03:38:26.106113': 0.4352482608420923,
 '2015-06-03 03:38:31.165578900': 2.3171035518656034,
 '2015-06-19 03:38:35.667616200': 5.282484499829515,
 '2015-07-05 03:38:39.611019400': None,
 '2015-07-21 03:38:43.890970800': 17.87546788794755,
 '2015-08-06 03:38:47.446540100': 17.156638793000266,
 '2015-08-22 03:38:49.835958': 16.446875297856305,
 '2015-09-23 03:38:56.625275': 7.142069118330452,
 '2015-10-09 03:39:03.514175400': 5.411897367921512,
 '2015-10-25 03:39:20.955381500': None,
 '2015-11-10 03:39:35.929538700': 0.5787855463790434,
 '2015-01-26 03:37:59.784796400': -18.29363267991298,
 '2015-02-27 03:38:13.544174600': -17.449794561610783,
 '2015-03-15 03:38:19.986550900': -10.282020891860048,
 '2015-03-31 03:38:26.153791400': -4.5889018430001824,
 '2015-05-02 03:38:41.658731100': -6.597104576151901,
 '2015-05-18 03:38:49.984207800': 1.9267096080067987,
 '2015-06-03 03:38:55.043476800': None,
 '2015-06-19 03:38:59.545691400': 8.41629292496407,
 '2015-07-05 03:39:03.490162': 4.7215032569048665,
 '2015-07-21 03:39:07.770194': 17.687279616608908,
 '2015-08-06 03:39:11.325895600': 16.59110172799297,
 '2015-08-22 03:39:13.714863400': 17.22781981611036,
 '2015-10-09 03:39:27.391914800': 7.230278291108429,
 '2015-02-02 03:43:26.840461200': -19.265702361757327,
 '2015-02-18 03:43:30.123369400': -21.10353792643862,
 '2015-03-06 03:43:38.890230': -20.405377930807056,
 '2015-03-22 03:43:47.507579': -16.524635578691548,
 '2015-04-07 03:43:54.805590': -9.719719085540074,
 '2015-05-09 03:44:08.650267': 0.18276616566416165,
 '2015-05-25 03:44:13.337693': 0.012970037472988994,
 '2015-06-10 03:44:21.408966800': -0.5564732939333601,
 '2015-06-26 03:44:21.975441400': None,
 '2015-07-28 03:44:30.987057500': 18.778682965816824,
 '2015-08-13 03:44:36.044932800': 16.780754664619423,
 '2015-08-29 03:44:36.361003800': 13.179251857041395,
 '2015-10-16 03:44:59.001130200': 1.6551872442224687,
 '2015-11-01 03:45:12.923584200': 2.7909447874816267,
 '2015-02-02 03:43:50.717999400': -17.446791497415333,
 '2015-02-18 03:43:54.001947': -21.929742182411015,
 '2015-03-06 03:44:02.768702200': -15.803227249067467,
 '2015-03-22 03:44:11.386732300': -10.552412917801112,
 '2015-04-07 03:44:18.684965400': -6.367198416173115,
 '2015-05-09 03:44:32.528405600': 0.13956516394204227,
 '2015-05-25 03:44:37.215070800': 1.214888146705882,
 '2015-06-10 03:44:45.286617200': 2.8259486371790548,
 '2015-06-26 03:44:45.853669100': None,
 '2015-07-12 03:44:53.251158900': None,
 '2015-07-28 03:44:54.865858400': 17.814009766414802,
 '2015-08-13 03:44:59.923903900': 13.860248306747636,
 '2015-08-29 03:45:00.239213800': 13.801361546449657,
 '2015-09-14 03:45:05.183983500': -3.1590864726471803,
 '2015-10-16 03:45:22.878912800': 4.92583772375729,
 '2015-11-01 03:45:36.802674500': 2.0438485182122936,
 '2015-11-17 03:45:54.316470100': -12.197363204953895,
 '2015-02-02 03:44:14.596091700': -15.366469969749467,
 '2015-02-18 03:44:17.880899900': -18.12939956773657,
 '2015-03-06 03:44:26.648044100': -12.839725227843672,
 '2015-03-22 03:44:35.266502600': -8.275897398552237,
 '2015-04-07 03:44:42.564925900': -3.606303338609343,
 '2015-04-23 03:44:47.986326200': -5.850442784447618,
 '2015-05-09 03:44:56.407339900': 1.4086788500369811,
 '2015-05-25 03:45:01.093029200': 2.914579116837905,
 '2015-06-10 03:45:09.164554800': 3.3824660899385948,
 '2015-06-26 03:45:09.732385700': None,
 '2015-07-12 03:45:17.130458900': None,
 '2015-07-28 03:45:18.745311': 15.893607549269019,
 '2015-08-13 03:45:23.803062600': 11.477789810370815,
 '2015-08-29 03:45:24.117754200': 13.628292596708098,
 '2015-09-14 03:45:29.062550500': None,
 '2015-10-16 03:45:46.757455800': 5.970004379545291,
 '2015-11-01 03:46:00.682192': 2.673184984082712,
 '2015-11-17 03:46:18.197508900': -5.49259066677066,
 '2015-02-02 03:44:38.474744500': -13.275432370322866,
 '2015-02-18 03:44:41.760679600': -15.959446335532133,
 '2015-03-06 03:44:50.527839700': -14.179491181711319,
 '2015-03-22 03:44:59.146966400': -6.916703503231859,
 '2015-04-07 03:45:06.445491300': -5.70994909341283,
 '2015-04-23 03:45:11.866303600': -5.580915041337094,
 '2015-05-09 03:45:20.286683': 2.114594984150745,
 '2015-05-25 03:45:24.971479200': 4.2129066200428245,
 '2015-06-10 03:45:33.043407700': 0.8667199536283879,
 '2015-07-12 03:45:41.010408600': None,
 '2015-07-28 03:45:42.625255800': 15.114184087027589,
 '2015-08-13 03:45:47.683191100': 13.960210274686695,
 '2015-08-29 03:45:47.997191400': 13.98591989007169,
 '2015-09-14 03:45:52.941525400': 0.6981783011869215,
 '2015-10-16 03:46:10.636740500': 6.553144960694164,
 '2015-11-01 03:46:24.562520300': 0.9559713246937098,
 '2015-11-17 03:46:42.079101600': -0.7697503503127228,
 '2015-12-03 03:46:54.275286': -7.142088175821373,
 '2015-02-09 03:50:02.104482400': -15.655711318257673,
 '2015-02-25 03:50:10.205064200': -16.558893392940963,
 '2015-03-13 03:50:15.946404900': -11.81358741889433,
 '2015-03-29 03:50:23.992590900': -3.0046885152604164,
 '2015-04-14 03:50:33.178595900': None,
 '2015-04-30 03:50:38.722896': -8.62723968988459,
 '2015-05-16 03:50:47.246431800': 0.16666811680927596,
 '2015-06-01 03:50:51.606178700': -10.723634045595295,
 '2015-06-17 03:50:57.323941100': 6.775616293756194,
 '2015-07-03 03:51:00.144045400': 11.711356252517527,
 '2015-07-19 03:51:05.342599900': 16.745900230822546,
 '2015-08-04 03:51:08.424064300': None,
 '2015-08-20 03:51:11.435919500': 17.23232028157725,
 '2015-09-05 03:51:12.237210100': 11.582719581061019,
 '2015-09-21 03:51:17.894346200': 5.561402060447544,
 '2015-10-07 03:51:22.554751100': None,
 '2015-10-23 03:51:40.873028300': 4.919145464036631,
 '2015-11-08 03:51:55.103523200': 1.512274863135904,
 '2015-02-09 03:50:25.982667200': -15.999639979058836,
 '2015-02-25 03:50:34.084014400': -14.416122127988048,
 '2015-03-13 03:50:39.826110700': -12.705563084370883,
 '2015-03-29 03:50:47.872254100': -4.904040741272987,
 '2015-04-14 03:50:57.058057': -13.934200833797762,
 '2015-04-30 03:51:02.601801900': None,
 '2015-05-16 03:51:11.124707600': 3.8800958371746352,
 '2015-06-01 03:51:15.484326600': 3.853045391983762,
 '2015-06-17 03:51:21.201922100': 7.175318377804173,
 '2015-07-03 03:51:24.022994100': 11.252089535915761,
 '2015-07-19 03:51:29.221969300': 16.30565105583957,
 '2015-08-04 03:51:32.303749600': 5.0205046781065015,
 '2015-08-20 03:51:35.314950800': 16.664850393697392,
 '2015-09-05 03:51:36.115952': 14.162223144803134,
 '2015-09-21 03:51:41.772422900': 6.356524771132649,
 '2015-10-07 03:51:46.432644600': 5.843558673779606,
 '2015-10-23 03:52:04.752048600': 6.128814469824558,
 '2015-11-08 03:52:18.984063600': 3.3212867704575832,
 '2015-11-24 03:52:34.777774400': -4.750964043252363,
 '2015-01-24 03:50:43.760346100': None,
 '2015-02-09 03:50:49.861443500': -16.117408017104417,
 '2015-02-25 03:50:57.963780500': -12.162722418386561,
 '2015-03-13 03:51:03.706512300': None,
 '2015-03-29 03:51:11.752443500': None,
 '2015-04-14 03:51:20.938225700': -4.939762300086028,
 '2015-04-30 03:51:26.481425100': None,
 '2015-05-16 03:51:35.003604800': 2.4086039075836614,
 '2015-06-01 03:51:39.362844200': 3.398107903235293,
 '2015-06-17 03:51:45.080631100': 7.266491490045747,
 '2015-07-03 03:51:47.902558900': 11.214774526960298,
 '2015-07-19 03:51:53.101900900': 14.470842090107435,
 '2015-08-04 03:51:56.183784400': 12.158539608100787,
 '2015-08-20 03:51:59.194532300': 14.231855662799022,
 '2015-09-05 03:51:59.995170100': None,
 '2015-09-21 03:52:05.651296': 6.371239664219436,
 '2015-10-07 03:52:10.311376200': 6.080914381138921,
 '2015-10-23 03:52:28.631405300': 6.864932247204749,
 '2015-11-08 03:52:42.865139700': 3.8493902675509175,
 '2015-11-24 03:52:58.659237300': -7.711498891326817,
 '2015-01-31 03:56:35.377532700': -11.844041982881244,
 '2015-02-16 03:56:37.490725800': -5.716138607920557,
 '2015-03-04 03:56:47.920411100': -5.325386321161304,
 '2015-03-20 03:56:55.968164600': -27.421162013231328,
 '2015-04-05 03:57:02.739177800': -4.23385035609488,
 '2015-04-21 03:57:08.555680600': -3.5178707832319764,
 '2015-05-07 03:57:16.372753100': 1.573968647057374,
 '2015-05-23 03:57:22.801290600': -0.27189903507107027,
 '2015-06-08 03:57:30.051637800': -20.835643498503117,
 '2015-06-24 03:57:31.429544400': 3.5746669740201535,
 '2015-07-10 03:57:38.106811800': 10.59990799341524,
 '2015-07-26 03:57:40.398585': 15.174181428969046,
 '2015-08-27 03:57:45.974446': 17.39710502120525,
 '2015-09-12 03:57:50.008884300': 15.554939516772583,
 '2015-09-28 03:57:53.567197700': 5.328870635463968,
 '2015-10-14 03:58:06.067248700': 5.39152316411188,
 '2015-10-30 03:58:21.027991': 5.806739109880711,
 '2015-11-15 03:58:37.859952500': -2.270111849681837,
 '2015-01-15 03:56:53.204912': -8.133549818823209,
 '2015-01-31 03:56:59.256374': -11.63470644459247,
 '2015-02-16 03:57:01.370396700': -10.145868778166841,
 '2015-03-04 03:57:11.800235900': -8.592121933638964,
 '2015-03-20 03:57:19.848706300': -19.351747637078766,
 '2015-04-05 03:57:26.619879900': -7.79506669719048,
 '2015-04-21 03:57:32.435382600': -11.51650640391594,
 '2015-05-07 03:57:40.252164400': 0.9155230004013006,
 '2015-05-23 03:57:46.679848900': 0.11812338514922009,
 '2015-06-08 03:57:53.930414100': 2.8487039893251787,
 '2015-06-24 03:57:55.308377900': 5.9548918378883915,
 '2015-07-10 03:58:01.986845': 12.515278266656637,
 '2015-07-26 03:58:04.278819500': 15.580538245785442,
 '2015-08-27 03:58:09.853808700': 17.151170662892717,
 '2015-09-12 03:58:13.887899': 14.927638032323976,
 '2015-09-28 03:58:17.446112600': 1.7120602794509698,
 '2015-10-14 03:58:29.946356400': 7.736553761415641,
 '2015-11-15 03:59:01.741345800': -3.8005443342238467,
 '2015-12-01 03:59:13.784063400': -3.6523002504692728,
 '2015-02-07 04:03:11.693624800': -12.608664705847634,
 '2015-02-23 04:03:18.308658500': -15.98922353673901,
 '2015-03-11 04:03:23.390261500': -20.161348575837547,
 '2015-03-27 04:03:33.139421200': -14.140528358311533,
 '2015-04-12 04:03:41.787077700': -9.478483570180146,
 '2015-05-14 04:03:55.903010400': 1.1889840148654052,
 '2015-06-15 04:04:06.584693600': 8.469942592837404,
 '2015-07-01 04:04:08.343325300': 10.073638153728286,
 '2015-08-02 04:04:17.063193300': 14.437448831940486,
 '2015-08-18 04:04:20.730347800': 11.601136415039747,
 '2015-09-03 04:04:20.582888': 16.217043262483365,
 '2015-09-19 04:04:26.838412600': -3.331713275558051,
 '2015-10-05 04:04:29.245286800': 10.123406742342045,
 '2015-10-21 04:04:48.405442800': -13.358407362029372,
 '2015-11-06 04:05:01.861552200': 3.3312407205913965,
 '2015-11-22 04:05:18.808228400': 1.8052188942713099,
'2016-02-06 03:38:36.341180': -15.336795103962487,
 '2016-02-22 03:38:28.602847': -21.63318738606259,
 '2016-03-09 03:38:27.259466': None,
 '2016-03-25 03:38:19.057025': -3.9622423753070932,
 '2016-04-10 03:38:12.125356': -8.224380470956953,
 '2016-04-26 03:38:06.604298': -12.505004730108956,
 '2016-05-12 03:38:06.811631': -1.795342215853378,
 '2016-05-28 03:38:13.526153': 1.5356318646488554,
 '2016-06-13 03:38:16.598968': 7.728902187839571,
 '2016-06-29 03:38:24.642736': 9.27855644684253,
 '2016-07-15 03:38:32.068885': 12.549615151571537,
 '2016-08-16 03:38:39.292750': 17.16191395236428,
 '2016-09-01 03:38:46.102961': 11.87746232399972,
 '2016-09-17 03:38:49.037776': 14.388177498871203,
 '2016-10-03 03:38:50.311288': 3.575935036590874,
 '2016-10-19 03:38:55.410519': 2.9413847578259937,
 '2016-11-04 03:38:55.601569': -18.136991514965022,
 '2016-02-06 03:39:00.223745': -14.71341634938681,
 '2016-02-22 03:38:52.481176': -20.155478719381964,
 '2016-03-09 03:38:51.133560': -21.19582638208276,
 '2016-03-25 03:38:42.935355': -3.811565408876545,
 '2016-04-10 03:38:35.999450': -12.667350061409962,
 '2016-04-26 03:38:30.478391': -2.6583537473149534,
 '2016-05-12 03:38:30.694196': -4.367220627658623,
 '2016-05-28 03:38:37.412955': 0.8906839920628034,
 '2016-06-13 03:38:40.481533': 5.802400542306966,
 '2016-06-29 03:38:48.529538': 8.336369301791107,
 '2016-07-15 03:38:55.947215': 8.911426913477072,
 '2016-08-16 03:39:03.171080': 16.714085915225482,
 '2016-09-01 03:39:09.981290': 13.601178174253164,
 '2016-09-17 03:39:12.920342': 14.598254201164337,
 '2016-10-03 03:39:14.198090': 4.970241123525353,
 '2016-10-19 03:39:19.293085': 3.685792677628015,
 '2016-11-04 03:39:19.479899': -6.05913781264693,
 '2016-11-20 03:39:19.359447': -22.472574863086837,
 '2016-01-21 03:39:27.961319': None,
 '2016-02-06 03:39:24.102074': -14.007637163196074,
 '2016-02-22 03:39:16.359506': -17.47188541686746,
 '2016-03-09 03:39:15.016126': -12.53034625702731,
 '2016-03-25 03:39:06.817920': -1.2074942302163536,
 '2016-04-10 03:38:59.877779': -12.502354439542145,
 '2016-04-26 03:38:54.352485': -0.7032061722285831,
 '2016-05-12 03:38:54.576762': -5.734042175105502,
 '2016-05-28 03:39:01.295520': 0.7512893179623154,
 '2016-06-13 03:39:04.368335': 5.806555792287961,
 '2016-06-29 03:39:12.412103': 11.019169350459851,
 '2016-07-15 03:39:19.829781': 12.411580628532839,
 '2016-07-31 03:39:23.673994': None,
 '2016-08-16 03:39:27.053645': 16.73379861538751,
 '2016-09-01 03:39:33.863856': 14.475562648036636,
 '2016-09-17 03:39:36.798672': 14.552194299736502,
 '2016-10-03 03:39:38.076419': 6.116081876318904,
 '2016-10-19 03:39:43.171414': 4.39308834846858,
 '2016-11-04 03:39:43.362465': -2.658492597403944,
 '2016-01-05 03:39:51.374121': None,
 '2016-01-21 03:39:51.843885': None,
 '2016-02-06 03:39:47.984640': -22.320684316051235,
 '2016-02-22 03:39:40.242071': -14.465460715155855,
 '2016-03-09 03:39:38.894455': -11.263203235733807,
 '2016-03-25 03:39:30.692014': -0.14648813651038553,
 '2016-04-10 03:39:23.751873': None,
 '2016-04-26 03:39:18.230814': -3.8313576481174305,
 '2016-05-12 03:39:18.459327': None,
 '2016-05-28 03:39:25.182322': 4.282028028498548,
 '2016-06-13 03:39:28.250900': None,
 '2016-06-29 03:39:36.298905': 15.85823886373167,
 '2016-07-15 03:39:43.716582': 16.135401763311165,
 '2016-07-31 03:39:47.556559': None,
 '2016-08-16 03:39:50.940447': 16.79740598395812,
 '2016-09-01 03:39:57.750658': 8.423520826779022,
 '2016-09-17 03:40:00.681237': 14.319187878535095,
 '2016-10-03 03:40:01.963221': 3.1257590993850433,
 '2016-10-19 03:40:07.053980': -1.8500781959022734,
 '2016-11-04 03:40:07.240794': None,
 '2016-11-20 03:40:07.116107': None,
 '2016-12-06 03:40:05.606337': None,
 '2016-01-28 03:44:50.029890': -33.31315961151065,
 '2016-02-13 03:44:43.332587': -23.27815282601416,
 '2016-02-29 03:44:39.522656': -14.925735433162593,
 '2016-03-16 03:44:35.270105': -6.503540088588545,
 '2016-04-01 03:44:24.941718': -8.075770200774485,
 '2016-04-17 03:44:20.225713': -3.084707640204186,
 '2016-05-03 03:44:23.723179': None,
 '2016-05-19 03:44:20.787735': None,
 '2016-06-04 03:44:26.202529': 3.2491540409594304,
 '2016-06-20 03:44:29.992102': 1.2822659618159913,
 '2016-07-06 03:44:39.128988': 12.093289678824682,
 '2016-07-22 03:44:44.996646': 14.72002496877967,
 '2016-08-07 03:44:47.490114': 9.837434160697525,
 '2016-08-23 03:44:53.485417': 12.758147094460524,
 '2016-09-08 03:44:58.690457': 13.175109252584354,
 '2016-09-24 03:44:59.862670': 10.666723466805031,
 '2016-10-10 03:45:04.004183': 4.624486785434019,
 '2016-10-26 03:45:06.867460': -3.7240101102215375,
 '2016-11-11 03:45:05.327021': None,
 '2016-01-28 03:45:13.908219': -25.629097316039545,
 '2016-02-13 03:45:07.206680': -20.25987696940284,
 '2016-02-29 03:45:03.400986': -10.089330254568956,
 '2016-03-16 03:44:59.148435': -6.045456829393078,
 '2016-04-01 03:44:48.820048': -6.693689608034818,
 '2016-04-17 03:44:44.099807': -2.065754973642462,
 '2016-05-03 03:44:47.605744': None,
 '2016-05-19 03:44:44.674537': None,
 '2016-06-04 03:44:50.085095': 3.8920265375754446,
 '2016-06-20 03:44:53.874668': 5.0870646637632175,
 '2016-07-06 03:45:03.011554': 8.90485420760225,
 '2016-07-22 03:45:08.879212': 12.19223666688499,
 '2016-08-07 03:45:11.376916': 10.469523969236048,
 '2016-08-23 03:45:17.363746': 14.256519867607897,
 '2016-09-08 03:45:22.577259': 12.931324352656988,
 '2016-09-24 03:45:23.745236': 12.300392752955208,
 '2016-10-10 03:45:27.882513': 3.618130795570366,
 '2016-10-26 03:45:30.750026': -0.06791868442152107,
 '2016-11-11 03:45:29.205350': None,
 '2016-01-28 03:45:37.782313': -20.84167341287803,
 '2016-02-13 03:45:31.089246': -17.29092301208674,
 '2016-02-29 03:45:27.279315': -7.247440763189628,
 '2016-03-16 03:45:23.031000': -7.611088080763127,
 '2016-04-01 03:45:12.694141': -5.365241694359452,
 '2016-04-17 03:45:07.978136': -1.4815426279312045,
 '2016-05-03 03:45:11.488310': -4.897260031638377,
 '2016-06-04 03:45:13.963425': 4.86505026087546,
 '2016-06-20 03:45:17.752997': 7.074755553327582,
 '2016-07-06 03:45:26.898355': None,
 '2016-07-22 03:45:32.766013': 6.874533948472766,
 '2016-08-07 03:45:35.259481': None,
 '2016-08-23 03:45:41.246312': 13.775713016162275,
 '2016-09-08 03:45:46.459825': None,
 '2016-09-24 03:45:47.627802': 11.78556786860966,
 '2016-10-10 03:45:51.765079': 6.680611704075898,
 '2016-10-26 03:45:54.632592': 0.6325973438477624,
 '2016-11-11 03:45:53.083680': -5.371128180452367,
 '2016-01-12 03:46:02.037589': -17.88657880648175,
 '2016-01-28 03:46:01.669114': -20.155347357420993,
 '2016-02-13 03:45:54.967575': -16.433358549510523,
 '2016-02-29 03:45:51.157644': -7.909581542017054,
 '2016-03-16 03:45:46.909330': None,
 '2016-04-01 03:45:36.572471': -12.28417560809208,
 '2016-04-17 03:45:31.852230': -2.5771833920356855,
 '2016-05-03 03:45:35.370875': -4.284510850712724,
 '2016-06-04 03:45:37.854462': 6.0338590585215055,
 '2016-06-20 03:45:41.644035': 7.8931401127017375,
 '2016-07-06 03:45:50.780921': None,
 '2016-07-22 03:45:56.648579': None,
 '2016-08-23 03:46:05.133114': None,
 '2016-09-08 03:46:10.338154': None,
 '2016-09-24 03:46:11.510368': 11.340884280283749,
 '2016-10-26 03:46:18.515157': -0.6423200629293098,
 '2016-11-11 03:46:16.966246': -2.4122675525921338,
 '2016-11-27 03:46:17.803216': -10.207190282922397,
 '2016-02-04 03:51:22.653279': -15.473469806505861,
 '2016-02-20 03:51:13.893633': -19.333171504711995,
 '2016-03-07 03:51:13.312396': -15.752291576905687,
 '2016-03-23 03:51:05.910167': -2.0248009503118176,
 '2016-04-08 03:50:58.224516': -4.996997434181391,
 '2016-04-24 03:50:51.233650': None,
 '2016-05-10 03:50:53.855388': 0.9956598129755347,
 '2016-05-26 03:50:58.375436': 2.0658354292314476,
 '2016-06-11 03:51:01.960733': 7.2421722327643625,
 '2016-06-27 03:51:09.042599': 8.82259573364008,
 '2016-07-13 03:51:16.862664': None,
 '2016-07-29 03:51:21.109028': None,
 '2016-08-14 03:51:23.734139': None,
 '2016-08-30 03:51:30.942487': None,
 '2016-09-15 03:51:34.398155': 11.751243997554612,
 '2016-10-01 03:51:34.854341': None,
 '2016-10-17 03:51:40.601547': 4.231603234859833,
 '2016-11-02 03:51:41.347256': 0.9406172045868482,
 '2016-11-18 03:51:40.904751': -5.178599913354455,
 '2016-01-19 03:51:49.619628': -26.853390844249663,
 '2016-02-04 03:51:46.535844': -13.252001385382746,
 '2016-02-20 03:51:37.776198': -22.312765358370118,
 '2016-03-07 03:51:37.186490': -17.000075948008444,
 '2016-03-23 03:51:29.788497': -1.7199633287388054,
 '2016-04-08 03:51:22.102845': -5.823385496114216,
 '2016-04-24 03:51:15.103507': -5.672661174323951,
 '2016-05-10 03:51:17.742190': 1.4206575607863783,
 '2016-05-26 03:51:22.258001': None,
 '2016-06-11 03:51:25.843299': 8.14476555551211,
 '2016-06-27 03:51:32.925165': 9.864415897979198,
 '2016-07-13 03:51:40.745230': 12.626952394420368,
 '2016-07-29 03:51:44.995830': 15.85508828121433,
 '2016-08-14 03:51:47.612469': 11.95963826885897,
 '2016-08-30 03:51:54.820817': 10.764359587789576,
 '2016-09-15 03:51:58.280720': 13.088620207710722,
 '2016-10-01 03:51:58.736907': 9.360246525211114,
 '2016-10-17 03:52:04.484113': 3.2101165147003785,
 '2016-11-02 03:52:05.225586': -1.2804408040436595,
 '2016-11-18 03:52:04.778845': -7.573180593975855,
 '2016-01-19 03:52:13.497958': -18.696696294265223,
 '2016-02-04 03:52:10.414174': -13.041610330288853,
 '2016-02-20 03:52:01.654528': None,
 '2016-03-07 03:52:01.069055': -17.88571378624424,
 '2016-03-23 03:51:53.671063': -1.6182591776320518,
 '2016-04-08 03:51:45.981175': -7.5110848501517635,
 '2016-04-24 03:51:38.986073': -0.9391933231606555,
 '2016-05-10 03:51:41.620519': 1.5858267041950955,
 '2016-05-26 03:51:46.144803': 5.813230473192698,
 '2016-06-11 03:51:49.725865': 6.5817067764889865,
 '2016-06-27 03:51:56.807730': 6.751406273928002,
 '2016-07-13 03:52:04.627796': 13.33887846823962,
 '2016-07-29 03:52:08.878396': 16.374706638366703,
 '2016-08-14 03:52:11.499270': 9.556585036795726,
 '2016-08-30 03:52:18.703383': 12.440705509511423,
 '2016-09-15 03:52:22.163286': 12.810260835338926,
 '2016-10-01 03:52:22.615236': 6.323106172520995,
 '2016-10-17 03:52:28.366678': 3.9969856061204285,
 '2016-11-02 03:52:29.112387': None,
 '2016-11-18 03:52:28.661411': -6.05690697366854,
 '2016-12-04 03:52:27.659982': -1.6256056839215398,
 '2016-02-11 03:57:53.988638': -15.211165665357274,
 '2016-02-27 03:57:48.992324': -13.88131519840867,
 '2016-03-14 03:57:45.590808': -2.9564981980086618,
 '2016-03-30 03:57:35.563263': -6.088747682084593,
 '2016-04-15 03:57:30.495694': None,
 '2016-05-01 03:57:31.832578': 0.6803667478532794,
 '2016-05-17 03:57:29.218951': 3.6853417483927053,
 '2016-06-02 03:57:35.146076': 5.153811736346765,
 '2016-06-18 03:57:38.028768': 4.925579065431099,
 '2016-07-20 03:57:53.871931': 9.834965751700873,
 '2016-08-21 03:58:02.000031': 8.040261455846649,
 '2016-09-06 03:58:07.683560': 12.019504504052152,
 '2016-09-22 03:58:09.343071': 13.841858713674956,
 '2016-10-08 03:58:12.671100': 4.021376013601175,
 '2016-10-24 03:58:16.157046': 4.6770047657440506,
 '2016-11-09 03:58:14.929758': None,
 '2016-11-25 03:58:15.673552': -0.03324346954466274,
 '2016-01-10 03:58:23.234316': -7.7568882548189,
 '2016-01-26 03:58:23.700314': None,
 '2016-02-11 03:58:17.875440': -16.576181064672877,
 '2016-02-27 03:58:12.870654': -13.180414803574266,
 '2016-03-14 03:58:09.469138': -3.9010773621135777,
 '2016-03-30 03:57:59.441592': None,
 '2016-04-15 03:57:54.378259': None,
 '2016-05-01 03:57:55.710908': -0.33856127682247,
 '2016-05-17 03:57:53.097281': 0.9444430109282769,
 '2016-06-02 03:57:59.032877': 4.814626424748449,
 '2016-06-18 03:58:01.915570': 5.770193605081465,
 '2016-07-04 03:58:11.429540': None,
 '2016-08-05 03:58:20.629190': None,
 '2016-08-21 03:58:25.878361': 14.322285190087758,
 '2016-09-06 03:58:31.561889': 9.99926378655764,
 '2016-09-22 03:58:33.221401': 12.1609990263027,
 '2016-10-08 03:58:36.549430': 7.451750903213107,
 '2016-10-24 03:58:40.035375': 3.319310030180863,
 '2016-11-25 03:58:39.556118': -6.488956777335015,
 '2016-12-11 03:58:36.724921': None,
 '2016-01-17 04:04:35.050444': -13.693493728065883,
 '2016-02-02 04:04:32.737700': -13.905659965861071,
 '2016-02-18 04:04:23.605015': -15.274522083496521,
 '2016-03-05 04:04:23.070157': -13.996817211956953,
 '2016-03-21 04:04:16.561155': -0.8740497262945877,
 '2016-04-06 04:04:08.108908': -2.3555572319657556,
 '2016-04-22 04:04:01.740950': -0.2585154359340289,
 '2016-05-08 04:04:04.688462': 2.0213569777524083,
 '2016-05-24 04:04:07.039479': None,
 '2016-06-25 04:04:17.274385': None,
 '2016-07-11 04:04:25.471121': None,
 '2016-07-27 04:04:30.157855': 12.068395338589706,
 '2016-08-28 04:04:39.600967': 12.800809296065118,
 '2016-09-13 04:04:43.569125': 11.301770901688846,
 '2016-09-29 04:04:43.517005': 11.569926902240427,
 '2016-10-31 04:04:50.899395': 5.785911259290539,
 '2016-11-16 04:04:50.138962': None,
 '2016-12-02 04:04:49.654348': 2.474105889729226,
 '2016-02-14 03:40:27.345025700': -18.306117195224335,
 '2016-03-17 03:40:41.297669500': -10.652215128826482,
 '2016-04-02 03:40:44.982084100': -11.226528877599106,
 '2016-04-18 03:40:52.472822500': -8.488542005707036,
 '2016-05-04 03:40:58.003590200': -0.9267679898941173,
 '2016-05-20 03:41:00.196821900': -3.960882343129584,
 '2016-06-21 03:41:08.668865300': 8.134669412284834,
 '2016-07-07 03:41:08.503982700': 15.61646320394756,
 '2016-07-23 03:41:10.439942300': 13.2247017181925,
 '2016-09-09 03:41:17.369903500': 12.508592981280445,
 '2016-09-25 03:41:19.624517700': 7.069849993466245,
 '2016-10-11 03:41:20.239336700': 2.319350452194952,
 '2016-10-27 03:41:16.606637100': None,
 '2016-11-12 03:41:09.087559100': -11.618390426250855,
 '2016-01-29 03:40:45.860982800': -24.01166629191795,
 '2016-02-14 03:40:51.222445400': -18.498176367114368,
 '2016-03-01 03:40:56.074618200': None,
 '2016-03-17 03:41:05.177654200': -11.795627877996326,
 '2016-04-02 03:41:08.861694': -6.555441996206261,
 '2016-04-18 03:41:16.351230800': -6.754182852011992,
 '2016-05-04 03:41:21.880303700': -1.0134613613965382,
 '2016-05-20 03:41:24.072996600': -0.9047683750090099,
 '2016-06-21 03:41:32.547291600': 3.8149410355811213,
 '2016-07-07 03:41:32.383399100': 8.174357209685017,
 '2016-07-23 03:41:34.319688900': 13.812472297216175,
 '2016-08-08 03:41:36.185272400': None,
 '2016-08-24 03:41:38.380045800': 13.452496419569032,
 '2016-09-09 03:41:41.246425600': 11.382753729631315,
 '2016-09-25 03:41:43.501552900': 10.58426502273143,
 '2016-10-11 03:41:44.117502': 2.8353788704334986,
 '2016-10-27 03:41:40.486068100': None,
 '2016-11-12 03:41:32.967300100': -1.580495536333552,
 '2016-01-29 03:41:09.738402900': -23.08194198107652,
 '2016-02-14 03:41:15.100643700': None,
 '2016-03-01 03:41:19.954332700': None,
 '2016-03-17 03:41:29.058284600': -11.535650574778623,
 '2016-04-02 03:41:32.741713400': -5.753835154692004,
 '2016-04-18 03:41:40.230196200': -6.960620861507659,
 '2016-05-04 03:41:45.757710800': -3.763987855297547,
 '2016-05-20 03:41:47.949877700': 0.566017310367726,
 '2016-06-21 03:41:56.426259200': 4.470420412929867,
 '2016-07-07 03:41:56.263599700': 6.509409868240198,
 '2016-07-23 03:41:58.200230500': 16.157294718358102,
 '2016-08-08 03:42:00.064731900': None,
 '2016-08-24 03:42:02.258045900': 13.489645121401784,
 '2016-09-09 03:42:05.123493': 9.34938599639601,
 '2016-09-25 03:42:07.378916900': 10.524472452689238,
 '2016-10-11 03:42:07.996001800': 6.011733013091362,
 '2016-10-27 03:42:04.365981300': -11.295364461373794,
 '2016-11-12 03:41:56.847749200': 0.5970413521640424,
 '2016-02-05 03:46:35.938324800': -13.503795134194204,
 '2016-02-21 03:46:38.165744': -21.6057713519586,
 '2016-03-08 03:46:47.839661200': -16.65957396968763,
 '2016-03-24 03:46:54.237583100': -5.5488885750421115,
 '2016-04-09 03:46:58.367541900': -7.739003879075339,
 '2016-04-25 03:47:06.190376': -2.2702540931258186,
 '2016-05-11 03:47:09.774180500': None,
 '2016-05-27 03:47:14.031844800': -0.43647713876632577,
 '2016-06-28 03:47:19.663847300': 7.920049865328492,
 '2016-07-14 03:47:19.090398': 11.985236890214091,
 '2016-07-30 03:47:22.322930': 13.376392581059843,
 '2016-08-15 03:47:23.112944400': 14.684878665419221,
 '2016-08-31 03:47:27.002054400': 12.29534544037816,
 '2016-09-16 03:47:28.549685400': 13.994668594557004,
 '2016-10-02 03:47:31.192801200': 6.736170846168947,
 '2016-10-18 03:47:29.985583600': 1.0775210623914777,
 '2016-02-05 03:46:59.815442': -12.224503423345062,
 '2016-02-21 03:47:02.043695500': -20.79852070020503,
 '2016-03-08 03:47:11.719505900': -16.224121204566604,
 '2016-03-24 03:47:18.117571100': -4.784604169690179,
 '2016-04-09 03:47:22.246963300': -7.584811820960278,
 '2016-04-25 03:47:30.068046': -1.8949077429134857,
 '2016-05-11 03:47:33.650481400': -3.3354926795850965,
 '2016-06-28 03:47:43.542909': 7.635510817413161,
 '2016-07-14 03:47:42.970527': 10.170363584087669,
 '2016-07-30 03:47:46.202512500': 9.318470384520307,
 '2016-08-15 03:47:46.991047900': 14.218790672614967,
 '2016-08-31 03:47:50.878879500': 15.23286829541249,
 '2016-09-16 03:47:52.426242700': 13.915142926822988,
 '2016-10-02 03:47:55.070195500': 6.597535476676667,
 '2016-10-18 03:47:53.864329500': -1.4894399899754918,
 '2016-11-19 03:47:41.567363500': -13.563146573460955,
 '2016-01-20 03:47:15.211284300': -23.86100588774326,
 '2016-02-05 03:47:23.693174800': -10.59578303269868,
 '2016-02-21 03:47:25.922409600': -18.082083024786858,
 '2016-03-08 03:47:35.599768900': -15.49342425959185,
 '2016-03-24 03:47:41.998181100': -2.6318770196735004,
 '2016-04-09 03:47:46.126891900': -5.951964787637328,
 '2016-04-25 03:47:53.946144100': -3.1738762217908687,
 '2016-05-11 03:47:57.527330200': -2.6147464706153802,
 '2016-06-28 03:48:07.422424900': 9.1495491768651,
 '2016-07-14 03:48:06.850941600': 12.504291225260122,
 '2016-07-30 03:48:10.082399600': 16.062725913453242,
 '2016-08-15 03:48:10.869583100': 13.853324337503983,
 '2016-08-31 03:48:14.756419800': 15.118698878846077,
 '2016-09-16 03:48:16.303485900': 13.489866780696817,
 '2016-10-02 03:48:18.948127100': 8.196935445829256,
 '2016-10-18 03:48:17.743627700': 3.4309222524084877,
 '2016-11-03 03:48:12.347052600': 0.9241413796075363,
 '2016-11-19 03:48:05.447769800': -13.647014481187774,
 '2016-01-20 03:47:39.089472': -21.303041984760622,
 '2016-02-05 03:47:47.571588700': -11.057580725595647,
 '2016-02-21 03:47:49.801641800': -19.014141120808766,
 '2016-03-08 03:47:59.480859500': -20.75063786283298,
 '2016-03-24 03:48:05.879413600': -2.2144861043970687,
 '2016-04-09 03:48:10.007269800': -5.526402980828386,
 '2016-04-25 03:48:17.825014800': -7.696781573759802,
 '2016-05-11 03:48:21.404801900': 0.015591840154424268,
 '2016-06-28 03:48:31.302778100': 7.774208391149391,
 '2016-07-14 03:48:30.732234300': 14.839961782020804,
 '2016-07-30 03:48:33.963186300': 16.344003781669112,
 '2016-08-15 03:48:34.748858600': 14.434866499018064,
 '2016-08-31 03:48:38.634407': 14.944373335888596,
 '2016-09-16 03:48:40.181203300': 13.008881856530316,
 '2016-10-02 03:48:42.826877300': 6.370179005922816,
 '2016-10-18 03:48:41.623447500': None,
 '2016-11-19 03:48:29.329024100': -16.437289337246074,
 '2016-01-27 03:53:06.448292300': -19.134370319633337,
 '2016-02-12 03:53:12.592596500': -15.55499599505723,
 '2016-02-28 03:53:16.177177700': -14.879850314409692,
 '2016-03-15 03:53:26.029840100': -1.535612082724208,
 '2016-03-31 03:53:30.258515300': -8.855626153895118,
 '2016-04-16 03:53:37.018751700': -1.0329283005788104,
 '2016-05-02 03:53:43.064834200': -4.985585015451227,
 '2016-07-05 03:53:54.148679600': 11.805553786475986,
 '2016-07-21 03:53:55.568919100': 3.2472111218547783,
 '2016-08-06 03:53:57.718977200': 15.240707021405456,
 '2016-09-07 03:54:02.724869400': 8.94851015492209,
 '2016-09-23 03:54:04.794036': 11.185663720637129,
 '2016-10-09 03:54:05.885703200': None,
 '2016-10-25 03:54:02.771144600': 0.27087033490513635,
 '2016-11-10 03:53:55.721808400': -6.130830465302172,
 '2016-01-27 03:53:30.325832800': -17.959315566665474,
 '2016-02-12 03:53:36.470720': -15.558788378147169,
 '2016-02-28 03:53:40.056937800': -13.720358893116869,
 '2016-03-15 03:53:49.910526700': -2.985217987699218,
 '2016-03-31 03:53:54.138851600': -7.321432887495444,
 '2016-04-16 03:54:00.897999600': -1.603777249579262,
 '2016-05-02 03:54:06.942254400': 1.0458639946801485,
 '2016-07-05 03:54:18.028709500': 14.127057506447938,
 '2016-07-21 03:54:19.449468800': 4.793158716210724,
 '2016-08-06 03:54:21.598373700': 17.166531116858742,
 '2016-09-07 03:54:26.602124600': 10.466830093872707,
 '2016-09-23 03:54:28.671632700': 10.194833760447814,
 '2016-10-09 03:54:29.764414300': None,
 '2016-10-25 03:54:26.650990800': 1.673307501563975,
 '2016-11-10 03:54:19.602170300': -1.9818655927202895,
 '2016-11-26 03:54:15.700585600': -5.220396065341486,
 '2016-01-27 03:53:54.203819500': -16.79441320911575,
 '2016-02-12 03:54:00.349337900': -18.02857571437741,
 '2016-02-28 03:54:03.937263200': -13.708758611255833,
 '2016-03-15 03:54:13.791825800': -4.118176067258805,
 '2016-03-31 03:54:18.019641200': None,
 '2016-04-16 03:54:24.777906800': -1.4252633285113703,
 '2016-05-02 03:54:30.820512': 0.16428385858977743,
 '2016-07-05 03:54:41.909404500': 12.452286915278167,
 '2016-07-21 03:54:43.330616900': None,
 '2016-08-06 03:54:45.478609800': 17.33218313997122,
 '2016-08-22 03:54:47.219485800': 8.339341960712268,
 '2016-09-07 03:54:50.479967900': 11.13556722216081,
 '2016-09-23 03:54:52.549663600': 2.098924963293887,
 '2016-10-25 03:54:50.531587900': 2.8761811820394683,
 '2016-11-10 03:54:43.483169300': 2.359942505197762,
 '2016-11-26 03:54:39.581237300': -5.2198024563845875,
 '2016-02-03 03:59:44.530785600': -12.430605391294936,
 '2016-02-19 03:59:47.702713100': -14.519992654170553,
 '2016-03-06 03:59:55.983695': -22.870864953398137,
 '2016-03-22 04:00:03.111941200': -3.5189728583062396,
 '2016-04-07 04:00:06.480634800': -3.5460512167518035,
 '2016-04-23 04:00:14.820442400': -16.466121485567335,
 '2016-05-09 04:00:19.018362400': -4.127613507537085,
 '2016-05-25 04:00:22.636549900': -18.395464630960944,
 '2016-06-26 04:00:29.061071900': -19.358516285702372,
 '2016-07-12 04:00:28.149005100': 15.056085013899926,
 '2016-07-28 04:00:31.448385800': -7.825966691870456,
 '2016-08-13 04:00:32.559219700': 5.180044811962009,
 '2016-08-29 04:00:35.957215400': -0.09538354884825784,
 '2016-09-14 04:00:37.788061700': 13.080901840053793,
 '2016-10-16 04:00:39.807066900': -5.318857885227758,
 '2016-11-01 04:00:34.860234900': 1.29593529761898,
 '2016-11-17 04:00:27.141855': -1.3270452768148275,
 '2016-01-18 03:59:59.271101400': -11.619415477576382,
 '2016-02-03 04:00:08.408979200': -13.795593858016565,
 '2016-02-19 04:00:11.581916400': -14.722293453649936,
 '2016-03-06 04:00:19.864583': -18.833875633358105,
 '2016-03-22 04:00:26.993215': -4.675441000908464,
 '2016-04-07 04:00:30.361353': -2.114125659670037,
 '2016-04-23 04:00:38.699435500': None,
 '2016-05-09 04:00:42.895972400': -4.348641328438275,
 '2016-05-25 04:00:46.514239': 1.1131790555810948,
 '2016-06-26 04:00:52.941298100': -18.356909838554003,
 '2016-07-12 04:00:52.029827400': 15.369946027811414,
 '2016-07-28 04:00:55.329050': None,
 '2016-08-13 04:00:56.438499': 3.9968248964762845,
 '2016-08-29 04:00:59.835363600': None,
 '2016-09-14 04:01:01.665850900': 12.04695769797536,
 '2016-09-30 04:01:04.321748600': 4.824152015061982,
 '2016-10-16 04:01:03.686931200': 3.3195506419118863,
 '2016-11-01 04:00:58.741025500': -5.179935835909518,
 '2016-11-17 04:00:51.023140600': -2.926747961071214,
 '2016-12-03 04:00:49.156984200': -1.4328000040381605,
 '2016-01-25 04:06:14.723092700': -20.06479828875262,
 '2016-02-10 04:06:21.570183800': -38.12624587232453,
 '2016-02-26 04:06:23.977161800': -11.023072131374487,
 '2016-03-13 04:06:34.535617700': -5.890807710242393,
 '2016-03-29 04:06:39.348012900': -3.341852538715724,
 '2016-04-30 04:06:51.898355200': -0.9691365565581221,
 '2016-05-16 04:06:53.948041700': None,
 '2016-06-17 04:07:03.219169300': 7.356508375983546,
 '2016-07-03 04:07:03.642051400': 5.06884176741997,
 '2016-07-19 04:07:04.506720600': None,
 '2016-08-04 04:07:06.971144100': None,
 '2016-08-20 04:07:08.246792600': 10.877716275840205,
 '2016-09-05 04:07:11.908733600': -21.020605717862207,
 '2016-09-21 04:07:13.813204700': 1.8697974265306105,
 '2016-10-07 04:07:15.339548200': 2.3152251485036857,
 '2016-11-08 04:07:06.174110700': None,
 '2016-11-24 04:07:01.437393700': -28.977335210640582,
'2017-02-08 03:38:33.565661': -23.06935731281107,
 '2017-02-24 03:38:28.486366': -13.722042540008534,
 '2017-03-12 03:38:20.426154': -14.359122642538688,
 '2017-03-28 03:38:11.172627900': -3.9409434876589935,
 '2017-04-29 03:37:52.904163': -4.508438643375923,
 '2017-05-15 03:38:00.762437900': None,
 '2017-05-31 03:38:10.708231900': 2.712594630151834,
 '2017-07-02 03:38:21.864061': None,
 '2017-07-18 03:38:26.003680900': 10.508500220591774,
 '2017-08-03 03:38:33.621184900': 18.441375953712782,
 '2017-08-19 03:38:38.910826': 17.692259407996236,
 '2017-09-04 03:38:41.508263': 10.447166621952414,
 '2017-10-06 03:38:50.622708': -7.521680769355522,
 '2017-10-22 03:38:53.191335': None,
 '2017-11-07 03:38:51.977366': -10.797046998573808,
 '2017-01-23 03:39:05.629705': -20.378317952286675,
 '2017-02-08 03:38:57.443990900': -21.583127452131514,
 '2017-02-24 03:38:52.364695': -9.804417243085332,
 '2017-03-12 03:38:44.308720': -12.287764291135767,
 '2017-03-28 03:38:35.055194': -4.4869991071619575,
 '2017-04-13 03:38:27.552269900': None,
 '2017-04-29 03:38:16.782493': -3.1260290536505573,
 '2017-05-15 03:38:24.645004': -1.7866889545962759,
 '2017-05-31 03:38:34.590798': 3.4417300413867293,
 '2017-06-16 03:38:41.491310': None,
 '2017-07-18 03:38:49.882010900': 12.527636706530911,
 '2017-08-03 03:38:57.503751': 18.65021386931432,
 '2017-08-19 03:39:02.789157': 15.34773717329044,
 '2017-09-04 03:39:05.390830': 10.438312756135037,
 '2017-09-20 03:39:08.128859': 5.2927516541192645,
 '2017-10-06 03:39:14.505273900': -4.937822413928506,
 '2017-10-22 03:39:17.069666': None,
 '2017-11-07 03:39:15.859933': None,
 '2017-11-23 03:39:10.727268': -7.949822665009055,
 '2017-01-23 03:39:29.512270900': -18.85051198599748,
 '2017-02-08 03:39:21.322320900': -20.41132208507788,
 '2017-02-24 03:39:16.238790': -5.858280236361073,
 '2017-03-12 03:39:08.178578': -10.68055875335116,
 '2017-03-28 03:38:58.925051900': -3.815450606597604,
 '2017-04-13 03:38:51.422128': -3.2306169681808496,
 '2017-04-29 03:38:40.673530900': -2.3232447295344576,
 '2017-05-15 03:38:48.523335': -3.961163490983928,
 '2017-05-31 03:38:58.477600900': 3.995750083877665,
 '2017-06-16 03:39:05.369640': 3.637840854857566,
 '2017-07-02 03:39:09.629194': None,
 '2017-07-18 03:39:13.764578': 12.164125503368272,
 '2017-08-03 03:39:21.382082': 18.69826052507265,
 '2017-08-19 03:39:26.671723': 12.576211996277017,
 '2017-09-04 03:39:29.277632': 11.516094880867616,
 '2017-09-20 03:39:32.011425': None,
 '2017-10-06 03:39:38.383605': 2.823911513304767,
 '2017-10-22 03:39:40.952232': -8.07564380053146,
 '2017-11-07 03:39:39.738263': None,
 '2017-11-23 03:39:34.605598900': -6.672408265004316,
 '2017-01-23 03:39:53.394837': -19.554791369338634,
 '2017-02-08 03:39:45.204887': -18.830844400293017,
 '2017-03-12 03:39:32.061144': -9.005285685304163,
 '2017-03-28 03:39:22.807618': -3.013241980668547,
 '2017-04-13 03:39:15.304693900': None,
 '2017-04-29 03:39:04.551860900': 0.09399173511424132,
 '2017-05-15 03:39:12.410136900': -4.665204905425594,
 '2017-05-31 03:39:22.360167': 3.9049946347391193,
 '2017-06-16 03:39:29.260679': 6.974410039389927,
 '2017-07-02 03:39:33.511761': None,
 '2017-07-18 03:39:37.651379900': 15.37887140184424,
 '2017-08-03 03:39:45.264648': 15.860480749567055,
 '2017-09-04 03:39:53.155962900': 13.107617713330962,
 '2017-09-20 03:39:55.889756': None,
 '2017-10-06 03:40:02.266171900': -0.8290572725271659,
 '2017-10-22 03:40:04.834798900': None,
 '2017-11-07 03:40:03.620829900': None,
 '2017-11-23 03:39:58.488166': -12.366253303874807,
 '2017-12-09 03:39:55.314832': None,
 '2017-02-15 03:44:42.187614': -22.524661335921508,
 '2017-03-03 03:44:36.257759900': -18.26381203004633,
 '2017-03-19 03:44:26.643438': -11.31687629310903,
 '2017-04-04 03:44:19.131335': -5.763887111534187,
 '2017-05-22 03:44:16.319245900': -0.926355370522889,
 '2017-06-07 03:44:24.867427': 1.4893034744061362,
 '2017-06-23 03:44:30.608063900': 4.753960748393003,
 '2017-07-09 03:44:33.799904900': 5.935049832203696,
 '2017-07-25 03:44:40.459951': 15.132816542822091,
 '2017-08-10 03:44:47.057299': 15.516005529328064,
 '2017-08-26 03:44:51.233915': None,
 '2017-09-11 03:44:52.336128': 10.951404011141062,
 '2017-09-27 03:44:58.346037': None,
 '2017-10-13 03:45:02.986059': -2.0883790006724827,
 '2017-10-29 03:45:03.936234': 0.32464304416281126,
 '2017-11-14 03:45:01.006933900': -6.9017364483830725,
 '2017-02-15 03:45:06.061708': -18.794238906957982,
 '2017-03-03 03:45:00.140326': -19.98410253372752,
 '2017-03-19 03:44:50.521768': -10.937458220794456,
 '2017-04-04 03:44:43.009665': -12.495252648225874,
 '2017-05-22 03:44:40.206048': -0.7992514758734974,
 '2017-06-07 03:44:48.749994': 2.653746040538438,
 '2017-06-23 03:44:54.490630': 6.792318907480517,
 '2017-07-09 03:44:57.686707': 6.732567932326761,
 '2017-07-25 03:45:04.342518': 13.533780952772727,
 '2017-08-10 03:45:10.944101900': 17.713649741172723,
 '2017-08-26 03:45:15.116480900': 11.952852904800531,
 '2017-09-11 03:45:16.222931': 10.517833398462308,
 '2017-10-29 03:45:27.823036': 2.616076274637544,
 '2017-11-14 03:45:24.885265': -9.2043514416623,
 '2017-01-30 03:45:37.197600': -19.583910890085498,
 '2017-02-15 03:45:29.944273900': -15.69636205813897,
 '2017-03-03 03:45:24.010183900': -14.744677486722747,
 '2017-03-19 03:45:14.395861': -9.581691387754262,
 '2017-04-04 03:45:06.883759': -11.231752929845733,
 '2017-04-20 03:44:57.890897': -12.178739840019034,
 '2017-05-06 03:44:52.443128900': None,
 '2017-05-22 03:45:04.092850': -0.018262023062890927,
 '2017-06-07 03:45:12.628324': 3.775005660141821,
 '2017-06-23 03:45:18.373196': 6.8241014117778915,
 '2017-07-09 03:45:21.569273900': 8.927481102609745,
 '2017-07-25 03:45:28.225083900': None,
 '2017-08-10 03:45:34.826668': 14.011855429904964,
 '2017-08-26 03:45:38.994810900': 9.040816448360898,
 '2017-09-11 03:45:40.101262': 9.877773404041243,
 '2017-09-27 03:45:46.106934': None,
 '2017-10-13 03:45:50.755428': None,
 '2017-10-29 03:45:51.701367': 4.136695993256434,
 '2017-11-14 03:45:48.763595900': -0.9581676965605234,
 '2017-01-14 03:46:07.348397': -16.959720847454967,
 '2017-01-30 03:46:01.075930': -20.231887799522163,
 '2017-02-15 03:45:53.818368': -14.476613753287232,
 '2017-03-03 03:45:47.892750': None,
 '2017-03-19 03:45:38.274191': -8.831921642699124,
 '2017-04-04 03:45:30.762089': None,
 '2017-04-20 03:45:21.769227': -19.910586245791503,
 '2017-05-06 03:45:16.329931': -0.8898551918325378,
 '2017-05-22 03:45:27.975417': -0.23194635081376264,
 '2017-06-07 03:45:36.519362': 5.759197157219517,
 '2017-06-23 03:45:42.259999': 12.30534816093454,
 '2017-07-09 03:45:45.451840': 12.743533389187236,
 '2017-08-10 03:45:58.704999': 14.456523362013536,
 '2017-08-26 03:46:02.881614': None,
 '2017-09-11 03:46:03.983827900': 10.198946423962147,
 '2017-09-27 03:46:09.993737': None,
 '2017-10-13 03:46:14.629523': -6.51008476883419,
 '2017-10-29 03:46:15.583933900': 4.645330497983437,
 '2017-11-14 03:46:12.646162': 1.7975781216607283,
 '2017-11-30 03:46:06.124393': -5.899447378762131,
 '2017-01-21 03:51:28.047219': -18.1421756779906,
 '2017-02-06 03:51:20.276366900': -16.825323726583928,
 '2017-02-22 03:51:14.726626': -15.097652412020429,
 '2017-03-10 03:51:07.136603': None,
 '2017-03-26 03:50:57.340641': -5.658467849129122,
 '2017-04-11 03:50:50.303553900': -6.181206063541911,
 '2017-04-27 03:50:39.719727': None,
 '2017-05-13 03:50:44.828666900': 1.8972614489746433,
 '2017-05-29 03:50:55.191226': -0.2684484193238172,
 '2017-06-14 03:51:02.413471900': 2.529010099674642,
 '2017-06-30 03:51:06.990744900': -0.4721117425819379,
 '2017-07-16 03:51:10.401861': 13.519876168876134,
 '2017-08-01 03:51:18.311515900': 16.32118918204699,
 '2017-08-17 03:51:23.884764': 16.52451581336773,
 '2017-09-02 03:51:26.859210': None,
 '2017-09-18 03:51:28.715742': 9.9076354821217,
 '2017-10-04 03:51:35.562035': 8.212468640789869,
 '2017-10-20 03:51:38.575510': 2.5879311379983148,
 '2017-11-05 03:51:37.848423': -5.704837166682456,
 '2017-01-21 03:51:51.925548': -17.07928829355428,
 '2017-02-06 03:51:44.154696900': -15.710220041555994,
 '2017-03-10 03:51:31.014933': -17.711878730525292,
 '2017-03-26 03:51:21.214735': -4.524389959427234,
 '2017-04-11 03:51:14.177647900': -3.056382629933793,
 '2017-04-27 03:51:03.589584900': -2.4223893370392693,
 '2017-05-13 03:51:08.711233': 2.407442570573112,
 '2017-05-29 03:51:19.078029': 3.111797366707418,
 '2017-06-14 03:51:26.304510': 5.06493104773076,
 '2017-06-30 03:51:30.873312': None,
 '2017-07-16 03:51:34.284428': 13.096647988412617,
 '2017-08-01 03:51:42.194082': 17.652077903081043,
 '2017-08-17 03:51:47.771566': 14.485728503966605,
 '2017-09-18 03:51:52.598309': 9.070289109171725,
 '2017-10-04 03:51:59.440366': 6.570302302991409,
 '2017-10-20 03:52:02.462312900': 5.289758914320316,
 '2017-11-05 03:52:01.735225': 3.2587893378327175,
 '2017-11-21 03:51:57.106756900': -14.248254102546511,
 '2017-01-05 03:52:20.306587': -3.386689796168058,
 '2017-01-21 03:52:15.803877900': -18.398420135034087,
 '2017-02-06 03:52:08.037263': -12.662070001432268,
 '2017-02-22 03:52:02.479050': -12.12274099151972,
 '2017-03-10 03:51:54.884791': None,
 '2017-03-26 03:51:45.088829': -3.7170348540362927,
 '2017-04-11 03:51:38.051742': -6.98460654395562,
 '2017-04-27 03:51:27.467914900': -5.195423184463249,
 '2017-05-13 03:51:32.598035': 2.398978309195083,
 '2017-05-29 03:51:42.956359': 4.06752227147106,
 '2017-06-14 03:51:50.187076900': 6.271926963397863,
 '2017-06-30 03:51:54.751642': None,
 '2017-07-16 03:51:58.171230': 12.924862852838155,
 '2017-08-01 03:52:06.076649': 18.666568119093174,
 '2017-08-17 03:52:11.654133': 13.860203474974774,
 '2017-09-18 03:52:16.476640': 7.2765616528413695,
 '2017-10-04 03:52:23.318696': 3.7355130102011285,
 '2017-10-20 03:52:26.344879': 5.247694860729347,
 '2017-11-05 03:52:25.613556': 4.166255542868769,
 '2017-11-21 03:52:20.980850900': None,
 '2017-12-07 03:52:17.053124': -13.47553163969888,
 '2017-01-28 03:57:59.797172': -19.974594654634696,
 '2017-02-13 03:57:52.077799': None,
 '2017-03-01 03:57:46.600999': None,
 '2017-03-17 03:57:37.405834': -5.505221113678374,
 '2017-04-02 03:57:29.410695': None,
 '2017-05-04 03:57:12.444641900': 1.275757850574788,
 '2017-05-20 03:57:24.439162': 3.109574198432271,
 '2017-06-21 03:57:39.426626': 12.05073285488608,
 '2017-07-23 03:57:48.872018900': 13.746343806143654,
 '2017-08-08 03:57:55.765620900': 18.16155323972221,
 '2017-08-24 03:58:00.272385': None,
 '2017-09-25 03:58:06.879910900': None,
 '2017-10-11 03:58:12.015574900': 2.9580132944786137,
 '2017-10-27 03:58:13.440179900': None,
 '2017-11-12 03:58:10.976840': -11.372372909663623,
 '2017-11-28 03:58:04.598889900': 1.6905211641806146,
 '2017-01-12 03:58:29.541531': -11.930044958040856,
 '2017-01-28 03:58:23.671266': -19.64278111988987,
 '2017-02-13 03:58:15.956128': None,
 '2017-03-01 03:58:10.479329': -14.749031046333911,
 '2017-03-17 03:58:01.275692': -6.654496541559977,
 '2017-04-02 03:57:53.289024900': None,
 '2017-05-04 03:57:36.331444': 0.16185164699709553,
 '2017-05-20 03:57:48.317493': 2.731996306379084,
 '2017-06-05 03:57:57.242692': None,
 '2017-06-21 03:58:03.313428': 12.277307857314293,
 '2017-07-07 03:58:06.801895': None,
 '2017-07-23 03:58:12.750348900': 16.402222550728066,
 '2017-08-08 03:58:19.648187': 19.538572262363918,
 '2017-08-24 03:58:24.159187': None,
 '2017-09-25 03:58:30.758242': None,
 '2017-10-11 03:58:35.893905': -0.657937121932062,
 '2017-10-27 03:58:37.318509900': -5.6103652792834575,
 '2017-11-12 03:58:34.855171': None,
 '2017-11-28 03:58:28.481456': -10.663043651037698,
 '2017-01-19 04:04:38.170650': None,
 '2017-02-04 04:04:30.827408': -13.091177652555531,
 '2017-02-20 04:04:24.790301': None,
 '2017-03-08 04:04:17.661822': -10.311608222879478,
 '2017-03-24 04:04:07.323616': -1.154336850661064,
 '2017-04-09 04:04:00.752225': -9.689003942178385,
 '2017-04-25 04:03:50.642445': -0.2926184315219581,
 '2017-05-11 04:03:52.730938': 0.4299647301025475,
 '2017-05-27 04:04:03.472100': None,
 '2017-06-12 04:04:11.058611': 9.922463833251228,
 '2017-07-14 04:04:18.635900': 14.34067643459988,
 '2017-07-30 04:04:26.850580900': None,
 '2017-08-15 04:04:32.720078': None,
 '2017-08-31 04:04:36.050244': 10.576904802287443,
 '2017-09-16 04:04:37.017028900': 7.692815268544577,
 '2017-10-02 04:04:44.311911': 6.984935908980802,
 '2017-10-18 04:04:47.782828': 6.484617842448434,
 '2017-11-03 04:04:47.542686': 5.616988239156938,
 '2017-11-19 04:04:43.401630': 2.1700293634590273,
 '2017-12-05 04:04:38.732154': -7.41002710671239,
 '2017-01-31 03:40:35.321672600': -26.745171282011093,
 '2017-02-16 03:40:32.709925600': -26.28807253958629,
 '2017-03-04 03:40:41.148019': -15.774672657548201,
 '2017-03-20 03:40:46.923512500': -11.16442040731262,
 '2017-04-05 03:40:50.514011700': -5.795943528382965,
 '2017-04-21 03:40:58.288101900': -5.190226724986996,
 '2017-05-07 03:41:02.916713500': 0.21072788586748303,
 '2017-05-23 03:41:05.273215500': None,
 '2017-06-08 03:41:10.292654700': -0.5391164391993093,
 '2017-06-24 03:41:13.426064': 7.96745821966242,
 '2017-07-26 03:41:14.033329400': 13.994900189669066,
 '2017-08-11 03:41:11.622631300': 18.09101896147308,
 '2017-09-12 03:41:10.563731900': 10.684524649172623,
 '2017-09-28 03:41:10.534835100': 2.718745813657689,
 '2017-10-14 03:41:12.710329600': None,
 '2017-10-30 03:41:13.223338900': -12.838369454024054,
 '2017-11-15 03:41:10.575287100': -9.857937654262734,
 '2017-01-31 03:40:59.199287600': -24.164204080506277,
 '2017-02-16 03:40:56.589028800': -20.94582425808577,
 '2017-03-04 03:41:05.027645700': -16.57358240963037,
 '2017-03-20 03:41:10.802739600': -11.304695354914593,
 '2017-04-05 03:41:14.392230500': -7.200888616376762,
 '2017-04-21 03:41:22.165149600': -5.454051069267529,
 '2017-05-07 03:41:26.793341500': 0.6383417686887509,
 '2017-05-23 03:41:29.150413300': -2.531676144629431,
 '2017-06-08 03:41:34.170771700': 1.5733628624930913,
 '2017-06-24 03:41:37.305139700': 6.375163038948459,
 '2017-07-10 03:41:38.544410100': 6.8321954430956024,
 '2017-07-26 03:41:37.911605': 13.626761008128224,
 '2017-08-11 03:41:35.499749300': 17.9658276169732,
 '2017-08-27 03:41:35.755994400': 8.084393373793175,
 '2017-09-12 03:41:34.440837100': 10.684697819840576,
 '2017-09-28 03:41:34.412900900': 4.21835883041294,
 '2017-10-14 03:41:36.589521700': -1.165283877590226,
 '2017-10-30 03:41:37.102679600': -7.51998130439869,
 '2017-11-15 03:41:34.453940100': -9.40082078490632,
 '2017-01-31 03:41:23.077787': -18.969943082874316,
 '2017-02-16 03:41:20.468710100': -17.023600695396844,
 '2017-03-04 03:41:28.907935600': -20.308575552763745,
 '2017-03-20 03:41:34.682526500': -11.121556675002138,
 '2017-04-05 03:41:38.270789700': -4.620061718046168,
 '2017-04-21 03:41:46.042938200': -3.1364981037541817,
 '2017-05-07 03:41:50.670501700': 0.7681502604089839,
 '2017-05-23 03:41:53.028235900': None,
 '2017-06-08 03:41:58.049685': 0.5657647587764509,
 '2017-06-24 03:42:01.184856600': 8.213890509858157,
 '2017-07-10 03:42:02.424095900': 7.62241841051124,
 '2017-07-26 03:42:01.790594300': 12.917904476481887,
 '2017-08-11 03:41:59.377867400': 16.33922636264435,
 '2017-08-27 03:41:59.633786400': 8.319709768338226,
 '2017-09-12 03:41:58.318740600': 10.07250959567447,
 '2017-09-28 03:41:58.291967200': 3.1531808841318787,
 '2017-10-14 03:42:00.469263700': 1.752998715033692,
 '2017-10-30 03:42:00.982817200': -2.5118438767696643,
 '2017-11-15 03:41:58.333523700': -4.395877563616057,
 '2017-02-07 03:46:44.067356900': -21.131671252364846,
 '2017-02-23 03:46:47.552101700': -14.232831360804136,
 '2017-03-11 03:46:54.770321600': -13.21746257938992,
 '2017-04-12 03:47:05.062627500': None,
 '2017-04-28 03:47:11.519150800': -1.775592198336637,
 '2017-05-30 03:47:18.489835200': 1.8544857372966705,
 '2017-06-15 03:47:22.728114400': -1.1741084210705504,
 '2017-07-01 03:47:24.974224400': 9.558173479027445,
 '2017-07-17 03:47:25.446855900': None,
 '2017-08-02 03:47:23.985868300': 14.962052760453032,
 '2017-09-03 03:47:22.517122600': None,
 '2017-09-19 03:47:19.987894800': None,
 '2017-10-21 03:47:24.145233700': 2.1121580717071797,
 '2017-11-06 03:47:23.293746300': 0.6812098526299146,
 '2017-02-07 03:47:07.945807500': -19.571614373668506,
 '2017-02-23 03:47:11.431678500': -14.004933455881298,
 '2017-03-11 03:47:18.649940': -13.086231081467766,
 '2017-03-27 03:47:23.490839800': None,
 '2017-04-12 03:47:28.940398800': None,
 '2017-04-28 03:47:35.395740500': 0.3094485035188919,
 '2017-05-14 03:47:38.911685400': -4.737122450729112,
 '2017-05-30 03:47:42.367599': 2.174435831471167,
 '2017-06-15 03:47:46.606839': 0.47296165334880674,
 '2017-07-01 03:47:48.853361700': 7.633088118439093,
 '2017-07-17 03:47:49.325443900': None,
 '2017-08-02 03:47:47.863679100': 16.85580219461272,
 '2017-08-18 03:47:46.165127600': 13.390937727721631,
 '2017-09-19 03:47:43.865820800': 9.245985635984509,
 '2017-10-05 03:47:45.860978': 1.7936821057903891,
 '2017-10-21 03:47:48.024436200': 3.623799369172042,
 '2017-11-06 03:47:47.173027800': 1.9490020405809674,
 '2017-01-22 03:47:37.321805500': -18.962550901333955,
 '2017-02-07 03:47:31.824714300': -16.71620776584006,
 '2017-02-23 03:47:35.311685500': -13.240283876445716,
 '2017-03-11 03:47:42.530168400': -16.71024663187297,
 '2017-03-27 03:47:47.370291200': -4.27569426392839,
 '2017-04-12 03:47:52.818520500': -6.979487064989096,
 '2017-04-28 03:47:59.273258800': 0.40569030006409096,
 '2017-05-14 03:48:02.789005500': -1.1683189262517815,
 '2017-05-30 03:48:06.245945600': 1.2710235238550542,
 '2017-06-15 03:48:10.486075100': 4.461870334298206,
 '2017-07-01 03:48:12.733283800': 7.477910364212813,
 '2017-08-02 03:48:11.742258300': 15.740240597747604,
 '2017-08-18 03:48:10.042920300': 13.294136929303113,
 '2017-09-19 03:48:07.744265300': 7.486508922518809,
 '2017-10-05 03:48:09.740453100': None,
 '2017-10-21 03:48:11.904599300': None,
 '2017-11-06 03:48:11.052982': 3.4178678812281733,
 '2017-11-22 03:48:06.996022800': -7.8011793889818035,
 '2017-01-22 03:48:01.199805700': -16.603789624254166,
 '2017-02-07 03:47:55.704282500': -14.85557185069582,
 '2017-02-23 03:47:59.192457500': -12.56255968092973,
 '2017-03-11 03:48:06.410916900': -12.918104568635744,
 '2017-03-27 03:48:11.250106': -1.0732265428191865,
 '2017-04-12 03:48:16.697439400': -4.728722504610667,
 '2017-04-28 03:48:23.151179500': 1.1107854309492142,
 '2017-05-14 03:48:26.666977700': 3.823607622954783,
 '2017-05-30 03:48:30.125045100': 1.0487826474763449,
 '2017-06-15 03:48:34.365972700': 6.830496705053148,
 '2017-07-01 03:48:36.613599700': 7.0490806993278445,
 '2017-08-02 03:48:35.621192500': 8.35421011316341,
 '2017-08-18 03:48:33.921474300': 14.21658409978087,
 '2017-09-19 03:48:31.623270800': 8.440924510602757,
 '2017-10-05 03:48:33.620421100': None,
 '2017-10-21 03:48:35.785043300': None,
 '2017-11-06 03:48:34.933347300': 3.8024754610433966,
 '2017-11-22 03:48:30.875766700': -11.097540579789365,
 '2017-01-29 03:53:21.340762400': -19.140066662555373,
 '2017-02-14 03:53:16.999500900': -14.948520166260549,
 '2017-03-02 03:53:25.784933200': -12.168657632773677,
 '2017-03-18 03:53:31.829950600': -14.437179235836922,
 '2017-04-03 03:53:35.584710100': -30.63277638299559,
 '2017-05-05 03:53:47.984668': -3.5826881905456873,
 '2017-05-21 03:53:50.618194800': 2.6800235190777175,
 '2017-06-06 03:53:55.281877': None,
 '2017-06-22 03:53:58.648519500': 8.02028602517973,
 '2017-07-08 03:54:00.104805200': 6.440171801129055,
 '2017-07-24 03:53:59.736361100': 8.208757222084826,
 '2017-08-09 03:53:57.535663100': 19.493989550203935,
 '2017-08-25 03:53:57.368917100': None,
 '2017-09-10 03:53:56.456290200': 9.869059291067856,
 '2017-10-12 03:53:57.959204600': -0.35309416463970544,
 '2017-10-28 03:53:58.846111600': None,
 '2017-11-13 03:53:56.586281': 0.72746906770207,
 '2017-01-29 03:53:45.219004100': -18.277978246865434,
 '2017-02-14 03:53:40.879137': -14.799174321879795,
 '2017-03-02 03:53:49.665261': -8.86637225960024,
 '2017-03-18 03:53:55.709985300': -10.521069805736804,
 '2017-04-19 03:54:06.854362': -1.3953206223818828,
 '2017-05-05 03:54:11.861877200': -1.215788769494607,
 '2017-05-21 03:54:14.496044100': 3.3754230470688458,
 '2017-06-06 03:54:19.160632400': 3.941606934589791,
 '2017-06-22 03:54:22.528282900': 8.98002122408809,
 '2017-07-08 03:54:23.984383500': 9.386082758329165,
 '2017-07-24 03:54:23.615326800': 6.673133361231909,
 '2017-08-09 03:54:21.413501500': 18.944524019580673,
 '2017-08-25 03:54:21.246410400': None,
 '2017-09-10 03:54:20.334000400': 9.306571742876496,
 '2017-09-26 03:54:19.689843200': -0.6429785217422086,
 '2017-10-12 03:54:21.838935300': -1.1657119152030078,
 '2017-10-28 03:54:22.726094300': 4.338680819370749,
 '2017-11-13 03:54:20.465685900': 1.6089391826849535,
 '2017-01-13 03:54:18.413973200': -11.596981638873897,
 '2017-01-29 03:54:09.097768400': -16.968337426212546,
 '2017-02-14 03:54:04.759482500': -13.495701304828145,
 '2017-03-02 03:54:13.546140': -8.14734975708536,
 '2017-03-18 03:54:19.590454200': -9.445898533881175,
 '2017-04-03 03:54:23.342716800': None,
 '2017-04-19 03:54:30.732811800': -1.2969679078875544,
 '2017-05-05 03:54:35.739796500': -1.3895352098131697,
 '2017-05-21 03:54:38.374389700': 3.310192441489571,
 '2017-06-06 03:54:43.040038700': 4.141449668988926,
 '2017-06-22 03:54:46.408712700': 10.409414175673872,
 '2017-07-08 03:54:47.864762500': 9.42463336476323,
 '2017-07-24 03:54:47.495132400': -29.20967676023345,
 '2017-08-09 03:54:45.292282400': 18.105249032460396,
 '2017-08-25 03:54:45.124847200': 2.625579188780405,
 '2017-09-10 03:54:44.212456700': 8.968663008014664,
 '2017-09-26 03:54:43.569446600': 4.14822443219083,
 '2017-10-12 03:54:45.719249600': 1.0086932213750999,
 '2017-10-28 03:54:46.606802100': 3.606376555873354,
 '2017-11-13 03:54:44.345958500': -1.1671034372342701,
 '2017-11-29 03:54:38.845349700': -15.76766006045823,
 '2017-01-20 04:00:00.422271200': -42.253708212871125,
 '2017-02-05 03:59:54.088109': -13.866235701067167,
 '2017-02-21 03:59:55.837298300': -10.853939836554853,
 '2017-03-09 04:00:03.411164800': -10.616593713508982,
 '2017-03-25 04:00:08.508460': -1.4547364452270488,
 '2017-04-10 04:00:13.449857900': -2.9616415733185835,
 '2017-04-26 04:00:20.300638600': -2.751708962916508,
 '2017-05-12 04:00:24.089267800': 0.593089347046108,
 '2017-05-28 04:00:27.214022900': 1.3998912327236452,
 '2017-06-29 04:00:34.200565800': 10.824218600332557,
 '2017-07-15 04:00:34.880201100': 7.948468070721022,
 '2017-07-31 04:00:33.653401900': 11.181789839391794,
 '2017-08-16 04:00:31.494027800': None,
 '2017-09-01 04:00:32.003590300': 3.5482154961018613,
 '2017-09-17 04:00:29.682059': None,
 '2017-10-03 04:00:31.055786800': -21.57456556929219,
 '2017-10-19 04:00:33.418842700': 5.098384076237577,
 '2017-11-04 04:00:32.956333600': None,
 '2017-01-20 04:00:24.300145800': -19.99907038822166,
 '2017-02-05 04:00:17.967550200': -15.307193145191766,
 '2017-02-21 04:00:19.717973300': -16.919663135186884,
 '2017-03-09 04:00:27.292018100': -11.38478357541205,
 '2017-03-25 04:00:32.388570': -2.1886653697390894,
 '2017-04-10 04:00:37.328621': -3.7271908566859167,
 '2017-04-26 04:00:44.178871300': -5.344190233097456,
 '2017-05-12 04:00:47.967295100': None,
 '2017-05-28 04:00:51.092827500': 2.806771394253977,
 '2017-06-29 04:00:58.081017700': 11.030621906372433,
 '2017-07-15 04:00:58.760290200': 5.386903442528738,
 '2017-07-31 04:00:57.532585': 16.444204009204554,
 '2017-08-16 04:00:55.372381': None,
 '2017-09-01 04:00:55.881787600': 10.71438276438525,
 '2017-09-17 04:00:53.560664100': None,
 '2017-10-03 04:00:54.935646500': None,
 '2017-10-19 04:00:57.299361700': 5.7338618737354095,
 '2017-11-04 04:00:56.836865300': 3.2122208243022214,
 '2017-02-12 04:06:25.143845400': -18.21593840483506,
 '2017-02-28 04:06:34.222440700': -9.136683001001037,
 '2017-04-01 04:06:44.669613900': -0.7682243496357674,
 '2017-04-17 04:06:51.502992200': -0.36127189829848316,
 '2017-06-04 04:07:04.100409800': None,
 '2017-06-20 04:07:07.712959400': 9.143758076442122,
 '2017-07-06 04:07:09.400469100': 12.484920923866449,
 '2017-07-22 04:07:09.265170900': 14.534205122120857,
 '2017-08-07 04:07:07.292764900': 16.333420098189222,
 '2017-08-23 04:07:06.702534300': 3.998688352687465,
 '2017-09-08 04:07:06.157421900': 6.776869807966933,
 '2017-09-24 04:07:04.935283300': None,
 '2017-10-10 04:07:07.047037400': 6.891730873084482,
 '2017-10-26 04:07:08.307675600': -37.25698122199681,
 '2017-11-11 04:07:06.428328600': -28.431777128707683,
 '2017-11-27 04:07:01.345443200': -26.009216846703193,
'2018-02-11 03:38:24.885847900': -24.374881202140877,
 '2018-02-27 03:38:18.290125': -20.796889775204615,
 '2018-03-15 03:38:08.952937': -20.465732090844146,
 '2018-03-31 03:38:02.445377': None,
 '2018-04-16 03:37:53.380486': -1.7992668260821036,
 '2018-05-02 03:37:44.561408': -5.469998184175051,
 '2018-05-18 03:37:33.827366': -0.05343377371877448,
 '2018-06-03 03:37:21.882172900': 3.2291671516191713,
 '2018-06-19 03:37:31.042084': 8.760139574464647,
 '2018-07-05 03:37:40.343950': 15.24618117647548,
 '2018-08-22 03:38:04.194243': 16.27627194545369,
 '2018-09-07 03:38:10.169814900': 10.586387630252473,
 '2018-09-23 03:38:14.865821': 6.715010829747521,
 '2018-10-09 03:38:22.947710': 2.631764749966456,
 '2018-10-25 03:38:28.340228': 0.16763017251270657,
 '2018-11-10 03:38:30.993233': -6.235237049885819,
 '2018-01-26 03:38:55.734882': -27.501359537895528,
 '2018-02-11 03:38:48.755707': -24.016993638458157,
 '2018-02-27 03:38:42.168456': -24.57074648624009,
 '2018-03-15 03:38:32.831268': -17.177045959047987,
 '2018-03-31 03:38:26.323707900': None,
 '2018-04-16 03:38:17.254581': -1.1946268992491322,
 '2018-05-02 03:38:08.431266': -2.1318332438183214,
 '2018-05-18 03:37:57.701461': 0.5089361467806653,
 '2018-06-03 03:37:45.760504': 2.8658831029288847,
 '2018-06-19 03:37:54.928887': 6.368835465039247,
 '2018-07-05 03:38:04.222281': 8.794148102242968,
 '2018-07-21 03:38:11.232466': None,
 '2018-08-22 03:38:28.076810': 17.070404945153232,
 '2018-09-07 03:38:34.052382': 10.659821834732693,
 '2018-09-23 03:38:38.744151900': 8.269587102279715,
 '2018-10-09 03:38:46.826041': 3.8933223558258887,
 '2018-10-25 03:38:52.227031': 1.1901751054357705,
 '2018-11-10 03:38:54.875800': -3.5944704606158453,
 '2018-01-26 03:39:19.617448': None,
 '2018-02-11 03:39:12.634037': -20.483096804326202,
 '2018-02-27 03:39:06.042551': -21.513886734908006,
 '2018-03-15 03:38:56.705363': -14.41306871276221,
 '2018-03-31 03:38:50.197803': None,
 '2018-04-16 03:38:41.132911': -0.6793807355816941,
 '2018-05-02 03:38:32.305361': -0.5378330530440124,
 '2018-05-18 03:38:21.579792': 0.4619567535287022,
 '2018-06-03 03:38:09.643070': 3.9242378036561076,
 '2018-06-19 03:38:18.815689': 7.504724874540913,
 '2018-07-05 03:38:28.113320': 7.013934159556202,
 '2018-07-21 03:38:35.119269': None,
 '2018-08-22 03:38:51.959376900': 16.96065942086206,
 '2018-09-07 03:38:57.939185': 11.723411478703198,
 '2018-09-23 03:39:02.626719': 10.052760998706148,
 '2018-10-09 03:39:10.708608': 5.3594654033029325,
 '2018-10-25 03:39:16.105361900': 3.6100014693434352,
 '2018-11-10 03:39:18.758366900': 1.4599629929724445,
 '2018-11-26 03:39:18.429109': None,
 '2018-01-10 03:39:52.229322': -10.90025723761894,
 '2018-01-26 03:39:43.500015': None,
 '2018-02-11 03:39:36.512368': -17.849997023284004,
 '2018-02-27 03:39:29.920882': None,
 '2018-03-15 03:39:20.583693': -11.877570157066165,
 '2018-03-31 03:39:14.076133': None,
 '2018-04-16 03:39:05.015478': 0.2646624610169987,
 '2018-05-02 03:38:56.187928': 1.4142058941142606,
 '2018-05-18 03:38:45.453887': None,
 '2018-06-03 03:38:33.525637': 5.987401287277933,
 '2018-06-19 03:38:42.698256': 13.102146027215946,
 '2018-07-05 03:38:51.991650900': None,
 '2018-08-22 03:39:15.846180': 17.18262090997577,
 '2018-09-07 03:39:21.821752': 14.584010673287008,
 '2018-09-23 03:39:26.513522': 10.455416269175643,
 '2018-10-09 03:39:34.595410900': 5.002952049707084,
 '2018-10-25 03:39:39.992165': 4.673535504241153,
 '2018-11-10 03:39:42.640934': 4.4998869639843075,
 '2018-12-12 03:39:39.038451': -5.901843025791968,
 '2018-02-02 03:44:38.180863': -26.215119196607887,
 '2018-02-18 03:44:33.189432': None,
 '2018-03-06 03:44:25.354465': -22.642561362398048,
 '2018-03-22 03:44:17.173516': -12.53922315670663,
 '2018-04-07 03:44:09.620064': -7.904778305051446,
 '2018-04-23 03:44:00.187086': -6.942051590270569,
 '2018-05-09 03:43:51.033231': -0.7424508073813489,
 '2018-05-25 03:43:39.019567': 1.4186285744720086,
 '2018-06-26 03:43:46.245358': 7.383047418167809,
 '2018-07-28 03:44:01.029394': 11.457007969922978,
 '2018-08-13 03:44:10.603506': 13.574119397631376,
 '2018-08-29 03:44:17.958831': 16.93532195807851,
 '2018-09-14 03:44:22.892338': None,
 '2018-09-30 03:44:29.516171': 4.617206062288353,
 '2018-10-16 03:44:36.450626': 0.16214075876612802,
 '2018-11-17 03:44:42.005893': None,
 '2018-02-02 03:45:02.059194': -23.93216867696623,
 '2018-02-18 03:44:57.067763': -24.6043880141175,
 '2018-03-06 03:44:49.224324': -20.834048736161947,
 '2018-03-22 03:44:41.051847': -10.010556899034823,
 '2018-04-07 03:44:33.498394': -6.649324878072274,
 '2018-04-23 03:44:24.056944': -2.3927966614299683,
 '2018-05-09 03:44:14.907326': -0.16258440279780884,
 '2018-05-25 03:44:02.889425': 0.8406374176253435,
 '2018-06-26 03:44:10.132161': 5.136554238342466,
 '2018-07-28 03:44:24.916196900': 11.21233297035415,
 '2018-08-13 03:44:34.481837': 10.141010267029579,
 '2018-08-29 03:44:41.841398': 17.144453101640202,
 '2018-09-14 03:44:46.770669': 7.642936130026782,
 '2018-09-30 03:44:53.398738': 7.554816649790581,
 '2018-10-16 03:45:00.333192900': 3.6379393938913864,
 '2018-11-01 03:45:04.531991': None,
 '2018-01-17 03:45:35.738654': -18.60901623971395,
 '2018-02-02 03:45:25.937525': -18.342390645000915,
 '2018-02-18 03:45:20.950330': -22.90270130766804,
 '2018-03-06 03:45:13.102654900': -17.242837995882283,
 '2018-03-22 03:45:04.930177900': -6.295971256283951,
 '2018-04-07 03:44:57.376725': -1.98734300434701,
 '2018-04-23 03:44:47.939510900': -5.554902447275882,
 '2018-05-09 03:44:38.785656900': 0.9699059550392174,
 '2018-05-25 03:44:26.763520': 1.8445774393909564,
 '2018-06-10 03:44:23.303579': 3.814018078020903,
 '2018-06-26 03:44:34.018963900': 5.4165326340074955,
 '2018-07-12 03:44:42.287550900': None,
 '2018-07-28 03:44:48.798764': 11.383215629152467,
 '2018-08-13 03:44:58.364404': 14.248652670781373,
 '2018-08-29 03:45:05.723964900': 17.113746801608936,
 '2018-09-14 03:45:10.653236': 8.311566047369663,
 '2018-09-30 03:45:17.285540900': 8.738490074076003,
 '2018-10-16 03:45:24.219996': 6.314963377581544,
 '2018-11-01 03:45:28.410322': -1.3200102688256947,
 '2018-11-17 03:45:29.771027': -5.944452570096138,
 '2018-01-17 03:45:59.621220900': -14.860323084339225,
 '2018-02-02 03:45:49.815855': -15.762316060965547,
 '2018-02-18 03:45:44.824424': -19.526113519055937,
 '2018-03-06 03:45:36.985222': -16.17611521305504,
 '2018-03-22 03:45:28.808509': -3.466967938399925,
 '2018-04-07 03:45:21.250820': -0.4878015101620912,
 '2018-04-23 03:45:11.809370': -3.466099545160255,
 '2018-05-09 03:45:02.659752': 2.1463284815689376,
 '2018-05-25 03:44:50.641850900': None,
 '2018-06-10 03:44:47.181910': 5.396119583945666,
 '2018-06-26 03:44:57.901530': 6.903285977343858,
 '2018-07-12 03:45:06.170118': None,
 '2018-07-28 03:45:12.685567': 9.049401803338997,
 '2018-08-13 03:45:22.251207': 16.005525984176575,
 '2018-08-29 03:45:29.610768': 16.52397801439064,
 '2018-09-14 03:45:34.540038900': 7.744328777071136,
 '2018-09-30 03:45:41.163872': 9.267959831929062,
 '2018-10-16 03:45:48.098327': 5.86846170230248,
 '2018-11-01 03:45:52.292889': None,
 '2018-11-17 03:45:53.649358': -12.269488357990122,
 '2018-12-03 03:45:52.040838': -10.752453668506178,
 '2018-01-24 03:51:18.633840': -34.70760135848096,
 '2018-02-09 03:51:11.014694': None,
 '2018-02-25 03:51:04.774447': -16.72415388085295,
 '2018-03-13 03:50:55.771760900': -10.574356241632083,
 '2018-03-29 03:50:48.891400': -1.161506190815513,
 '2018-04-14 03:50:40.173607': -0.5917808361732145,
 '2018-04-30 03:50:31.202152': -5.889047347122807,
 '2018-05-16 03:50:20.832559': 2.0038696028756204,
 '2018-06-01 03:50:07.806937900': 1.610687683623515,
 '2018-06-17 03:50:15.234097': 4.234349439351881,
 '2018-07-03 03:50:24.840688': 1.502973264141239,
 '2018-07-19 03:50:32.130451': 12.52632650771645,
 '2018-08-04 03:50:40.171358': 12.350465775358257,
 '2018-08-20 03:50:48.792523': 15.14072294499515,
 '2018-09-05 03:50:55.068660': None,
 '2018-09-21 03:50:59.201349': 8.120223583543938,
 '2018-10-07 03:51:07.579595': 5.954343369206366,
 '2018-10-23 03:51:13.323535': 1.1052420850889677,
 '2018-11-08 03:51:16.336633': -4.517456082200314,
 '2018-01-24 03:51:42.512170': -33.13925982253183,
 '2018-02-09 03:51:34.893025': -21.53527832258443,
 '2018-02-25 03:51:28.652778': -17.296787391973197,
 '2018-03-13 03:51:19.645856': -13.139608760213788,
 '2018-03-29 03:51:12.769730900': -1.9412035298757486,
 '2018-04-14 03:51:04.043466': -2.8215930868557524,
 '2018-04-30 03:50:55.080483': -0.19057444763752282,
 '2018-05-16 03:50:44.710889900': 3.276823620791645,
 '2018-06-01 03:50:31.685269': 3.7765968166002097,
 '2018-06-17 03:50:39.120900': 4.366303599839344,
 '2018-07-03 03:50:48.727491': 9.421324729342722,
 '2018-07-19 03:50:56.013016900': 13.140190057612559,
 '2018-08-04 03:51:04.053924': 14.722742380263014,
 '2018-08-20 03:51:12.675090': 15.54756965022327,
 '2018-09-21 03:51:23.083915900': 7.812226318357349,
 '2018-10-07 03:51:31.462162': 6.0461357560653735,
 '2018-10-23 03:51:37.210338': 3.113746700648774,
 '2018-11-08 03:51:40.219200': 0.1688193228463122,
 '2018-11-24 03:51:40.271129': None,
 '2018-01-08 03:52:14.747288': -10.3305516831001,
 '2018-01-24 03:52:06.390501': -25.635721468363748,
 '2018-02-09 03:51:58.771356': -26.755838896925482,
 '2018-02-25 03:51:52.526872': None,
 '2018-03-13 03:51:43.524186': None,
 '2018-03-29 03:51:36.643826': -0.6415235228088884,
 '2018-04-14 03:51:27.921797': -1.235521965478747,
 '2018-04-30 03:51:18.954578': 0.14912767208199273,
 '2018-05-16 03:51:08.584985': 3.194244119190405,
 '2018-06-01 03:50:55.567835900': 1.2314368928346278,
 '2018-06-17 03:51:03.003467': 3.760493807218921,
 '2018-07-03 03:51:12.614293900': 9.067229716484437,
 '2018-07-19 03:51:19.899820': 8.667595056921748,
 '2018-08-04 03:51:27.936491': 14.00858233271719,
 '2018-08-20 03:51:36.561893': 15.590245475049498,
 '2018-09-05 03:51:42.829558': 4.505127256014009,
 '2018-09-21 03:51:46.962247': 6.748076578007931,
 '2018-10-07 03:51:55.344728900': 5.959160668693024,
 '2018-10-23 03:52:01.092904': 4.705907628215042,
 '2018-11-08 03:52:04.106002900': 1.4722370588985771,
 '2018-11-24 03:52:04.149460': -10.789499713016493,
 '2018-12-10 03:52:01.248999': -4.625292253359275,
 '2018-01-15 03:57:58.404597': None,
 '2018-01-31 03:57:48.734728': -17.932735198708777,
 '2018-02-16 03:57:43.357707': -18.97183806226326,
 '2018-03-04 03:57:35.873701': -14.822823214210954,
 '2018-03-20 03:57:27.311486': -11.067797822667256,
 '2018-04-05 03:57:20.101066': -4.357194210558661,
 '2018-04-21 03:57:10.528196': -1.8171647108808489,
 '2018-05-07 03:57:01.709103900': None,
 '2018-05-23 03:56:50.042696': 1.8602396072572625,
 '2018-06-08 03:56:43.439549': 2.8664282776464796,
 '2018-06-24 03:56:54.451234900': -0.3742056353482986,
 '2018-07-10 03:57:02.999354': None,
 '2018-08-11 03:57:18.919256': 18.49451998227439,
 '2018-08-27 03:57:26.592139': 16.173232661245247,
 '2018-09-12 03:57:31.805146900': 9.581853960214897,
 '2018-10-14 03:57:45.147252': 7.986444852365785,
 '2018-10-30 03:57:49.684642': None,
 '2018-11-15 03:57:51.409730': 1.2439709518012205,
 '2018-01-15 03:58:22.287164': -5.558482594955189,
 '2018-01-31 03:58:12.613059': -15.693915849501307,
 '2018-02-16 03:58:07.240273': -19.6718037086442,
 '2018-03-04 03:57:59.752032': -15.895715039027769,
 '2018-03-20 03:57:51.189817': -6.949798340907011,
 '2018-04-05 03:57:43.979396900': -8.37181767218709,
 '2018-04-21 03:57:34.406527': -4.269315981383736,
 '2018-05-07 03:57:25.591670': None,
 '2018-05-23 03:57:13.916791': 1.9694218650554562,
 '2018-06-08 03:57:07.326352': 4.195752203436029,
 '2018-06-24 03:57:18.329566': 6.603239438353143,
 '2018-07-26 03:57:33.193722': 10.106359596334636,
 '2018-08-11 03:57:42.806059': 16.1025358980402,
 '2018-08-27 03:57:50.474705900': 16.64187787937958,
 '2018-09-12 03:57:55.687714': 10.632743620192155,
 '2018-10-14 03:58:09.034055': 7.020028578507092,
 '2018-10-30 03:58:13.562971900': -3.410722055188431,
 '2018-11-15 03:58:15.288061': -0.7542548246787784,
 '2018-01-06 04:04:37.214342': 0.83076603882715,
 '2018-01-22 04:04:29.238557900': None,
 '2018-02-07 04:04:20.975392': -14.160224265382297,
 '2018-02-23 04:04:15.090706': -16.693222511023748,
 '2018-03-11 04:04:06.418090900': -9.18910261865023,
 '2018-03-27 04:03:59.173404900': -6.384235875926884,
 '2018-05-30 04:03:18.800333': 4.102844530368559,
 '2018-06-15 04:03:23.270493': 4.195833984437539,
 '2018-07-01 04:03:33.186016': 10.253349630678366,
 '2018-08-02 04:03:48.355783900': None,
 '2018-08-18 04:03:57.239516': 15.414952713993085,
 '2018-09-03 04:04:03.820336': 14.45547290989182,
 '2018-09-19 04:04:08.042110': None,
 '2018-10-05 04:04:16.060115': None,
 '2018-10-21 04:04:22.147130': None,
 '2018-11-06 04:04:25.507584': 1.4261429415166322,
 '2018-11-22 04:04:25.932312': None,
 '2018-12-08 04:04:23.408656': -1.018572099575545,
 '2018-02-03 03:40:14.342866600': -25.745321454632595,
 '2018-02-19 03:40:03.683234900': -28.45837022872712,
 '2018-03-07 03:39:53.220272300': -23.379768859310232,
 '2018-04-08 03:39:26.629248100': -12.707857524208826,
 '2018-04-24 03:39:10.314292400': None,
 '2018-05-10 03:38:52.471769800': -4.411671419586916,
 '2018-05-26 03:38:38.108523400': None,
 '2018-06-11 03:38:21.813437200': 6.232454345390751,
 '2018-06-27 03:38:03.675140100': 9.233818975201338,
 '2018-07-29 03:37:28.208374900': 14.756993276966327,
 '2018-08-14 03:37:10.415325500': 11.23340602017235,
 '2018-08-30 03:36:51.138380200': None,
 '2018-09-15 03:36:30.218561100': None,
 '2018-10-17 03:35:43.656194300': 1.2785946649939317,
 '2018-11-02 03:35:23.565579600': -0.5984340228281838,
 '2018-02-03 03:40:38.221619300': -24.19563227043343,
 '2018-02-19 03:40:27.562528400': -27.2427437634316,
 '2018-03-07 03:40:17.098971300': -21.683426969376224,
 '2018-03-23 03:40:04.811863800': -5.725468245942899,
 '2018-04-08 03:39:50.505858500': -11.697971027769125,
 '2018-04-24 03:39:34.190586600': -7.72645539305272,
 '2018-05-10 03:39:16.348856800': -5.285251300444251,
 '2018-06-11 03:38:45.692399600': 4.597322404141493,
 '2018-06-27 03:38:27.553885500': 7.036560955044585,
 '2018-07-13 03:38:08.386611500': 2.9418563187563875,
 '2018-07-29 03:37:52.085140700': 12.66577945384237,
 '2018-08-14 03:37:34.291778': 14.473608571059147,
 '2018-08-30 03:37:15.015243100': 7.088727111639111,
 '2018-10-01 03:36:31.616208': -3.633826051466211,
 '2018-10-17 03:36:07.535328300': 3.0635025032653185,
 '2018-11-02 03:35:47.443785500': 1.8386113637395671,
 '2018-11-18 03:35:25.183196': -20.239299443813252,
 '2018-02-03 03:41:02.100878200': -20.095698313461636,
 '2018-02-19 03:40:51.442446400': -24.95286491444457,
 '2018-03-07 03:40:40.978317400': -17.74829554177444,
 '2018-03-23 03:40:28.689864': -3.4670833794064735,
 '2018-04-08 03:40:14.383029100': -16.99662353144452,
 '2018-04-24 03:39:58.067409100': -3.3807750698155465,
 '2018-05-10 03:39:40.226544100': -9.890549541266541,
 '2018-05-26 03:39:25.865650100': None,
 '2018-06-27 03:38:51.433141200': 8.579313845249462,
 '2018-07-13 03:38:32.265004700': None,
 '2018-07-29 03:38:15.962278100': 12.945145975932991,
 '2018-08-14 03:37:58.168525200': 14.320314612553068,
 '2018-08-30 03:37:38.892858300': 5.374452354925026,
 '2018-09-15 03:37:17.975368700': None,
 '2018-10-01 03:36:55.495585500': 4.523289195648695,
 '2018-10-17 03:36:31.415220200': 5.252697052050865,
 '2018-11-02 03:36:11.322946500': 2.2557978759923625,
 '2018-11-18 03:35:49.060986': -13.860638765017386,
 '2018-02-10 03:46:19.529655600': -26.028794966859195,
 '2018-02-26 03:46:10.130250': -21.290755963949557,
 '2018-03-14 03:45:58.914279600': -17.669776970278257,
 '2018-03-30 03:45:45.727549100': -15.56173758675084,
 '2018-05-01 03:45:13.331447300': -0.720856106683072,
 '2018-05-17 03:44:57.270954600': -0.10922658475353027,
 '2018-06-18 03:44:24.922444500': 7.212264965112005,
 '2018-07-04 03:44:05.937475': 6.466851006885028,
 '2018-07-20 03:43:48.364807': 13.201814963410705,
 '2018-08-05 03:43:31.389386700': 8.510707251306872,
 '2018-08-21 03:43:12.972877800': 14.045258443545581,
 '2018-09-22 03:42:31.395573800': 6.667321896057937,
 '2018-10-08 03:42:08.195923800': 3.0330981225705753,
 '2018-10-24 03:41:45.947403600': -2.153174049090749,
 '2018-11-09 03:41:24.952727300': -2.436293775823701,
 '2018-01-25 03:46:58.209342400': -26.275258742451364,
 '2018-02-10 03:46:43.408984500': -24.583467301172035,
 '2018-02-26 03:46:34.009571': -22.18708133531743,
 '2018-03-14 03:46:22.792637500': -15.818435274869366,
 '2018-03-30 03:46:09.604654900': None,
 '2018-04-15 03:45:54.468479500': -6.755734974753722,
 '2018-05-01 03:45:37.207894200': -0.7593814802944799,
 '2018-05-17 03:45:21.148581200': 0.6947089527446071,
 '2018-06-18 03:44:48.801540': 4.3928436782802684,
 '2018-07-04 03:44:29.815859300': 5.327805102173893,
 '2018-07-20 03:44:12.242042100': None,
 '2018-08-05 03:43:55.265840700': 8.030317634467606,
 '2018-08-21 03:43:36.849495700': 15.439515146583943,
 '2018-09-22 03:42:55.274077600': 7.991367849198407,
 '2018-10-08 03:42:32.075123100': 4.094884988184502,
 '2018-11-09 03:41:48.830531300': 1.660264880144824,
 '2018-01-25 03:47:22.088353300': -27.35039191436207,
 '2018-02-10 03:47:07.288888500': -23.021488252361276,
 '2018-02-26 03:46:57.889341200': -20.662218120202542,
 '2018-03-14 03:46:46.671530300': -12.628191723067072,
 '2018-03-30 03:46:33.482365900': None,
 '2018-04-15 03:46:18.345430600': -4.305740994483395,
 '2018-05-01 03:46:01.085047400': -0.2373099913605797,
 '2018-05-17 03:45:45.026916600': 2.5411766489739875,
 '2018-06-02 03:45:29.817581600': 1.3042011802672213,
 '2018-06-18 03:45:12.681122700': 5.824772177365719,
 '2018-07-04 03:44:53.694841600': 10.91334664673296,
 '2018-08-21 03:44:00.726475300': 13.007806120985375,
 '2018-09-06 03:43:40.720287600': 10.217078612178337,
 '2018-09-22 03:43:19.153338500': 8.974741402567696,
 '2018-10-08 03:42:55.954544600': 6.586364105705047,
 '2018-10-24 03:42:33.705933500': None,
 '2018-11-09 03:42:12.709158500': 3.5013934466222913,
 '2018-01-25 03:47:45.967787': -22.638983700770154,
 '2018-02-10 03:47:31.169466900': None,
 '2018-02-26 03:47:21.769834200': -18.935042528900922,
 '2018-03-14 03:47:10.551049200': -13.05347231266929,
 '2018-03-30 03:46:57.360546300': -10.41595392673156,
 '2018-04-15 03:46:42.222841100': -3.1154348799626965,
 '2018-05-01 03:46:24.962721400': 0.7509162975029827,
 '2018-05-17 03:46:08.905885900': 3.804989672892568,
 '2018-06-02 03:45:53.697566100': 1.6710164654379343,
 '2018-06-18 03:45:36.561403200': 7.013851111192001,
 '2018-07-04 03:45:17.574265': 14.309180851956837,
 '2018-07-20 03:44:59.998624': None,
 '2018-08-21 03:44:24.604337300': 9.80075887085616,
 '2018-09-06 03:44:04.599066200': 9.703440054411157,
 '2018-09-22 03:43:43.032990800': 8.752033146488733,
 '2018-10-08 03:43:19.835227600': 6.661235781481772,
 '2018-11-09 03:42:36.588071800': 3.741454845892374,
 '2018-02-01 03:53:01.946052300': -22.29798949098511,
 '2018-02-17 03:52:50.367395900': -21.1946182190904,
 '2018-03-05 03:52:40.145592300': -15.03620419645712,
 '2018-03-21 03:52:28.105828700': -8.997202034873487,
 '2018-04-06 03:52:14.022483100': None,
 '2018-04-22 03:51:57.988436500': -28.62459343119716,
 '2018-05-24 03:51:25.516368500': -0.7940689274533819,
 '2018-06-09 03:51:09.451744700': 3.4322277380201993,
 '2018-06-25 03:50:51.549875900': -17.904538329174024,
 '2018-07-11 03:50:31.927745200': -20.480393013257416,
 '2018-07-27 03:50:15.822336600': 7.623799120749848,
 '2018-08-12 03:49:58.223922700': 14.377739488731118,
 '2018-08-28 03:49:39.151255800': 17.469615508261196,
 '2018-09-13 03:49:18.429054800': None,
 '2018-09-29 03:48:56.144283700': 8.735266347882964,
 '2018-10-15 03:48:32.126582': 7.23508839968747,
 '2018-10-31 03:48:11.696175400': None,
 '2018-02-01 03:53:25.825373800': -18.224027024157987,
 '2018-02-17 03:53:14.247301100': -21.545063732247662,
 '2018-03-05 03:53:04.025049': -15.775784757638846,
 '2018-03-21 03:52:51.984166': -4.89446804009736,
 '2018-04-06 03:52:37.899598400': None,
 '2018-04-22 03:52:21.865325600': -14.132044861358848,
 '2018-05-24 03:51:49.395211600': 0.15098445854875353,
 '2018-06-09 03:51:33.331337900': 6.040185363185315,
 '2018-06-25 03:51:15.429397900': 5.294042044872969,
 '2018-07-27 03:50:39.699759200': 11.656880294427019,
 '2018-08-12 03:50:22.100966400': 14.550806928982622,
 '2018-08-28 03:50:03.028628300': 16.758004826265587,
 '2018-09-29 03:49:20.023768100': 8.348148448793507,
 '2018-10-15 03:48:56.006233600': 7.10272465945703,
 '2018-10-31 03:48:35.575147900': 3.3899122732953315,
 '2018-11-16 03:48:13.589272800': -16.551655097308043,
 '2018-01-16 03:54:05.378745700': -14.847559241056617,
 '2018-02-01 03:53:49.705307900': -16.24821390317842,
 '2018-02-17 03:53:38.127962500': -16.003647351287725,
 '2018-03-05 03:53:27.905208900': -16.811958490275256,
 '2018-03-21 03:53:15.863059100': -1.8134771505489597,
 '2018-04-06 03:53:01.777269800': -14.810938680067324,
 '2018-04-22 03:52:45.742605': -11.82341967785337,
 '2018-05-24 03:52:13.274481800': 0.0834968740751083,
 '2018-06-09 03:51:57.211589300': 5.532092282025741,
 '2018-06-25 03:51:39.309235300': 5.867555807751044,
 '2018-07-27 03:51:03.577655900': 11.73635638649833,
 '2018-08-12 03:50:45.978373800': 14.127057071062195,
 '2018-08-28 03:50:26.906714600': 15.59678487618569,
 '2018-09-29 03:49:43.903823700': 6.993229629877836,
 '2018-10-15 03:49:19.886568700': 6.748445364912692,
 '2018-10-31 03:48:59.454975700': 2.0932873994815875,
 '2018-11-16 03:48:37.467654900': -8.212959538648642,
 '2018-12-02 03:48:13.206376300': -6.069659954481478,
 '2018-01-23 03:59:45.677752200': -14.904726333285511,
 '2018-02-24 03:59:20.786371200': -12.114306843723815,
 '2018-03-28 03:58:56.865815800': -1.48579544506741,
 '2018-04-13 03:58:41.963194200': -1.5003095338593504,
 '2018-04-29 03:58:24.980784200': -17.061256829624124,
 '2018-05-15 03:58:08.411901700': None,
 '2018-05-31 03:57:53.446167800': 4.109240829875174,
 '2018-06-16 03:57:36.540232': None,
 '2018-07-02 03:57:17.801807500': 9.28077861998745,
 '2018-07-18 03:56:59.774197300': 7.664249483845669,
 '2018-08-03 03:56:42.973029700': 12.93317049762397,
 '2018-08-19 03:56:24.739509100': -3.261253405722458,
 '2018-10-06 03:55:20.553683700': -7.804237707821381,
 '2018-10-22 03:54:57.803398700': 6.24829244953404,
 '2018-11-07 03:54:37.077277700': 4.854676358187795,
 '2018-11-23 03:54:14.086395600': -5.499997943271443,
 '2018-01-23 04:00:09.557144900': -19.273781514642486,
 '2018-02-24 03:59:44.666844600': -14.953029964583322,
 '2018-03-28 03:59:20.743999200': -1.1637371616931764,
 '2018-04-13 03:59:05.840959100': -0.7441515542392608,
 '2018-04-29 03:58:48.858644': -1.9641505032047721,
 '2018-05-15 03:58:32.290857': None,
 '2018-05-31 03:58:17.326189100': 4.545204091560907,
 '2018-06-16 03:58:00.420636800': None,
 '2018-07-02 03:57:41.681581800': 11.298135732887253,
 '2018-07-18 03:57:23.652812700': 11.99523844681269,
 '2018-08-03 03:57:06.850657200': 14.291995656484573,
 '2018-08-19 03:56:48.617275800': None,
 '2018-10-22 03:55:21.683341600': 5.476005505927172,
 '2018-11-07 03:55:00.956655500': 3.3716685572034013,
 '2018-11-23 03:54:37.964456400': -9.190786593262924,
 '2018-01-14 04:06:28.805108700': -3.0131932791627065,
 '2018-02-15 04:06:00.896909800': -13.681074804030823,
 '2018-03-03 04:05:50.905122800': -18.023061508499293,
 '2018-03-19 04:05:39.116907300': -6.051994080112451,
 '2018-04-04 04:05:25.269203900': None,
 '2018-04-20 04:05:09.491752100': -1.2203728278837802,
 '2018-05-06 04:04:51.602843100': 1.7942262048291275,
 '2018-05-22 04:04:36.762172900': -15.245988705423429,
 '2018-06-07 04:04:20.956383200': 4.684437145476969,
 '2018-06-23 04:04:03.257576400': 7.536849779498935,
 '2018-07-09 04:03:43.715646900': 3.0285816554381793,
 '2018-07-25 04:03:27.294756100': 3.4998244528188134,
 '2018-08-26 04:02:51.009893200': 15.356509760285293,
 '2018-09-11 04:02:30.477778100': 3.616949324488742,
 '2018-09-27 04:02:08.405496500': 4.424708215000566,
 '2018-10-13 04:01:44.623862700': 6.990847966748033,
 '2018-10-29 04:01:23.677945300': 6.525255313406258,
 '2018-11-14 04:01:01.970327200': -9.582570893859039,
 '2018-11-30 04:00:37.971397700': None,
'2019-01-29 03:38:21.672019': None,
 '2019-02-14 03:38:19.686404900': -23.11714865773235,
 '2019-03-02 03:38:15.677750': -10.026745787203934,
 '2019-03-18 03:38:10.377701': -8.922763945614378,
 '2019-05-05 03:38:02.340906': -6.9767176715738835,
 '2019-05-21 03:38:12.802170': -4.1280849087801625,
 '2019-06-06 03:38:20.896248900': -1.3154694862311496,
 '2019-07-08 03:38:30.628016': 11.266176988795886,
 '2019-07-24 03:38:34.678822': 18.39606907164742,
 '2019-08-09 03:38:41.109326': 16.2282672333631,
 '2019-08-25 03:38:45.566234900': None,
 '2019-09-10 03:38:49.477258': 14.197293065836062,
 '2019-10-12 03:38:58.132558900': 3.6355291683151223,
 '2019-10-28 03:38:58.734130': None,
 '2019-01-29 03:38:45.554585900': -23.79010487864802,
 '2019-02-14 03:38:43.564736': -22.997497551052138,
 '2019-03-02 03:38:39.556082': -9.953476555536003,
 '2019-03-18 03:38:34.256032': None,
 '2019-05-05 03:38:26.231946': -5.652945195274134,
 '2019-05-21 03:38:36.693208900': -1.4638145156498599,
 '2019-06-06 03:38:44.778816': 1.3155095715659673,
 '2019-06-22 03:38:50.642108': 7.792739143341868,
 '2019-07-08 03:38:54.510583': 7.373699984072494,
 '2019-07-24 03:38:58.552917': 18.95974936011464,
 '2019-08-09 03:39:04.991892900': 16.002982682031895,
 '2019-09-10 03:39:13.355589': 13.769213067923117,
 '2019-09-26 03:39:18.847558': None,
 '2019-10-12 03:39:22.015126': 3.98337858418068,
 '2019-10-28 03:39:22.612461': None,
 '2019-01-29 03:39:09.432917': -20.736261685876688,
 '2019-02-14 03:39:07.447302900': -20.960273864491572,
 '2019-03-02 03:39:03.434413': -8.988071366669837,
 '2019-03-18 03:38:58.130127': None,
 '2019-04-03 03:38:54.818750': None,
 '2019-05-05 03:38:50.114513': -7.462863550372567,
 '2019-05-21 03:39:00.571540': 1.766309247834495,
 '2019-06-06 03:39:08.665619': 2.72396104691957,
 '2019-06-22 03:39:14.524675': 9.318313091977132,
 '2019-07-08 03:39:18.393150': 9.399988565861273,
 '2019-07-24 03:39:22.443956': 17.505842326338957,
 '2019-08-09 03:39:28.870224': 15.798049451395418,
 '2019-08-25 03:39:33.335606': None,
 '2019-09-10 03:39:37.242391900': 12.896514705493157,
 '2019-09-26 03:39:42.725890': None,
 '2019-10-12 03:39:45.901930': 2.7109428980350345,
 '2019-10-28 03:39:46.490792': None,
 '2019-11-29 03:39:42.008521': -16.516947596721632,
 '2019-01-13 03:39:37.289784': -16.124435211074495,
 '2019-01-29 03:39:33.315483900': -18.273550995067232,
 '2019-02-14 03:39:31.325634': -19.369564768161776,
 '2019-03-02 03:39:27.312744': -9.925564258561746,
 '2019-03-18 03:39:22.008458': None,
 '2019-04-03 03:39:18.701317': None,
 '2019-05-05 03:39:13.992844': None,
 '2019-05-21 03:39:24.462579': 4.288021056836078,
 '2019-06-06 03:39:32.548185900': None,
 '2019-06-22 03:39:38.411478': 4.286910322493377,
 '2019-07-08 03:39:42.271481': 13.69293299600582,
 '2019-07-24 03:39:46.318051': 18.64458276573672,
 '2019-08-09 03:39:52.752792': 16.008968822384368,
 '2019-08-25 03:39:57.218173': 14.508186074112977,
 '2019-09-10 03:40:01.124959': None,
 '2019-09-26 03:40:06.612693': None,
 '2019-10-28 03:40:10.377596': None,
 '2019-11-29 03:40:05.891088': -17.913327004461145,
 '2019-02-05 03:44:31.661052': None,
 '2019-03-09 03:44:23.948239': -12.30547433048109,
 '2019-03-25 03:44:20.042454': -11.71281519157483,
 '2019-04-10 03:44:15.651490': -3.732175298652922,
 '2019-04-26 03:44:08.904398': -8.269793060129581,
 '2019-05-12 03:44:18.067899900': -11.566351755102431,
 '2019-06-13 03:44:34.515026': 3.5242493357964504,
 '2019-06-29 03:44:39.501580': 7.299678128278633,
 '2019-07-15 03:44:42.485118': 5.366238281114711,
 '2019-07-31 03:44:48.555822': 16.93919669598405,
 '2019-08-16 03:44:54.135041': 15.041069611426439,
 '2019-09-01 03:44:57.762105': 12.609286889585377,
 '2019-09-17 03:45:02.973838': None,
 '2019-11-04 03:45:08.868760900': 1.2217555000191749,
 '2019-02-05 03:44:55.539382900': -32.95182234252765,
 '2019-03-09 03:44:47.830806': -11.311142797541187,
 '2019-03-25 03:44:43.912313': -14.659631088066792,
 '2019-04-10 03:44:39.529822': -2.977399606993929,
 '2019-04-26 03:44:32.782729': -5.1603016707359215,
 '2019-05-12 03:44:41.954703': -7.994320109995988,
 '2019-06-13 03:44:58.397593': 3.8720544719713654,
 '2019-06-29 03:45:03.388383': 4.762247458616524,
 '2019-07-15 03:45:06.367685': 6.671340700447011,
 '2019-07-31 03:45:12.438388900': 15.90296093573515,
 '2019-08-16 03:45:18.017607900': 14.792600203188583,
 '2019-09-01 03:45:21.644672': 16.023654533388097,
 '2019-09-17 03:45:26.856405': 8.104204763237565,
 '2019-10-03 03:45:31.361928900': 7.8717686636767885,
 '2019-11-04 03:45:32.747092': 1.7984657593323714,
 '2019-11-20 03:45:29.156685900': None,
 '2019-01-20 03:45:22.535719': -10.812000247855362,
 '2019-02-05 03:45:19.417714': -26.066663791953903,
 '2019-02-21 03:45:16.831687900': -11.672051898337012,
 '2019-03-09 03:45:11.704901': -8.243634368994002,
 '2019-03-25 03:45:07.794880': -13.802770301282033,
 '2019-04-10 03:45:03.403917': -0.9454059844413645,
 '2019-04-26 03:44:56.665296': -2.813726750233023,
 '2019-05-12 03:45:05.837270': -0.8853408971335422,
 '2019-06-13 03:45:22.288632': 5.42560230824176,
 '2019-06-29 03:45:27.270950': 6.8022701566958315,
 '2019-07-15 03:45:30.246016': 11.676232983405338,
 '2019-07-31 03:45:36.325192': 14.540482300703513,
 '2019-08-16 03:45:41.904411': 14.591382755712123,
 '2019-09-01 03:45:45.531476': 15.86471922375454,
 '2019-09-17 03:45:50.738971900': 9.258967608005905,
 '2019-10-03 03:45:55.244496': 8.95896987686715,
 '2019-11-04 03:45:56.629660': 2.8748584404402218,
 '2019-11-20 03:45:53.039253': None,
 '2019-01-20 03:45:46.422521': None,
 '2019-02-05 03:45:43.304518': -25.15949065180426,
 '2019-02-21 03:45:40.714255': -10.909719490725678,
 '2019-03-09 03:45:35.587468': -8.435796826994467,
 '2019-03-25 03:45:31.673211': -9.979396525356666,
 '2019-04-10 03:45:27.286484': -0.3792880808164646,
 '2019-04-26 03:45:20.552099': -4.269049817808126,
 '2019-05-12 03:45:29.724074': -0.8315161542196648,
 '2019-05-28 03:45:39.071870': None,
 '2019-06-13 03:45:46.166963900': 6.7390953035059695,
 '2019-06-29 03:45:51.157753': 9.244031074193225,
 '2019-07-15 03:45:54.128584': 8.97053423405462,
 '2019-07-31 03:46:00.207759': 14.300237785047106,
 '2019-08-16 03:46:05.778506': 13.758696027544996,
 '2019-09-01 03:46:09.409807': 15.242387877642498,
 '2019-09-17 03:46:14.621539': 9.71752262294518,
 '2019-10-03 03:46:19.131299': 8.987428199538158,
 '2019-11-04 03:46:20.507991': 3.0683581685257137,
 '2019-11-20 03:46:16.913349': None,
 '2019-12-06 03:46:16.377996': -2.5391504959837707,
 '2019-01-27 03:51:07.629266': -12.887591842976635,
 '2019-02-12 03:51:05.533265': -21.25787553874627,
 '2019-03-16 03:50:56.126673900': -1.8938953267729441,
 '2019-04-01 03:50:53.111770': -3.738084807984027,
 '2019-04-17 03:50:47.555473': -1.6711905734196855,
 '2019-05-03 03:50:46.662418': -8.593409455083956,
 '2019-05-19 03:50:57.165671': 2.261756489228678,
 '2019-06-04 03:51:05.522259': None,
 '2019-06-20 03:51:11.665024': 3.002392766526875,
 '2019-07-06 03:51:15.783388': 11.595115893078793,
 '2019-07-22 03:51:19.262448': 17.891020075213042,
 '2019-08-07 03:51:25.938576': 6.735459117603156,
 '2019-08-23 03:51:30.641151': 14.924997952701611,
 '2019-09-08 03:51:34.158168': 8.902966115038152,
 '2019-09-24 03:51:39.908353': 12.217416275813937,
 '2019-10-10 03:51:43.380790': 3.8314594228060805,
 '2019-10-26 03:51:44.316965900': None,
 '2019-01-27 03:51:31.507597': -12.696273446281475,
 '2019-02-12 03:51:29.411595900': -21.475014234451606,
 '2019-02-28 03:51:25.699309900': -8.070982433311855,
 '2019-03-16 03:51:20.005005': -1.6128678708476774,
 '2019-04-01 03:51:16.994337': -2.19830795371409,
 '2019-04-17 03:51:11.429567900': -1.6077348996589147,
 '2019-05-03 03:51:10.540750': None,
 '2019-05-19 03:51:21.048239': 2.9810937961453003,
 '2019-06-04 03:51:29.409062': 1.8563073321988295,
 '2019-06-20 03:51:35.547591': 5.504799132189237,
 '2019-07-06 03:51:39.670190900': 12.530367332394212,
 '2019-07-22 03:51:43.145016': 17.63204321993373,
 '2019-08-07 03:51:49.825379': 13.750501941419497,
 '2019-08-23 03:51:54.523718': 14.95251384874171,
 '2019-09-08 03:51:58.040735': 10.205601119911533,
 '2019-09-24 03:52:03.790921': 11.604006335879658,
 '2019-10-10 03:52:07.267593': 3.4702037619350636,
 '2019-10-26 03:52:08.199533': 3.6201811817832548,
 '2019-11-11 03:52:06.247609': -1.705944223999634,
 '2019-11-27 03:52:03.632287': -0.21775545744413055,
 '2019-01-11 03:51:59.330724': None,
 '2019-01-27 03:51:55.390164': -13.884163559102241,
 '2019-02-12 03:51:53.285691': -21.34566673777039,
 '2019-02-28 03:51:49.577641': -9.043019028262117,
 '2019-03-16 03:51:43.883335900': -0.935438400343036,
 '2019-04-01 03:51:40.872668': -1.5198518854621643,
 '2019-04-17 03:51:35.303663': -2.566586657177236,
 '2019-05-19 03:51:44.930806': 2.0907028627660913,
 '2019-06-04 03:51:53.295864900': 2.7814711116558093,
 '2019-06-20 03:51:59.434395': 6.098189554206958,
 '2019-07-06 03:52:03.552758': 12.747062799685075,
 '2019-08-07 03:52:13.707945900': 12.81026081183459,
 '2019-08-23 03:52:18.402048900': 14.486443222110921,
 '2019-09-08 03:52:21.923303': 11.860485126406552,
 '2019-09-24 03:52:27.673488': 10.833488579048847,
 '2019-10-10 03:52:31.150160': 1.6166602660109475,
 '2019-10-26 03:52:32.082100': 4.6818823185915255,
 '2019-11-11 03:52:30.130175900': -11.780921649662806,
 '2019-11-27 03:52:27.510619': -0.5925955814445957,
 '2019-12-13 03:52:26.382261': -1.315160115470674,
 '2019-01-18 03:57:44.690959': -13.43476307667082,
 '2019-02-03 03:57:41.187214': -21.015885598246168,
 '2019-02-19 03:57:38.927158': -11.338333360331735,
 '2019-03-23 03:57:29.809648': -10.1356843424475,
 '2019-04-08 03:57:25.736220': -1.6722728952803747,
 '2019-04-24 03:57:18.976512': -1.6563837158780934,
 '2019-05-26 03:57:35.776119900': 3.743934280335355,
 '2019-06-27 03:57:48.391253': None,
 '2019-07-13 03:57:51.624557': 14.29220062735969,
 '2019-08-30 03:58:06.808479': 15.412452082210498,
 '2019-09-15 03:58:11.655903': 8.458631357347508,
 '2019-10-01 03:58:16.457729900': None,
 '2019-10-17 03:58:18.829044900': 7.772493144008672,
 '2019-11-02 03:58:18.520460': 5.327362533751378,
 '2019-11-18 03:58:15.285627': 1.8327380597233198,
 '2019-01-18 03:58:08.573526': -12.10013675626636,
 '2019-02-03 03:58:05.065545': -22.712396650796357,
 '2019-02-19 03:58:02.805488900': -13.31763367483935,
 '2019-03-07 03:57:57.996569': -11.442952511050319,
 '2019-03-23 03:57:53.687979900': -13.514077855413191,
 '2019-04-08 03:57:49.614550900': -4.4045977745519655,
 '2019-04-24 03:57:42.854843': -4.727912788273763,
 '2019-05-10 03:57:50.014529': -2.160464507735502,
 '2019-05-26 03:57:59.662923900': 3.1426218175728002,
 '2019-06-11 03:58:07.045893': None,
 '2019-06-27 03:58:12.278056': 9.54336284701196,
 '2019-08-14 03:58:26.826885900': None,
 '2019-08-30 03:58:30.695282900': 15.992873797387354,
 '2019-09-15 03:58:35.542706': 8.040164139474122,
 '2019-10-17 03:58:42.715848': 6.528380039781288,
 '2019-11-02 03:58:42.403026900': 4.341190148263351,
 '2019-11-18 03:58:39.168193900': -1.1381419975461071,
 '2019-12-04 03:58:38.158376900': -5.596797358519855,
 '2019-01-09 04:04:21.320816': -1.3816757172857996,
 '2019-01-25 04:04:17.418354': -13.585161382823314,
 '2019-02-10 04:04:15.211895900': None,
 '2019-02-26 04:04:11.804504': -7.467178666552967,
 '2019-03-14 04:04:05.885322900': -17.26776404714259,
 '2019-03-30 04:04:02.997453900': -0.7138203028953812,
 '2019-04-15 04:03:57.771538': -0.9062605249155754,
 '2019-05-01 04:03:55.226444': 0.6115143681242378,
 '2019-05-17 04:04:05.360850': -0.056310690573233665,
 '2019-06-02 04:04:13.992807': 4.03969951450492,
 '2019-06-18 04:04:20.410695': None,
 '2019-07-04 04:04:24.779043': 8.364065544614231,
 '2019-07-20 04:04:27.694733': 14.928011205777292,
 '2019-08-05 04:04:34.620770': 7.167618657696422,
 '2019-08-21 04:04:39.564716900': 13.40030695305224,
 '2019-09-22 04:04:48.708959900': -0.9518316503903598,
 '2019-10-08 04:04:52.477771': 9.591378440069255,
 '2019-10-24 04:04:53.735737': 6.131250003696836,
 '2019-11-09 04:04:52.147992': None,
 '2019-11-25 04:04:49.083348': None,
 '2019-12-11 04:04:48.294065': -12.924982203818255,
 '2019-02-06 03:32:40.815105500': -33.92821842008643,
 '2019-02-22 03:32:12.138157': -18.79453092308201,
 '2019-03-10 03:31:41.880714800': -19.99428570367991,
 '2019-03-26 03:31:09.998782200': -11.50674225123738,
 '2019-04-11 03:30:36.548465': -3.9606602557134143,
 '2019-04-27 03:30:01.237470': -6.166639328337138,
 '2019-05-29 03:28:48.312630500': -1.1971446837073674,
 '2019-06-14 03:28:14.096798600': -3.700276768965272,
 '2019-06-30 03:27:38.399165100': 10.253644376729838,
 '2019-07-16 03:27:01.266973800': 15.623766265389625,
 '2019-08-01 03:26:22.783400': None,
 '2019-09-02 03:25:02.030176600': 15.50228819779778,
 '2019-09-18 03:24:19.650808500': 4.7233061119241535,
 '2019-10-04 03:23:38.983474400': None,
 '2019-11-05 03:22:16.546740100': -13.46684950527307,
 '2019-02-06 03:33:04.694345100': -29.18914642525217,
 '2019-02-22 03:32:36.016679500': -15.948596965618613,
 '2019-03-10 03:32:05.758076100': -12.581126341893873,
 '2019-03-26 03:31:33.874978700': -11.621860263503706,
 '2019-04-11 03:31:00.424493300': -4.66822893708353,
 '2019-04-27 03:30:25.114195500': -7.837699533981107,
 '2019-05-13 03:29:48.149652200': -15.200172275120183,
 '2019-05-29 03:29:12.191387': 0.050650165009705664,
 '2019-06-14 03:28:37.975281': 1.0108598620486084,
 '2019-06-30 03:28:02.276500900': 5.88528765315409,
 '2019-07-16 03:27:25.143009700': 10.359726687880578,
 '2019-08-01 03:26:46.658837400': 14.368424598000507,
 '2019-09-02 03:25:25.907861400': 13.279122451195974,
 '2019-09-18 03:24:43.529769': 2.056607664377929,
 '2019-10-04 03:24:02.862362800': -1.5349132006200834,
 '2019-10-20 03:23:22.666446600': None,
 '2019-11-05 03:22:40.423116200': -3.596539401185387,
 '2019-01-21 03:33:58.395758100': -13.427031180723958,
 '2019-02-06 03:33:28.574252100': -25.445129391166297,
 '2019-02-22 03:32:59.895768': -13.093346185671052,
 '2019-03-10 03:32:29.635862900': -10.054848451657735,
 '2019-03-26 03:31:57.751464100': -9.184240307887533,
 '2019-04-11 03:31:24.300834800': -7.592232243788035,
 '2019-04-27 03:30:48.991505600': -12.730017676083301,
 '2019-05-13 03:30:12.028134200': -12.935192924876171,
 '2019-05-29 03:29:36.070888600': 2.2862982222371504,
 '2019-06-14 03:29:01.854190500': 3.1828969919135837,
 '2019-06-30 03:28:26.154240100': 7.1046757335046875,
 '2019-07-16 03:27:49.019506400': 10.502515740278715,
 '2019-08-01 03:27:10.535049400': 12.617799007470579,
 '2019-09-02 03:25:49.786296600': 13.465469423000973,
 '2019-09-18 03:25:07.408978200': 2.3619111607282597,
 '2019-10-04 03:24:26.741936600': 6.988996354700822,
 '2019-10-20 03:23:46.544775': 5.742034359315035,
 '2019-11-05 03:23:04.300370300': -0.42912903470343783,
 '2019-11-21 03:22:19.849461300': None,
 '2019-01-21 03:34:22.275679700': None,
 '2019-02-06 03:33:52.454723700': -25.83903560861466,
 '2019-03-10 03:32:53.514175700': -8.275255029729351,
 '2019-03-26 03:32:21.628822200': -6.797214513110692,
 '2019-04-11 03:31:48.177967700': None,
 '2019-04-27 03:31:12.869547400': -13.92602893088132,
 '2019-05-13 03:30:35.907186100': None,
 '2019-05-29 03:29:59.950971900': -4.23739221186625,
 '2019-06-14 03:29:25.734051200': -2.0077606214972024,
 '2019-06-30 03:28:50.032724600': 8.110248282576645,
 '2019-07-16 03:28:12.896702100': None,
 '2019-08-01 03:27:34.411712600': 12.932481795582822,
 '2019-09-02 03:26:13.665202800': 13.287263980304946,
 '2019-09-18 03:25:31.289133': None,
 '2019-10-04 03:24:50.622104400': 7.33252086196742,
 '2019-10-20 03:24:10.423916300': 6.8100467522396775,
 '2019-11-05 03:23:28.178066900': 0.3166922566020632,
 '2019-11-21 03:22:43.726557900': -27.472193512225882,
 '2019-02-13 03:38:39.259132800': -31.054729091918777,
 '2019-03-01 03:38:09.891723800': -12.667788037248236,
 '2019-04-02 03:37:06.384833400': -3.9078411739066543,
 '2019-04-18 03:36:32.128529100': -3.4632212771369306,
 '2019-05-04 03:35:56.106909300': -1.3767163068289496,
 '2019-05-20 03:35:18.298915500': -0.4245052409966941,
 '2019-07-07 03:33:33.145498700': 6.432239567539355,
 '2019-07-23 03:32:55.380108800': 16.551288500590907,
 '2019-08-08 03:32:16.325164400': 14.049374605741354,
 '2019-08-24 03:31:36.008199500': 13.243952896289382,
 '2019-09-09 03:30:54.445434': 14.257049777761049,
 '2019-09-25 03:30:11.615679200': 11.236746559494328,
 '2019-10-27 03:28:51.415109': -11.134150934735361,
 '2019-02-13 03:39:03.138159900': -25.21646534731961,
 '2019-03-01 03:38:33.769655900': -15.643310758706658,
 '2019-03-17 03:38:02.816216800': None,
 '2019-04-02 03:37:30.260712300': -7.783397516171989,
 '2019-04-18 03:36:56.004663300': -4.99803110966005,
 '2019-05-04 03:36:19.984092200': -0.8589272451056681,
 '2019-05-20 03:35:42.177282800': -0.9923972459473069,
 '2019-06-05 03:35:08.251294800': None,
 '2019-06-21 03:34:33.342983100': None,
 '2019-07-07 03:33:57.022316400': 6.726108858994442,
 '2019-07-23 03:33:19.255674900': 12.984433908238836,
 '2019-08-08 03:32:40.201189200': 12.554755600729855,
 '2019-08-24 03:31:59.885191500': 15.722441931969348,
 '2019-09-09 03:31:18.323903300': 13.437187992545029,
 '2019-09-25 03:30:35.494738800': 11.17452852202611,
 '2019-10-11 03:29:56.312235300': 1.9378913796597748,
 '2019-10-27 03:29:15.292386200': None,
 '2019-01-28 03:39:55.263270400': None,
 '2019-02-13 03:39:27.017801200': -21.906571157612433,
 '2019-03-01 03:38:57.648246700': -11.650340236973227,
 '2019-03-17 03:38:26.693263800': -6.9067437932135,
 '2019-04-02 03:37:54.137210300': -6.303969367854092,
 '2019-04-18 03:37:19.881375700': -3.056077182878288,
 '2019-05-04 03:36:43.862015900': -0.28864636915770475,
 '2019-05-20 03:36:06.056137800': 2.0273238556289965,
 '2019-06-05 03:35:32.130875500': 2.8081541725534493,
 '2019-06-21 03:34:57.221608100': 3.0959382052658753,
 '2019-07-07 03:34:20.899688': 9.431721940157429,
 '2019-07-23 03:33:43.132139600': None,
 '2019-08-24 03:32:23.762889200': 14.545558365652502,
 '2019-09-09 03:31:42.202930900': 12.213270262726208,
 '2019-09-25 03:30:59.374562300': 11.520173489660634,
 '2019-10-11 03:30:20.191295200': 5.030144190215685,
 '2019-11-12 03:28:55.950765800': -4.6766607214570906,
 '2019-01-28 03:40:19.143493800': -20.190397706270854,
 '2019-02-13 03:39:50.897950400': -20.50108024701618,
 '2019-03-01 03:39:21.527166400': -11.943468061025365,
 '2019-03-17 03:38:50.570952400': -13.327978449493669,
 '2019-04-02 03:38:18.014108100': -1.8054310359579884,
 '2019-04-18 03:37:43.758795': -1.7025749428922101,
 '2019-05-04 03:37:07.740444700': 0.23907054319174778,
 '2019-05-20 03:36:29.935827': 2.877362018057548,
 '2019-06-05 03:35:56.010997200': 4.090375274396119,
 '2019-06-21 03:35:21.100978200': 6.412367793058886,
 '2019-07-07 03:34:44.777480100': 2.3662569286821524,
 '2019-08-08 03:33:27.954729400': 11.500154368444141,
 '2019-08-24 03:32:47.641062400': 11.099013917706024,
 '2019-09-09 03:32:06.082593900': 11.427889617171791,
 '2019-09-25 03:31:23.254964700': 10.93876049995213,
 '2019-10-11 03:30:44.071184700': 6.063152491241115,
 '2019-11-12 03:29:19.828046700': -1.3897742122144017,
 '2019-11-28 03:28:34.397088600': -1.907997337756593,
 '2019-02-04 03:45:29.781027700': None,
 '2019-03-08 03:44:31.247452600': -6.218888804701288,
 '2019-03-24 03:43:59.576388100': -5.700894040144931,
 '2019-04-09 03:43:26.328231400': -26.344167659644768,
 '2019-04-25 03:42:51.239543700': -0.7368650747618909,
 '2019-05-11 03:42:14.488405800': 0.3647886074395423,
 '2019-05-27 03:41:37.972417200': None,
 '2019-06-12 03:41:03.954978800': -9.114021791918425,
 '2019-06-28 03:40:28.419067800': 7.3543654196406365,
 '2019-07-14 03:39:51.474173700': 15.873930733057612,
 '2019-07-30 03:39:13.137590': 14.177084517859244,
 '2019-08-15 03:38:33.536205500': 13.493740634865379,
 '2019-08-31 03:37:52.712301300': 15.592228354330027,
 '2019-09-16 03:37:10.521399900': 7.549230069246436,
 '2019-10-02 03:36:29.390119500': 7.38778763748376,
 '2019-10-18 03:35:49.411579900': -2.575044119359164,
 '2019-11-03 03:35:07.447016500': None,
 '2019-03-08 03:44:55.125455700': -5.776952676115165,
 '2019-03-24 03:44:23.453132300': -3.5331322152656277,
 '2019-04-09 03:43:50.204815500': -6.223944294680932,
 '2019-05-11 03:42:38.366766': 0.27895453700984557,
 '2019-05-27 03:42:01.851745700': -1.0070736172105512,
 '2019-06-12 03:41:27.834160400': -8.724831197018325,
 '2019-06-28 03:40:52.297206700': 8.37496371884057,
 '2019-07-14 03:40:15.350931': 15.446424526337175,
 '2019-07-30 03:39:37.013665': None,
 '2019-08-15 03:38:57.413074300': 13.333617578137716,
 '2019-08-31 03:38:16.590478800': 15.35865297996938,
 '2019-09-16 03:37:34.400873600': None,
 '2019-10-02 03:36:53.269476400': 8.284797123695018,
 '2019-10-18 03:36:13.290272200': 6.3268268165876,
 '2019-11-03 03:35:31.324164300': 1.180948157384546,
 '2019-11-19 03:34:47.136805200': -9.864373683053339,
 '2019-01-19 03:46:47.796811900': -11.17082391856807,
 '2019-02-04 03:46:17.541419300': -27.89346476351947,
 '2019-02-20 03:45:49.069040800': -16.56177056026463,
 '2019-03-08 03:45:19.003998': -6.4473804793980705,
 '2019-03-24 03:44:47.330552100': -6.139104401263432,
 '2019-04-09 03:44:14.081844800': -10.187089110357649,
 '2019-05-11 03:43:02.245861700': -0.11314047331492467,
 '2019-05-27 03:42:25.731887200': 1.6658517775542054,
 '2019-06-12 03:41:51.713892400': 0.7345646140509289,
 '2019-06-28 03:41:16.175743600': 9.174627065012267,
 '2019-07-14 03:40:39.228044300': 11.184872985370369,
 '2019-07-30 03:40:00.890615900': 10.225883114467814,
 '2019-08-15 03:39:21.290542': 11.765086074380466,
 '2019-08-31 03:38:40.469410100': 15.630759900595107,
 '2019-09-16 03:37:58.280768200': 6.581459418549733,
 '2019-10-02 03:37:17.150036200': 6.22529831830018,
 '2019-10-18 03:36:37.169559800': 5.870711852230107,
 '2019-11-03 03:35:55.202144': 0.8732668920194843,
 '2019-11-19 03:35:11.013717800': -5.726177998837809,
 '2019-02-11 03:51:52.204917700': -19.378312905168524,
 '2019-01-26 03:52:44.774270600': -16.400470213517966,
 '2019-02-11 03:52:16.085158300': -20.541903994648667,
 '2019-02-27 03:51:46.895095800': -8.761552404307956,
 '2019-03-31 03:50:43.779691600': -7.610537520786422,
 '2019-04-16 03:50:09.749678100': None,
 '2019-05-02 03:49:33.941277200': -0.6513643326987614,
 '2019-05-18 03:48:56.378445800': -31.37714143304406,
 '2019-06-19 03:47:47.163158500': 9.60145392033434,
 '2019-07-05 03:47:11.023471100': 13.155605433872827,
 '2019-07-21 03:46:33.427742600': 14.448845876455941,
 '2019-08-06 03:45:54.528081600': 13.359578353179543,
 '2019-08-22 03:45:14.365262800': 11.120665316924237,
 '2019-09-07 03:44:32.962401600': -15.661004484499975,
 '2019-09-23 03:43:50.169213100': 12.18712045397427,
 '2019-10-09 03:43:10.674350200': 2.6515730589086086,
 '2019-10-25 03:42:29.893473500': 2.645873623586656,
 '2019-11-10 03:41:46.955014400': -4.603678929714651,
 '2019-11-26 03:41:01.806599700': -3.384251360100878,
 '2019-01-17 03:59:13.295021500': -10.453841719748777,
 '2019-02-02 03:58:42.585676500': None,
 '2019-02-18 03:58:14.329635200': -14.336290957046739,
 '2019-03-06 03:57:44.464957900': -15.61969844532517,
 '2019-03-22 03:57:12.996391600': -5.150549613797294,
 '2019-04-07 03:56:39.952406400': -2.732057903141149,
 '2019-04-23 03:56:05.086456600': -3.3788836462973184,
 '2019-05-09 03:55:28.554350800': -10.879758208561967,
 '2019-05-25 03:54:51.482242500': 3.060967104317473,
 '2019-06-10 03:54:17.672516900': None,
 '2019-06-26 03:53:42.305266900': None,
 '2019-08-13 03:51:47.907261200': 10.148418685319603,
 '2019-08-29 03:51:07.239307700': 11.523163808011645,
 '2019-09-14 03:50:25.231134800': 8.674728114676032,
 '2019-09-30 03:49:43.641894900': -6.855444426812102,
 '2019-10-16 03:49:03.884036300': 7.42584362096473,
 '2019-11-01 03:48:22.195120200': 5.620723756390764,
 '2019-11-17 03:47:38.275259800': None,
'2020-02-01 03:38:40.204649900': -30.79461569070477,
 '2020-02-17 03:38:36.600225900': -19.51441507215975,
 '2020-03-04 03:38:31.576398': -17.493136709678847,
 '2020-03-20 03:38:24.046222900': None,
 '2020-04-05 03:38:14.881641': -1.3257519218342098,
 '2020-04-21 03:38:08.352742': -1.2832176635328778,
 '2020-05-07 03:37:59.274654': -1.4287523549590448,
 '2020-05-23 03:38:01.641060': 3.817413141582875,
 '2020-06-08 03:38:09.506976': None,
 '2020-06-24 03:38:18.502481': 13.85463224126283,
 '2020-07-26 03:38:30.500270': 14.147152255726226,
 '2020-08-11 03:38:33.454368900': 12.07843328169171,
 '2020-08-27 03:38:41.935876': 16.58239837266628,
 '2020-09-12 03:38:48.620809': 10.128274939879532,
 '2020-10-14 03:38:55.125571': 5.379767095456007,
 '2020-10-30 03:38:54.357690900': -4.047864272030778,
 '2020-11-15 03:38:52.069507': 0.7316603148616903,
 '2020-02-01 03:39:04.082981900': -24.389092207587325,
 '2020-02-17 03:39:00.478557': -18.272177630179463,
 '2020-03-04 03:38:55.454729': -15.209737860291556,
 '2020-04-05 03:38:38.755736900': -1.60402274569897,
 '2020-04-21 03:38:32.231074': -1.539623381228587,
 '2020-05-07 03:38:23.148749900': -1.8770370496533444,
 '2020-05-23 03:38:25.523627': 2.5645915265787718,
 '2020-06-08 03:38:33.393780': None,
 '2020-06-24 03:38:42.389284': 10.318166759069078,
 '2020-07-26 03:38:54.378602': 10.653476219471184,
 '2020-08-11 03:38:57.341172': 12.464394499367053,
 '2020-08-27 03:39:05.814208': 15.66723609610377,
 '2020-09-12 03:39:12.503376': 9.26126444294834,
 '2020-09-28 03:39:17.057354900': None,
 '2020-10-14 03:39:19.008137900': 5.190083226790615,
 '2020-10-30 03:39:18.236022': -3.5556846542342657,
 '2020-11-15 03:39:15.943603': -1.4460233702106018,
 '2020-01-16 03:39:33.061211900': -21.317716458799556,
 '2020-02-01 03:39:27.957077': -22.51575541205317,
 '2020-02-17 03:39:24.356887900': -15.58122794662507,
 '2020-03-04 03:39:19.333060': -14.026604885166128,
 '2020-03-20 03:39:11.794414': -6.840673562601282,
 '2020-04-05 03:39:02.629832': -3.09730238002817,
 '2020-04-21 03:38:56.105168900': -1.3686442526511775,
 '2020-05-07 03:38:47.027081': -0.8450111094751418,
 '2020-05-23 03:38:49.410430': 2.708362979437577,
 '2020-06-08 03:38:57.272110900': 3.533050178624597,
 '2020-06-24 03:39:06.276088': 11.131487303436879,
 '2020-07-26 03:39:18.261169': 10.754490670025358,
 '2020-08-11 03:39:21.223739': 12.020681014895786,
 '2020-09-12 03:39:36.390180': 4.934298979784745,
 '2020-09-28 03:39:40.939922900': None,
 '2020-10-14 03:39:42.886470': 5.470162171415698,
 '2020-10-30 03:39:42.122826': -0.8049813741485274,
 '2020-11-15 03:39:39.826170': -1.3146047987101703,
 '2020-01-16 03:39:56.935308': -17.713621644574065,
 '2020-02-01 03:39:51.843879900': -23.87978988724944,
 '2020-02-17 03:39:48.235220': -14.971396453187937,
 '2020-03-04 03:39:43.207155': -14.86788618333163,
 '2020-03-20 03:39:35.668509': -17.515746635587185,
 '2020-04-05 03:39:26.508163': -3.117161820575626,
 '2020-04-21 03:39:19.987736': -0.7874277483566267,
 '2020-05-07 03:39:10.901176900': 0.4694277284545003,
 '2020-05-23 03:39:13.297233900': 1.4288338500738784,
 '2020-06-08 03:39:21.163151': 5.030067413852141,
 '2020-06-24 03:39:30.154418900': None,
 '2020-08-11 03:39:45.097834900': 16.986679564288146,
 '2020-09-12 03:40:00.268510900': None,
 '2020-09-28 03:40:04.818254': 8.47648348113673,
 '2020-10-14 03:40:06.769037': 2.365158038590187,
 '2020-10-30 03:40:05.996920900': None,
 '2020-11-15 03:40:03.708738': 3.013120319695333,
 '2020-12-01 03:40:07.609355900': None,
 '2020-02-24 03:44:45.571808': -16.98770602303198,
 '2020-03-11 03:44:39.416739': -14.292819064183375,
 '2020-03-27 03:44:30.790211': None,
 '2020-04-12 03:44:23.154822900': None,
 '2020-04-28 03:44:15.512624': 0.09133020198020694,
 '2020-05-14 03:44:10.513552': None,
 '2020-05-30 03:44:14.273551': 1.5835453611837413,
 '2020-06-15 03:44:24.491800': 9.499374234788018,
 '2020-07-01 03:44:32.606465': 19.159669195448657,
 '2020-07-17 03:44:38.730511': None,
 '2020-08-02 03:44:42.875798': 14.13975141527326,
 '2020-08-18 03:44:48.074612': 15.968446502462148,
 '2020-09-19 03:45:01.724355': 10.824034115422192,
 '2020-10-05 03:45:05.190290900': 7.234817859584618,
 '2020-01-23 03:45:18.082050': -25.624498070868658,
 '2020-02-08 03:45:12.999236': None,
 '2020-02-24 03:45:09.445904': -17.23442430358736,
 '2020-03-11 03:45:03.290834': -13.123404802098097,
 '2020-03-27 03:44:54.660071': None,
 '2020-04-12 03:44:47.033155': -3.6187869806988116,
 '2020-04-28 03:44:39.386719900': 0.11051257567443107,
 '2020-05-14 03:44:34.396120': None,
 '2020-05-30 03:44:38.160354': -1.2776728831322521,
 '2020-06-15 03:44:48.378603': 6.87591587761244,
 '2020-07-01 03:44:56.484797': 16.182249369480218,
 '2020-08-02 03:45:06.754128900': 15.094948831036,
 '2020-08-18 03:45:11.961414900': 16.41869066866436,
 '2020-09-19 03:45:25.611158': 10.054862828416526,
 '2020-10-05 03:45:29.068622': 7.465601669199461,
 '2020-10-21 03:45:29.867674900': 3.3113617446616876,
 '2020-01-23 03:45:41.960381': -20.436293955490562,
 '2020-02-08 03:45:36.877566900': -18.898571760867387,
 '2020-02-24 03:45:33.324235': -13.657830112312851,
 '2020-03-11 03:45:27.169165': -9.1634153417786,
 '2020-03-27 03:45:18.534166': None,
 '2020-04-12 03:45:10.903013900': -2.6496674496877777,
 '2020-04-28 03:45:03.265051': 0.17093239572434518,
 '2020-05-14 03:44:58.282923': None,
 '2020-05-30 03:45:02.047158': -0.6597059495714196,
 '2020-06-15 03:45:12.261170': 7.1718772146757015,
 '2020-07-01 03:45:20.375836': 15.442343666617417,
 '2020-07-17 03:45:26.495646': 13.7640042773971,
 '2020-08-18 03:45:35.839746': 16.386420543707022,
 '2020-09-03 03:45:43.740562': 12.894911503415763,
 '2020-09-19 03:45:49.489490': 9.540880281673166,
 '2020-10-05 03:45:52.951190': 8.102856745033646,
 '2020-10-21 03:45:53.746006': -2.4394151358381175,
 '2020-11-22 03:45:52.841912': -0.4411379650472817,
 '2020-01-07 03:46:09.499076': -11.241346540782446,
 '2020-01-23 03:46:05.838712': -18.26414221306661,
 '2020-02-08 03:46:00.751663': -16.442382731077448,
 '2020-02-24 03:45:57.202566': -13.330523870073648,
 '2020-03-11 03:45:51.043260900': -7.114994566066513,
 '2020-03-27 03:45:42.412497': None,
 '2020-04-12 03:45:34.785581900': -4.196411987994366,
 '2020-04-28 03:45:27.143382': -2.4477574527986787,
 '2020-05-14 03:45:22.161253900': -1.8566761983509954,
 '2020-05-30 03:45:25.925488900': 4.423494140629993,
 '2020-06-15 03:45:36.147974': 9.184811481848675,
 '2020-07-01 03:45:44.254167': 16.382231686525103,
 '2020-07-17 03:45:50.378214': 14.751015937124064,
 '2020-08-18 03:45:59.722314': 16.461692272949513,
 '2020-09-03 03:46:07.618894': 12.741863350496189,
 '2020-09-19 03:46:13.376292900': 5.0953945322877034,
 '2020-10-05 03:46:16.837993': 7.497048520832194,
 '2020-10-21 03:46:17.628574': -7.802367736285824,
 '2020-12-08 03:46:18.701769': -4.86693783999411,
 '2020-01-30 03:51:26.502728': -19.23859364969576,
 '2020-02-15 03:51:22.563616': -15.79596181037682,
 '2020-03-02 03:51:17.857677900': -17.55962816004236,
 '2020-03-18 03:51:10.619535': -4.218655042672432,
 '2020-04-03 03:51:01.022903': -0.6785525542677953,
 '2020-04-19 03:50:54.811291900': -2.027889424202207,
 '2020-05-05 03:50:46.038311': None,
 '2020-05-21 03:50:46.786726': None,
 '2020-06-22 03:51:03.016490900': 6.783822500496808,
 '2020-07-24 03:51:15.509801': 13.935337638592955,
 '2020-08-09 03:51:18.731053': None,
 '2020-08-25 03:51:26.471347900': 15.82012270969689,
 '2020-09-10 03:51:33.427263': 13.16038703370349,
 '2020-10-12 03:51:40.537519900': None,
 '2020-10-28 03:51:40.129777': 0.46262366308087516,
 '2020-01-30 03:51:50.376822900': -14.836541370551792,
 '2020-02-15 03:51:46.437711': -16.444716262351452,
 '2020-03-02 03:51:41.740245': -14.255138404667532,
 '2020-03-18 03:51:34.497867': -3.774728157583882,
 '2020-04-03 03:51:24.892761900': -0.26325700146440706,
 '2020-04-19 03:51:18.681152': -1.4580560599780756,
 '2020-05-05 03:51:09.916642': None,
 '2020-06-22 03:51:26.899058': 8.638224477034933,
 '2020-07-08 03:51:34.132833': 14.208421429837925,
 '2020-07-24 03:51:39.388132900': 10.797186354857468,
 '2020-08-09 03:51:42.613621': 12.982673774984342,
 '2020-08-25 03:51:50.353915': 14.387026517953057,
 '2020-09-10 03:51:57.314067': 13.873663952307636,
 '2020-10-12 03:52:04.420087900': 4.980090140025602,
 '2020-10-28 03:52:04.016580': 2.0161726387175682,
 '2020-11-29 03:52:05.112129': -2.895181105510488,
 '2020-01-14 03:52:19.046024': -18.814295075868994,
 '2020-01-30 03:52:14.259391': None,
 '2020-02-15 03:52:10.316041900': -18.263430603383956,
 '2020-03-02 03:52:05.614339900': -11.510981100208847,
 '2020-03-18 03:51:58.371962': -2.7504247824165553,
 '2020-04-03 03:51:48.771093': -0.2856822230967567,
 '2020-04-19 03:51:42.559482900': -1.48118090851366,
 '2020-05-05 03:51:33.794974': None,
 '2020-06-06 03:51:41.519449900': -2.5250251247467554,
 '2020-06-22 03:51:50.781626': 6.907839369240973,
 '2020-07-08 03:51:58.019636': 14.103633967513833,
 '2020-07-24 03:52:03.274936': 10.122196789542286,
 '2020-08-09 03:52:06.491951900': 12.435395406706482,
 '2020-08-25 03:52:14.240718': 12.098049967476687,
 '2020-09-10 03:52:21.192398': 12.753288590718311,
 '2020-09-26 03:52:26.017511': None,
 '2020-10-12 03:52:28.298419': 4.051858788345633,
 '2020-10-28 03:52:27.894912': 3.1084545098922187,
 '2020-11-29 03:52:28.990461': -2.8804028934358814,
 '2020-01-21 03:58:04.181146': -13.953630288542536,
 '2020-02-06 03:57:58.759505': -14.801781637547677,
 '2020-03-09 03:57:49.703251': -9.450372833077116,
 '2020-03-25 03:57:41.385820': -16.98683086128033,
 '2020-04-10 03:57:33.326840': 0.5475478645308668,
 '2020-04-26 03:57:25.997907': 1.1022030158396738,
 '2020-05-12 03:57:19.397761900': 1.2205657964270111,
 '2020-06-13 03:57:32.748613': 4.347193919985178,
 '2020-06-29 03:57:41.113150': 11.411492795819957,
 '2020-07-15 03:57:47.470232': None,
 '2020-07-31 03:57:51.856990': 13.547398836069512,
 '2020-08-16 03:57:56.344139900': 12.639885163915713,
 '2020-09-01 03:58:04.499176': 13.9212273661125,
 '2020-09-17 03:58:10.523688': 12.869356425628865,
 '2020-10-03 03:58:14.302775': None,
 '2020-10-19 03:58:15.436386900': None,
 '2020-11-20 03:58:13.922156': -8.706242771721653,
 '2020-01-05 03:58:31.398083900': -4.618619850448387,
 '2020-01-21 03:58:28.059477': -15.222088936579613,
 '2020-02-06 03:58:22.642072': -16.71142516113629,
 '2020-02-22 03:58:19.423121900': None,
 '2020-03-09 03:58:13.581582': -10.137473517705374,
 '2020-03-25 03:58:05.259915': -14.104334126468896,
 '2020-04-10 03:57:57.196700': 0.14595936792117828,
 '2020-04-26 03:57:49.867766': 0.23726961060638396,
 '2020-05-12 03:57:43.284565900': 3.3194476259618644,
 '2020-06-13 03:57:56.635417': 6.405120794781495,
 '2020-06-29 03:58:04.995716900': 12.402259566010217,
 '2020-07-15 03:58:11.352798900': 14.381940316636587,
 '2020-07-31 03:58:15.735322': 15.28567971400536,
 '2020-08-16 03:58:20.222471': 9.851825877307041,
 '2020-09-01 03:58:28.385978900': 13.00699997747297,
 '2020-09-17 03:58:34.402020': 12.119848660886069,
 '2020-10-03 03:58:38.189577900': 5.256928986128202,
 '2020-10-19 03:58:39.314718': 2.6410850585271293,
 '2020-11-20 03:58:37.808959': -8.515929011252078,
 '2020-12-06 03:58:40.345541': None,
 '2020-01-12 04:04:41.101660': -10.605145266046856,
 '2020-01-28 04:04:36.628417': -15.987246329060545,
 '2020-02-13 04:04:32.358853900': -15.073168061079572,
 '2020-02-29 04:04:27.983470': -11.473425559816805,
 '2020-03-16 04:04:21.041432': -3.5890498619998996,
 '2020-04-01 04:04:11.627046900': -0.45772738458946644,
 '2020-04-17 04:04:05.097399': -7.0386939067597805,
 '2020-05-03 04:03:56.646382900': -1.1224333402023592,
 '2020-05-19 04:03:55.776842': 3.8258464443956326,
 '2020-06-04 04:04:01.837714': None,
 '2020-06-20 04:04:11.362175': 8.960328569083467,
 '2020-07-06 04:04:18.858583900': 15.506240339909093,
 '2020-07-22 04:04:24.359566900': 5.317049581866981,
 '2020-08-07 04:04:27.843753900': None,
 '2020-09-08 04:04:42.086737': None,
 '2020-09-24 04:04:47.174621900': None,
 '2020-10-10 04:04:49.798249': 7.23799832059602,
 '2020-10-26 04:04:49.737850': 0.11478789837320044,
 '2020-11-11 04:04:46.523376': None,
 '2020-11-27 04:04:50.291067': 2.5151447221480847,
 '2020-10-15 02:56:46.860988100': None,
 '2020-11-16 02:55:01.524153900': None,
 '2020-02-25 03:16:40.875893100': -19.48718457843882,
 '2020-03-28 03:14:55.757481400': -14.466886995268615,
 '2020-04-13 03:14:00.787170800': -5.269250393732461,
 '2020-04-29 03:13:04.101422600': -6.904160770921085,
 '2020-05-15 03:12:05.663679': -2.033334625981967,
 '2020-05-31 03:11:07.333166400': -2.4259672459513193,
 '2020-06-16 03:10:11.842931500': 5.846100179714124,
 '2020-07-02 03:09:14.857312500': 19.705208060458233,
 '2020-07-18 03:08:16.485880900': None,
 '2020-08-03 03:07:16.816401400': 18.250086488108906,
 '2020-08-19 03:06:15.739911700': 15.751283272933085,
 '2020-09-04 03:05:13.351751800': 14.42249969035783,
 '2020-10-06 03:03:08.502463400': 7.167636711936054,
 '2020-11-07 03:01:01.936240500': 1.0126883644030575,
 '2020-02-25 03:17:04.752585400': -18.744219577137574,
 '2020-03-12 03:16:12.968432500': -17.99793493605413,
 '2020-03-28 03:15:19.632672500': -5.616850335078471,
 '2020-04-13 03:14:24.663731900': -3.47264605985371,
 '2020-04-29 03:13:27.979247600': -4.0753968583570925,
 '2020-05-15 03:12:29.542321200': None,
 '2020-05-31 03:11:31.211247400': -0.7432882290874372,
 '2020-06-16 03:10:35.719691200': 4.43526875566089,
 '2020-07-02 03:09:38.732747400': 16.080820669905496,
 '2020-07-18 03:08:40.361038200': 12.466158564853815,
 '2020-08-03 03:07:40.692271800': 18.046399122676487,
 '2020-08-19 03:06:39.617614500': 13.71176635484426,
 '2020-09-04 03:05:37.230583900': 15.211408600944347,
 '2020-10-06 03:03:32.379895300': 6.853018115497433,
 '2020-10-22 03:02:30.221879100': 3.551424041181952,
 '2020-11-07 03:01:25.811481500': 1.1968667737507852,
 '2020-02-09 03:18:18.781890300': -19.51673134879389,
 '2020-02-25 03:17:28.629631': -17.986469460231945,
 '2020-03-12 03:16:36.844198200': -14.345874386993708,
 '2020-03-28 03:15:43.508611300': -4.285161108359119,
 '2020-04-13 03:14:48.540655400': -2.192165565568765,
 '2020-04-29 03:13:51.857865500': -1.885697167062951,
 '2020-05-31 03:11:55.090043600': -0.7654724831297278,
 '2020-06-16 03:10:59.597099100': 2.813358105738247,
 '2020-07-02 03:10:02.608729100': 14.472267557575396,
 '2020-07-18 03:09:04.236751900': 13.334261538164961,
 '2020-08-03 03:08:04.568925900': 16.94924494094579,
 '2020-08-19 03:07:03.495725700': 15.69815496712759,
 '2020-09-04 03:06:01.109698800': 14.859154772424832,
 '2020-09-20 03:04:57.294181400': 11.160495071377442,
 '2020-10-06 03:03:56.257799600': 3.1747688611995297,
 '2020-10-22 03:02:54.098408900': None,
 '2020-11-07 03:01:49.687372900': 2.210167576595196,
 '2020-01-24 03:19:31.130799200': -19.525171251423632,
 '2020-02-09 03:18:42.661325': -18.97178795700932,
 '2020-02-25 03:17:52.507407600': -22.71358819367109,
 '2020-03-12 03:17:00.720628400': -17.05784657581046,
 '2020-03-28 03:16:07.385115100': -4.550720326220787,
 '2020-04-13 03:15:12.418464100': -0.3431117795597046,
 '2020-04-29 03:14:15.737025800': None,
 '2020-05-31 03:12:18.969233': 4.40534760607362,
 '2020-06-16 03:11:23.475035700': None,
 '2020-07-02 03:10:26.485306': 10.072364185699668,
 '2020-07-18 03:09:28.113183600': 16.45065269837923,
 '2020-08-19 03:07:27.374558600': 15.36777748424103,
 '2020-09-04 03:06:24.989689500': 12.8735015668946,
 '2020-09-20 03:05:21.174034800': 10.819287459696508,
 '2020-10-06 03:04:20.136241100': 7.1601741485722785,
 '2020-10-22 03:03:17.975357': 5.141960107112112,
 '2020-11-07 03:02:13.563472200': -0.25241133546526234,
 '2020-02-16 03:23:20.063994400': -19.567993205407443,
 '2020-03-03 03:22:29.195356800': -21.634227045577624,
 '2020-03-19 03:21:36.747580700': -12.083264966765071,
 '2020-04-04 03:20:42.715710400': -2.4602718180037426,
 '2020-04-20 03:19:47.012488400': -3.613321374369783,
 '2020-05-06 03:18:49.558304700': None,
 '2020-05-22 03:17:50.404904200': 3.528925181723031,
 '2020-06-07 03:16:54.087419500': 6.361437482834978,
 '2020-06-23 03:15:57.900703300': 13.01592766161943,
 '2020-07-09 03:15:00.295325': 14.010146442579016,
 '2020-07-25 03:14:01.343558400': 16.818491899285355,
 '2020-08-10 03:13:01.048117700': 15.455246860338145,
 '2020-08-26 03:11:59.388653800': 15.403689737704456,
 '2020-09-27 03:09:53.392013300': None,
 '2020-10-13 03:08:52.333867100': 3.7691604775049674,
 '2020-01-31 03:24:33.162645500': -18.940858818769613,
 '2020-02-16 03:23:43.941576900': -18.49217557394516,
 '2020-03-03 03:22:53.071338400': -20.352810017522188,
 '2020-03-19 03:22:00.622712800': -11.65210495190071,
 '2020-04-04 03:21:06.591525700': -14.382554382517567,
 '2020-04-20 03:20:10.889694900': -4.298224705927998,
 '2020-05-06 03:19:13.436773': None,
 '2020-05-22 03:18:14.283380900': 2.7040059582362184,
 '2020-06-07 03:17:17.965050300': 3.379843649572162,
 '2020-06-23 03:16:21.776610100': 8.51971220201253,
 '2020-07-09 03:15:24.170289500': 14.184076109385245,
 '2020-07-25 03:14:25.218789500': 15.434167785038245,
 '2020-08-10 03:13:24.924600800': 15.791329740405446,
 '2020-08-26 03:12:23.266847600': 15.280450416315997,
 '2020-09-27 03:10:17.270269300': 9.056881209772158,
 '2020-10-13 03:09:16.210571500': 4.696949800220333,
 '2020-10-29 03:08:13.172767500': -7.2459986948469695,
 '2020-01-31 03:24:57.042181600': -16.987928430742187,
 '2020-02-16 03:24:07.819718100': -16.292725317572557,
 '2020-03-03 03:23:16.947912300': -18.0614479542663,
 '2020-03-19 03:22:24.498496100': None,
 '2020-04-04 03:21:30.467987': -7.2473219657817305,
 '2020-04-20 03:20:34.767580200': -2.449613100101403,
 '2020-05-22 03:18:38.162358200': 2.7252236215499934,
 '2020-06-07 03:17:41.843184700': 3.9723830568306453,
 '2020-06-23 03:16:45.653290900': 8.289142597918048,
 '2020-07-09 03:15:48.045915800': 13.006065129540877,
 '2020-07-25 03:14:49.094774600': 10.960517870015325,
 '2020-08-10 03:13:48.801879700': 12.585416904747174,
 '2020-08-26 03:12:47.145562200': 14.47639643286948,
 '2020-09-27 03:10:41.149123900': 6.872919326830981,
 '2020-10-13 03:09:40.087939900': 5.380003379424537,
 '2020-10-29 03:08:37.049010700': -0.20349161920667264,
 '2020-01-31 03:25:20.922166800': -18.790978608490757,
 '2020-02-16 03:24:31.698403300': -17.25778629454712,
 '2020-03-03 03:23:40.825038100': -15.185749792384208,
 '2020-03-19 03:22:48.374935300': -21.982887883352497,
 '2020-04-04 03:21:54.344946500': -3.455783548765169,
 '2020-04-20 03:20:58.645965200': -1.4113918517685538,
 '2020-05-22 03:19:02.042092300': 3.6400820371272693,
 '2020-06-07 03:18:05.721921900': 5.11199892537066,
 '2020-06-23 03:17:09.530422400': 9.995622917638968,
 '2020-07-09 03:16:11.921941600': None,
 '2020-07-25 03:15:12.971337100': None,
 '2020-08-10 03:14:12.679691100': 11.291807377343348,
 '2020-08-26 03:13:11.025051300': 7.536210534910266,
 '2020-09-27 03:11:05.028640100': 7.923939099788573,
 '2020-10-13 03:10:03.965798100': 4.716076312325017,
 '2020-10-29 03:09:00.925592800': 0.2949871253865055,
 '2020-11-14 03:07:55.371765600': -2.5420738152920705,
 '2020-02-07 03:30:22.658094': -18.96900176512254,
 '2020-02-23 03:29:32.724485600': -15.010110024367503,
 '2020-03-10 03:28:41.135455300': None,
 '2020-04-11 03:26:53.237901300': -0.06382211576568307,
 '2020-04-27 03:25:56.775010500': None,
 '2020-05-13 03:24:58.538850500': -22.637102907367627,
 '2020-05-29 03:23:59.637323600': 2.462056329952021,
 '2020-06-14 03:23:04.349450800': 5.347952820039594,
 '2020-06-30 03:22:07.526610100': 13.287464739756526,
 '2020-07-16 03:21:09.331985400': None,
 '2020-08-17 03:19:08.920358100': 15.20931636048214,
 '2020-09-02 03:18:06.712578400': 15.833977030233992,
 '2020-09-18 03:17:03.077924300': 14.002352290037772,
 '2020-10-04 03:16:01.618252': 8.113936448893602,
 '2020-10-20 03:14:59.706920800': None,
 '2020-11-05 03:13:55.613715800': -4.250797253605255,
 '2020-02-07 03:30:46.537156100': -18.5009831361145,
 '2020-02-23 03:29:56.601783100': -11.639013285957388,
 '2020-03-10 03:29:05.011376': -16.53182646498824,
 '2020-03-26 03:28:11.876931400': -13.63878225735205,
 '2020-04-11 03:27:17.114695100': 0.08630192804780579,
 '2020-04-27 03:26:20.653609500': -10.955390217208203,
 '2020-05-13 03:25:22.417875800': 1.2704843282931546,
 '2020-05-29 03:24:23.516235500': 3.6964183332697576,
 '2020-06-14 03:23:28.227068700': 8.289335307932028,
 '2020-06-30 03:22:31.402755200': 13.733077000316888,
 '2020-07-16 03:21:33.207659800': -14.031566856350143,
 '2020-08-01 03:20:33.690319500': None,
 '2020-08-17 03:19:32.798405200': 15.569177455224112,
 '2020-09-02 03:18:30.591727': 14.130696822726666,
 '2020-09-18 03:17:26.956933900': 12.48557117487962,
 '2020-10-04 03:16:25.496324400': 7.64325464474993,
 '2020-10-20 03:15:23.583527900': 3.998796402382618,
 '2020-11-05 03:14:19.489574100': 2.375978690119813,
 '2020-01-22 03:31:58.669298500': -18.487139892675618,
 '2020-02-07 03:31:10.416789400': -19.00308650365694,
 '2020-02-23 03:30:20.479716500': -11.43986580421044,
 '2020-03-10 03:29:28.887894200': -12.66970059770811,
 '2020-03-26 03:28:35.753358800': None,
 '2020-04-11 03:27:40.992303600': -0.5110998811456517,
 '2020-04-27 03:26:44.532426700': -3.257025522544405,
 '2020-05-13 03:25:46.297755800': 1.65844027045629,
 '2020-05-29 03:24:47.395785200': 3.477656958566047,
 '2020-06-14 03:23:52.105112': 9.347964960938251,
 '2020-06-30 03:22:55.279366700': 13.736749350141864,
 '2020-07-16 03:21:57.083997900': None,
 '2020-08-17 03:19:56.677096200': 15.17454328578225,
 '2020-09-02 03:18:54.471630200': 11.984622266906277,
 '2020-09-18 03:17:50.836853300': 10.945356576999814,
 '2020-10-04 03:16:49.375196': 7.073891166848969,
 '2020-10-20 03:15:47.460600200': 3.413711452119128,
 '2020-11-05 03:14:43.365813': 2.406502858523295,
 '2020-11-21 03:13:36.629640200': -5.594222156212343,
 '2020-01-29 03:37:48.574030500': -14.429329833082798,
 '2020-02-14 03:36:59.557775100': -19.5024804124037,
 '2020-03-17 03:35:16.611663400': -5.4199498811427915,
 '2020-04-02 03:34:22.793387800': -0.8198568295811991,
 '2020-04-18 03:33:27.295155800': -14.201285078497788,
 '2020-05-04 03:32:30.069617500': 2.447920592812114,
 '2020-05-20 03:31:31.121046': 2.4534644649898554,
 '2020-06-05 03:30:34.232673': 3.2591776807769683,
 '2020-06-21 03:29:38.240621200': 5.9774115831866865,
 '2020-07-07 03:28:40.805053300': 15.423112681322781,
 '2020-08-24 03:25:40.412222100': 13.320305286036072,
 '2020-09-09 03:24:37.578098600': 10.076825226634316,
 '2020-09-25 03:23:34.114374900': 2.698376006537801,
 '2020-10-11 03:22:33.301795600': 5.967306297436887,
 '2020-10-27 03:21:30.522649100': 4.704304621386244,
 '2020-11-12 03:20:25.304103700': 2.3006020588982605,
 '2020-02-05 03:43:38.131746400': -26.84187921371503,
 '2020-02-21 03:42:48.414122400': -9.087419672266266,
 '2020-03-08 03:41:57.021597600': None,
 '2020-04-09 03:40:09.542080900': -0.12586097809179358,
 '2020-04-25 03:39:13.285996900': 0.6337426358043866,
 '2020-05-11 03:38:15.276722100': 4.113767383594891,
 '2020-05-27 03:37:15.787682400': None,
 '2020-06-12 03:36:20.716063400': -25.469736529115742,
 '2020-06-28 03:35:24.065297700': 14.768102540884902,
 '2020-07-14 03:34:26.024801100': 1.736901865191021,
 '2020-07-30 03:33:26.671771700': 12.639252946387392,
 '2020-08-15 03:32:25.946072400': -7.728725875473591,
 '2020-08-31 03:31:23.917546700': -16.928402506496024,
 '2020-09-16 03:30:20.454039800': 9.265036006800567,
 '2020-10-02 03:29:18.581854700': -15.343038402411642,
 '2020-10-18 03:28:16.911824800': -6.811159304225497,
 '2020-11-03 03:27:13.118505500': -1.705122257169515,
'2021-02-03 03:38:42.508109900': -34.24286302751392,
 '2021-02-19 03:38:38.393934': -24.089783053822377,
 '2021-03-07 03:38:31.039613': None,
 '2021-03-23 03:38:23.636803': -6.063901968744997,
 '2021-04-08 03:38:19.314401': -0.9592101995478517,
 '2021-04-24 03:38:11.971036': -14.15794138033982,
 '2021-05-10 03:38:06.446226': -0.09109331029984932,
 '2021-05-26 03:38:16.619892': 0.8095339649892275,
 '2021-06-11 03:38:23.779109': None,
 '2021-06-27 03:38:28.094259': None,
 '2021-07-13 03:38:29.542497': 13.954065167674177,
 '2021-07-29 03:38:36.985352': 17.08608760565811,
 '2021-08-14 03:38:43.365828': 18.223726883741993,
 '2021-08-30 03:38:47.908964': 15.30724011157935,
 '2021-09-15 03:38:52.004235': 8.543514279576064,
 '2021-10-01 03:38:56.498076': None,
 '2021-10-17 03:39:00.628840': 1.4881763343098482,
 '2021-11-02 03:39:00.409889900': 1.667088039811655,
 '2021-11-18 03:38:54.605502': 0.4256978365319014,
 '2021-02-03 03:39:06.386441': None,
 '2021-02-19 03:39:02.272266': -21.881585736226977,
 '2021-03-07 03:38:54.922179900': None,
 '2021-03-23 03:38:47.510898': -3.5702097259501366,
 '2021-04-08 03:38:43.192732': -1.1293955215222649,
 '2021-04-24 03:38:35.849368': -15.218719103733536,
 '2021-05-10 03:38:30.337265': -0.8593778140997795,
 '2021-05-26 03:38:40.502458900': -0.1491650799255512,
 '2021-06-11 03:38:47.665913': -0.30359058047316423,
 '2021-06-27 03:38:51.976826': 0.7535279887348632,
 '2021-07-13 03:38:53.420828900': 7.737800104502916,
 '2021-07-29 03:39:00.872156': 14.39356986998303,
 '2021-08-14 03:39:07.244159': 16.60445670900777,
 '2021-08-30 03:39:11.795768': 14.099274651106448,
 '2021-09-15 03:39:15.891039': 10.096280511331532,
 '2021-10-01 03:39:20.376408': None,
 '2021-10-17 03:39:24.511408': 3.096842510424797,
 '2021-11-02 03:39:24.288222': 2.6568820176520225,
 '2021-11-18 03:39:18.488068900': -0.5820025470211659,
 '2021-01-18 03:39:31.614190900': None,
 '2021-02-03 03:39:30.269008': None,
 '2021-02-19 03:39:26.154833': -17.17238224974751,
 '2021-03-07 03:39:18.800512': -21.129961335689888,
 '2021-03-23 03:39:11.389230': -5.668515376538701,
 '2021-04-08 03:39:07.066828': -1.8769548463092003,
 '2021-04-24 03:38:59.731935': -12.649910747389129,
 '2021-05-26 03:39:04.389263': -1.387542429999583,
 '2021-06-11 03:39:11.548480': -2.3953701072086546,
 '2021-06-27 03:39:15.859394': 0.07476656486818449,
 '2021-07-13 03:39:17.303396': 5.986013211706839,
 '2021-07-29 03:39:24.750487': 14.04966618137907,
 '2021-08-14 03:39:31.126727': 15.277877645020379,
 '2021-08-30 03:39:35.674100': 11.992849395531126,
 '2021-09-15 03:39:39.773607': 9.970143428673012,
 '2021-10-17 03:39:48.393975': 4.403437283477871,
 '2021-11-02 03:39:48.175025900': 3.1523069267325874,
 '2021-11-18 03:39:42.370636900': 0.31347265571741867,
 '2021-01-18 03:39:55.488286': None,
 '2021-02-03 03:39:54.147339900': None,
 '2021-02-19 03:39:50.033164': -15.080407790135109,
 '2021-03-07 03:39:42.674607': None,
 '2021-03-23 03:39:35.263325900': -7.584824107368585,
 '2021-04-08 03:39:30.940923': -0.1139807135598115,
 '2021-04-24 03:39:23.601795': -9.041362124934913,
 '2021-05-26 03:39:28.271830': None,
 '2021-06-11 03:39:35.435284': -1.7506404728603768,
 '2021-06-27 03:39:39.741961': 3.538828215836998,
 '2021-07-13 03:39:41.185964': None,
 '2021-07-29 03:39:48.637291': 15.451506393150892,
 '2021-08-14 03:39:55.013530': 15.167638382020469,
 '2021-08-30 03:39:59.556667': None,
 '2021-09-15 03:40:03.660409900': 11.703193274761846,
 '2021-10-01 03:40:08.145779': 5.880589843562789,
 '2021-10-17 03:40:12.276543': 5.962805025560098,
 '2021-11-02 03:40:12.053358': -2.4296294257363313,
 '2021-11-18 03:40:06.253204900': 2.547337126617395,
 '2021-12-04 03:40:08.441746': -4.006443298463127,
 '2021-02-10 03:44:51.884989': -13.907771787840462,
 '2021-03-30 03:44:32.956046': -5.781007744556832,
 '2021-04-15 03:44:27.341156': -6.717106878432747,
 '2021-05-01 03:44:18.481701': None,
 '2021-05-17 03:44:22.123698': -2.3538014742187463,
 '2021-06-02 03:44:30.900149': 1.2614866732416876,
 '2021-06-18 03:44:36.827762': -1.2572468980043634,
 '2021-07-04 03:44:39.881215': 10.032529844585273,
 '2021-07-20 03:44:42.964522': 10.481646423716414,
 '2021-08-21 03:44:56.186762': 8.724900927090648,
 '2021-09-06 03:45:00.925472': 7.8035445607047,
 '2021-09-22 03:45:03.648788900': 7.878031370989413,
 '2021-11-09 03:45:09.285818900': 1.77029581318222,
 '2021-02-26 03:45:10.315063': None,
 '2021-03-30 03:44:56.834378': -5.490178042331981,
 '2021-04-15 03:44:51.219488': -3.7608759024059246,
 '2021-05-17 03:44:46.002030': -3.4648746766438783,
 '2021-06-02 03:44:54.778479900': 0.1833796783318922,
 '2021-06-18 03:45:00.710330': -1.118916909935705,
 '2021-07-04 03:45:03.763782': 10.720329249929032,
 '2021-07-20 03:45:06.847090': 6.893014485472335,
 '2021-08-05 03:45:14.790486900': 9.238016658681591,
 '2021-08-21 03:45:20.065094': 13.778046352800311,
 '2021-09-22 03:45:27.535593': 8.80086148035439,
 '2021-10-08 03:45:33.554912': -0.3052141350233437,
 '2021-10-24 03:45:35.800974': 1.1278609853611694,
 '2021-11-09 03:45:33.164151': 1.9669362103241363,
 '2021-01-25 03:45:42.058111': None,
 '2021-03-30 03:45:20.712709': -3.150928959262841,
 '2021-04-15 03:45:15.093583': -3.029468079896108,
 '2021-05-17 03:45:09.893069': -1.310039036436344,
 '2021-06-02 03:45:18.661047900': 2.452668556881242,
 '2021-06-18 03:45:24.588661': 3.4747949693324576,
 '2021-07-04 03:45:27.642114': 10.47882736476148,
 '2021-07-20 03:45:30.733893': 9.983205577220652,
 '2021-08-05 03:45:38.673054': 11.89695014983205,
 '2021-08-21 03:45:43.947661': 13.952801817637893,
 '2021-09-22 03:45:51.413924900': 9.121855227386135,
 '2021-10-08 03:45:57.433244': 4.204201407136129,
 '2021-10-24 03:45:59.675069900': 4.683280486350277,
 '2021-11-09 03:45:57.046719': 3.123672934027617,
 '2021-11-25 03:45:54.694384': 0.2734563929365492,
 '2021-01-09 03:46:10.919439': -7.361969936255767,
 '2021-02-10 03:46:03.519984': None,
 '2021-03-30 03:45:44.586805': -3.241961121224956,
 '2021-04-15 03:45:38.971915': -8.603560475516527,
 '2021-05-17 03:45:33.771400900': None,
 '2021-06-02 03:45:42.547851': None,
 '2021-06-18 03:45:48.471229': 5.0592748743501055,
 '2021-07-04 03:45:51.520445900': 9.75956027032111,
 '2021-07-20 03:45:54.616461': 13.062358070269532,
 '2021-08-05 03:46:02.551386': 14.68050448832413,
 '2021-08-21 03:46:07.834465': 11.58397000568351,
 '2021-09-22 03:46:15.296492': 8.911405982004709,
 '2021-10-08 03:46:21.315812': 2.288182931409672,
 '2021-10-24 03:46:23.561873': 5.462738921968812,
 '2021-11-09 03:46:20.925051': 4.317989467660854,
 '2021-11-25 03:46:18.581188': 0.44724644176264367,
 '2021-12-11 03:46:18.719520900': -0.8571147296845917,
 '2021-02-01 03:51:28.331038': -22.251357631462877,
 '2021-03-05 03:51:17.662951': -20.059579955941302,
 '2021-03-21 03:51:09.459281': -13.588259592268619,
 '2021-04-06 03:51:05.522467': -0.9388146491074033,
 '2021-04-22 03:50:58.585719': None,
 '2021-05-08 03:50:50.480908': -0.13247579984414934,
 '2021-05-24 03:51:01.039979': -1.2218714609626626,
 '2021-06-09 03:51:08.550581': -3.3953704493240147,
 '2021-06-25 03:51:13.229880900': 3.3365039156445273,
 '2021-07-27 03:51:21.523342': 13.022841721949856,
 '2021-08-12 03:51:28.234301': 12.915551998465185,
 '2021-08-28 03:51:32.705151': 14.011223565263796,
 '2021-09-13 03:51:37.198507': 3.980099765022603,
 '2021-10-15 03:51:45.865229': 4.581479458116335,
 '2021-10-31 03:51:46.230626': -2.015337351451028,
 '2021-01-16 03:51:54.355081': None,
 '2021-02-01 03:51:52.213604900': -22.841202299213666,
 '2021-02-17 03:51:48.463663': None,
 '2021-03-05 03:51:41.537046': -19.004207265032772,
 '2021-03-21 03:51:33.337612': None,
 '2021-04-06 03:51:29.400798': -0.6965913214724541,
 '2021-04-22 03:51:22.464051': None,
 '2021-05-08 03:51:14.367712': -1.7027601432749027,
 '2021-05-24 03:51:24.918311': -0.844337833478134,
 '2021-06-09 03:51:32.433148': 3.8065497022489136,
 '2021-06-25 03:51:37.108212': 1.5194159107107648,
 '2021-07-11 03:51:38.908062': 8.004947155084553,
 '2021-07-27 03:51:45.405910': 13.85450583764709,
 '2021-08-12 03:51:52.116868': 14.423220448795627,
 '2021-08-28 03:51:56.587719': 14.280442075575635,
 '2021-09-13 03:52:01.081075': 9.147466836051068,
 '2021-09-29 03:52:05.079317': None,
 '2021-10-15 03:52:09.752033': 4.641292507201712,
 '2021-10-31 03:52:10.108956900': 1.7216022381511575,
 '2021-02-01 03:52:16.096172': -23.726755510818233,
 '2021-02-17 03:52:12.346231': None,
 '2021-03-05 03:52:05.419614': -18.152879164123735,
 '2021-03-21 03:51:57.215943900': -10.222951674510616,
 '2021-04-06 03:51:53.279130': -0.8455415679595467,
 '2021-05-08 03:51:38.250278900': None,
 '2021-05-24 03:51:48.805114': 1.4417670264973712,
 '2021-06-09 03:51:56.319952': 4.686133422221778,
 '2021-06-25 03:52:00.995016': 6.650543428931018,
 '2021-07-11 03:52:02.790630': 5.551947266194364,
 '2021-07-27 03:52:09.292712900': 12.959204881487807,
 '2021-08-12 03:52:15.999436': 14.493183020874186,
 '2021-08-28 03:52:20.470286': 13.098351979884178,
 '2021-09-13 03:52:24.972115': 9.618497360125447,
 '2021-10-15 03:52:33.630365': 4.3529907066170495,
 '2021-10-31 03:52:33.987289': 1.721513367687636,
 '2021-11-16 03:52:28.822493': None,
 '2021-12-02 03:52:30.053731': -5.504032122316742,
 '2021-02-08 03:58:01.734363': -18.29756046816923,
 '2021-02-24 03:57:56.684299': -14.035970690818946,
 '2021-03-28 03:57:42.839000900': -2.6743223212271543,
 '2021-04-13 03:57:37.579892': None,
 '2021-04-29 03:57:29.169288': None,
 '2021-05-15 03:57:30.214794': 1.0753402241647865,
 '2021-08-03 03:57:59.468273': 1.6906608732234008,
 '2021-08-19 03:58:05.051975': 11.041726517629355,
 '2021-09-04 03:58:09.756750': 11.602098935027486,
 '2021-09-20 03:58:12.873857': 11.346095446891475,
 '2021-10-06 03:58:18.482209': 8.226923756730379,
 '2021-10-22 03:58:21.257787900': 6.898127875793253,
 '2021-11-07 03:58:19.374651': None,
 '2021-11-23 03:58:15.993036': -4.181665362917358,
 '2021-01-07 03:58:33.444334': -25.820744502120476,
 '2021-01-23 03:58:27.635430': None,
 '2021-02-08 03:58:25.616930': -15.697742145944572,
 '2021-02-24 03:58:20.562629900': -17.44844092571993,
 '2021-03-12 03:58:12.162452': -9.245052851527088,
 '2021-03-28 03:58:06.713096': -3.908759939709209,
 '2021-04-13 03:58:01.458223900': -5.506916425787359,
 '2021-04-29 03:57:53.043384': None,
 '2021-05-15 03:57:54.101597': 1.007751976788858,
 '2021-05-31 03:58:03.263289900': -0.2924302432286503,
 '2021-08-03 03:58:23.350840': None,
 '2021-08-19 03:58:28.938778': 12.731345514087131,
 '2021-09-04 03:58:33.639318': 10.305207896822003,
 '2021-09-20 03:58:36.752188': 10.399032170952964,
 '2021-10-06 03:58:42.369013': 7.7964424677585455,
 '2021-10-22 03:58:45.144591900': 6.07445144547827,
 '2021-11-07 03:58:43.252983': -5.590855674656627,
 '2021-11-23 03:58:39.871367900': -5.342420664868742,
 '2021-01-14 04:04:40.931910': None,
 '2021-01-30 04:04:37.985804': None,
 '2021-02-15 04:04:34.604162900': -18.254635757350663,
 '2021-03-03 04:04:28.113851': -14.569902673184142,
 '2021-03-19 04:04:19.096418': -4.710128702064604,
 '2021-04-04 04:04:15.562102': -0.585055843072924,
 '2021-04-20 04:04:09.023616': None,
 '2021-05-06 04:03:59.084384': None,
 '2021-05-22 04:04:09.296150': None,
 '2021-06-07 04:04:17.162308900': None,
 '2021-06-23 04:04:22.184404': 15.549368747292533,
 '2021-07-09 04:04:24.348480900': None,
 '2021-07-25 04:04:29.893028': 16.786103805680973,
 '2021-08-10 04:04:36.942893': None,
 '2021-08-26 04:04:41.421899': None,
 '2021-09-11 04:04:46.228698': 7.783253665838826,
 '2021-09-27 04:04:49.718699': None,
 '2021-10-13 04:04:54.929134': -0.3885130321975019,
 '2021-10-29 04:04:55.828225': 5.826525040324767,
 '2021-11-14 04:04:51.353709900': 4.045806388455895,
 '2021-11-30 04:04:51.589458900': -15.965683855002288,
 '2021-12-12 03:40:06.156204': None,
 '2021-11-11 03:44:30.893919': None,
 '2021-11-16 03:46:58.972080900': None,
 '2021-11-11 03:44:54.738338900': -0.6620301820805967,
 '2021-11-16 03:47:22.808028': None,
 '2021-11-11 03:45:18.578522': 2.9096278075421287,
 '2021-11-16 03:47:46.656684': None,
 '2021-11-11 03:45:42.427178': 3.437834765423855,
 '2021-11-16 03:48:10.492631': None,
 '2021-11-21 03:49:50.415866': 0.2238715540475018,
 '2021-11-21 03:50:14.256049': 0.6970554479120257,
 '2021-11-21 03:50:38.087760900': 1.736159181895002,
 '2021-12-10 03:52:29.183087': -3.3255433788898827,
 '2021-11-09 04:04:27.049091900': 4.771511449074289,
 '2021-12-08 04:04:52.146327': -4.634052725773781,
 '2021-02-20 02:47:39.157570400': -20.128266150209637,
 '2021-03-08 02:46:24.864856200': -18.234646588350675,
 '2021-03-24 02:45:08.270216100': None,
 '2021-04-09 02:43:49.334106800': -5.476996017108043,
 '2021-04-25 02:42:28.224969600': None,
 '2021-05-11 02:41:08.158142': 0.09990825550684584,
 '2021-05-27 02:39:46.465003300': 0.7968002727545758,
 '2021-06-12 02:38:27.037644300': 2.398382770135999,
 '2021-06-28 02:37:07.911676200': None,
 '2021-07-14 02:35:46.592098300': None,
 '2021-07-30 02:34:23.285448700': 15.271273892579162,
 '2021-08-15 02:32:57.906007500': 17.920254385691475,
 '2021-08-31 02:31:35.081801500': 13.73402437919961,
 '2021-10-02 02:28:45.015165800': 6.908479347642912,
 '2021-10-18 02:27:20.179170700': 2.227243505672065,
 '2021-11-03 02:25:51.769183900': None,
 '2021-02-20 02:48:03.033302100': -18.2701020405786,
 '2021-03-08 02:46:48.740205100': -16.3066409670261,
 '2021-03-24 02:45:32.146329': -14.329252646274805,
 '2021-04-09 02:44:13.211912800': None,
 '2021-05-11 02:41:32.036919200': 0.023057800586978306,
 '2021-05-27 02:40:10.342121600': 1.0364683510526502,
 '2021-06-12 02:38:50.913594700': 1.8808990009136886,
 '2021-06-28 02:37:31.786893600': None,
 '2021-07-30 02:34:47.162931': 15.118256168541272,
 '2021-08-15 02:33:21.784601300': 17.62687906408549,
 '2021-08-31 02:31:58.960939700': 14.012844452559397,
 '2021-09-16 02:30:34.624602400': None,
 '2021-10-02 02:29:08.891536800': 6.783207524130318,
 '2021-10-18 02:27:44.054587700': 2.7077243041460193,
 '2021-11-03 02:26:15.644864': None,
 '2021-02-11 02:54:06.832065500': -18.993350780189566,
 '2021-02-27 02:52:53.899156200': -20.570327208427972,
 '2021-03-15 02:51:38.573387900': -15.903746086673987,
 '2021-03-31 02:50:20.918563800': -5.285791554534722,
 '2021-04-16 02:49:01.077080600': -11.4598391584196,
 '2021-05-02 02:47:39.978518800': -6.239835015036649,
 '2021-05-18 02:46:19.635529200': -1.9157652915220467,
 '2021-06-03 02:44:57.464143200': 3.789376921928492,
 '2021-06-19 02:43:39.583859600': 4.752863865359275,
 '2021-07-05 02:42:19.507529900': 12.267278956365503,
 '2021-07-21 02:40:57.333690800': 1.1025567246219619,
 '2021-08-06 02:39:33.089146100': None,
 '2021-08-22 02:38:08.381583500': None,
 '2021-09-07 02:36:45.429425900': None,
 '2021-09-23 02:35:20.019836900': 7.236199915815384,
 '2021-10-09 02:33:55.250377700': None,
 '2021-02-11 02:54:30.708185300': -17.742968223144793,
 '2021-02-27 02:53:17.773957800': -19.566875774908016,
 '2021-03-15 02:52:02.448219500': -15.240751842979227,
 '2021-03-31 02:50:44.794676400': -4.438492115976909,
 '2021-04-16 02:49:24.954845700': -9.980920108832843,
 '2021-05-02 02:48:03.856890600': -12.833103417085885,
 '2021-05-18 02:46:43.513251600': -1.6297182850212122,
 '2021-06-03 02:45:21.340420900': 2.627769265404863,
 '2021-06-19 02:44:03.458853600': 3.8427045995812663,
 '2021-07-05 02:42:43.382194300': 7.741303966237886,
 '2021-07-21 02:41:21.209358900': -1.4985131467037014,
 '2021-08-06 02:39:56.966412500': 10.609537467338267,
 '2021-08-22 02:38:32.259988': 10.556580662210846,
 '2021-09-23 02:35:43.896501': 7.346937022766541,
 '2021-10-09 02:34:19.125742': 5.541465735476014,
 '2021-10-25 02:32:52.752530900': -4.332881395420404,
 '2021-02-11 02:54:54.584702200': -14.435922264179693,
 '2021-02-27 02:53:41.649238700': -17.01539990543497,
 '2021-03-15 02:52:26.323753100': -12.397327821180921,
 '2021-03-31 02:51:08.671423': -9.040518684026695,
 '2021-04-16 02:49:48.833117400': -5.544599097504982,
 '2021-05-02 02:48:27.736016100': None,
 '2021-05-18 02:47:07.391278900': -0.5853762758428712,
 '2021-06-03 02:45:45.217160900': 2.8059822187035537,
 '2021-06-19 02:44:27.334203200': 4.640221607343624,
 '2021-07-05 02:43:07.257482500': 9.640910558952445,
 '2021-07-21 02:41:45.085707600': None,
 '2021-08-06 02:40:20.844000300': 10.551370701816037,
 '2021-09-23 02:36:07.773845500': 9.064975828690006,
 '2021-10-09 02:34:43.001696900': 6.450700448746612,
 '2021-10-25 02:33:16.627737': 2.7056649833139765,
 '2021-02-11 02:55:18.462005800': -15.49455391314119,
 '2021-02-27 02:54:05.525182400': -18.810830770532196,
 '2021-03-15 02:52:50.199842400': -12.322029650807295,
 '2021-03-31 02:51:32.548783800': -4.872612967637298,
 '2021-04-16 02:50:12.711942400': -1.6969954402927858,
 '2021-05-18 02:47:31.270000600': 0.3723665106674763,
 '2021-06-03 02:46:09.094523600': 3.811360436326445,
 '2021-06-19 02:44:51.210340700': 8.031473235464603,
 '2021-07-05 02:43:31.133324900': 12.246094837448116,
 '2021-08-06 02:40:44.722622700': 14.915573809015344,
 '2021-08-22 02:39:20.018306700': 8.6203609668584,
 '2021-09-23 02:36:31.651571800': 7.20465761538319,
 '2021-10-25 02:33:40.503913600': None,
 '2021-11-10 02:32:10.023084900': 3.469575359043268,
 '2021-03-06 02:58:31.992711700': None,
 '2021-04-07 02:55:57.016059800': -6.586100389971743,
 '2021-04-23 02:54:36.217388500': -2.6404930301372196,
 '2021-05-09 02:53:15.902754300': -6.233051361463005,
 '2021-05-25 02:51:54.516889300': 0.5108810145434299,
 '2021-06-10 02:50:34.484462700': 3.606738033492834,
 '2021-06-26 02:49:15.637280900': 10.19466657142783,
 '2021-07-12 02:47:54.590547400': 10.651267627213867,
 '2021-07-28 02:46:31.543122600': 16.449812725821708,
 '2021-08-13 02:45:06.390233': 16.620954636231577,
 '2021-08-29 02:43:43.175072300': 12.491007304788509,
 '2021-10-16 02:39:28.674273900': None,
 '2021-11-01 02:38:00.767554500': -4.368088171548592,
 '2021-03-06 02:58:55.867368600': -17.677289701334953,
 '2021-03-22 02:57:39.566193600': None,
 '2021-04-07 02:56:20.893159800': -4.184391096741269,
 '2021-04-23 02:55:00.095499600': -5.075725971726257,
 '2021-05-09 02:53:39.780862200': -3.2873880943667473,
 '2021-05-25 02:52:18.393631300': None,
 '2021-06-10 02:50:58.359851500': 3.2797682151476266,
 '2021-06-26 02:49:39.511784700': 7.87852239706259,
 '2021-07-12 02:48:18.465503400': 4.690964948021506,
 '2021-07-28 02:46:55.419508400': 12.886652047724397,
 '2021-08-13 02:45:30.267938200': 15.165633936999457,
 '2021-08-29 02:44:07.053343100': 14.299656108194501,
 '2021-09-14 02:42:43.005130700': 8.08946408062932,
 '2021-09-30 02:41:16.961447900': None,
 '2021-10-16 02:39:52.549179500': -7.090372055730691,
 '2021-11-01 02:38:24.642223900': -0.3453218384016905,
 '2021-02-02 03:01:45.474224': -24.76931417014109,
 '2021-02-18 03:00:33.737349200': -15.523215097631892,
 '2021-03-06 02:59:19.742676800': -16.325377275054855,
 '2021-03-22 02:58:03.442312900': None,
 '2021-04-07 02:56:44.770759700': -2.414434580382953,
 '2021-04-23 02:55:23.974355500': -10.17222781642105,
 '2021-05-09 02:54:03.659584800': -6.465888802661429,
 '2021-06-10 02:51:22.236017100': 3.113687805130206,
 '2021-06-26 02:50:03.387062100': 7.226243428404703,
 '2021-07-12 02:48:42.341146400': 3.5269196940650156,
 '2021-07-28 02:47:19.296578200': 12.443046431790977,
 '2021-08-13 02:45:54.146524': 13.529283161866978,
 '2021-08-29 02:44:30.932480200': 14.310128043288442,
 '2021-09-14 02:43:06.883256600': 9.156846350090557,
 '2021-09-30 02:41:40.837877600': None,
 '2021-10-16 02:40:16.424633100': None,
 '2021-11-01 02:38:48.517840': -3.6556392752630806,
 '2021-02-02 03:02:09.352174400': -23.35256279862977,
 '2021-02-18 03:00:57.613858200': -17.628371904746093,
 '2021-03-06 02:59:43.618444200': -15.614088503637511,
 '2021-03-22 02:58:27.318834900': None,
 '2021-04-07 02:57:08.648924500': -1.4260557085476966,
 '2021-04-23 02:55:47.853813700': None,
 '2021-05-09 02:54:27.539016200': -1.489153597497248,
 '2021-05-25 02:53:06.149067600': None,
 '2021-06-10 02:51:46.112518700': 2.6336598740958603,
 '2021-06-26 02:50:27.262652800': 6.741088215281039,
 '2021-07-28 02:47:43.174195': 13.207075257855296,
 '2021-08-13 02:46:18.025526600': 14.089500220572695,
 '2021-08-29 02:44:54.812067900': 13.140664335876055,
 '2021-09-14 02:43:30.762074700': None,
 '2021-09-30 02:42:04.715303400': None,
 '2021-10-16 02:40:40.300719600': -4.673643590785214,
 '2021-11-01 02:39:12.393778500': -1.000973520313403,
 '2021-02-25 03:05:48.620909500': -17.787706963070896,
 '2021-03-13 03:04:33.579541': -22.077105224695543,
 '2021-03-29 03:03:16.235712600': -8.451227906753479,
 '2021-04-14 03:01:56.639438500': -1.6313687565670805,
 '2021-04-30 03:00:35.343490600': -0.8622790846847707,
 '2021-05-16 02:59:15.285751100': 0.536600281286514,
 '2021-06-01 02:57:52.785920300': None,
 '2021-06-17 02:56:34.922247500': 3.066515531392855,
 '2021-07-03 02:55:15.098513700': 3.5234797090290257,
 '2021-07-19 02:53:53.165720500': None,
 '2021-08-04 02:52:29.195060300': 2.05039992719858,
 '2021-08-20 02:51:04.046309300': 13.31439003663827,
 '2021-10-07 02:46:51.277819800': 6.3816982721789115,
 '2021-10-23 02:45:25.318082300': 4.1766167835604255,
 '2021-02-25 03:06:12.496436100': -18.787149800010887,
 '2021-03-13 03:04:57.454880900': -9.143967288348744,
 '2021-03-29 03:03:40.112362700': -7.936550188548184,
 '2021-04-14 03:02:20.517704300': -6.280568727338212,
 '2021-04-30 03:00:59.222480800': -0.1961716053691346,
 '2021-06-17 02:56:58.797566400': 4.530094661488598,
 '2021-07-03 02:55:38.973796900': 6.450543225257981,
 '2021-07-19 02:54:17.041899200': None,
 '2021-08-04 02:52:53.072588600': 11.219805887265853,
 '2021-08-20 02:51:27.925017700': 13.35680200747558,
 '2021-09-05 02:50:05.277082400': 10.323767147757348,
 '2021-10-07 02:47:15.154022800': 6.729508236951574,
 '2021-10-23 02:45:49.193607500': 3.460093829781762,
 '2021-11-08 02:44:19.344060300': -6.369891193395919,
 '2021-02-25 03:06:36.372463100': -18.685112675393682,
 '2021-03-13 03:05:21.330955100': -7.927659307329463,
 '2021-03-29 03:04:03.989502900': -8.152499840035071,
 '2021-04-14 03:02:44.396539200': -23.776332659386224,
 '2021-04-30 03:01:23.101857': -0.01813493284678565,
 '2021-05-16 03:00:03.042937200': -2.8455119358105194,
 '2021-06-17 02:57:22.673812800': 4.513866805844718,
 '2021-07-03 02:56:02.849738900': 5.693690337068201,
 '2021-08-04 02:53:16.951049600': 10.027229789995282,
 '2021-08-20 02:51:51.804602': 12.997966360551294,
 '2021-09-05 02:50:29.156378100': 8.9497429177827,
 '2021-10-07 02:47:39.030514900': 6.248058680793403,
 '2021-10-23 02:46:13.069633200': 5.560934968478009,
 '2021-11-08 02:44:43.220740300': -0.1757457770781038,
 '2021-01-31 03:14:39.785383500': -30.46541231915223,
 '2021-02-16 03:13:28.288321300': -16.38039624837893,
 '2021-03-04 03:12:14.606915400': -15.728303213643553,
 '2021-03-20 03:10:58.572347400': -7.85920626502722,
 '2021-04-05 03:09:40.187445700': -1.6066420635495609,
 '2021-05-07 03:06:59.133525400': -1.6641812817907868,
 '2021-05-23 03:05:38.048169600': 0.5281072822585532,
 '2021-06-08 03:04:17.395047400': 2.286097239218393,
 '2021-07-10 03:01:38.049638300': 6.059437931029175,
 '2021-07-26 03:00:15.278371400': 13.501157311647585,
 '2021-08-11 02:58:50.373526500': 11.837674068621794,
 '2021-08-27 02:57:26.730737800': 8.127370470702944,
 '2021-09-12 02:56:03.008130700': 10.37297680915437,
 '2021-09-28 02:54:36.785967100': 6.397183191957372,
 '2021-10-14 02:53:12.636595300': -4.134496636557215,
 '2021-10-30 02:51:45.172909800': 5.553379908975853,
 '2021-11-15 02:50:13.183352200': -1.8672016642783953,
'2022-02-22 03:38:36.807675': -19.237903367654184,
 '2022-03-10 03:38:33.763891': -14.570733921686182,
 '2022-03-26 03:38:22.701322': -8.841500702277639,
 '2022-04-11 03:38:24.241418': -10.146592283006768,
 '2022-04-27 03:38:20.992388': -6.580525553688998,
 '2022-05-13 03:38:29.898254': -3.3076359777405027,
 '2022-05-29 03:38:31.627830': None,
 '2022-06-30 03:38:49.575905': 12.423419342807426,
 '2022-07-16 03:38:48.214447': None,
 '2022-08-17 03:39:06.730827': 11.36003463661665,
 '2022-09-02 03:39:06.500146': None,
 '2022-09-18 03:39:13.556481': 10.070390076206321,
 '2022-10-04 03:39:11.947605': 5.693309002606289,
 '2022-10-20 03:39:09.254504': 3.7680283367579537,
 '2022-11-05 03:39:13.420473': -4.188275753693434,
 '2022-02-22 03:39:00.681771': -19.737767916369982,
 '2022-03-10 03:38:57.642223': -13.578527684615196,
 '2022-03-26 03:38:46.575418': -8.391859522572306,
 '2022-04-11 03:38:48.119749': -9.28070854789369,
 '2022-04-27 03:38:44.879191': -10.523823074559576,
 '2022-05-13 03:38:53.785058': -4.994105542538505,
 '2022-05-29 03:38:55.510398': None,
 '2022-06-30 03:39:13.462709': 8.307192121672651,
 '2022-08-01 03:39:24.332987': 11.193700690569633,
 '2022-08-17 03:39:30.617631': 11.78726733039408,
 '2022-09-02 03:39:30.378477900': None,
 '2022-09-18 03:39:37.434813': 9.655400978783309,
 '2022-10-04 03:39:35.830173': 4.769551148486064,
 '2022-10-20 03:39:33.141308': 4.662970437572901,
 '2022-11-05 03:39:37.303041': 0.629724393722933,
 '2022-11-21 03:39:30.887587': -10.18814974411057,
 '2022-01-21 03:39:34.125978900': -15.654763116999634,
 '2022-02-06 03:39:30.484028': -19.63296988386907,
 '2022-02-22 03:39:24.560102900': -18.608673308398867,
 '2022-03-10 03:39:21.516319': -10.75884051185792,
 '2022-03-26 03:39:10.453748900': -7.880489559280802,
 '2022-04-11 03:39:11.993845': -9.472810579718118,
 '2022-04-27 03:39:08.761759': -11.171860701067905,
 '2022-05-13 03:39:17.663390': -4.305236739332849,
 '2022-05-29 03:39:19.397200900': -1.8643130239145382,
 '2022-06-30 03:39:37.345276': 11.521529972008748,
 '2022-08-01 03:39:48.211319': 11.020540447721986,
 '2022-08-17 03:39:54.500199': 12.087639104922355,
 '2022-09-02 03:39:54.265282': 10.389151372132945,
 '2022-09-18 03:40:01.317381': 10.204967837071809,
 '2022-10-04 03:39:59.716977': 4.096494192995133,
 '2022-10-20 03:39:57.023876': 3.1178809880764122,
 '2022-11-05 03:40:01.185609': 2.4543033270055417,
 '2022-11-21 03:39:54.765919': -8.189200370505617,
 '2022-01-05 03:40:01.390119': -2.0299353697572253,
 '2022-01-21 03:39:58.008546': -13.841020213297146,
 '2022-02-06 03:39:54.362360': -17.29103440242674,
 '2022-02-22 03:39:48.434199': -18.604987841319158,
 '2022-03-10 03:39:45.390414': -7.989754681152041,
 '2022-03-26 03:39:34.327845': -5.6448670688126095,
 '2022-04-11 03:39:35.872177': -4.558025624009162,
 '2022-04-27 03:39:32.644326900': -2.6483890351587047,
 '2022-05-13 03:39:41.550194': -3.755364742710286,
 '2022-05-29 03:39:43.284004900': 2.259147576171482,
 '2022-06-30 03:40:01.223608': None,
 '2022-07-16 03:39:59.862150': None,
 '2022-08-01 03:40:12.093887': 16.01690087053557,
 '2022-08-17 03:40:18.378531': None,
 '2022-09-02 03:40:18.147849': 13.758073008668687,
 '2022-09-18 03:40:25.195713': 10.338574997458695,
 '2022-10-04 03:40:23.595309': None,
 '2022-10-20 03:40:20.902208': None,
 '2022-11-05 03:40:25.063941': 0.9557857542636934,
 '2022-11-21 03:40:18.652723': None,
 '2022-01-28 03:44:54.085030': -22.64088976744019,
 '2022-02-13 03:44:51.312274900': -29.75080836684526,
 '2022-03-01 03:44:47.095782': -11.051863074348208,
 '2022-03-17 03:44:40.692637': -17.390594043761084,
 '2022-05-04 03:44:35.378023': -5.888335457538912,
 '2022-05-20 03:44:42.449764': -3.8985408685841985,
 '2022-06-05 03:44:48.991438': None,
 '2022-07-07 03:45:00.369753': 4.863097752414608,
 '2022-07-23 03:45:05.290702900': 15.461032132437131,
 '2022-08-24 03:45:18.075611': None,
 '2022-09-09 03:45:21.216815': 11.601927237659359,
 '2022-09-25 03:45:24.864071': 8.361393909575716,
 '2022-10-27 03:45:23.506412': 4.757295932791258,
 '2022-11-12 03:45:21.038976': -5.385273521608204,
 '2022-02-13 03:45:15.190607': -29.479201841329633,
 '2022-03-01 03:45:10.969878': -11.773615825885061,
 '2022-03-17 03:45:04.566733': -17.59231082157864,
 '2022-04-02 03:44:53.724340': -5.112732628460438,
 '2022-04-18 03:44:59.475697': -8.251448225756182,
 '2022-05-04 03:44:59.264825900': -1.1650627112916103,
 '2022-05-20 03:45:06.332332': 0.6457064267122681,
 '2022-06-05 03:45:12.874005': -6.777957241574664,
 '2022-06-21 03:45:22.598390900': 4.326511271410359,
 '2022-07-07 03:45:24.248085': 6.985314954207807,
 '2022-08-08 03:45:38.723196': None,
 '2022-09-09 03:45:45.099383': 10.09482778585404,
 '2022-09-25 03:45:48.742403': 7.703400378895409,
 '2022-10-11 03:45:41.576046': None,
 '2022-10-27 03:45:47.384743900': 4.587549426291918,
 '2022-11-12 03:45:44.913072': -1.6755543826010604,
 '2022-01-28 03:45:41.845929': -20.69865583760966,
 '2022-02-13 03:45:39.064702': -24.881817321675285,
 '2022-03-17 03:45:28.445065': -15.258132972036043,
 '2022-04-02 03:45:17.598435': -3.791148730523597,
 '2022-04-18 03:45:23.366737': -5.465437912010575,
 '2022-05-04 03:45:23.143158': -1.4606340866990537,
 '2022-05-20 03:45:30.214898900': 3.1421162854070173,
 '2022-06-05 03:45:36.756573': -1.8535132056094745,
 '2022-06-21 03:45:46.476723': 5.488815804805854,
 '2022-07-07 03:45:48.130653': 8.913372577221171,
 '2022-07-23 03:45:53.051603': None,
 '2022-08-08 03:46:02.605763': 12.73478687065195,
 '2022-08-24 03:46:05.840745900': None,
 '2022-09-09 03:46:08.977715': 8.229670256534353,
 '2022-09-25 03:46:12.624971': 7.155829666955369,
 '2022-10-11 03:46:05.454378': None,
 '2022-10-27 03:46:11.271548': 4.890144529907874,
 '2022-11-12 03:46:08.799875': 0.9988846247285653,
 '2022-11-28 03:46:07.310354900': -11.534782977382283,
 '2022-01-12 03:46:11.396812': -13.032449193139522,
 '2022-01-28 03:46:05.724260900': -14.087799682572136,
 '2022-02-13 03:46:02.951506': -22.460289351164942,
 '2022-03-17 03:45:52.327632': -13.874781112305566,
 '2022-04-02 03:45:41.476766900': -0.6792086202193571,
 '2022-04-18 03:45:47.249303900': -0.9040473590551423,
 '2022-05-04 03:45:47.034197900': -7.2752651145135845,
 '2022-05-20 03:45:54.097466900': 3.253586139193895,
 '2022-06-05 03:46:00.634905': 4.110783901759906,
 '2022-06-21 03:46:10.359291': 4.522722567126123,
 '2022-07-07 03:46:12.013221': 11.222839988516922,
 '2022-08-08 03:46:26.484095': 14.955953614898993,
 '2022-08-24 03:46:29.719077900': 8.949632747398285,
 '2022-09-09 03:46:32.860283': 9.527343338647867,
 '2022-09-25 03:46:36.507539': 7.017367264816566,
 '2022-10-11 03:46:29.336946': 1.026933808488652,
 '2022-10-27 03:46:35.149879900': 4.422513373397076,
 '2022-11-12 03:46:32.682443': 2.043640159370818,
 '2022-11-28 03:46:31.184451': -18.417860606004954,
 '2022-02-04 03:51:28.602820': -24.597156169362215,
 '2022-02-20 03:51:22.272091': -27.014918016674695,
 '2022-03-08 03:51:20.050261': None,
 '2022-03-24 03:51:09.983081900': None,
 '2022-04-09 03:51:08.816234900': None,
 '2022-04-25 03:51:07.672033': -12.741244862588891,
 '2022-05-11 03:51:14.476766900': -0.974831022346637,
 '2022-05-27 03:51:15.812647': 1.7821858952560397,
 '2022-06-12 03:51:28.983992900': 4.688623218079951,
 '2022-06-28 03:51:34.904459': 8.037378937929578,
 '2022-07-14 03:51:33.703637': None,
 '2022-07-30 03:51:44.779364': 12.254557275295104,
 '2022-08-15 03:51:51.855719': 13.996956906216418,
 '2022-08-31 03:51:52.341102': 8.802977646219036,
 '2022-09-16 03:51:58.638918': None,
 '2022-10-18 03:51:53.430331': -0.6368325581131438,
 '2022-11-03 03:51:59.231253': 2.3472904315883554,
 '2022-11-19 03:51:51.710202': 1.5849123450698088,
 '2022-01-19 03:51:56.530059': -11.767464896320705,
 '2022-02-04 03:51:52.481151900': -20.84438010232914,
 '2022-03-24 03:51:33.857177': -13.453768405800416,
 '2022-04-25 03:51:31.558836900': -12.976911778331932,
 '2022-05-11 03:51:38.363570': -0.5425234363623699,
 '2022-05-27 03:51:39.690979': 2.7418930634573937,
 '2022-06-12 03:51:52.862325': 6.443987561558264,
 '2022-06-28 03:51:58.791263': 9.930264697422912,
 '2022-07-30 03:52:08.661932': 11.529804066015393,
 '2022-08-15 03:52:15.729815': 15.796748540504689,
 '2022-08-31 03:52:16.227906': 7.5549902717195705,
 '2022-09-16 03:52:22.521486': None,
 '2022-10-18 03:52:17.312899': 1.3117389655364624,
 '2022-11-03 03:52:23.113821': 3.2421561726984267,
 '2022-11-19 03:52:15.592770': 1.8096452384922042,
 '2022-01-19 03:52:20.408391': -10.84203603065206,
 '2022-02-04 03:52:16.355248': -18.38947527971558,
 '2022-03-08 03:52:07.811160': None,
 '2022-03-24 03:51:57.739745': -9.34933487631017,
 '2022-04-25 03:51:55.441404900': None,
 '2022-05-11 03:52:02.246138': 0.06720695111337033,
 '2022-05-27 03:52:03.582019': 3.373921186742561,
 '2022-06-12 03:52:16.749128900': 7.041675399961881,
 '2022-06-28 03:52:22.669595': 12.099542180418084,
 '2022-07-14 03:52:21.468773': None,
 '2022-07-30 03:52:32.544500': 11.707814387476061,
 '2022-08-15 03:52:39.620855': 16.653362734193138,
 '2022-08-31 03:52:40.106238': 7.74499663381265,
 '2022-10-02 03:52:46.023443': 3.8634663757135463,
 '2022-10-18 03:52:41.195467': 2.1202925914737505,
 '2022-11-03 03:52:46.996389': 3.3638766395260995,
 '2022-11-19 03:52:39.475338': -0.4493029751149723,
 '2022-12-05 03:52:41.821945': -2.0390607941079715,
 '2022-01-26 03:58:04.254168': -14.853845850533123,
 '2022-02-11 03:58:01.502324': -16.60076512615758,
 '2022-03-15 03:57:51.386809900': -11.178238351757614,
 '2022-03-31 03:57:39.481298': -11.716381422046744,
 '2022-04-16 03:57:45.380597900': -0.6663867576974111,
 '2022-05-02 03:57:43.195545': 2.264593376992764,
 '2022-05-18 03:57:51.660836900': None,
 '2022-06-19 03:58:07.409951': -1.9406133117094042,
 '2022-07-05 03:58:09.898510900': 2.5486486500612067,
 '2022-07-21 03:58:13.103839': -4.523509556314618,
 '2022-08-06 03:58:23.314517': 10.456113204481326,
 '2022-08-22 03:58:27.443280': 8.823336967220685,
 '2022-09-07 03:58:29.491519': None,
 '2022-09-23 03:58:34.273963': 0.1218333393720546,
 '2022-10-09 03:58:28.933208': 7.168955186647483,
 '2022-10-25 03:58:32.136616': 5.552521421902486,
 '2022-01-10 03:58:33.343439900': -12.065920738818823,
 '2022-01-26 03:58:28.132499900': -15.40607873163491,
 '2022-02-11 03:58:25.380656': -22.42724962682675,
 '2022-02-27 03:58:20.617731900': -9.85067331444513,
 '2022-03-15 03:58:15.265142': -14.29236755298249,
 '2022-03-31 03:58:03.359630': -18.406046324370465,
 '2022-04-16 03:58:09.263165': -1.6498833341202692,
 '2022-05-02 03:58:07.082348900': 1.1803505175224256,
 '2022-05-18 03:58:15.547641': 0.40310896743950503,
 '2022-06-19 03:58:31.296754': 5.01832711102177,
 '2022-07-05 03:58:33.776843': None,
 '2022-07-21 03:58:36.982170900': None,
 '2022-08-06 03:58:47.201321': None,
 '2022-08-22 03:58:51.321610900': 12.348510038309302,
 '2022-09-07 03:58:53.369851': 8.447699360544487,
 '2022-09-23 03:58:58.156531': 3.7756990233257244,
 '2022-10-09 03:58:52.820012': 3.4291114013665704,
 '2022-10-25 03:58:56.019184': 4.184843788258099,
 '2022-11-26 03:58:52.524148': None,
 '2022-12-12 03:58:50.163501': -19.03338664520792,
 '2022-01-17 04:04:42.698082': None,
 '2022-02-02 04:04:38.187108': -15.114321434936267,
 '2022-02-18 04:04:32.851822': -13.663858012743177,
 '2022-03-06 04:04:30.096337': -11.810926790852674,
 '2022-03-22 04:04:21.037310': -2.701213900951908,
 '2022-04-07 04:04:17.061805': -6.082428297087929,
 '2022-04-23 04:04:18.039670900': -15.942482976763614,
 '2022-05-09 04:04:22.793888900': None,
 '2022-05-25 04:04:25.862569': 4.618269806688011,
 '2022-06-26 04:04:44.022329': 8.572303406221316,
 '2022-07-12 04:04:43.651572900': 9.5804965729582,
 '2022-07-28 04:04:52.918692': 6.407174768673167,
 '2022-08-13 04:05:00.740393': 8.06615583340849,
 '2022-08-29 04:05:01.971504': -6.089436044734297,
 '2022-09-30 04:05:08.239882900': None,
 '2022-10-16 04:05:01.310729': 5.817925060132562,
 '2022-11-01 04:05:08.632257': 4.92632525143843,
 '2022-11-17 04:05:01.123891900': None,
 '2022-12-03 04:05:03.695612': -21.042797211133635,
 '2022-01-29 03:38:51.490945': -20.77715661498818,
 '2022-02-14 03:38:47.412617': -25.00123358292799,
 '2022-03-02 03:38:35.911121': None,
 '2022-04-03 03:38:31.835844': -3.5035472703774513,
 '2022-04-19 03:38:22.666142900': -2.884892222555353,
 '2022-05-05 03:38:17.766090': -3.4776721384305906,
 '2022-06-06 03:38:07.008211': 5.154822923712615,
 '2022-06-22 03:38:23.032519900': 7.188158401623313,
 '2022-07-08 03:38:25.543254': 10.571184729491703,
 '2022-07-24 03:38:32.505510900': 11.568307627219573,
 '2022-08-09 03:38:44.106441': None,
 '2022-08-25 03:38:48.916880': 12.107025908421734,
 '2022-09-10 03:38:48.995245': 11.682620057306265,
 '2022-09-26 03:38:56.278808': None,
 '2022-10-12 03:38:59.695080': 4.6551402136593465,
 '2022-10-28 03:39:01.810388': 3.600462193498363,
 '2022-11-13 03:39:03.205155': -5.30982081911591,
 '2022-01-29 03:39:15.369252': -22.95095441943869,
 '2022-02-14 03:39:11.286688': -25.442260283427853,
 '2022-03-02 03:38:59.793664': None,
 '2022-04-03 03:38:55.709915': -3.216658969549493,
 '2022-04-19 03:38:46.540213900': None,
 '2022-05-05 03:38:41.635926': -1.546583320884838,
 '2022-05-21 03:38:29.543294': 0.24833365513206662,
 '2022-06-06 03:38:30.886520': 4.2933929389669485,
 '2022-06-22 03:38:46.915064': 5.037493823836342,
 '2022-07-08 03:38:49.430034': 7.707665853756402,
 '2022-07-24 03:38:56.388054': 10.18063206811342,
 '2022-08-09 03:39:07.984749': 13.086417885646425,
 '2022-08-25 03:39:12.790952': 11.238957783567365,
 '2022-09-10 03:39:12.873552': 10.42264783798455,
 '2022-09-26 03:39:20.161351': None,
 '2022-10-12 03:39:23.581859': 2.0553450926417463,
 '2022-10-28 03:39:25.692930900': 4.513657080238184,
 '2022-11-13 03:39:27.087698900': -2.026286532368629,
 '2022-01-29 03:39:39.251796': -21.743090789559723,
 '2022-02-14 03:39:35.173467': -23.203947409867094,
 '2022-03-02 03:39:23.671972': -14.235718511345963,
 '2022-04-03 03:39:19.583986': -7.10449505297349,
 '2022-04-19 03:39:10.410049': None,
 '2022-05-05 03:39:05.509997': -1.1611656094540208,
 '2022-05-21 03:38:53.413129': 0.05933881453077634,
 '2022-06-06 03:38:54.769063': 4.007675671865239,
 '2022-06-22 03:39:10.797606900': 5.762869059653545,
 '2022-07-08 03:39:13.312577': 9.326354680807201,
 '2022-07-24 03:39:20.270597900': 11.010209237365007,
 '2022-08-09 03:39:31.867291900': 13.543625845795948,
 '2022-08-25 03:39:36.673494900': 11.474291042456747,
 '2022-09-10 03:39:36.756096': 11.191174352902452,
 '2022-09-26 03:39:44.048130': None,
 '2022-10-12 03:39:47.460166900': 3.745421087194529,
 '2022-10-28 03:39:49.567003': 4.953021887700585,
 '2022-11-13 03:39:50.970243': 0.4547714034950256,
 '2022-01-13 03:40:00.607376': None,
 '2022-01-29 03:40:03.130103': -21.601215251474006,
 '2022-02-14 03:39:59.056011': -22.757891893905985,
 '2022-03-02 03:39:47.550279': -10.62358090657867,
 '2022-04-03 03:39:43.462294': None,
 '2022-04-19 03:39:34.284121': None,
 '2022-05-05 03:39:29.384068': None,
 '2022-05-21 03:39:17.291436': 1.307314724721782,
 '2022-06-06 03:39:18.655841900': -1.3725900413162893,
 '2022-06-22 03:39:34.684387': 3.420472898076773,
 '2022-07-08 03:39:37.199356': 15.544387098454937,
 '2022-07-24 03:39:44.153141': None,
 '2022-08-09 03:39:55.745600': 16.356548139104827,
 '2022-08-25 03:40:00.556038': 13.775434064594775,
 '2022-09-10 03:40:00.642875': None,
 '2022-09-26 03:40:07.934911': None,
 '2022-10-12 03:40:11.346946': 5.564645834311257,
 '2022-10-28 03:40:13.453782': 4.661335589258929,
 '2022-11-13 03:40:14.848550': 2.2352056565069005,
 '2022-02-05 03:45:01.588230': -19.323679923528807,
 '2022-02-21 03:44:53.849411': -22.44054832240763,
 '2022-03-25 03:44:43.467741': -16.183587256810362,
 '2022-04-10 03:44:40.118436900': -8.64349833959853,
 '2022-04-26 03:44:31.291574': -2.1745344875919854,
 '2022-05-12 03:44:23.985755': -0.09839497487607686,
 '2022-05-28 03:44:16.323661': 0.8323165332332741,
 '2022-06-13 03:44:21.224682900': None,
 '2022-06-29 03:44:38.536461': 2.6947064926937707,
 '2022-08-16 03:44:57.949605': 11.758648875385367,
 '2022-09-17 03:45:04.159662': 11.461030467949323,
 '2022-10-03 03:45:07.186103': None,
 '2022-11-04 03:45:09.989365': 0.5727802356297633,
 '2022-02-05 03:45:25.466537': -19.620349086728517,
 '2022-02-21 03:45:17.731954': -22.08381463136537,
 '2022-03-25 03:45:07.341813': -16.761810246642707,
 '2022-04-10 03:45:03.992509': None,
 '2022-04-26 03:44:55.165646': -1.2436467140043994,
 '2022-05-12 03:44:47.859827': -0.1903015987663273,
 '2022-05-28 03:44:40.206203900': 1.5563509807311398,
 '2022-06-13 03:44:45.111462': None,
 '2022-06-29 03:45:02.423239900': 4.640808081140373,
 '2022-08-16 03:45:21.832148': 13.348554028491414,
 '2022-09-01 03:45:23.812835900': None,
 '2022-09-17 03:45:28.046441': 10.190246128916078,
 '2022-10-03 03:45:31.068646': 4.905574280279511,
 '2022-10-19 03:45:36.839990': 5.413702626020491,
 '2022-11-04 03:45:33.871907900': None,
 '2022-01-20 03:45:49.190778': -12.04373885534125,
 '2022-02-05 03:45:49.349081': -16.152562540191422,
 '2022-02-21 03:45:41.610261': -18.76752628977043,
 '2022-03-09 03:45:35.470953': -2.0389691875700824,
 '2022-03-25 03:45:31.220120': -7.942871205851693,
 '2022-04-10 03:45:27.866580': -7.234316279237418,
 '2022-04-26 03:45:19.039717': -0.7995261277405746,
 '2022-05-12 03:45:11.733898': -0.4704518700574317,
 '2022-05-28 03:45:04.088747': 2.827098594528825,
 '2022-06-13 03:45:08.998242': None,
 '2022-06-29 03:45:26.305784': 6.703311793839133,
 '2022-07-15 03:45:21.521698': None,
 '2022-07-31 03:45:36.812068900': None,
 '2022-08-16 03:45:45.710454900': 9.13163220033884,
 '2022-09-01 03:45:47.695379': 10.147868106726722,
 '2022-09-17 03:45:51.924750': 9.847665660578484,
 '2022-10-03 03:45:54.942718': 2.9634980332691185,
 '2022-10-19 03:46:00.722533': 6.081572578300697,
 '2022-11-04 03:45:57.750214900': 1.8760143104604017,
 '2022-01-20 03:46:13.069086': -11.18683321832911,
 '2022-02-05 03:46:13.231624': -12.774737423720472,
 '2022-02-21 03:46:05.492805': -16.7441406036345,
 '2022-03-09 03:45:59.345024': -3.0577368281342494,
 '2022-03-25 03:45:55.094191': -5.210286193194899,
 '2022-04-10 03:45:51.740652': -3.649726794801078,
 '2022-04-26 03:45:42.918025': -0.4848524240586297,
 '2022-05-12 03:45:35.603732900': 0.46996535341648943,
 '2022-05-28 03:45:27.971292': 4.278683248047588,
 '2022-06-13 03:45:32.876550': 1.6135682546611867,
 '2022-06-29 03:45:50.192562900': 10.069441502368806,
 '2022-07-15 03:45:45.400005900': 10.055790464330409,
 '2022-07-31 03:46:00.694612': 12.11069358098323,
 '2022-08-16 03:46:09.592999': 8.776860805438295,
 '2022-09-01 03:46:11.582158': 11.058885954931089,
 '2022-09-17 03:46:15.807292900': 7.45548410476492,
 '2022-10-03 03:46:18.833733': -2.6419422273186606,
 '2022-10-19 03:46:24.605076': 4.955398957868555,
 '2022-11-04 03:46:21.632758': 3.9157879387543906,
 '2022-01-27 03:51:36.956355': -18.171158620958316,
 '2022-02-12 03:51:33.915979': -21.493829198314216,
 '2022-04-01 03:51:17.551724': -3.7841998967751613,
 '2022-04-17 03:51:09.932433': None,
 '2022-05-03 03:51:04.235993': -7.1730842407836235,
 '2022-05-19 03:50:51.957014': None,
 '2022-06-04 03:50:52.381147': 0.8809659867608614,
 '2022-06-20 03:51:04.398312': 4.52703354089867,
 '2022-07-06 03:51:11.717029': 7.013671845706927,
 '2022-08-07 03:51:28.479879': 7.587921981455449,
 '2022-08-23 03:51:34.192573': 11.345081586367522,
 '2022-09-08 03:51:33.491632': 9.453273742057162,
 '2022-09-24 03:51:41.385144900': 8.388873796120162,
 '2022-10-10 03:51:43.937243': 6.06048710457202,
 '2022-10-26 03:51:47.691993': 4.6183294232007315,
 '2022-11-11 03:51:48.298942': -2.1820794919134063,
 '2022-01-27 03:52:00.830426900': -15.91492976617135,
 '2022-02-12 03:51:57.794286': -20.400947400030606,
 '2022-02-28 03:51:46.792868': -10.572439280021324,
 '2022-04-01 03:51:41.421559900': -5.690009247353528,
 '2022-04-17 03:51:33.806504': -5.014542393924761,
 '2022-05-03 03:51:28.110065': -2.590610675784424,
 '2022-05-19 03:51:15.835320900': 1.75677139881835,
 '2022-06-04 03:51:16.267926': 1.0480884878682062,
 '2022-06-20 03:51:28.276619': 5.591791568692941,
 '2022-07-06 03:51:35.603807900': 9.993995910243338,
 '2022-08-07 03:51:52.353950': 10.88662193504071,
 '2022-08-23 03:51:58.079352': 8.895047891223939,
 '2022-09-08 03:51:57.374176': 10.523920547511034,
 '2022-09-24 03:52:05.271924': 8.091188300092021,
 '2022-10-10 03:52:07.819786': 5.36645015818366,
 '2022-10-26 03:52:11.574536': 5.041578947748905,
 '2022-11-11 03:52:12.181484900': 1.4281692596881852,
 '2022-11-27 03:52:10.784265': -20.939530056899294,
 '2022-01-11 03:52:22.952493': -7.335317784644796,
 '2022-01-27 03:52:24.717206': -11.922679278937233,
 '2022-02-12 03:52:21.672594': -18.988642348596446,
 '2022-02-28 03:52:10.671175': -12.44282520561433,
 '2022-04-01 03:52:05.304103': -10.277351220231623,
 '2022-04-17 03:51:57.680576': -2.3436346655233335,
 '2022-05-03 03:51:51.984136900': 0.23251306653743845,
 '2022-05-19 03:51:39.705156': 3.494633276562415,
 '2022-06-04 03:51:40.150468900': 3.2294698247950886,
 '2022-06-20 03:51:52.167635': 5.962030247797849,
 '2022-07-06 03:51:59.486351': 12.18589186327492,
 '2022-08-07 03:52:16.240729': 13.164280407330686,
 '2022-08-23 03:52:21.953424': 11.7087503650583,
 '2022-09-08 03:52:21.256719': 10.488423053834577,
 '2022-09-24 03:52:29.154467': 5.8439990963749775,
 '2022-10-10 03:52:31.706565': 5.315830351238704,
 '2022-10-26 03:52:35.461315': 4.663496478263079,
 '2022-11-11 03:52:36.055555900': -1.213169094286591,
 '2022-11-27 03:52:34.662573': -20.436720586409137,
 '2022-12-13 03:52:35.408003900': -4.955526812707558,
 '2022-01-18 03:58:10.379580': -8.447056010706719,
 '2022-02-03 03:58:11.359816': -13.290818824808026,
 '2022-02-19 03:58:04.633615': -14.011099959685698,
 '2022-03-07 03:57:57.003372': -5.598407157176362,
 '2022-03-23 03:57:53.811582': -5.3671040999901125,
 '2022-04-08 03:57:50.479341': -1.016367689497441,
 '2022-04-24 03:57:40.648599': -13.774137652958826,
 '2022-05-10 03:57:34.846491': None,
 '2022-05-26 03:57:25.490127': 4.608031349665433,
 '2022-06-11 03:57:28.806986': 4.255003203875773,
 '2022-06-27 03:57:48.173305': 9.133810209535362,
 '2022-07-13 03:57:43.533314': 10.084053196177566,
 '2022-07-29 03:57:56.925880': 8.620048217438542,
 '2022-08-14 03:58:06.641902': 13.431195912770704,
 '2022-08-30 03:58:09.397795': None,
 '2022-09-15 03:58:12.538587': 7.1254740514975,
 '2022-10-17 03:58:21.959111': 6.348196534981244,
 '2022-11-02 03:58:20.020440': 4.882055656696438,
 '2022-01-18 03:58:34.257888': -8.214809466986482,
 '2022-02-03 03:58:35.238124': -14.599224219997852,
 '2022-02-19 03:58:28.507686': -17.925350856096227,
 '2022-03-07 03:58:20.881680': -8.749433439864129,
 '2022-03-23 03:58:17.689890': -8.532466868861462,
 '2022-04-08 03:58:14.361885': -0.3754584171367628,
 '2022-04-24 03:58:04.522671': -9.120275808319645,
 '2022-05-10 03:57:58.720562': None,
 '2022-05-26 03:57:49.364198': 4.6001233514551965,
 '2022-06-11 03:57:52.698001': 4.300991295649797,
 '2022-06-27 03:58:12.051612': 13.379481366795792,
 '2022-07-13 03:58:07.411621900': 13.330386436759193,
 '2022-07-29 03:58:20.804188': 11.485126604712963,
 '2022-08-14 03:58:30.520210': 13.840376909606178,
 '2022-08-30 03:58:33.276101900': None,
 '2022-09-15 03:58:36.425366': 7.25351706608117,
 '2022-10-01 03:58:40.675956': None,
 '2022-10-17 03:58:45.841653900': 5.390015698210744,
 '2022-11-02 03:58:43.898748': 4.1452951476122335,
 '2022-12-04 03:58:44.471854': -2.8118886318692495,
 '2022-01-09 04:04:45.221271': -0.979846228207453,
 '2022-01-25 04:04:46.219529': -17.509419588729937,
 '2022-02-10 04:04:44.132424': -21.885256589936294,
 '2022-02-26 04:04:34.008104': -14.363774721510998,
 '2022-03-30 04:04:26.849413': -4.773643337489876,
 '2022-04-15 04:04:20.916093': -1.5425605955724293,
 '2022-05-01 04:04:14.406312': 0.8311128781883207,
 '2022-05-17 04:04:03.546411': None,
 '2022-06-02 04:04:01.543391': 1.9588229973557723,
 '2022-06-18 04:04:09.485666': None,
 '2022-07-20 04:04:23.593009900': 11.701089777443425,
 '2022-08-05 04:04:36.621493': 5.53329220219418,
 '2022-08-21 04:04:43.287295': -5.875272825785892,
 '2022-09-06 04:04:43.391304': None,
 '2022-09-22 04:04:50.225770': -4.842115059814508,
 '2022-10-08 04:04:51.714584': 6.22786477454604,
 '2022-11-25 04:04:57.017589900': 1.9444674064084773,
 '2022-12-11 04:04:57.051424': None,
 '2022-05-18 02:07:09.462138700': 0.9394381291935036,
 '2022-06-04 02:06:20.770209600': None,
 '2022-07-08 02:04:13.902498900': 11.263914914503392,
 '2022-07-25 02:02:56.870849200': 12.794911104022518,
 '2022-10-18 01:54:24.328081800': -6.089363196411898,
 '2022-02-23 02:14:57.556158700': -24.64626555064873,
 '2022-03-11 02:13:23.645796700': -24.658170330546138,
 '2022-05-18 02:07:33.296461100': 1.2274707872718096,
 '2022-06-04 02:06:44.603252100': None,
 '2022-07-08 02:04:37.736661': 7.097931145116632,
 '2022-10-01 01:56:52.686792400': None,
 '2022-10-18 01:54:48.160700900': -2.427406545471182,
 '2022-02-23 02:15:21.431668600': -24.23941219296164,
 '2022-03-11 02:13:47.522213900': -24.89858995859601,
 '2022-05-18 02:07:57.131649800': 2.4817073427416756,
 '2022-06-04 02:07:08.436982500': None,
 '2022-06-21 02:06:09.707778': 4.665265469369649,
 '2022-07-08 02:05:01.571500700': 10.416271240413218,
 '2022-07-25 02:03:44.542368600': None,
 '2022-10-01 01:57:16.519968': 0.3885546802364597,
 '2022-10-18 01:55:11.994309400': -2.2286736602013177,
 '2022-03-18 02:18:27.489779300': -23.978442026597314,
 '2022-04-03 02:16:56.473810300': -3.6255301778530784,
 '2022-05-06 02:13:22.611682500': -1.0913989432010909,
 '2022-05-23 02:12:45.124342': -0.24016081379213106,
 '2022-06-26 02:10:51.164515600': 5.688840409912959,
 '2022-07-13 02:09:40.772467100': 12.142979027759939,
 '2022-07-30 02:08:20.767248700': 11.66499816371357,
 '2022-08-11 02:01:31.678290400': 12.157177289614262,
 '2022-08-16 02:06:53.571747600': 12.445397840104121,
 '2022-08-28 01:59:59.038939700': None,
 '2022-09-02 02:05:18.924899100': None,
 '2022-09-14 01:58:19.107958300': 6.903365949965157,
 '2022-09-19 02:03:36.073304500': None,
 '2022-10-06 02:01:42.511104800': -6.82972871107242,
 '2022-02-14 02:22:00.273755500': -30.19331260496025,
 '2022-03-18 02:18:51.366647': -25.092837935041217,
 '2022-04-03 02:17:20.351504900': -3.7039698909919556,
 '2022-05-06 02:13:46.447177100': -1.0056668025088964,
 '2022-05-23 02:13:08.958325800': 0.420731869997723,
 '2022-06-09 02:12:16.673025300': 1.6707031423855982,
 '2022-06-26 02:11:14.997949': 4.597063532033573,
 '2022-07-13 02:10:04.607164700': 10.303936933815969,
 '2022-07-30 02:08:44.602918900': 9.664932021961402,
 '2022-08-11 02:01:55.514187500': 11.642165271023128,
 '2022-08-16 02:07:17.407335500': 11.43951914316153,
 '2022-08-28 02:00:22.873703100': None,
 '2022-09-14 01:58:42.941331600': 2.240982322991033,
 '2022-09-19 02:03:59.906000700': 0.6978097498105211,
 '2022-10-06 02:02:06.343305200': 1.7345592226541067,
 '2022-10-23 01:59:59.235285500': 3.0323310947164,
 '2022-02-14 02:22:24.148893500': -25.787394122571314,
 '2022-03-02 02:20:51.829448500': -16.08669122376605,
 '2022-03-18 02:19:15.244062100': -18.46039675092889,
 '2022-04-03 02:17:44.229913700': None,
 '2022-05-06 02:14:10.283499300': -2.0316814552692817,
 '2022-05-23 02:13:32.792760600': 2.0487009578285353,
 '2022-06-09 02:12:40.506530200': 2.1428784767644076,
 '2022-06-26 02:11:38.831918700': 6.024390817780088,
 '2022-07-13 02:10:28.442420900': 11.345397670906765,
 '2022-07-30 02:09:08.439473300': 10.369308680370002,
 '2022-08-11 02:02:19.350491': 13.18016526528281,
 '2022-08-16 02:07:41.243744800': 12.761161805149703,
 '2022-08-28 02:00:46.709378': 0.7930048282958171,
 '2022-09-02 02:06:06.594307400': 8.173924237256669,
 '2022-09-14 01:59:06.775244500': None,
 '2022-09-19 02:04:23.739510200': 9.734744380412423,
 '2022-10-06 02:02:30.176088500': 4.8875790521460845,
 '2022-10-23 02:00:23.068924600': 4.756567471566625,
 '2022-02-14 02:22:48.024544': -24.912842566867127,
 '2022-03-02 02:21:15.705971300': -14.032572436102573,
 '2022-03-18 02:19:39.122181400': -16.29473627973535,
 '2022-04-03 02:18:08.108946900': -24.707996146679605,
 '2022-05-06 02:14:34.120211800': -7.091174129686454,
 '2022-05-23 02:13:56.627944200': 0.07431590770245702,
 '2022-06-26 02:12:02.666371100': 9.858809735209721,
 '2022-07-13 02:10:52.278399200': 13.0459066637244,
 '2022-07-30 02:09:32.276410500': 12.119664091794105,
 '2022-08-16 02:08:05.080443200': None,
 '2022-09-02 02:06:30.429924800': 12.670570658334427,
 '2022-09-19 02:04:47.573420700': 9.205302032079656,
 '2022-10-06 02:02:54.009578300': 2.1826898366228793,
 '2022-10-23 02:00:46.903254200': 3.8120041199609322,
 '2022-02-21 02:27:06.650813500': -26.460964483472253,
 '2022-03-25 02:23:59.344283100': -19.91475393095036,
 '2022-05-11 02:19:01.395970300': -7.689567391520564,
 '2022-05-28 02:18:19.237186400': 0.520661422660157,
 '2022-06-14 02:17:24.224688100': 6.167764816179718,
 '2022-07-01 02:16:19.550745400': 14.342986170751884,
 '2022-07-18 02:15:06.680904300': 15.149154527717396,
 '2022-08-04 02:13:44.191168': 13.399058547898141,
 '2022-09-07 02:10:38.171759100': None,
 '2022-02-21 02:27:30.525533700': -26.27573791038839,
 '2022-03-25 02:24:23.221667': -19.78409342228291,
 '2022-05-11 02:19:25.231065900': -2.1214823699665555,
 '2022-05-28 02:18:43.070594900': 1.4045944683589147,
 '2022-06-14 02:17:48.057413900': 2.400496194138182,
 '2022-07-01 02:16:43.384405400': 9.640798722915722,
 '2022-07-18 02:15:30.515871500': 12.601714224288138,
 '2022-08-04 02:14:08.026778800': 12.245749556512104,
 '2022-08-21 02:12:38.604307': 10.81999172339354,
 '2022-09-24 02:09:16.038719800': 8.148994366845976,
 '2022-10-28 02:05:09.204380600': 3.667279191433786,
 '2022-02-21 02:27:54.401066': -22.152512527405623,
 '2022-03-09 02:26:21.162589500': -1.8875335111501754,
 '2022-03-25 02:24:47.099698400': -9.163008555845673,
 '2022-05-11 02:19:49.066866300': -0.9773223184358149,
 '2022-05-28 02:19:06.904758300': 2.5927660850854433,
 '2022-06-14 02:18:11.890949900': 2.639011409131839,
 '2022-07-01 02:17:07.218572900': 11.085956037123056,
 '2022-07-18 02:15:54.351276': 12.114824088130824,
 '2022-08-04 02:14:31.863481': 12.752032683947865,
 '2022-08-21 02:13:02.440342': 11.35578882553327,
 '2022-09-07 02:11:25.840328300': 2.6962133505925028,
 '2022-09-24 02:09:39.871691200': 8.578010512631664,
 '2022-10-11 02:07:41.992129200': 4.617319876541472,
 '2022-10-28 02:05:33.038376500': 4.679126854527554,
 '2022-02-21 02:28:18.277030200': -18.510996607769936,
 '2022-03-09 02:26:45.039452400': -1.952509738701557,
 '2022-03-25 02:25:10.978280300': -7.831436580228111,
 '2022-05-11 02:20:12.903139800': 0.7708986583240098,
 '2022-05-28 02:19:30.739393100': 4.45614542023,
 '2022-06-14 02:18:35.724922200': 4.527031247910136,
 '2022-07-01 02:17:31.053511500': 12.895537903319351,
 '2022-07-18 02:16:18.187678400': 9.200627460898218,
 '2022-08-04 02:14:55.700454500': 12.473505405698535,
 '2022-08-21 02:13:26.276921500': None,
 '2022-09-07 02:11:49.675572': 6.174082314783745,
 '2022-09-24 02:10:03.705745200': 5.988910207360995,
 '2022-10-11 02:08:05.825759800': 2.4761781278607775,
 '2022-10-28 02:05:56.872902500': 4.087517063759459,
 '2022-02-28 02:33:01.177390800': None,
 '2022-03-16 02:31:25.239231400': -17.440877133789662,
 '2022-04-01 02:29:53.840377600': -3.968627618152863,
 '2022-05-16 02:25:03.048703900': None,
 '2022-02-28 02:33:25.053167700': -10.908074149812576,
 '2022-03-16 02:31:49.116039': -15.759876124557453,
 '2022-04-01 02:30:17.718708100': -6.428069283501738,
 '2022-05-16 02:25:26.883806400': None,
 '2022-06-19 02:23:42.297496300': -1.5353874830318435,
 '2022-07-06 02:22:35.058917200': 9.235053539314238,
 '2022-08-09 02:19:54.819905800': 14.162182386149723,
 '2022-08-26 02:18:23.030063300': 9.245561941971037,
 '2022-09-12 02:16:44.172024600': None,
 '2022-09-29 02:14:55.125175400': 7.340821556956067,
 '2022-10-16 02:12:52.056094': 5.93831393779628,
 '2022-02-12 02:35:22.296004200': -21.91604541837759,
 '2022-02-28 02:33:48.929551900': -12.008164347786538,
 '2022-03-16 02:32:12.993589800': -17.229198713538604,
 '2022-04-01 02:30:41.597721300': -9.464198775305505,
 '2022-05-16 02:25:50.719633400': 0.7436937806437599,
 '2022-06-02 02:25:03.754003100': None,
 '2022-06-19 02:24:06.131636': 3.3249974465497547,
 '2022-07-06 02:22:58.894165700': 11.463338438469819,
 '2022-08-09 02:20:18.656973700': 13.965614037732319,
 '2022-09-12 02:17:08.006412700': 5.6659194585698005,
 '2022-09-29 02:15:18.958823200': 6.828553713499919,
 '2022-10-16 02:13:15.889876400': 4.934093074366348,
 '2022-11-02 02:11:05.630149': 2.5547179789113375,
 '2022-02-19 02:40:51.171952100': -20.513644862518593,
 '2022-03-07 02:39:18.609416300': -12.636337949394056,
 '2022-03-23 02:37:43.800008800': -19.985445076139655,
 '2022-05-21 02:31:27.051262200': 2.27769236775588,
 '2022-06-24 02:29:35.569155900': 3.169426037318818,
 '2022-07-11 02:28:26.091784500': 7.309128815817029,
 '2022-07-28 02:27:07.235563500': 9.700966524373195,
 '2022-08-14 02:25:40.860498400': 8.976305085323594,
 '2022-08-31 02:24:06.989559800': 7.971355126652362,
 '2022-09-17 02:22:25.315248400': -1.9165476075082248,
 '2022-10-04 02:20:33.158142600': 2.114249074715227,
 '2022-10-21 02:18:26.998295700': 4.010686549950119,
 '2022-05-09 02:37:41.994127900': -2.57162572100713,
 '2022-05-26 02:37:01.723595100': 6.490771726094918,
'2023-02-09 03:38:55.699333': -18.053085428464662,
 '2023-02-25 03:38:38.248751': -23.68293646704412,
 '2023-03-29 03:38:23.682526900': None,
 '2023-04-14 03:38:08.465232': -3.419263936605489,
 '2023-04-30 03:38:04.164276': -2.733640106863044,
 '2023-05-16 03:37:51.043571900': -7.279272741030206,
 '2023-06-01 03:37:53.362560': 2.4638682057448964,
 '2023-06-17 03:38:02.931816900': 2.2557302390125002,
 '2023-07-03 03:38:12.907015': 6.42357654640637,
 '2023-07-19 03:38:16.942645': 11.561857901611745,
 '2023-08-04 03:38:26.332271900': None,
 '2023-09-05 03:38:33.167884900': 13.936217216785131,
 '2023-09-21 03:38:39.124751': 11.73769239227733,
 '2023-10-07 03:38:42.593923': 7.568588317658089,
 '2023-10-23 03:38:45.483417': 6.7813327115123485,
 '2023-11-08 03:38:43.926064': None,
 '2023-01-24 03:39:20.241473': -33.75410884802514,
 '2023-02-09 03:39:19.581901': -19.873253246397944,
 '2023-02-25 03:39:02.127082900': -17.838724472585515,
 '2023-03-13 03:38:57.473483': None,
 '2023-03-29 03:38:47.560859': -9.576898469313743,
 '2023-04-14 03:38:32.335091900': -3.289757925540632,
 '2023-04-30 03:38:28.034135900': -2.3029333774910294,
 '2023-05-16 03:38:14.921904': -4.5320281599219365,
 '2023-06-01 03:38:17.245128': 2.463758333169399,
 '2023-06-17 03:38:26.818622': -0.5214147383941081,
 '2023-07-03 03:38:36.793819': -2.671018564369512,
 '2023-07-19 03:38:40.820977': 9.268045147811865,
 '2023-08-04 03:38:50.210604': 13.359196274873886,
 '2023-08-20 03:38:57.033172': None,
 '2023-09-05 03:38:57.046217': 12.008532647020989,
 '2023-09-21 03:39:03.003083': 12.651548186787023,
 '2023-10-07 03:39:06.476491': 9.37658095550921,
 '2023-10-23 03:39:09.370221': 6.407569116062506,
 '2023-11-08 03:39:07.808632': -1.3636765257483892,
 '2023-11-24 03:39:13.956270': -9.62661912094328,
 '2023-01-24 03:39:44.119805': None,
 '2023-02-09 03:39:43.460233': -18.29573758989605,
 '2023-02-25 03:39:26.009650900': -13.95051527857919,
 '2023-03-13 03:39:21.351814': None,
 '2023-03-29 03:39:11.430717900': -2.0711395716648653,
 '2023-04-14 03:38:56.213424': -1.4841461965128033,
 '2023-04-30 03:38:51.908232': -0.9461172995449052,
 '2023-05-16 03:38:38.800236': -0.8084961075853199,
 '2023-06-01 03:38:41.131932': 3.7411691617956127,
 '2023-06-17 03:38:50.705425': None,
 '2023-07-03 03:39:00.680622900': None,
 '2023-07-19 03:39:04.707781': 12.726855779216356,
 '2023-08-04 03:39:14.097407900': 12.269423792433331,
 '2023-08-20 03:39:20.919976': 8.011749691506552,
 '2023-09-05 03:39:20.933020900': 12.70887173836174,
 '2023-09-21 03:39:26.889887': 13.512628914543278,
 '2023-10-07 03:39:30.363295': 10.33879472899057,
 '2023-10-23 03:39:33.248553': 6.914154000575287,
 '2023-11-08 03:39:31.686964': None,
 '2023-11-24 03:39:37.834602': -8.999552605283691,
 '2023-01-08 03:40:10.462625': -6.908719635390656,
 '2023-01-24 03:40:08.002373': None,
 '2023-02-09 03:40:07.334329': -18.39662387128564,
 '2023-02-25 03:39:49.887983': -11.865290071790437,
 '2023-03-13 03:39:45.221674': None,
 '2023-03-29 03:39:35.309051': -2.616130451874992,
 '2023-04-14 03:39:20.091756': -1.368423799900695,
 '2023-04-30 03:39:15.786564': -0.2991805387497691,
 '2023-05-16 03:39:02.674332': 2.486682959355363,
 '2023-06-01 03:39:05.018736': 4.025356467776923,
 '2023-07-03 03:39:24.563190900': None,
 '2023-07-19 03:39:28.590349': 15.585451983827793,
 '2023-08-04 03:39:37.979975900': None,
 '2023-08-20 03:39:44.802544': None,
 '2023-09-05 03:39:44.815589': 14.747909593760129,
 '2023-09-21 03:39:50.768219': 14.49191247409703,
 '2023-10-07 03:39:54.241627': 11.64896324117084,
 '2023-10-23 03:39:57.131121': 3.617042807562551,
 '2023-11-08 03:39:55.569532': None,
 '2023-11-24 03:40:01.721406': 0.16883096634799186,
 '2023-12-10 03:39:54.453615': -13.709008079410136,
 '2023-02-16 03:45:00.915728': -20.922266325125737,
 '2023-03-20 03:44:39.808262': -8.805556179745599,
 '2023-04-05 03:44:27.986426900': -15.21207267835015,
 '2023-04-21 03:44:16.636884900': -5.91027994727421,
 '2023-05-07 03:44:09.531544': -1.0392566457108787,
 '2023-05-23 03:44:03.624216': -3.7431205042249998,
 '2023-06-08 03:44:09.748144': 3.954837034560447,
 '2023-06-24 03:44:15.424616900': -0.2492685428839733,
 '2023-07-10 03:44:27.242282': 9.548189596716718,
 '2023-07-26 03:44:32.683656': 14.55733230113959,
 '2023-08-27 03:44:45.816968': 15.271718499023525,
 '2023-09-12 03:44:47.703092': None,
 '2023-09-28 03:44:52.858894900': 9.827804082661945,
 '2023-10-14 03:44:55.874850': 7.883289556125665,
 '2023-10-30 03:44:58.896296': 1.2899145995458972,
 '2023-11-15 03:44:57.385043': 0.9371882751633775,
 '2023-02-16 03:45:24.794060': -18.524679170533577,
 '2023-03-20 03:45:03.682358': -6.166464414169668,
 '2023-04-21 03:44:40.506745': -6.082472543141194,
 '2023-05-07 03:44:33.405640': -0.6079435489516516,
 '2023-05-23 03:44:27.502548': -2.652324000605466,
 '2023-06-08 03:44:33.630712': 4.0285566057790945,
 '2023-06-24 03:44:39.307184900': 0.7491284358150674,
 '2023-07-10 03:44:51.124850': 6.174909060538942,
 '2023-07-26 03:44:56.566224': 7.423856453516192,
 '2023-08-27 03:45:09.703771900': 15.510186483849989,
 '2023-09-12 03:45:11.585660': 11.767949272007227,
 '2023-09-28 03:45:16.741462900': 10.523145978979343,
 '2023-10-14 03:45:19.753181900': 7.386493099728081,
 '2023-10-30 03:45:22.783100': 2.5531063399135605,
 '2023-11-15 03:45:21.263375': 0.9171650503584616,
 '2023-01-15 03:45:52.596777': -19.957610397921258,
 '2023-02-16 03:45:48.676628': -15.336753001173385,
 '2023-03-20 03:45:27.556453900': -3.3971862378737527,
 '2023-04-05 03:45:15.743091': -4.382995558287087,
 '2023-04-21 03:45:04.380841': -5.80241150177571,
 '2023-05-07 03:44:57.283972': 0.16900498240556597,
 '2023-05-23 03:44:51.393587900': 2.455309364008339,
 '2023-06-08 03:44:57.513280': 5.445362319684321,
 '2023-06-24 03:45:03.193989': 6.135458389003891,
 '2023-07-10 03:45:15.015890': None,
 '2023-07-26 03:45:20.453028': None,
 '2023-08-11 03:45:24.754975': 14.327991008680456,
 '2023-08-27 03:45:33.582104': 15.751296738059287,
 '2023-09-12 03:45:35.468228': 13.148151596426084,
 '2023-09-28 03:45:40.624030900': 8.908095943883401,
 '2023-10-14 03:45:43.635749900': 7.854745449043296,
 '2023-10-30 03:45:46.661432': 1.1182092328146598,
 '2023-11-15 03:45:45.150179': 1.2393090912083473,
 '2023-12-01 03:45:45.515829': -2.8997338884841386,
 '2023-01-15 03:46:16.483580900': -23.46501408522509,
 '2023-01-31 03:46:20.488397': -13.252876719714846,
 '2023-02-16 03:46:12.550724': -16.245184522715814,
 '2023-03-04 03:45:59.827769': None,
 '2023-03-20 03:45:51.434786': -5.668464270424245,
 '2023-04-05 03:45:39.617187': None,
 '2023-04-21 03:45:28.259173': -9.149030746522936,
 '2023-05-07 03:45:21.158067900': 1.3045541931245783,
 '2023-05-23 03:45:15.271920': 2.8153365586978656,
 '2023-06-08 03:45:21.404320': 6.266260174680598,
 '2023-06-24 03:45:27.076557': 7.751297313792834,
 '2023-07-10 03:45:38.898458': None,
 '2023-07-26 03:45:44.335596': None,
 '2023-08-11 03:45:48.641778': 15.2710775026988,
 '2023-08-27 03:45:57.468908': 16.500434289479355,
 '2023-09-12 03:45:59.355032': 13.589690021785422,
 '2023-09-28 03:46:04.506598900': None,
 '2023-10-14 03:46:07.518317900': 7.474152380006995,
 '2023-10-30 03:46:10.548236': -3.4683959351037155,
 '2023-11-15 03:46:09.028510900': 0.9411552256020848,
 '2023-12-01 03:46:09.402633': -3.8634725801858134,
 '2023-01-22 03:51:40.695612': None,
 '2023-02-07 03:51:42.315125900': -11.170116101452402,
 '2023-02-23 03:51:26.533358900': -13.057296627979532,
 '2023-03-11 03:51:19.401752900': -11.763226736959977,
 '2023-03-27 03:51:10.060581': -3.8830632572860413,
 '2023-04-12 03:50:54.296623': -8.117775627225978,
 '2023-04-28 03:50:50.631140': -1.8204995665996848,
 '2023-05-14 03:50:37.633178': -0.9273685975149147,
 '2023-05-30 03:50:39.122035': 0.956590395233223,
 '2023-06-15 03:50:47.991938': 2.1421210413910132,
 '2023-07-01 03:50:56.967660': None,
 '2023-08-02 03:51:11.337659': None,
 '2023-09-03 03:51:19.549607': 14.36397874749419,
 '2023-09-19 03:51:24.803226': 12.965406911746276,
 '2023-10-05 03:51:27.226341': 10.261499215344596,
 '2023-10-21 03:51:29.594532': 5.286754973035284,
 '2023-01-22 03:52:04.569708': -31.587294873598896,
 '2023-02-07 03:52:06.193458': -12.76239708477481,
 '2023-02-23 03:51:50.411690': -12.262000183967658,
 '2023-03-11 03:51:43.280085': -11.753710944459943,
 '2023-03-27 03:51:33.938913': -5.638648172560899,
 '2023-04-12 03:51:18.166482900': -9.231286388256905,
 '2023-04-28 03:51:14.505236': -1.2879464376343872,
 '2023-05-14 03:51:01.511510': -2.265805745465629,
 '2023-05-30 03:51:03.004603': -0.8754616800373592,
 '2023-06-15 03:51:11.878742': 4.013346870645975,
 '2023-07-01 03:51:20.854463900': None,
 '2023-08-18 03:51:41.470762': None,
 '2023-09-03 03:51:43.432175': 14.328234702671192,
 '2023-09-19 03:51:48.690030': 13.532953957235327,
 '2023-10-05 03:51:51.113145': 10.040400604750037,
 '2023-10-21 03:51:53.477100': 6.512422761061256,
 '2023-11-22 03:51:59.240764': -6.15560140627222,
 '2023-01-06 03:52:32.793723': -12.827422582087726,
 '2023-01-22 03:52:28.456512': -30.62555360558923,
 '2023-02-07 03:52:30.076026': -13.675185323627584,
 '2023-02-23 03:52:14.290022': -12.934533536805,
 '2023-03-11 03:52:07.158417': -11.836859048510773,
 '2023-03-27 03:51:57.813009': -4.675257560268673,
 '2023-04-12 03:51:42.049050900': -6.8968392277729045,
 '2023-04-28 03:51:38.383567900': -6.188526657308552,
 '2023-05-14 03:51:25.389842': -13.657764531490892,
 '2023-05-30 03:51:26.891407': None,
 '2023-06-15 03:51:35.761310': 4.114364153917958,
 '2023-07-01 03:51:44.737031900': None,
 '2023-07-17 03:51:50.573071': 13.388116770997192,
 '2023-08-18 03:52:05.353330': 14.062517613435901,
 '2023-09-03 03:52:07.314743': 12.048164180467978,
 '2023-09-19 03:52:12.568362': 13.288439860130651,
 '2023-10-05 03:52:14.991477': 6.936091586049644,
 '2023-10-21 03:52:17.359668': 7.674678074671215,
 '2023-11-22 03:52:23.123332': -3.0797339727069435,
 '2023-01-29 03:58:18.072607': -13.129417104445714,
 '2023-02-14 03:58:11.909652': -11.46337680368785,
 '2023-03-02 03:57:58.398775': -6.798916490148422,
 '2023-03-18 03:57:50.983961900': -6.174692044691291,
 '2023-04-03 03:57:39.742131': -5.738319827437723,
 '2023-04-19 03:57:26.083986': -3.668644347993297,
 '2023-05-05 03:57:20.812755': -3.765381767738206,
 '2023-05-21 03:57:12.270889': 3.2766619546457703,
 '2023-06-06 03:57:17.593819': 5.710725599828257,
 '2023-06-22 03:57:23.096622900': 1.970354666941772,
 '2023-07-08 03:57:35.922526': 4.7153366389663605,
 '2023-07-24 03:57:40.368621900': None,
 '2023-08-25 03:57:54.984383900': 13.580860803389749,
 '2023-09-10 03:57:55.675791': 9.855708396240468,
 '2023-09-26 03:58:01.475646': None,
 '2023-10-12 03:58:05.194559900': 10.194629531873094,
 '2023-11-13 03:58:05.315467': 3.6393408505164824,
 '2023-11-29 03:58:08.671684': -9.131735822207338,
 '2023-01-13 03:58:39.932714900': -23.098648898170776,
 '2023-01-29 03:58:41.950938900': -15.569384202245931,
 '2023-02-14 03:58:35.787984': -14.112657562185435,
 '2023-03-02 03:58:22.285578': -7.342414670312558,
 '2023-03-18 03:58:14.862294': -6.069134067213348,
 '2023-04-03 03:58:03.620463': -6.07261196528481,
 '2023-04-19 03:57:49.962317900': -3.174566440927872,
 '2023-05-05 03:57:44.691087': -0.9527752150154719,
 '2023-05-21 03:57:36.157693': 1.8207802521877878,
 '2023-06-06 03:57:41.480623': 4.510514487484046,
 '2023-06-22 03:57:46.983426900': 2.2000951217967843,
 '2023-07-08 03:57:59.800858': 11.402781828243299,
 '2023-07-24 03:58:04.255425900': 13.25084484630404,
 '2023-08-25 03:58:18.871187900': 16.190900460234303,
 '2023-09-10 03:58:19.562595': None,
 '2023-10-12 03:58:29.072892': 10.540320082771675,
 '2023-10-28 03:58:31.895111': None,
 '2023-11-13 03:58:29.198035': 2.5664240012151427,
 '2023-11-29 03:58:32.554252': -10.355948526976032,
 '2023-01-20 04:04:48.474384': None,
 '2023-02-05 04:04:52.377084': -18.080896676226498,
 '2023-02-21 04:04:38.912277900': -14.085221474785877,
 '2023-03-25 04:04:19.948643': -5.566426060467007,
 '2023-04-10 04:04:04.633324': -14.397284092436946,
 '2023-05-12 04:03:47.821664900': 0.10092492097896652,
 '2023-05-28 04:03:48.552277': 2.461009164912572,
 '2023-06-13 04:03:56.676303': None,
 '2023-06-29 04:04:04.716073': 11.05098439308308,
 '2023-07-15 04:04:12.272007': 15.332769119025153,
 '2023-07-31 04:04:19.929016900': None,
 '2023-09-01 04:04:29.525867': None,
 '2023-09-17 04:04:33.906903': 13.256558172798378,
 '2023-10-03 04:04:37.257877': 0.017952648016555463,
 '2023-10-19 04:04:38.261868': 6.71320329260072,
 '2023-11-04 04:04:41.991272900': -10.203276103284423,
 '2023-11-20 04:04:44.097455': None,
 '2023-12-06 04:04:35.762349': None,
 '2023-02-01 03:39:03.137990': -20.512627203584,
 '2023-02-17 03:39:00.638950': -19.834252176175294,
 '2023-03-05 03:38:48.261439': -14.541230606782833,
 '2023-03-21 03:38:41.831363': -13.286687021975155,
 '2023-04-06 03:38:31.813144': -7.5756310344820985,
 '2023-04-22 03:38:25.658347': -6.475488830763987,
 '2023-05-08 03:38:14.255452': -4.3150931512832935,
 '2023-05-24 03:38:03.852121900': None,
 '2023-06-09 03:37:59.713905': 3.8948641933208354,
 '2023-06-25 03:37:53.791594': 4.740960393788648,
 '2023-07-27 03:38:12.136894': 8.268637605852453,
 '2023-08-12 03:38:21.462135': 15.609958191414474,
 '2023-08-28 03:38:29.854715': 16.12699164288429,
 '2023-09-13 03:38:39.246624': 10.913980625165502,
 '2023-09-29 03:38:35.820479': 10.886632401995808,
 '2023-10-15 03:38:48.712214900': -2.7747975942840997,
 '2023-10-31 03:38:44.672487': -4.244030386370571,
 '2023-11-16 03:38:45.339610': -2.0305770322378653,
 '2023-02-01 03:39:27.007826': -17.170674527571173,
 '2023-02-17 03:39:24.513021900': -17.23820395877543,
 '2023-03-05 03:39:12.139746': -13.509691883521723,
 '2023-03-21 03:39:05.709669900': -19.796353666296056,
 '2023-04-06 03:38:55.691451': -7.04186543544545,
 '2023-04-22 03:38:49.536654': -6.414018006319614,
 '2023-05-08 03:38:38.129523': -0.0747197478865129,
 '2023-05-24 03:38:27.726193': None,
 '2023-06-09 03:38:23.583740900': 3.531619296123707,
 '2023-06-25 03:38:17.674137900': 4.98514739009334,
 '2023-07-11 03:38:33.937452': 9.301996107788979,
 '2023-08-12 03:38:45.344678': 15.860121227784377,
 '2023-08-28 03:38:53.737258': 14.595232354940556,
 '2023-09-13 03:39:03.124931': 11.688402321186247,
 '2023-09-29 03:38:59.707258': 10.275975547989827,
 '2023-10-15 03:39:12.594758': 5.824794261650376,
 '2023-10-31 03:39:08.546559': 0.00212217866582188,
 '2023-11-16 03:39:09.222152900': -3.2458120433501483,
 '2023-02-01 03:39:50.890370': -16.861865886118327,
 '2023-02-17 03:39:48.391328900': -15.00675487537363,
 '2023-03-05 03:39:36.022289': -13.42578285813555,
 '2023-03-21 03:39:29.587976900': None,
 '2023-04-06 03:39:19.565523': -4.754338765397968,
 '2023-04-22 03:39:13.406489': -2.0261381945279475,
 '2023-05-08 03:39:02.007830': 0.4610330771236112,
 '2023-06-09 03:38:47.462047900': 4.404981829147363,
 '2023-06-25 03:38:41.560917': 5.292748758185546,
 '2023-07-11 03:38:57.815758900': 11.60302326595117,
 '2023-08-12 03:39:09.218748900': 16.29233064708058,
 '2023-08-28 03:39:17.615566': 12.258344209307626,
 '2023-09-13 03:39:27.007473900': 11.923281909882462,
 '2023-09-29 03:39:23.585565': 11.041315021962118,
 '2023-10-15 03:39:36.477301': 8.174679422627158,
 '2023-10-31 03:39:32.429101900': 4.142761891890361,
 '2023-11-16 03:39:33.104697': -0.4882470681161492,
 '2023-12-02 03:39:40.436162': -6.419714283174714,
 '2023-01-16 03:40:08.723537': None,
 '2023-02-01 03:40:14.772913': -14.937898062892986,
 '2023-02-17 03:40:12.269636': -14.675369971508067,
 '2023-03-05 03:39:59.900597900': None,
 '2023-04-06 03:39:43.443830': -4.081905565362609,
 '2023-04-22 03:39:37.284796': -2.7285464876903878,
 '2023-05-08 03:39:25.877666': 0.8790182466126044,
 '2023-05-24 03:39:15.478572': None,
 '2023-06-09 03:39:11.336118900': 5.980800828737084,
 '2023-06-25 03:39:05.447696': 9.588278695752898,
 '2023-07-11 03:39:21.702538': 15.706181158139588,
 '2023-07-27 03:39:23.780288900': 14.354010146412136,
 '2023-08-12 03:39:33.105529': 15.204606262554499,
 '2023-08-28 03:39:41.502345': 8.920064977651109,
 '2023-09-13 03:39:50.894254': None,
 '2023-09-29 03:39:47.472344': 12.074069414018883,
 '2023-10-15 03:40:00.359844': 8.249130138482519,
 '2023-10-31 03:39:56.311645': 6.032931050366408,
 '2023-11-16 03:39:56.978768900': -0.10386413292246652,
 '2023-12-02 03:40:04.322940900': -1.7717337677378566,
 '2023-02-08 03:45:16.594479': -20.553240517762077,
 '2023-02-24 03:45:01.747588': -17.01372181358459,
 '2023-03-12 03:44:54.605778': -17.525693036839847,
 '2023-04-13 03:44:37.958338': -3.534136150635795,
 '2023-05-15 03:44:22.777125': -0.6523887957071893,
 '2023-05-31 03:44:14.402972900': 0.37760038863915774,
 '2023-06-16 03:44:04.676986': 4.058958810593818,
 '2023-07-02 03:44:15.021489': 6.189668404017104,
 '2023-07-18 03:44:22.173787': 5.8095377776700845,
 '2023-08-03 03:44:29.355881900': 10.733499304372097,
 '2023-09-04 03:44:44.040156': 8.14160320261831,
 '2023-09-20 03:44:50.280512': 11.087744901729065,
 '2023-10-22 03:45:00.636007': 3.2666644438211763,
 '2023-11-07 03:44:55.863531900': -8.375637862955848,
 '2023-02-08 03:45:40.472787': -18.432031046771737,
 '2023-02-24 03:45:25.625896': -16.397177163201096,
 '2023-03-12 03:45:18.479849': -17.443365170656968,
 '2023-03-28 03:45:08.682018': -6.248534246144679,
 '2023-04-13 03:45:01.836646': -3.789568153810651,
 '2023-05-15 03:44:46.655432': -0.31391272174434237,
 '2023-05-31 03:44:38.277043900': 0.6212318299085646,
 '2023-06-16 03:44:28.559528900': 4.1167235766896315,
 '2023-07-02 03:44:38.908268900': 2.7549228785081117,
 '2023-07-18 03:44:46.052094': 6.861315012017084,
 '2023-08-03 03:44:53.238425': 2.645171743019943,
 '2023-09-04 03:45:07.922699900': 13.830195491131153,
 '2023-09-20 03:45:14.163056': 12.135972676294863,
 '2023-10-06 03:45:16.701239': 8.766041841379142,
 '2023-10-22 03:45:24.510078': 5.21862846109262,
 '2023-02-08 03:46:04.351094': -14.749597653829323,
 '2023-02-24 03:45:49.499967': -12.951719279355615,
 '2023-03-12 03:45:42.358156': -14.996304295745913,
 '2023-03-28 03:45:32.564561': -4.505715250077134,
 '2023-04-13 03:45:25.710717': -2.438851619838986,
 '2023-05-15 03:45:10.529503': 1.2399110398644826,
 '2023-05-31 03:45:02.155351': -0.9954186518025446,
 '2023-06-16 03:44:52.433601': 4.998304201856099,
 '2023-07-02 03:45:02.790812': 5.451747752580712,
 '2023-07-18 03:45:09.934639': 11.698904675882199,
 '2023-08-03 03:45:17.116732': None,
 '2023-08-19 03:45:20.672238900': 10.530435322587724,
 '2023-09-04 03:45:31.809479': 14.318113615511733,
 '2023-09-20 03:45:38.045599': 13.069000865746212,
 '2023-10-06 03:45:40.588017900': 9.694140562108874,
 '2023-10-22 03:45:48.392622': 7.06635178508406,
 '2023-11-07 03:45:43.632854900': None,
 '2023-02-08 03:46:28.233637': -13.625806218556862,
 '2023-02-24 03:46:13.378274': -12.31382671362081,
 '2023-03-12 03:46:06.236463': -18.637300142443998,
 '2023-03-28 03:45:56.434396': -2.11202308714423,
 '2023-04-13 03:45:49.584788': -2.6343770812948275,
 '2023-04-29 03:45:42.887876': None,
 '2023-05-15 03:45:34.407810': -3.3618783934246914,
 '2023-05-31 03:45:26.029421900': None,
 '2023-06-16 03:45:16.324616': 4.160483037342655,
 '2023-07-02 03:45:26.677592': 6.419225808828309,
 '2023-07-18 03:45:33.817181900': None,
 '2023-08-19 03:45:44.554782': 13.272408434263681,
 '2023-09-04 03:45:55.692022': 13.824215484550962,
 '2023-09-20 03:46:01.923907': 12.613593388901572,
 '2023-10-06 03:46:04.470562': 9.26743183422044,
 '2023-10-22 03:46:12.275165': 7.073192174127375,
 '2023-11-07 03:46:07.515398': None,
 '2023-12-09 03:46:11.702111': -13.530354103405328,
 '2023-03-03 03:51:35.236105': -4.335255653011388,
 '2023-03-19 03:51:28.475600': -1.892705024491943,
 '2023-04-04 03:51:17.894035900': None,
 '2023-04-20 03:51:11.696846': -7.346241434510141,
 '2023-05-06 03:50:59.620386': None,
 '2023-05-22 03:50:51.669693': 1.7523400150911808,
 '2023-06-07 03:50:46.540300': 4.116643009202385,
 '2023-06-23 03:50:39.851225': None,
 '2023-07-09 03:50:54.657136': 10.44668333913281,
 '2023-07-25 03:50:55.209893900': -4.540898644099968,
 '2023-08-26 03:51:14.075879': 13.847272171673094,
 '2023-09-11 03:51:23.891436900': 12.125993327919742,
 '2023-09-27 03:51:20.456804': 12.178906224173371,
 '2023-10-13 03:51:33.009465': None,
 '2023-10-29 03:51:32.007050': -3.4941110213312183,
 '2023-11-14 03:51:30.861039': None,
 '2023-02-15 03:52:12.139972': -18.234841302073086,
 '2023-03-03 03:51:59.110177': -4.745220871124056,
 '2023-03-19 03:51:52.349671': -3.5099322706607574,
 '2023-04-04 03:51:41.772343': -3.5300734452005753,
 '2023-04-20 03:51:35.579389900': -6.655489710612953,
 '2023-05-06 03:51:23.498693': -0.3030232818419959,
 '2023-05-22 03:51:15.543764900': 1.2775347081809967,
 '2023-06-07 03:51:10.410135': 5.103874627403658,
 '2023-06-23 03:51:03.733768': None,
 '2023-07-09 03:51:18.543916': 11.649388766770988,
 '2023-07-25 03:51:19.092437': 15.839084123374183,
 '2023-08-10 03:51:30.679798': None,
 '2023-08-26 03:51:37.962660': 12.661591917887911,
 '2023-09-11 03:51:47.773980': 13.049563880109316,
 '2023-09-27 03:51:44.335111': 11.66264341561217,
 '2023-10-13 03:51:56.896246': 5.032955208316823,
 '2023-10-29 03:51:55.885358900': 3.969565515942947,
 '2023-11-14 03:51:54.739346': None,
 '2023-11-30 03:52:02.867074': None,
 '2023-01-14 03:52:32.163083': -27.02511461216996,
 '2023-01-30 03:52:35.069268': -15.194194435123917,
 '2023-02-15 03:52:36.014043900': None,
 '2023-03-03 03:52:22.988485': -4.765000138515096,
 '2023-03-19 03:52:16.232214': -4.117174149489162,
 '2023-04-04 03:52:05.646414': -2.806113352777776,
 '2023-04-20 03:51:59.453460900': -5.774018501078674,
 '2023-05-06 03:51:47.372764': 0.05294327900970242,
 '2023-05-22 03:51:39.417836': 0.549915092293678,
 '2023-06-07 03:51:34.284206': 4.468653899315429,
 '2023-06-23 03:51:27.616311': None,
 '2023-07-09 03:51:42.426459': 9.559822299042345,
 '2023-07-25 03:51:42.979216900': 13.628909351461877,
 '2023-08-26 03:52:01.836731': 15.129901653969418,
 '2023-09-11 03:52:11.656523900': 12.607175248071176,
 '2023-09-27 03:52:08.221890': 10.56052964689242,
 '2023-10-13 03:52:20.778789': 5.939905443669894,
 '2023-10-29 03:52:19.767902': 3.8805236831245655,
 '2023-11-14 03:52:18.621889': 1.8868890804277354,
 '2023-11-30 03:52:26.753853': -8.885046242134026,
 '2023-01-21 03:58:12.604237': -20.45660135568806,
 '2023-02-06 03:58:25.630199': -13.044603353052755,
 '2023-02-22 03:58:13.727345900': -8.445492689747713,
 '2023-03-10 03:58:03.107754': -8.960696375813342,
 '2023-04-11 03:57:49.099319900': -2.5123400614936164,
 '2023-04-27 03:57:42.576061': -1.8261556420313745,
 '2023-05-13 03:57:33.240366': -1.8421834057965492,
 '2023-05-29 03:57:23.963923900': 3.6419981856354746,
 '2023-06-14 03:57:15.479265900': 5.650705942757438,
 '2023-06-30 03:57:22.269520': 3.7427742456762263,
 '2023-07-16 03:57:31.531227900': 6.847214940093566,
 '2023-08-01 03:57:37.285787': None,
 '2023-08-17 03:57:41.027766900': None,
 '2023-09-18 03:58:00.078599900': 12.951409590538105,
 '2023-10-04 03:58:01.057863': 10.57081227505213,
 '2023-10-20 03:58:10.239141': 7.156535806948099,
 '2023-11-05 03:58:04.047565900': 2.441118663971025,
 '2023-11-21 03:58:10.725912': None,
 '2023-01-05 03:58:46.445568': None,
 '2023-01-21 03:58:36.478308': -22.81096036436027,
 '2023-02-06 03:58:49.508505900': -13.383211093036033,
 '2023-02-22 03:58:37.601416900': -9.060228122832434,
 '2023-03-10 03:58:26.986061900': -11.820465956387565,
 '2023-03-26 03:58:20.958290900': -3.685205783406272,
 '2023-04-11 03:58:12.977628': -6.540120327907256,
 '2023-04-27 03:58:06.454368': -4.201441816043878,
 '2023-05-13 03:57:57.118674': -2.012878725949612,
 '2023-05-29 03:57:47.833759': 2.2848834005713416,
 '2023-06-14 03:57:39.357573': 5.253626134079417,
 '2023-06-30 03:57:46.152063': 6.166642061504135,
 '2023-07-16 03:57:55.418007': 13.016621368312153,
 '2023-08-17 03:58:04.906075': 14.61574477524208,
 '2023-09-18 03:58:23.956908900': 11.614472249764768,
 '2023-10-04 03:58:24.944643': 10.84158426591971,
 '2023-10-20 03:58:34.121685': 8.193420433351,
 '2023-11-05 03:58:27.930110900': 3.418908230916846,
 '2023-11-21 03:58:34.608454900': -8.63030020834089,
 '2023-01-12 04:04:55.145139900': None,
 '2023-01-28 04:04:55.153811': -17.047521789048158,
 '2023-02-13 04:04:59.190838900': -12.659046171460894,
 '2023-03-01 04:04:45.555315': None,
 '2023-03-17 04:04:38.638095': None,
 '2023-04-02 04:04:27.463505': -11.950806112923976,
 '2023-04-18 04:04:21.219674900': -1.1693047202850553,
 '2023-05-04 04:04:09.223671': None,
 '2023-06-05 04:03:56.965435': None,
 '2023-06-21 04:03:49.471445': 9.494259553880104,
 '2023-07-07 04:04:02.942809900': None,
 '2023-07-23 04:04:05.084059': 14.042996724427177,
 '2023-08-08 04:04:15.769264900': None,
 '2023-08-24 04:04:21.946632': 15.166907550846233,
 '2023-09-09 04:04:32.126469': 3.468536983312781,
 '2023-10-11 04:04:40.638592': 10.935399897717515,
 '2023-10-27 04:04:42.766692': -4.641838137474494,
 '2023-11-12 04:04:40.599661': 4.018593418015598,
 '2023-11-28 04:04:48.655247': -15.850223096195961,
 '2023-04-21 01:32:54.832983600': -15.096719438893462,
 '2023-05-13 01:30:41.391099600': None,
 '2023-06-04 01:27:51.703848200': None,
 '2023-07-23 01:24:08.955545': 12.71651131636209,
 '2023-08-14 01:19:35.188131400': 16.890969889626067,
 '2023-10-07 01:14:56.435056500': 0.17144106137034493,
 '2023-03-30 01:34:47.153507100': 0.1037623560039217,
 '2023-04-21 01:33:18.663022200': -12.438896298863606,
 '2023-06-04 01:28:15.531107900': 1.6932054583625942,
 '2023-06-26 01:24:54.396861600': 5.955423980641685,
 '2023-07-23 01:24:32.784894700': 10.53552114020668,
 '2023-08-14 01:19:59.016096400': 14.725060470024932,
 '2023-09-10 01:18:07.442508300': 10.803461889270386,
 '2023-10-07 01:15:20.260981300': 8.940977881978844,
 '2023-03-30 01:35:10.985926600': 0.10913270387902088,
 '2023-04-21 01:33:42.493742900': -11.466014939055453,
 '2023-06-04 01:28:39.359013100': 1.2381053019871653,
 '2023-06-26 01:25:18.226124100': None,
 '2023-07-23 01:24:56.615022300': 14.02551398423732,
 '2023-08-14 01:20:22.844542': 14.362795019547491,
 '2023-09-10 01:18:31.268703600': 12.17937624578229,
 '2023-10-07 01:15:44.087459': 10.028183162105783,
 '2023-04-04 01:38:36.291574300': -10.166077222329042,
 '2023-04-26 01:36:57.144078600': -2.6903028214227658,
 '2023-05-18 01:34:34.704692900': -9.02559313716817,
 '2023-06-09 01:31:38.017232500': 1.782600920454868,
 '2023-07-06 01:31:46.179032500': 3.413648327743004,
 '2023-07-28 01:27:38.904923300': 9.285919995234948,
 '2023-08-24 01:26:17.850142800': 11.893444882153393,
 '2023-09-15 01:20:57.015355100': 11.986490502290518,
 '2023-10-12 01:17:56.185348800': 8.332488225393629,
 '2023-04-04 01:39:00.123083300': -11.253844626935146,
 '2023-04-26 01:37:20.973436200': -3.7807575509494726,
 '2023-05-18 01:34:58.532173100': -4.655821468072034,
 '2023-06-09 01:32:01.844752900': 2.1152045134393753,
 '2023-07-01 01:28:32.912825400': 4.7869509721326695,
 '2023-07-06 01:32:10.008313100': -1.160266643703811,
 '2023-08-24 01:26:41.677051300': 11.139675830049931,
 '2023-09-15 01:21:20.840792600': 11.024050922878782,
 '2023-10-12 01:18:20.011678500': 9.669685930004116,
 '2023-02-24 01:44:41.006597700': -17.57232651034189,
 '2023-03-13 01:40:14.143623500': -14.041057528445402,
 '2023-03-18 01:44:36.471296400': -10.435961770517608,
 '2023-04-04 01:39:23.955121800': -11.463508847320576,
 '2023-04-26 01:37:44.803438100': -2.8790371419395013,
 '2023-05-18 01:35:22.360360100': -5.255708062831256,
 '2023-06-09 01:32:25.672975700': 3.5015059332419027,
 '2023-07-01 01:28:56.742342700': 4.749191685313264,
 '2023-07-06 01:32:33.838346700': None,
 '2023-07-28 01:28:26.563969100': 10.487971015506002,
 '2023-08-19 01:23:45.055603300': 8.73319851870111,
 '2023-08-24 01:27:05.504272900': 12.916686127293493,
 '2023-09-15 01:21:44.666840900': 8.130108498927207,
 '2023-10-12 01:18:43.838397100': 10.37911373141287,
 '2023-02-24 01:45:04.839294900': -16.30752524169293,
 '2023-03-18 01:45:00.304710300': -11.28035177988592,
 '2023-04-04 01:39:47.787782600': -11.927926849592662,
 '2023-04-26 01:38:08.633839300': -6.112254387663724,
 '2023-05-18 01:35:46.188980900': None,
 '2023-06-09 01:32:49.501860600': 4.803955938043468,
 '2023-07-01 01:29:20.572538800': 1.9432414901950725,
 '2023-07-28 01:28:50.394273500': 8.046852931590294,
 '2023-08-19 01:24:08.884233800': 12.881045704000146,
 '2023-08-24 01:27:29.332365600': 14.043140269848191,
 '2023-09-15 01:22:08.493825100': -6.776776034568818,
 '2023-10-12 01:19:07.665898900': 10.962610961874509,
 '2023-04-09 01:42:46.858189300': -12.613809162352497,
 '2023-04-14 01:46:55.380856600': -7.678467304776889,
 '2023-05-01 01:40:57.193948900': -3.235051905957232,
 '2023-05-23 01:38:26.358723100': -1.7334590706134507,
 '2023-05-28 01:42:16.334819100': -1.4427336692408281,
 '2023-06-14 01:35:22.693208400': 8.2767671437866,
 '2023-06-19 01:39:05.680022700': 3.4038087096074783,
 '2023-08-02 01:31:07.131840400': 14.474526355314188,
 '2023-08-07 01:34:33.430192900': 15.74740110804841,
 '2023-08-29 01:29:36.743573400': 15.547870375167149,
 '2023-09-20 01:24:08.342330900': 11.353453010238793,
 '2023-09-25 01:27:17.387498600': 9.951642506540356,
 '2023-04-09 01:43:10.689401600': -12.28739568535354,
 '2023-04-14 01:47:19.211829100': -5.564183855214969,
 '2023-05-01 01:41:21.022671300': -4.927666696962283,
 '2023-05-06 01:45:19.306222200': -17.505989425340328,
 '2023-05-23 01:38:50.186046300': 0.551580572237034,
 '2023-05-28 01:42:40.162058900': 0.18715656192131058,
 '2023-06-14 01:35:46.520998800': 6.028549655410059,
 '2023-06-19 01:39:29.508270700': 3.8161290586429177,
 '2023-07-11 01:35:45.735322400': 7.9546011556238225,
 '2023-08-02 01:31:30.960850500': 9.499914368340184,
 '2023-08-07 01:34:57.258893100': None,
 '2023-08-29 01:30:00.569829800': 15.457457619700822,
 '2023-09-20 01:24:32.167765100': 11.667191284116624,
 '2023-09-25 01:27:41.213031700': 10.442047371129531,
 '2023-03-23 01:48:56.382786900': -14.508853836658142,
 '2023-04-09 01:43:34.521218500': -11.02152837146896,
 '2023-04-14 01:47:43.043101100': -4.367366120782712,
 '2023-05-01 01:41:44.852122300': -3.713959163175341,
 '2023-05-06 01:45:43.135203100': -1.8645357369712743,
 '2023-05-23 01:39:14.013817': 2.6869715026604957,
 '2023-05-28 01:43:03.990002800': 0.6870470014964211,
 '2023-06-14 01:36:10.349505600': 3.3912234334341704,
 '2023-06-19 01:39:53.336791300': 4.705938032309795,
 '2023-07-11 01:36:09.565343400': 12.687078761915478,
 '2023-08-07 01:35:21.087857400': 13.995890371346865,
 '2023-08-29 01:30:24.396923700': 16.085632872810045,
 '2023-09-20 01:24:55.993687600': 12.57491975264749,
 '2023-10-17 01:21:40.530139800': 4.851468230092118,
 '2023-03-01 01:49:38.567514400': None,
 '2023-03-23 01:49:20.216109300': -28.662448285910315,
 '2023-04-09 01:43:58.353512': -11.797703091133322,
 '2023-04-14 01:48:06.875154': -7.071632991125896,
 '2023-05-01 01:42:08.682248800': -4.661986773760305,
 '2023-05-06 01:46:06.964741400': -0.012702727449793099,
 '2023-05-23 01:39:37.842317800': 3.331553682431897,
 '2023-05-28 01:43:27.818433': 1.3036092437473017,
 '2023-06-14 01:36:34.178616800': 6.443709833173018,
 '2023-06-19 01:40:17.166279300': 3.6852695444787726,
 '2023-07-11 01:36:33.396125600': 10.465544430830244,
 '2023-08-07 01:35:44.917684300': 15.203343331857525,
 '2023-08-29 01:30:48.224382600': 15.981879008506892,
 '2023-09-20 01:25:19.820368': 12.700417756172595,
 '2023-03-06 01:52:57.925046400': -19.88888109794715,
 '2023-03-06 01:53:21.756852800': -16.819072343974014,
 '2023-03-28 01:52:49.398226400': -5.25684908537394,
 '2023-06-24 01:43:10.653056900': None,
 '2023-07-16 01:39:19.984948900': 11.9107486801238,
 '2023-09-03 01:33:17.926394400': 14.578346166111183,
 '2023-09-30 01:30:47.662550300': 8.327659824627071,
 '2023-03-11 01:58:12.652882800': -13.774284496104142,
 '2023-03-28 01:53:13.230626900': -4.807604020563106,
 '2023-04-19 01:51:49.098878600': -4.768355226467674,
 '2023-05-11 01:49:39.383561600': -15.641687488396727,
 '2023-06-24 01:43:34.482264900': 5.463683848239088,
 '2023-07-16 01:39:43.815096700': 12.292380445002717,
 '2023-07-21 01:43:16.266251800': 11.466696452425031,
 '2023-08-12 01:38:45.614646100': 17.874882917858915,
 '2023-09-03 01:33:41.753055300': 14.070857877244393,
 '2023-09-30 01:31:11.489027100': None,
 '2023-10-05 01:34:15.412047200': 8.71910587476668,
 '2023-03-11 01:58:36.486070800': -16.018494286273434,
 '2023-03-28 01:53:37.063781900': -3.2745046923798338,
 '2023-04-19 01:52:12.930215900': -6.307913645416273,
 '2023-05-11 01:50:03.212491900': None,
 '2023-06-02 01:47:16.397109100': -0.8677002545429682,
 '2023-06-24 01:43:58.312157200': 5.6322682566854345,
 '2023-07-16 01:40:07.645728300': 7.927880155702873,
 '2023-07-21 01:43:40.096791700': 8.657748458680231,
 '2023-08-12 01:39:09.443672600': 16.088540920330015,
 '2023-09-03 01:34:05.580299200': 13.885954532807832,
 '2023-09-08 01:37:21.296531900': 12.881552961625781,
 '2023-09-30 01:31:35.315857200': -0.6976108635152425,
 '2023-10-05 01:34:39.239089100': 5.715583709067673,
 '2023-04-02 01:57:27.345988': -11.055526112978837,
 '2023-06-07 01:50:39.538512300': 4.360029918495757,
 '2023-06-29 01:47:13.683790600': -17.78596171974746,
 '2023-08-17 01:42:08.497550300': -2.964755952749535,
 '2023-02-22 02:02:59.053548600': -12.573814980693422,
 '2023-03-16 02:03:00.008742600': -16.862186520205128,
 '2023-04-02 01:57:51.179009400': -11.379078375036373,
 '2023-04-07 02:02:02.624153200': -5.7973938168457115,
 '2023-04-24 01:56:16.358635500': -2.762561440192654,
 '2023-05-16 01:53:57.266857300': -2.305062525022502,
 '2023-06-07 01:51:03.367179300': 2.028305047040115,
 '2023-06-29 01:47:37.513878900': 5.452014237227318,
 '2023-07-26 01:47:10.783130800': 13.216503961661603,
 '2023-08-17 01:42:32.326230200': 13.903400827781038,
 '2023-08-22 01:45:53.456207100': 13.253258351609574,
 '2023-10-10 01:37:40.205509500': 6.297137298406887,
 '2023-02-27 02:07:33.740437500': -15.114075559115742,
 '2023-04-29 02:00:17.107140300': -11.9282463758655,
 '2023-05-21 01:57:49.636403900': 1.5924270133758476,
 '2023-07-31 01:50:39.694026900': None,
 '2023-09-18 01:43:47.665374500': -1.8134788917693525,
 '2023-10-15 01:40:38.046264400': 10.141722341394594,
'2024-02-12 03:38:40.672768': -15.45819306200891,
 '2024-02-28 03:38:35.081515': -24.33041847241545,
 '2024-03-15 03:38:24.674052': -13.042325379244787,
 '2024-03-31 03:38:11.574590': -10.838406854281457,
 '2024-05-02 03:37:54.855427': None,
 '2024-05-18 03:37:42.281395': -9.205196104687088,
 '2024-06-03 03:37:39.213443': None,
 '2024-07-05 03:37:54.671798': 14.12598655963079,
 '2024-07-21 03:38:10.108827': 9.668481738133913,
 '2024-08-06 03:38:08.315491': 18.669581438136536,
 '2024-08-22 03:38:20.413397': 17.739524764942843,
 '2024-09-07 03:38:29.041204900': 13.384628565407512,
 '2024-10-09 03:38:30.006423': 5.117829469387378,
 '2024-10-25 03:38:38.252754900': -0.7116668984688245,
 '2024-11-10 03:38:36.087661': -6.998043828177535,
 '2024-01-27 03:38:58.850155': -16.28003077160668,
 '2024-02-12 03:39:04.555336': -17.22417222128011,
 '2024-02-28 03:38:58.955611': -22.266494121486193,
 '2024-03-15 03:38:48.556620': -9.98025132235533,
 '2024-03-31 03:38:35.448686': -13.2289218124458,
 '2024-04-16 03:38:30.174179': -7.375535886784434,
 '2024-05-02 03:38:18.729524': None,
 '2024-05-18 03:38:06.155491': -8.325382999865944,
 '2024-06-03 03:38:03.096011': None,
 '2024-06-19 03:38:10.365116': 2.316120299102856,
 '2024-07-05 03:38:18.554366': 7.719943767066718,
 '2024-07-21 03:38:33.999868': 13.023760122345761,
 '2024-08-06 03:38:32.202296': 18.735441402777074,
 '2024-08-22 03:38:44.295966': 18.33669004547086,
 '2024-09-07 03:38:52.923772900': 12.663913088629142,
 '2024-09-23 03:38:53.018214900': 8.000311613647744,
 '2024-10-09 03:38:53.884756': 4.863155789434729,
 '2024-10-25 03:39:02.135322900': 2.0584587014444446,
 '2024-11-10 03:38:59.965994': -0.3376900336358569,
 '2024-02-12 03:39:28.433668': -18.42138475255395,
 '2024-02-28 03:39:22.838179': -19.535074005130934,
 '2024-03-15 03:39:12.430716': -9.763923621394168,
 '2024-03-31 03:38:59.322782': -9.527286397448144,
 '2024-04-16 03:38:54.048275': -4.859206080276395,
 '2024-05-02 03:38:42.612092': None,
 '2024-05-18 03:38:30.038059': 0.6314687689335724,
 '2024-06-03 03:38:26.982815': -2.2456329534665276,
 '2024-06-19 03:38:34.251920900': 3.8784692810953887,
 '2024-07-05 03:38:42.441170': 8.160217024193877,
 '2024-07-21 03:38:57.882436': 10.578088613839132,
 '2024-08-06 03:38:56.084864': 19.785933815717367,
 '2024-08-22 03:39:08.178534': 19.384142600065978,
 '2024-09-07 03:39:16.806340900': 12.282350617647479,
 '2024-09-23 03:39:16.900782900': 9.631542205062606,
 '2024-10-09 03:39:17.775795900': 5.323305967521896,
 '2024-10-25 03:39:26.022127': 1.9415331407630774,
 '2024-11-10 03:39:23.848562': 1.3358246678836045,
 '2024-01-11 03:39:53.884730': -8.383304968350023,
 '2024-01-27 03:39:46.615291': -17.935658099646027,
 '2024-02-28 03:39:46.712275': -17.50592200242872,
 '2024-03-15 03:39:36.304812': -12.878983373406554,
 '2024-03-31 03:39:23.205350': -7.028770781749275,
 '2024-04-16 03:39:17.930843': 0.09658963975580785,
 '2024-05-02 03:39:06.490424': None,
 '2024-05-18 03:38:53.912155': 2.1750733421637563,
 '2024-06-03 03:38:50.873855': None,
 '2024-07-05 03:39:06.332209900': None,
 '2024-07-21 03:39:21.765004': 12.960254829691568,
 '2024-08-06 03:39:19.963196': 22.087688371368,
 '2024-08-22 03:39:32.065338': 19.595509334922,
 '2024-09-07 03:39:40.688908900': 8.075487919523534,
 '2024-09-23 03:39:40.783352': 11.28363701691084,
 '2024-10-09 03:39:41.654128': 6.080448265527483,
 '2024-10-25 03:39:49.900460': 4.98010568524514,
 '2024-11-10 03:39:47.731131': None,
 '2024-12-12 03:39:45.333913': -0.8783083505636314,
 '2024-02-19 03:44:49.846462': -27.458847415232963,
 '2024-03-22 03:44:29.820586': -23.566177190612567,
 '2024-04-07 03:44:22.185416900': -4.196317695512457,
 '2024-05-09 03:44:02.574737': -4.825893444038575,
 '2024-05-25 03:43:49.107347': 1.1192362993643694,
 '2024-06-10 03:43:50.746305': None,
 '2024-06-26 03:44:02.238912': 4.615048631342087,
 '2024-07-12 03:44:09.940721': 14.487435882477484,
 '2024-07-28 03:44:23.146559': 17.643368959525976,
 '2024-09-14 03:44:40.389601': 7.694255199900794,
 '2024-10-16 03:44:49.239020900': None,
 '2024-11-01 03:44:39.690120900': -5.1776609229570125,
 '2024-11-17 03:44:47.594367': -7.593958273633429,
 '2024-02-19 03:45:13.733266': -24.340564526818476,
 '2024-03-22 03:44:53.694682': None,
 '2024-04-07 03:44:46.055277': -3.302859773791314,
 '2024-05-09 03:44:26.448833': -2.0786613034205934,
 '2024-05-25 03:44:12.977206900': 1.5041303480998633,
 '2024-06-26 03:44:26.125716': 0.8397272200940903,
 '2024-07-12 03:44:33.823289': 9.487463121005703,
 '2024-07-28 03:44:47.033363': 16.09932471395763,
 '2024-09-14 03:45:04.267934': 10.79922380032089,
 '2024-09-30 03:44:58.004715': 4.2806158403633265,
 '2024-10-16 03:45:13.121590': 0.9559452107217744,
 '2024-11-01 03:45:03.572688900': -0.25974107792291895,
 '2024-02-03 03:45:38.488683': -19.85321552842988,
 '2024-02-19 03:45:37.607362': -23.227598935535983,
 '2024-03-06 03:45:26.365510900': -12.753264608465063,
 '2024-03-22 03:45:17.577250': -16.85372277350214,
 '2024-04-07 03:45:09.937845900': -1.2802369074022177,
 '2024-04-23 03:45:01.343735900': -7.33246785441577,
 '2024-05-09 03:44:50.327166': -2.6664852973207,
 '2024-05-25 03:44:36.851303900': 3.256213455959035,
 '2024-06-10 03:44:38.515677900': 1.568510146625485,
 '2024-06-26 03:44:50.012520': 5.461322890586188,
 '2024-07-12 03:44:57.710094': 13.318156635484124,
 '2024-07-28 03:45:10.915931': 18.25392871363674,
 '2024-08-29 03:45:18.293381900': 15.288701739789564,
 '2024-09-14 03:45:28.154737900': 9.495986870458859,
 '2024-09-30 03:45:21.887283': 7.112019817181526,
 '2024-10-16 03:45:37.004158': -1.5315157870921574,
 '2024-11-01 03:45:27.451022': -1.0945519717850771,
 '2024-11-17 03:45:35.359504': -3.9348487242788766,
 '2024-01-18 03:46:04.244455900': -15.753165542085613,
 '2024-02-03 03:46:02.371251': -19.60359121465742,
 '2024-02-19 03:46:01.485693900': -24.462637319620114,
 '2024-03-06 03:45:50.235371': -11.304036395768065,
 '2024-04-07 03:45:33.807706': -0.44110410938319156,
 '2024-04-23 03:45:25.226305': -7.583949254575768,
 '2024-05-09 03:45:14.201262': -0.8830659380182888,
 '2024-05-25 03:45:00.729636': 4.925161756672804,
 '2024-06-10 03:45:02.406718': 0.9189616757717962,
 '2024-06-26 03:45:13.899325': 8.539623024997681,
 '2024-07-12 03:45:21.592662': 14.863026349935872,
 '2024-07-28 03:45:34.798499900': 19.661860843080223,
 '2024-08-13 03:45:36.059442': 14.153364334653553,
 '2024-08-29 03:45:42.171713900': 14.484134752351135,
 '2024-09-14 03:45:52.041541900': 11.291583601676855,
 '2024-09-30 03:45:45.769852': 7.703963515418248,
 '2024-10-16 03:46:00.890962': 2.027706371119802,
 '2024-11-01 03:45:51.333590': None,
 '2024-11-17 03:45:59.237836': 0.8365463351210415,
 '2024-12-03 03:46:07.463990900': -1.0953341184030065,
 '2024-01-25 03:51:21.793777900': -16.812588508719077,
 '2024-02-26 03:51:22.034698': None,
 '2024-03-13 03:51:11.156876': None,
 '2024-03-29 03:50:56.490083900': -8.212646148985094,
 '2024-04-14 03:50:53.506786': -16.374604691429628,
 '2024-04-30 03:50:42.405246': None,
 '2024-06-01 03:50:23.658044900': None,
 '2024-06-17 03:50:29.990723': None,
 '2024-07-03 03:50:38.197032': 11.133612427538742,
 '2024-07-19 03:50:53.981182': 15.096706826740903,
 '2024-08-04 03:50:55.534133': 18.0447910822361,
 '2024-09-05 03:51:13.908779': 13.590107613453943,
 '2024-09-21 03:51:15.604087900': 9.957140360710735,
 '2024-10-07 03:51:12.755576': 7.014873948394901,
 '2024-11-24 03:51:30.752439': -22.123008840057565,
 '2024-01-25 03:51:45.680581900': -15.263394584926935,
 '2024-02-26 03:51:45.908793900': -19.26955478958469,
 '2024-03-13 03:51:35.030973': -14.168491311633852,
 '2024-03-29 03:51:20.368417': -7.011767041513515,
 '2024-04-14 03:51:17.380882': -11.006903851127333,
 '2024-04-30 03:51:06.283578': -5.183871685079418,
 '2024-05-16 03:50:51.476965': None,
 '2024-06-01 03:50:47.544849': 4.035477832386059,
 '2024-06-17 03:50:53.877527': 3.6629215804969255,
 '2024-07-03 03:51:02.083837': 14.59294793650165,
 '2024-07-19 03:51:17.867986': 16.281399471448044,
 '2024-08-04 03:51:19.412465900': 18.758405899652985,
 '2024-08-20 03:51:29.125212': 13.95452316931203,
 '2024-09-05 03:51:37.787112': 13.796835242507933,
 '2024-09-21 03:51:39.486657': 11.378397338555121,
 '2024-10-07 03:51:36.638145': 8.40303613784193,
 '2024-10-23 03:51:48.705323': 3.635574163700517,
 '2024-11-24 03:51:54.639244': -8.500036610837169,
 '2024-01-09 03:52:14.714489900': None,
 '2024-01-25 03:52:09.563151': -15.885544175443306,
 '2024-02-10 03:52:14.471886': -10.706039825841199,
 '2024-02-26 03:52:09.791363': -22.518737655412462,
 '2024-03-13 03:51:58.909305': -7.296968255597339,
 '2024-03-29 03:51:44.242512900': -6.362354757455861,
 '2024-04-14 03:51:41.259214': -2.5122554073290493,
 '2024-04-30 03:51:30.161909900': -2.897161202349577,
 '2024-05-16 03:51:15.351061': -0.16344050712509844,
 '2024-06-01 03:51:11.427417': 5.131447644431995,
 '2024-06-17 03:51:17.768567': 6.076391437867507,
 '2024-07-03 03:51:25.962168900': 15.763775453397157,
 '2024-07-19 03:51:41.754789900': 17.17090069961714,
 '2024-08-04 03:51:43.295033900': 17.73930275619708,
 '2024-08-20 03:51:53.007781': 16.048889808958126,
 '2024-09-05 03:52:01.673916': 11.816892970559856,
 '2024-09-21 03:52:03.373461': 11.06804758612312,
 '2024-10-07 03:52:00.520713': 8.311151048830483,
 '2024-10-23 03:52:12.587891': 4.043148864730415,
 '2024-11-24 03:52:18.517576': -2.325707117590952,
 '2024-12-10 03:52:09.088590': -0.8682933698732734,
 '2024-01-16 03:58:02.695534900': -16.14453258316645,
 '2024-02-01 03:57:59.068688': -20.687970136077645,
 '2024-02-17 03:57:59.246237': None,
 '2024-03-04 03:57:50.503641': -11.48330001756438,
 '2024-03-20 03:57:41.265906': -1.9551141678826196,
 '2024-05-23 03:56:59.887152': None,
 '2024-06-08 03:57:00.517767': 3.4569890056601813,
 '2024-06-24 03:57:10.586980900': 5.745592367220591,
 '2024-07-10 03:57:18.475191': 14.307635011137446,
 '2024-07-26 03:57:32.409516': 13.287691904110613,
 '2024-08-11 03:57:32.022361': 18.96637622012427,
 '2024-08-27 03:57:40.041006': 16.95234052700212,
 '2024-09-28 03:57:44.612969': None,
 '2024-10-14 03:57:57.124622900': 4.683795143552516,
 '2024-10-30 03:57:52.209883': None,
 '2024-11-15 03:57:53.658368': -13.15945509859728,
 '2024-12-01 03:58:06.480446900': 2.244951633984357,
 '2024-01-16 03:58:26.569631': -16.524621404246663,
 '2024-02-01 03:58:22.955492': -22.231364051481954,
 '2024-02-17 03:58:23.124568900': None,
 '2024-03-04 03:58:14.381973': -14.602377374277689,
 '2024-03-20 03:58:05.144238900': -6.4252428388947385,
 '2024-05-07 03:57:37.110127': -1.0708094373061101,
 '2024-05-23 03:57:23.765484': None,
 '2024-06-08 03:57:24.400335': 3.947045578831869,
 '2024-06-24 03:57:34.478021': 5.723141942351091,
 '2024-07-10 03:57:42.357759': 12.479570561816923,
 '2024-08-11 03:57:55.904929': 19.405574920941934,
 '2024-08-27 03:58:03.923575': 14.389842320341874,
 '2024-09-12 03:58:11.924927': 11.153076729641002,
 '2024-10-14 03:58:21.011426900': 4.297607939924868,
 '2024-11-15 03:58:17.545172900': -1.2714722029376735,
 '2024-12-01 03:58:30.363016': 1.0605562544948395,
 '2024-01-07 04:04:35.230701': None,
 '2024-01-23 04:04:33.243796': -15.407208563101397,
 '2024-02-08 04:04:36.203930': -14.602533483470168,
 '2024-02-24 04:04:32.510379': None,
 '2024-03-11 04:04:21.234349': -16.6554014984766,
 '2024-03-27 04:04:05.902326900': -4.447627433273652,
 '2024-04-12 04:04:04.197886': -0.40337651050717926,
 '2024-04-28 04:03:54.138528900': -0.21773459237372145,
 '2024-05-14 04:03:39.852605': 2.234290905479217,
 '2024-05-30 04:03:32.285903': 2.66913894464571,
 '2024-06-15 04:03:37.110468': 7.4780001565377345,
 '2024-07-01 04:03:47.227124': 16.205272759370658,
 '2024-07-17 04:04:01.231892900': 15.872233539595774,
 '2024-08-02 04:04:06.097528': None,
 '2024-08-18 04:04:13.293609': 15.750135066876265,
 '2024-09-03 04:04:21.765204': None,
 '2024-09-19 04:04:25.565424': 11.63544955975163,
 '2024-10-05 04:04:22.369543': 8.510122685671144,
 '2024-10-21 04:04:34.652699900': 4.354204757225886,
 '2024-11-22 04:04:38.362555': -4.999143604794852,
 '2024-12-08 04:04:34.020831': None,
 '2024-02-04 03:38:53.721015': -21.643315913041686,
 '2024-02-20 03:38:43.479138': -25.475308887059416,
 '2024-03-07 03:38:46.250536': -17.9320949876342,
 '2024-03-23 03:38:42.380595': -11.767798581378422,
 '2024-04-08 03:38:33.744956': -14.302077653479332,
 '2024-05-26 03:37:59.166587900': 1.736510367499784,
 '2024-06-11 03:37:53.250860900': 3.6854878088723426,
 '2024-06-27 03:37:38.316267': 9.023852793772875,
 '2024-07-13 03:37:59.968033': None,
 '2024-07-29 03:38:02.482214900': 18.13480979004114,
 '2024-08-14 03:38:10.418516': 16.132579386335944,
 '2024-08-30 03:38:21.540936': 15.492847667385163,
 '2024-09-15 03:38:25.110886900': None,
 '2024-10-17 03:38:39.189868900': 3.9794657379420246,
 '2024-11-02 03:38:38.951600': 2.8215580249688266,
 '2024-11-18 03:38:43.936919900': None,
 '2024-02-04 03:39:17.599322': -23.034503017633842,
 '2024-02-20 03:39:07.353209': -22.638148725485557,
 '2024-03-07 03:39:10.124607': -14.536266126392114,
 '2024-03-23 03:39:06.254667': -10.730477446478204,
 '2024-04-08 03:38:57.627499900': -12.830245526251195,
 '2024-04-24 03:38:35.999473': None,
 '2024-05-26 03:38:23.049132': 0.23131372018536758,
 '2024-06-27 03:38:02.190338': -15.509461598524629,
 '2024-07-29 03:38:26.364759': 17.210599648518336,
 '2024-08-14 03:38:34.296823': 16.339335372554174,
 '2024-08-30 03:38:45.427714900': 16.180982722815326,
 '2024-10-01 03:38:54.223932': 2.2316061142315204,
 '2024-10-17 03:39:03.072412': 3.6999982958008215,
 '2024-11-02 03:39:02.834144': -1.6937524264892727,
 '2024-11-18 03:39:07.815227': None,
 '2024-01-19 03:39:38.567065': -15.78088389015335,
 '2024-02-04 03:39:41.477630': -23.929619228036458,
 '2024-02-20 03:39:31.227280': -21.53594111334876,
 '2024-03-07 03:39:34.007151': -11.52134332281755,
 '2024-03-23 03:39:30.128737900': -11.060173348672667,
 '2024-04-08 03:39:21.501570900': -10.211951467350497,
 '2024-04-24 03:38:59.873544': -2.8506021779419703,
 '2024-05-10 03:38:58.929999900': -1.1225188114049045,
 '2024-05-26 03:38:46.927439900': 0.08239221521196652,
 '2024-06-27 03:38:26.072881': 7.229578212919496,
 '2024-07-29 03:38:50.247301900': 18.256526890913026,
 '2024-08-14 03:38:58.183602900': 17.346942368629644,
 '2024-08-30 03:39:09.310258': 16.107630722871086,
 '2024-10-01 03:39:18.102239': 0.21157602454409644,
 '2024-10-17 03:39:26.950720': 4.377414011935241,
 '2024-11-02 03:39:26.712451': None,
 '2024-11-18 03:39:31.697770': -7.12344093538351,
 '2024-01-19 03:40:02.445372': -14.025564295280612,
 '2024-02-04 03:40:05.364409': -21.038236336215625,
 '2024-02-20 03:39:55.109823900': -21.745550433655634,
 '2024-03-07 03:39:57.881223': -9.30901445134867,
 '2024-03-23 03:39:54.002810': -10.29807799792484,
 '2024-04-08 03:39:45.375641900': None,
 '2024-04-24 03:39:23.747615': -0.5442913124288401,
 '2024-05-10 03:39:22.808308': -1.7852218554440054,
 '2024-05-26 03:39:10.801512': -0.41597289815249255,
 '2024-06-27 03:38:49.946954': 9.687396806703338,
 '2024-07-29 03:39:14.129846': 19.891685803336877,
 '2024-08-14 03:39:22.066146': None,
 '2024-08-30 03:39:33.188566': 4.526204188395748,
 '2024-09-15 03:39:36.766989900': None,
 '2024-10-01 03:39:41.984782900': None,
 '2024-10-17 03:39:50.833264': 5.687095920302356,
 '2024-12-04 03:39:59.054448': -17.580273547939875,
 '2024-02-27 03:44:53.570254': -21.3936538794159,
 '2024-03-30 03:44:46.495267': -10.948199494028511,
 '2024-05-01 03:44:22.326795': -1.3572754229403106,
 '2024-05-17 03:44:13.873179': None,
 '2024-06-02 03:44:08.034014900': None,
 '2024-06-18 03:43:55.353246900': 3.621599298080647,
 '2024-07-04 03:44:00.907998': 13.993035650753548,
 '2024-07-20 03:44:12.961297': 16.080254912368822,
 '2024-08-05 03:44:16.564481': 17.13527103240484,
 '2024-08-21 03:44:28.898124900': 13.292879746765106,
 '2024-09-06 03:44:26.795609': 13.539447188689739,
 '2024-09-22 03:44:31.890373900': 4.8034600124400475,
 '2024-10-24 03:44:55.686728': 0.5887675088111108,
 '2024-02-27 03:45:17.444325': -21.901333790317157,
 '2024-03-14 03:45:19.182354900': -7.003582465153254,
 '2024-03-30 03:45:10.369339': -10.948768608600819,
 '2024-05-01 03:44:46.200866900': -1.1565246570856464,
 '2024-05-17 03:44:37.747251': -1.157479741501897,
 '2024-06-18 03:44:19.235790': 1.2577593217535237,
 '2024-07-04 03:44:24.790540900': 7.519481256674099,
 '2024-07-20 03:44:36.843840': 8.902890848363992,
 '2024-08-05 03:44:40.447024': 17.970143818164164,
 '2024-08-21 03:44:52.780668': 10.95594477919261,
 '2024-09-06 03:44:50.673916': 14.115514938204063,
 '2024-09-22 03:44:55.772917': 5.494049405402733,
 '2024-10-08 03:45:03.037061': 6.260850825701762,
 '2024-10-24 03:45:19.577742900': 2.6892990951440034,
 '2024-11-09 03:45:14.506483': -2.688361563266377,
 '2024-01-26 03:45:47.620110': -14.67802246031901,
 '2024-02-27 03:45:41.322632': None,
 '2024-03-14 03:45:43.060662': -5.274569500657545,
 '2024-03-30 03:45:34.247646': -9.43571367235417,
 '2024-04-15 03:45:26.950037': -0.6463762935927374,
 '2024-05-01 03:45:10.074937900': -2.660167303730755,
 '2024-05-17 03:45:01.625558': -1.1246397519209037,
 '2024-06-02 03:44:55.786395': -0.5719763604131244,
 '2024-06-18 03:44:43.109862': 4.650067732886574,
 '2024-07-04 03:44:48.668848': 9.03290755790269,
 '2024-07-20 03:45:00.726384': 11.456095463824736,
 '2024-08-05 03:45:04.329567': 18.755820533310708,
 '2024-08-21 03:45:16.667448': None,
 '2024-09-06 03:45:14.556460': 13.858976467751262,
 '2024-09-22 03:45:19.659696900': 8.312767893236147,
 '2024-10-08 03:45:26.919606': 5.454018119283654,
 '2024-10-24 03:45:43.460286': 3.566485557026833,
 '2024-11-09 03:45:38.393263': None,
 '2024-11-25 03:45:49.301049': -12.282907684980959,
 '2024-01-10 03:46:13.302753': -12.349775591186994,
 '2024-01-26 03:46:11.502653': -15.32659283674065,
 '2024-02-27 03:46:05.205176': None,
 '2024-03-14 03:46:06.934734900': -6.212976792735054,
 '2024-03-30 03:45:58.121717': -10.046073387386555,
 '2024-04-15 03:45:50.828344': -0.4931915192208914,
 '2024-05-01 03:45:33.957482': -11.273424488546576,
 '2024-05-17 03:45:25.499629': 0.26417138605657897,
 '2024-06-02 03:45:19.664701900': -0.698994616548286,
 '2024-06-18 03:45:06.983933900': 5.834417278826492,
 '2024-07-04 03:45:12.559864': 13.848038377430308,
 '2024-07-20 03:45:24.613163': 11.701760804930245,
 '2024-08-05 03:45:28.220583': 17.53765559681097,
 '2024-09-06 03:45:38.447475': 13.379042359366151,
 '2024-09-22 03:45:43.542240': 10.154990746005977,
 '2024-10-08 03:45:50.802148900': 4.075808045835154,
 '2024-10-24 03:46:07.342829900': 1.6117044721642921,
 '2024-11-09 03:46:02.271570': None,
 '2024-11-25 03:46:13.183592': None,
 '2024-12-11 03:46:03.170228': -0.8727163872028884,
 '2024-02-02 03:51:38.704144': -18.451283571439557,
 '2024-02-18 03:51:31.817192': -22.274198040731687,
 '2024-03-05 03:51:31.597873900': -14.496175281316734,
 '2024-03-21 03:51:29.057933': -6.511594756264319,
 '2024-04-22 03:51:01.818931': -1.5168334543356727,
 '2024-05-08 03:50:57.003621': 0.9266498969396136,
 '2024-05-24 03:50:46.483523': 3.0217252569213313,
 '2024-06-09 03:50:40.440850': 1.8818250884880963,
 '2024-06-25 03:50:23.278151': 6.3554517643477055,
 '2024-07-11 03:50:43.892051900': 13.960970717871515,
 '2024-07-27 03:50:46.185823': 17.64618321277025,
 '2024-08-12 03:50:55.452284': 19.100243960724445,
 '2024-08-28 03:51:07.316000900': 16.72695354500782,
 '2024-09-13 03:51:10.343624900': 12.389880261396911,
 '2024-09-29 03:51:14.095698': None,
 '2024-10-31 03:51:26.874185': -1.529675218714771,
 '2024-11-16 03:51:26.496577': 2.22624862151199,
 '2024-01-17 03:52:00.997825': -10.702294555543562,
 '2024-02-18 03:51:55.695498900': -24.945448753750917,
 '2024-03-05 03:51:55.476181': -12.357274166038817,
 '2024-03-21 03:51:52.936242': -1.5555211144797911,
 '2024-04-22 03:51:25.693002900': -0.5215387244437828,
 '2024-05-08 03:51:20.877692900': 1.1346338859502225,
 '2024-05-24 03:51:10.366066': 2.5909783134185256,
 '2024-06-09 03:51:04.310685': 2.0165166499123113,
 '2024-06-25 03:50:47.152222': 8.198808121835917,
 '2024-07-11 03:51:07.770359': 16.996645109171563,
 '2024-07-27 03:51:10.068367': 15.987796524827377,
 '2024-08-12 03:51:19.339064': 17.053124917335726,
 '2024-08-28 03:51:31.194308': 16.353142730390374,
 '2024-09-13 03:51:34.226169': 12.191104757539062,
 '2024-09-29 03:51:37.978243': 1.5573809483898882,
 '2024-10-15 03:51:45.644919': 5.4628855219519705,
 '2024-10-31 03:51:50.752492': None,
 '2024-11-16 03:51:50.379120': 0.6084750033139439,
 '2024-12-02 03:51:58.779704': 1.153872792839593,
 '2024-01-17 03:52:24.880368': -15.972430411631228,
 '2024-02-02 03:52:26.460758': -19.899514012012535,
 '2024-02-18 03:52:19.573805900': -25.757446474646564,
 '2024-03-05 03:52:19.358725': -13.999128273403539,
 '2024-03-21 03:52:16.818785': -1.355941391480679,
 '2024-04-22 03:51:49.567073900': -0.7213685350661532,
 '2024-05-08 03:51:44.760236': 1.08533054711041,
 '2024-05-24 03:51:34.240137900': 3.101865090479399,
 '2024-06-09 03:51:28.193228': 4.300437190253915,
 '2024-06-25 03:51:11.030529': 8.80494518968951,
 '2024-07-11 03:51:31.657138': 13.903998289559436,
 '2024-07-27 03:51:33.955147': 15.188802809582308,
 '2024-08-12 03:51:43.217371': 15.105948128001968,
 '2024-08-28 03:51:55.081087900': 14.287540730634719,
 '2024-09-13 03:51:58.104476': 9.03818224364565,
 '2024-09-29 03:52:01.860786': None,
 '2024-10-15 03:52:09.531698900': 5.437924295757467,
 '2024-10-31 03:52:14.643508': None,
 '2024-11-16 03:52:14.265900': -2.219956085948996,
 '2024-12-02 03:52:22.662246900': -1.0750804713424007,
 '2024-01-24 03:58:07.322855900': -16.043355196527596,
 '2024-02-09 03:58:13.804546': -14.102107025074908,
 '2024-02-25 03:58:01.334536': -15.927588177904596,
 '2024-03-12 03:58:05.461607': -7.013530504649243,
 '2024-03-28 03:57:56.593443': -9.007479077904874,
 '2024-04-13 03:57:50.511716': None,
 '2024-04-29 03:57:32.467495': -0.6535205040080247,
 '2024-05-15 03:57:26.106279': None,
 '2024-05-31 03:57:17.229957900': 3.1373369709216967,
 '2024-06-16 03:57:07.624566': 5.914106378726045,
 '2024-07-02 03:57:07.185393': 8.636981770665317,
 '2024-07-18 03:57:22.390157900': 14.087861021068422,
 '2024-08-03 03:57:25.612067': 14.777404508051669,
 '2024-08-19 03:57:35.958977900': 17.991336417266325,
 '2024-09-04 03:57:37.880669': None,
 '2024-09-20 03:57:43.106774': 10.888151507494541,
 '2024-10-06 03:57:50.586815': 9.362699705624216,
 '2024-10-22 03:58:04.200496': None,
 '2024-11-07 03:58:01.103177': 4.098125245538436,
 '2024-01-08 03:58:33.784940': -10.508576918500152,
 '2024-01-24 03:58:31.201164': -16.551525820438687,
 '2024-02-09 03:58:37.678617': -14.551231657745864,
 '2024-02-25 03:58:25.212842900': -21.364913788924536,
 '2024-03-12 03:58:29.339914': -6.120973031411977,
 '2024-03-28 03:58:20.471750': -7.220936244299829,
 '2024-04-29 03:57:56.345803': -3.1536434911390923,
 '2024-05-15 03:57:49.980351': -0.3464808549303585,
 '2024-05-31 03:57:41.108265': 4.300726337094356,
 '2024-06-16 03:57:31.502872900': 7.310869963494005,
 '2024-07-02 03:57:31.067937': 12.508081410557361,
 '2024-07-18 03:57:46.268465': 15.731846248110529,
 '2024-08-03 03:57:49.490374': 11.628291600248609,
 '2024-08-19 03:57:59.845757': 18.427708323364687,
 '2024-09-04 03:58:01.763213': 9.46841008242247,
 '2024-09-20 03:58:06.989318': 10.384888883039979,
 '2024-10-06 03:58:14.473594': 6.772715179265955,
 '2024-10-22 03:58:28.083040': 3.4466587034368117,
 '2024-11-07 03:58:24.985721': 3.276201030068682,
 '2024-12-09 03:58:23.804365': -2.002085051177192,
 '2024-01-15 04:04:46.883311': None,
 '2024-01-31 04:04:47.302985': None,
 '2024-02-16 04:04:43.635342': -20.610025727130633,
 '2024-03-03 04:04:40.446543': -14.736370800704195,
 '2024-03-19 04:04:39.122201': -2.259490677038629,
 '2024-04-04 04:04:30.152367900': -4.441774858942446,
 '2024-04-20 04:04:15.704219900': None,
 '2024-05-06 04:04:06.339489900': 1.05871044390556,
 '2024-05-22 04:03:57.187407': None,
 '2024-06-07 04:03:51.043154': 4.438986919021157,
 '2024-06-23 04:03:31.766742900': 7.479473549466729,
 '2024-07-09 04:03:51.342806900': 12.986798200161044,
 '2024-07-25 04:03:54.729378900': 3.3819015418901057,
 '2024-08-10 04:04:02.970777': 17.43043280104714,
 '2024-08-26 04:04:16.291672': None,
 '2024-09-11 04:04:18.361766900': -0.43724242819280335,
 '2024-10-13 04:04:28.823139': None,
 '2024-10-29 04:04:38.611524': -7.812347148738695,
 '2024-11-14 04:04:32.485525': -1.5386377907815916,
'2025-01-29 03:38:38.158157900': -22.69257025278951,
 '2025-02-14 03:38:34.556797': None,
 '2025-03-02 03:38:26.145587': -25.2241073885202,
 '2025-03-18 03:38:18.085606': -20.10484725398851,
 '2025-04-03 03:38:14.656055900': -3.7530039677593194,
 '2025-04-19 03:38:04.276009': -8.837141991576301,
 '2025-01-29 03:39:02.036490': -19.346795431339782,
 '2025-02-14 03:38:58.443601': None,
 '2025-03-02 03:38:50.023919': -18.847302587922695,
 '2025-03-18 03:38:41.968174900': -11.945517116727558,
 '2025-04-03 03:38:38.530153': -4.470334994367721,
 '2025-04-19 03:38:28.158577': -8.964933156810115,
 '2025-01-29 03:39:25.914822900': -18.787859763117435,
 '2025-03-02 03:39:13.898016': -15.399688929079147,
 '2025-03-18 03:39:05.842271': -5.782651852112389,
 '2025-04-03 03:39:02.412721': None,
 '2025-04-19 03:38:52.032674': -9.826516278738666,
 '2025-01-29 03:39:49.801627': -16.21970274984916,
 '2025-03-02 03:39:37.784820': -14.037172760998638,
 '2025-03-18 03:39:29.724839': -3.597505312554706,
 '2025-04-03 03:39:26.286818': None,
 '2025-04-19 03:39:15.906770': -2.2152529224701043,
 '2025-02-05 03:44:50.096423': -21.736726717314323,
 '2025-02-21 03:44:42.162583': -20.166724586601564,
 '2025-03-09 03:44:36.979157': None,
 '2025-03-25 03:44:26.538802': -7.729367820222767,
 '2025-04-10 03:44:21.135511': -7.572361476617985,
 '2025-02-05 03:45:13.974754900': -19.958550568099596,
 '2025-02-21 03:45:06.040916': -18.011828888765216,
 '2025-03-25 03:44:50.417133900': -9.451355839289167,
 '2025-04-10 03:44:45.018079': -3.6413652604974773,
 '2025-01-20 03:45:36.112996': -4.509667945588248,
 '2025-02-21 03:45:29.919248': -16.381401275406777,
 '2025-03-25 03:45:14.299703': None,
 '2025-04-10 03:45:08.892176': None,
 '2025-01-04 03:46:00.529376': None,
 '2025-01-20 03:45:59.991328': -4.939363894545746,
 '2025-02-21 03:45:53.797580': -15.152743839336955,
 '2025-03-09 03:45:48.614154900': None,
 '2025-03-25 03:45:38.178035': None,
 '2025-01-27 03:51:22.220413': -29.855307330339485,
 '2025-02-28 03:51:10.842872': -14.946985755303396,
 '2025-04-01 03:51:00.353024': None,
 '2025-04-17 03:50:51.917240': 0.0434480063833318,
 '2025-01-27 03:51:46.098745': -25.519573298905012,
 '2025-02-12 03:51:44.471300': -10.389842073870165,
 '2025-02-28 03:51:34.716969': -13.451942414794729,
 '2025-03-16 03:51:30.291495': -4.363215834230804,
 '2025-04-01 03:51:24.231356': None,
 '2025-04-17 03:51:15.787100': -0.20788400119096434,
 '2025-01-11 03:52:13.967757': -5.109362407699226,
 '2025-01-27 03:52:09.981313900': -11.938762266754033,
 '2025-02-12 03:52:08.349632': -12.35224408808502,
 '2025-02-28 03:51:58.599537': -14.406853209205387,
 '2025-03-16 03:51:54.174064': -4.220922404156831,
 '2025-04-01 03:51:48.105453': -9.022187371564353,
 '2025-04-17 03:51:39.669669': 0.05438221282187079,
 '2025-01-18 03:57:59.219646900': -6.930225807078713,
 '2025-02-03 03:57:59.803638900': -20.919394435597688,
 '2025-02-19 03:57:53.127389': -17.68750772876324,
 '2025-03-07 03:57:46.783358900': -6.625077513030736,
 '2025-03-23 03:57:36.499617': -3.1103616346964853,
 '2025-04-08 03:57:32.045163': -2.109433642334531,
 '2025-01-18 03:58:23.102214900': -6.773346212613465,
 '2025-02-03 03:58:23.686206900': -20.736766649011408,
 '2025-02-19 03:58:17.001485': -14.851527023139024,
 '2025-03-07 03:58:10.665926900': -7.290484843017567,
 '2025-03-23 03:58:00.373713': -3.9258390303830386,
 '2025-04-08 03:57:55.927732': -2.98392543692771,
 '2025-01-09 04:04:35.236030': 1.0897548091357723,
 '2025-01-25 04:04:29.703493': None,
 '2025-02-10 04:04:31.697829': None,
 '2025-02-26 04:04:20.151348': -17.223637736601162,
 '2025-03-14 04:04:18.013307': -7.3798916444293585,
 '2025-03-30 04:04:09.525998': -4.328532546467077,
 '2025-04-15 04:04:02.962469': -5.290084645334292,
 '2025-02-06 03:38:41.614395': -21.917046956484278,
 '2025-02-22 03:38:44.093886': -19.29416705684538,
 '2025-03-10 03:38:38.728562': -9.547097932976795,
 '2025-03-26 03:38:27.738663': -15.129970706492308,
 '2025-04-11 03:38:27.762852': -4.391934743343753,
 '2025-01-21 03:39:11.362206900': -7.699090568769711,
 '2025-02-06 03:39:05.492703': -19.0928991041505,
 '2025-02-22 03:39:07.967958': -16.900669289939934,
 '2025-03-10 03:39:02.602633': -10.636579854358965,
 '2025-03-26 03:38:51.616971': -8.40161792218574,
 '2025-04-11 03:38:51.645395': -3.4588336317301267,
 '2025-01-21 03:39:35.240514': -5.206624516182372,
 '2025-02-06 03:39:29.379482': -19.562865633018045,
 '2025-02-22 03:39:31.850502': -15.19057942980007,
 '2025-03-10 03:39:26.485177': -6.928042704692308,
 '2025-03-26 03:39:15.495278': -7.890872413797909,
 '2025-04-11 03:39:15.519467': -2.471023454170598,
 '2025-01-05 03:39:52.327858': None,
 '2025-01-21 03:39:59.127293900': -5.448760588305762,
 '2025-02-06 03:39:53.257790': -19.014902519399612,
 '2025-02-22 03:39:55.724573': -15.055089059538528,
 '2025-03-10 03:39:50.359248': -4.712710762199969,
 '2025-03-26 03:39:39.377822': None,
 '2025-04-11 03:39:39.402010': -3.1234257225444955,
 '2025-01-28 03:44:53.447060': -20.064664981844256,
 '2025-03-01 03:44:49.176085': -18.10172119308008,
 '2025-03-17 03:44:43.357648': -6.714977565229188,
 '2025-04-02 03:44:33.948164900': -2.7060754466494834,
 '2025-04-18 03:44:36.153991': -0.6399546262013691,
 '2025-01-28 03:45:17.329603': -17.07403916312794,
 '2025-03-01 03:45:13.058629': -16.59833283535458,
 '2025-03-17 03:45:07.231720': -7.33981717899529,
 '2025-04-02 03:44:57.826473': -2.995949813851212,
 '2025-04-18 03:45:00.032298': -0.8094457581483453,
 '2025-01-28 03:45:41.207911': -16.077873556955804,
 '2025-02-13 03:45:44.428792': -14.216394577448423,
 '2025-03-01 03:45:36.936937': -14.341175873239457,
 '2025-03-17 03:45:31.110027': -4.224706854167792,
 '2025-04-02 03:45:21.700544': -4.731563505175199,
 '2025-04-18 03:45:23.914843': -0.10844532064488657,
 '2025-01-12 03:46:06.559011': -18.828330885397264,
 '2025-01-28 03:46:05.090454': -18.924797440044408,
 '2025-02-13 03:46:08.307099': -14.096022358650938,
 '2025-03-01 03:46:00.811008': -14.010794981426415,
 '2025-03-17 03:45:54.984100': -3.122769154134251,
 '2025-04-02 03:45:45.578851900': -2.4074350439732437,
 '2025-04-18 03:45:47.793149900': 0.7951238659243509,
 '2025-02-04 03:51:27.482201': None,
 '2025-02-20 03:51:30.919037900': -14.53384838636143,
 '2025-03-08 03:51:24.825092': -14.185593555803264,
 '2025-03-24 03:51:13.212361900': -5.27006752211954,
 '2025-04-09 03:51:12.897656': -1.3479698149939896,
 '2025-04-25 03:51:03.865508': None,
 '2025-02-04 03:51:51.360508': -19.37768978704054,
 '2025-02-20 03:51:54.793108900': -14.992743909754616,
 '2025-03-08 03:51:48.703400': -10.708736927324425,
 '2025-03-24 03:51:37.094905': -4.60008781865771,
 '2025-04-09 03:51:36.771727': None,
 '2025-04-25 03:51:27.748052': None,
 '2025-02-04 03:52:15.243050900': -17.28625362803587,
 '2025-02-20 03:52:18.671417': -13.864671058616619,
 '2025-03-08 03:52:12.577471': -10.608137946092164,
 '2025-03-24 03:52:00.968977900': -5.586599088809324,
 '2025-04-09 03:52:00.654271': -6.570600754331859,
 '2025-04-25 03:51:51.626359': None,
 '2025-01-26 03:58:04.853691': -10.918152422894133,
 '2025-02-11 03:58:05.126326': -10.076324040199188,
 '2025-02-27 03:57:58.608740900': -7.11352398245034,
 '2025-03-15 03:57:55.285195900': -8.731292980637946,
 '2025-04-16 03:57:46.831803': 0.7995344657960587,
 '2025-01-10 03:58:26.307847': -16.101366592701172,
 '2025-01-26 03:58:28.736233900': -8.495809283057104,
 '2025-02-11 03:58:29.004632900': -11.261556670807511,
 '2025-02-27 03:58:22.487049': -7.594203305018301,
 '2025-03-15 03:58:19.159266900': -8.948367603996493,
 '2025-03-31 03:58:09.076216': None,
 '2025-04-16 03:58:10.705874': -0.3639130619148062,
 '2025-01-17 04:04:42.072771': 0.44000656913713504,
 '2025-02-02 04:04:36.779321': -15.890321319499963,
 '2025-02-18 04:04:41.101483900': -13.954308507436942,
 '2025-03-06 04:04:34.376336': -10.979416159877593,
 '2025-03-22 04:04:21.823077': None,
 '2025-04-07 04:04:21.372790900': -6.76003390407444,
 '2025-04-23 04:04:15.212662900': 0.5222731027015969}

In [7]:
dates_to_del = []
for date in all_years_landsat:
   if all_years_landsat[date] is None:
      dates_to_del.append(date)
for date in dates_to_del:
   del all_years_landsat[date]

all_years_landsat

{'1986-07-13 03:01:24.924031': 16.27339768998559,
 '1986-07-29 03:00:51.344056': 16.573165869364228,
 '1986-11-02 02:57:32.929000': -13.227672344186969,
 '1986-07-13 03:01:48.785013': 12.091803148713247,
 '1986-07-29 03:01:15.206081': 15.148326739520522,
 '1986-11-02 02:57:56.660081': -18.596356014020028,
 '1986-07-13 03:02:12.788081': 7.4542387361959594,
 '1986-07-29 03:01:39.212006': 13.97481009402597,
 '1986-11-02 02:58:20.821050': -26.57226152621019,
 '1986-11-18 02:57:43.266088': -3.4592302068156173,
 '1986-11-18 02:58:07': 2.5347998448313156,
 '1986-06-02 03:08:55.313094': -3.5995585078764716,
 '1986-06-18 03:08:25.396050': 6.847358187443933,
 '1986-11-09 03:03:27.547094': 0.30937971240183865,
 '1986-06-02 03:09:19.171038': -0.5565752948732023,
 '1986-06-18 03:08:49.255038': 2.6156504669638636,
 '1986-11-09 03:03:51.423019': -2.229586178007629,
 '1986-06-18 03:09:13.257025': 4.177805599950558,
 '1986-11-09 03:04:15.441050': 0.7690070096073587,
 '1986-06-18 03:09:36.973031': 7.274

In [16]:
all_years_landsat_df = pd.DataFrame(columns=["Date", "Image T"])
for (i, (date, ts)) in enumerate(all_years_landsat.items()):
   all_years_landsat_df.loc[i] = [date, ts]

all_years_landsat_df = all_years_landsat_df.sort_values(by=['Date'])
all_years_landsat_df

,Date,Image T
30,1986-04-29 03:23:24.899000,-1.539700
31,1986-05-31 03:22:32.065063,2.058033
11,1986-06-02 03:08:55.313094,-3.599559
14,1986-06-02 03:09:19.171038,-0.556575
26,1986-06-09 03:16:04.867050,-0.163581
...,...,...
9884,2025-04-19 03:38:04.276009,-8.837142
9889,2025-04-19 03:38:28.158577,-8.964933
9893,2025-04-19 03:38:52.032674,-9.826516
9897,2025-04-19 03:39:15.906770,-2.215253


In [17]:
all_years_landsat_df.to_csv('Данные_валидации/landsat_baikal_lst.csv', index=False)